# File: Scrape pitcher data from BaseballSavant
* Matchup histories between all active pitchers & hitters
* Pitch-by-pitch sequencing for relief pitchers in the 2024 regular season

## Imports for selenium web scraper and CSVs

In [2]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Chrome Driver Settings
opts = Options()
opts.add_argument("--headless=new")
service = Service("/path/to/chromedriver")  # Replace with actual path
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

## 2024 bullpen staffs of all 30 MLB teams

In [41]:
staffs_2024 = {108: ['702674', '519326', '681806', '669721', '670046', '690829', '672282', '660761', '656288', '681217', '670871', '682989', '663776', '668968', '668943', '665896', '691951', '656271', '650671', '519043', '608334', '672710', '667755', '666171', '686799', '542881'], 109: ['669459', '674072', '668678', '679885', '694297', '686826', '677944', '593958', '657044', '691441', '573009', '640462', '518876', '656464', '669194', '656756', '543518', '700270', '676714', '623149', '686796'], 110: ['681882', '680694', '572143', '607644', '666974', '621199', '672335', '543056', '642397', '680570', '669203', '694646', '700249', '665152', '669432', '657097', '544150', '621107', '669330', '641329', '669211', '622554'], 111: ['677161', '663767', '682990', '686580', '690544', '601713', '572020', '683068', '670174', '669711', '592155', '445276', '690916', '656557', '676477', '681867', '692285', '455119', '663462', '676710', '678394', '657514', '472610', '608371'], 112: ['640470', '622259', '687863', '680891', '622075', '657240', '684007', '668873', '543294', '668338', '696136', '681799', '668970', '592767', '669713', '676962', '665871', '660600', '681432', '663878', '694037', '605347', '592791', '623474', '624522', '657006', '605155'], 113: ['680689', '668933', '518585', '686730', '664139', '594580', '664747', '571656', '687924', '682227', '608718', '596001', '691172', '666157', '695076', '668984', '458677', '671096', '668881', '607259', '622088', '663574', '641941', '605335'], 114: ['689958', '669212', '682120', '676282', '571510', '671922', '681870', '661403', '676440', '502171', '687362', '658648', '663474', '656529', '671106', '594902', '657265', '669456', '683769', '680704', '668909'], 115: ['676105', '596295', '622608', '656234', '683627', '680767', '664875', '677955', '608566', '607536', '678020', '663567', '687145', '641755', '685299', '656458', '663372', '678821', '615698', '683409', '687134', '641712'], 116: ['669438', '669724', '669373', '681857', '676684', '671345', '676428', '656412', '663947', '656638', '676568', '689225', '674944', '676614', '672456', '666159', '663554', '680744', '628317', '695549'], 117: ['681293', '605463', '650556', '519151', '672391', '666215', '663321', '687911', '681869', '593576', '686613', '623352', '579328', '661527', '669854', '641585', '664351', '657571', '664285', '664299', '434378'], 118: ['663738', '672582', '666142', '608379', '663460', '607625', '674444', '663704', '679525', '670167', '663903', '519293', '640451', '672578', '608717', '547179', '680742', '669674', '668674'], 119: ['523260', '607192', '669165', '676272', '689981', '641401', '660813', '623465', '656427', '676508', '518489', '680736', '681911', '683618', '641703', '689017', '656657', '656629', '808967', '694813', '621111', '675627', '595014', '607455', '543339', '477132'], 120: ['678606', '571882', '663432', '669022', '606930', '640448', '687377', '686747', '676395', '680686', '687792', '686294', '592866', '674285', '663623', '694350', '672851', '571578', '671131', '680730'], 121: ['622098', '657585', '640455', '664208', '656887', '656849', '621242', '679358', '673380', '664192', '681035', '622663', '548384', '676130', '493603', '500779', '656731', '622065', '701643', '592773', '676604', '621112', '664849', '673540', '623211'], 133: ['694361', '669620', '674370', '622072', '669372', '613534', '680880', '621053', '666205', '687765', '519008', '548389', '695243', '671212', '665622', '676664', '518397', '683155', '666204', '671305', '663687'], 134: ['594835', '682847', '656605', '621294', '670059', '683003', '669387', '666214', '670280', '621366', '694973', '676755', '681347', '663559', '693312', '642701', '547973', '676702', '641726', '656222'], 135: ['666745', '663568', '669093', '605397', '670970', '663773', '676680', '689690', '663158', '506433', '593974', '527048', '673513', '592094', '650633', '656302', '669308', '681190', '670810', '656945', '663362'], 136: ['669923', '685116', '608723', '700187', '682243', '646242', '675989', '622491', '662253', '663423', '608638', '676106', '642100', '676092', '666619', '642048', '669302', '554340', '693433', '660825'], 137: ['605483', '690986', '686790', '702352', '666808', '592662', '676254', '681810', '678906', '643511', '669062', '678495', '676775', '694738', '663546', '657277', '573124', '696131', '663941', '806185', '663855'], 138: ['669467', '700241', '458681', '676617', '502043', '668868', '607200', '680572', '667463', '702795', '669461', '688297', '571927', '664854', '681676', '668941', '552640', '700669', '571945', '543243', '681517'], 139: ['656876', '671737', '686993', '664076', '687330', '642121', '682254', '621097', '641793', '605488', '669358', '655889', '663992', '621363', '669270', '686752', '664126', '641302', '682052', '670955', '685801'], 140: ['679156', '592351', '502085', '669391', '605177', '489446', '543135', '594798', '641816', '600917', '670036', '656641', '681982', '677958', '571760', '622250', '642520', '687847', '642016', '453286', '683004', '641540', '570632', '668390', '674003'], 141: ['621114', '687798', '642232', '667297', '643338', '685126', '814005', '605135', '647315', '622253', '684320', '666201', '663893', '687922', '676969', '592332', '605447', '657024', '621244', '650893', '642629', '670102'], 142: ['661395', '680573', '657746', '663978', '641154', '681892', '641927', '650960', '623437', '686973', '660896', '592779', '678692', '573204', '543859', '663485', '687396', '805673', '643377', '680739', '656240', '677976', '701581'], 143: ['592836', '676661', '675650', '663969', '621381', '605400', '624133', '621237', '656457', '656546', '605513', '608032', '689147', '656814', '628708', '686751', '667725', '554430', '650911', '614179', '663465', '621016', '621383'], 144: ['675911', '592426', '680885', '608331', '669276', '450203', '641729', '605154', '693821', '621345', '689266', '572955', '656550', '678061', '700363', '445926', '625643', '519242', '694462', '678226', '628452', '642216', '607481'], 145: ['680732', '694363', '608678', '676979', '681402', '623167', '671111', '660853', '686563', '621051', '647336', '680897', '641582', '672860', '681066', '683568', '605121', '663436', '686642', '605182', '689672', '676760', '673929', '671109'], 146: ['682610', '669622', '669199', '641312', '678368', '687424', '685107', '664350', '676974', '670124', '669111', '666129', '622786', '670766', '665795', '656848', '675540', '666277', '676534', '686539', '677053', '800049', '677960', '657508', '666200'], 147: ['657612', '596133', '641482', '701542', '657376', '547001', '573186', '664776', '607074', '643410', '641835', '592454', '661563', '664123', '642528', '605280', '641656', '690925', '672782', '543037'], 158: ['489119', '682842', '692230', '683232', '657649', '571948', '669060', '607067', '656266', '642207', '676879', '688107', '650644', '664079', '669084', '593423', '665625', '606303', '663542', '643361', '656730', '605452', '668964', '642239', '642547']}

## Retrieve matchup histories between all pitchers & hitters
* Matchup histories between all active pitchers & hitters
* Pitch-by-pitch sequencing for relief pitchers in the 2024 regular season

In [228]:
# BaseballSavant ID numbers of all 30 MLB teams
team_ids = [108,109,110,111,112,113,114,115,116,117,118,119,120,121,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,158]

# Create an empty list to store
data = []

# For all teams and pitchers on those teams:
for team, pitchers in staffs_2024.items():
    # Iterate over all pitchers on the current team:
    for pitcher_id in pitchers:
        # Iterate over all opposing teams:
        for hitting_team in team_ids:
            
            # Construct the URL for the matchup
            url = f'https://baseballsavant.mlb.com/player_matchup?type=batter&teamPitching={team}&teamBatting={hitting_team}&player_id={pitcher_id}'
            
            # Open the webpage
            driver.get(url)
            
            try:
                # Wait for player_table and team_table to load
                #  - BaseballSavant is dynamically loaded so data cannot be obtained from the page source
                #  - Graphics & the quantity of data per page cause long loading times
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'player_table')))
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'team_table')))
                
                # Extract player and team tables
                #  - Targets:
                #      - The pitcher's data table of overall metrics against the team
                #      - The team's data table of individual stats against the pitcher
                player_table = driver.find_element(By.ID, 'player_table').text
                team_table = driver.find_element(By.ID, 'team_table').text
                
                
                pitcher = ""
                lines = player_table.split('\n')
                # Get pitcher name from the first row of the player table
                if len(lines) > 2:
                    first_data_row = lines[2].split()
                    if len(first_data_row) >= 2:
                        pitcher = first_data_row[0] + " " + first_data_row[1]
                
                # Extract headers to determine stat columns
                headers = team_table.split('\n')[1].split()
                
                # All team tables contain the same metrics, but not all players have sufficient data
                expected_length = len(headers)
                
                # Process matchup data of all hitters in the team's table row-by-row
                for line in team_table.split('\n')[1:]:
                    stats = line.split()
                    
                    # Column alignment is off for players with 3 names, so skip AB
                    #  - We can still retrieve the number of ABs in this scenario 
                    if stats[2] == 'AB':
                        continue

                    # If a player is missing any of the expected stats in the team table, we must skip them
                    #  - BaseballSavant does not fill in the missing stats with placeholders such as 'NaN'
                    #  - This throws off the alignment of all metrics which would skew the data & leave empty fields
                    if len(stats) < expected_length:
                        print(f"Skipping player with insufficient stats: {stats[0]} {stats[1]}")
                        continue


                    # Create temporary dictionary to define and manage all metrics for the current player
                    stat_dict = {
                        'Pitcher Name': pitcher,
                        'Pitcher ID': pitcher_id,
                        'Batting Team': hitting_team,
                        'Player Name': stats[0] + " " + stats[1],
                        'PA': stats[2],
                        'AB': stats[3],
                        'H': stats[4],
                        '2B': stats[5],
                        '3B': stats[6],
                        'HR': stats[7],
                        'SO': stats[8],
                        'K%': stats[9],
                        'Whiff%': stats[10] if len(stats) > 10 else 'N/A',
                        'BB': stats[11] if len(stats) > 11 else 'N/A',
                        'BB%': stats[12] if len(stats) > 12 else 'N/A',
                        'BA': stats[13] if len(stats) > 13 else 'N/A',
                        'SLG': stats[14] if len(stats) > 14 else 'N/A',
                        'wOBA': stats[15] if len(stats) > 15 else 'N/A',
                        'xBA': stats[16] if len(stats) > 16 else 'N/A',
                        'xSLG': stats[17] if len(stats) > 17 else 'N/A',
                        'xwOBA': stats[18] if len(stats) > 18 else 'N/A',
                        'EV': stats[19] if len(stats) > 19 else 'N/A',
                        'LA': stats[20] if len(stats) > 20 else 'N/A'
                    }

                    # Append the player's data
                    data.append(stat_dict)

                print(f"Data scraped for pitcher {pitcher_id} vs. Team {hitting_team}")
            
            except Exception as e:
                print(f"Failed to scrape data for pitcher {pitcher_id} vs. Team {hitting_team}: {e}")

# After obtaining all matchups between every active pitcher and every active hitter,
# convert the array to a dataframe
df = pd.DataFrame(data)

# Save the DataFrame to a CSV
df.to_csv('pitcher_matchups.csv', index=False)

print("Data scraping complete. Data saved to 'pitcher_matchups.csv'.")

# Close the web driver
driver.quit()

Failed to scrape data for pitcher 702674 vs. Team 144: Message: 

Data scraped for pitcher 702674 vs. Team 145
Failed to scrape data for pitcher 702674 vs. Team 146: Message: 

Failed to scrape data for pitcher 702674 vs. Team 147: Message: 

Failed to scrape data for pitcher 702674 vs. Team 158: Message: 

Skipping player with insufficient stats: Williams, Luke
Data scraped for pitcher 519326 vs. Team 144
Skipping player with insufficient stats: Sosa, Lenyn
Skipping player with insufficient stats: Lee, Korey
Skipping player with insufficient stats: Robert Jr.,
Data scraped for pitcher 519326 vs. Team 145
Data scraped for pitcher 519326 vs. Team 146
Skipping player with insufficient stats: Grisham, Trent
Skipping player with insufficient stats: Chisholm Jr.,
Data scraped for pitcher 519326 vs. Team 147
Data scraped for pitcher 519326 vs. Team 158
Skipping player with insufficient stats: Riley, Austin
Data scraped for pitcher 681806 vs. Team 144
Data scraped for pitcher 681806 vs. Team 

Data scraped for pitcher 650671 vs. Team 146
Skipping player with insufficient stats: Volpe, Anthony
Data scraped for pitcher 650671 vs. Team 147
Skipping player with insufficient stats: Yelich, Christian
Skipping player with insufficient stats: Haase, Eric
Data scraped for pitcher 650671 vs. Team 158
Skipping player with insufficient stats: Murphy, Sean
Skipping player with insufficient stats: De La
Data scraped for pitcher 519043 vs. Team 144
Skipping player with insufficient stats: Thaiss, Matt
Skipping player with insufficient stats: Robert Jr.,
Data scraped for pitcher 519043 vs. Team 145
Skipping player with insufficient stats: Bride, Jonah
Data scraped for pitcher 519043 vs. Team 146
Skipping player with insufficient stats: Peraza, Oswald
Skipping player with insufficient stats: Weaver, Luke
Data scraped for pitcher 519043 vs. Team 147
Skipping player with insufficient stats: Ortiz, Joey
Data scraped for pitcher 519043 vs. Team 158
Data scraped for pitcher 608334 vs. Team 144
Sk

Data scraped for pitcher 640462 vs. Team 144
Skipping player with insufficient stats: Robert Jr.,
Skipping player with insufficient stats: Tauchman, Mike
Skipping player with insufficient stats: Lee, Korey
Data scraped for pitcher 640462 vs. Team 145
Skipping player with insufficient stats: Mervis, Matt
Data scraped for pitcher 640462 vs. Team 146
Skipping player with insufficient stats: Cabrera, Oswaldo
Data scraped for pitcher 640462 vs. Team 147
Skipping player with insufficient stats: Hoskins, Rhys
Skipping player with insufficient stats: Collins, Isaac
Data scraped for pitcher 640462 vs. Team 158
Data scraped for pitcher 518876 vs. Team 144
Data scraped for pitcher 518876 vs. Team 145
Data scraped for pitcher 518876 vs. Team 146
Data scraped for pitcher 518876 vs. Team 147
Data scraped for pitcher 518876 vs. Team 158
Skipping player with insufficient stats: Harris II,
Data scraped for pitcher 656464 vs. Team 144
Skipping player with insufficient stats: Slater, Austin
Skipping play

Skipping player with insufficient stats: LeMahieu, DJ
Skipping player with insufficient stats: Stanton, Giancarlo
Skipping player with insufficient stats: Bellinger, Cody
Data scraped for pitcher 543056 vs. Team 147
Skipping player with insufficient stats: Ortiz, Joey
Skipping player with insufficient stats: Haase, Eric
Data scraped for pitcher 543056 vs. Team 158
Skipping player with insufficient stats: White, Eli
Skipping player with insufficient stats: Arcia, Orlando
Data scraped for pitcher 642397 vs. Team 144
Skipping player with insufficient stats: Fletcher, Dominic
Skipping player with insufficient stats: Slater, Austin
Skipping player with insufficient stats: Colás, Oscar
Data scraped for pitcher 642397 vs. Team 145
Skipping player with insufficient stats: Myers, Dane
Data scraped for pitcher 642397 vs. Team 146
Skipping player with insufficient stats: Judge, Aaron
Data scraped for pitcher 642397 vs. Team 147
Skipping player with insufficient stats: Turang, Brice
Data scraped f

Skipping player with insufficient stats: Bride, Jonah
Data scraped for pitcher 686580 vs. Team 146
Skipping player with insufficient stats: Grisham, Trent
Skipping player with insufficient stats: Cabrera, Oswaldo
Data scraped for pitcher 686580 vs. Team 147
Failed to scrape data for pitcher 686580 vs. Team 158: Message: 

Skipping player with insufficient stats: De La
Data scraped for pitcher 690544 vs. Team 144
Data scraped for pitcher 690544 vs. Team 145
Data scraped for pitcher 690544 vs. Team 146
Skipping player with insufficient stats: Volpe, Anthony
Skipping player with insufficient stats: Judge, Aaron
Skipping player with insufficient stats: Stanton, Giancarlo
Data scraped for pitcher 690544 vs. Team 147
Failed to scrape data for pitcher 690544 vs. Team 158: Message: 

Data scraped for pitcher 601713 vs. Team 144
Skipping player with insufficient stats: Thaiss, Matt
Skipping player with insufficient stats: Pérez, Martín
Data scraped for pitcher 601713 vs. Team 145
Skipping playe

...

Data scraped for pitcher 676979 vs. Team 144
Data scraped for pitcher 676979 vs. Team 145
Skipping player with insufficient stats: Hill, Derek
Data scraped for pitcher 676979 vs. Team 146
Skipping player with insufficient stats: Chisholm Jr.,
Skipping player with insufficient stats: Stanton, Giancarlo
Data scraped for pitcher 676979 vs. Team 147
Data scraped for pitcher 676979 vs. Team 158
Skipping player with insufficient stats: Olson, Matt
Skipping player with insufficient stats: Harris II,
Data scraped for pitcher 681402 vs. Team 144
Skipping player with insufficient stats: Vargas, Miguel
Skipping player with insufficient stats: Thaiss, Matt
Data scraped for pitcher 681402 vs. Team 145
Skipping player with insufficient stats: Myers, Dane
Data scraped for pitcher 681402 vs. Team 146
Data scraped for pitcher 681402 vs. Team 147
Skipping player with insufficient stats: Yelich, Christian
Skipping player with insufficient stats: Contreras, William
Data scraped for pitcher 681402 vs. Team

Skipping player with insufficient stats: Yelich, Christian
Skipping player with insufficient stats: Perkins, Blake
Skipping player with insufficient stats: Frelick, Sal
Data scraped for pitcher 673929 vs. Team 158
Failed to scrape data for pitcher 671109 vs. Team 144: Message: 

Failed to scrape data for pitcher 671109 vs. Team 145: Message: 

Data scraped for pitcher 671109 vs. Team 146
Failed to scrape data for pitcher 671109 vs. Team 147: Message: 

Failed to scrape data for pitcher 671109 vs. Team 158: Message: 

Data scraped for pitcher 682610 vs. Team 144
Data scraped for pitcher 682610 vs. Team 145
Data scraped for pitcher 682610 vs. Team 146
Data scraped for pitcher 682610 vs. Team 147
Failed to scrape data for pitcher 682610 vs. Team 158: Message: 

Data scraped for pitcher 669622 vs. Team 144
Skipping player with insufficient stats: Vaughn, Andrew
Data scraped for pitcher 669622 vs. Team 145
Failed to scrape data for pitcher 669622 vs. Team 146: Message: 

Skipping player wit

Skipping player with insufficient stats: Perkins, Blake
Skipping player with insufficient stats: Yelich, Christian
Data scraped for pitcher 675540 vs. Team 158
Data scraped for pitcher 666277 vs. Team 144
Data scraped for pitcher 666277 vs. Team 145
Failed to scrape data for pitcher 666277 vs. Team 146: Message: 

Skipping player with insufficient stats: Judge, Aaron
Skipping player with insufficient stats: Cabrera, Oswaldo
Data scraped for pitcher 666277 vs. Team 147
Data scraped for pitcher 666277 vs. Team 158
Skipping player with insufficient stats: Profar, Jurickson
Data scraped for pitcher 676534 vs. Team 144
Skipping player with insufficient stats: Vargas, Miguel
Skipping player with insufficient stats: Sosa, Lenyn
Data scraped for pitcher 676534 vs. Team 145
Skipping player with insufficient stats: Myers, Dane
Data scraped for pitcher 676534 vs. Team 146
Skipping player with insufficient stats: Judge, Aaron
Skipping player with insufficient stats: Goldschmidt, Paul
Skipping play

Skipping player with insufficient stats: Fletcher, Dominic
Skipping player with insufficient stats: Colás, Oscar
Skipping player with insufficient stats: Thaiss, Matt
Data scraped for pitcher 641835 vs. Team 145
Data scraped for pitcher 641835 vs. Team 146
Skipping player with insufficient stats: Grisham, Trent
Skipping player with insufficient stats: Chisholm Jr.,
Data scraped for pitcher 641835 vs. Team 147
Data scraped for pitcher 641835 vs. Team 158
Skipping player with insufficient stats: De La
Skipping player with insufficient stats: Murphy, Sean
Skipping player with insufficient stats: Albies, Ozzie
Skipping player with insufficient stats: Arcia, Orlando
Skipping player with insufficient stats: Riley, Austin
Data scraped for pitcher 592454 vs. Team 144
Skipping player with insufficient stats: Vaughn, Andrew
Skipping player with insufficient stats: Tauchman, Mike
Skipping player with insufficient stats: Vargas, Miguel
Data scraped for pitcher 592454 vs. Team 145
Failed to scrape 

Data scraped for pitcher 607067 vs. Team 146
Data scraped for pitcher 607067 vs. Team 147
Data scraped for pitcher 607067 vs. Team 158
Data scraped for pitcher 656266 vs. Team 144
Data scraped for pitcher 656266 vs. Team 145
Data scraped for pitcher 656266 vs. Team 146
Data scraped for pitcher 656266 vs. Team 147
Failed to scrape data for pitcher 656266 vs. Team 158: Message: 

Skipping player with insufficient stats: Acuña Jr.,
Skipping player with insufficient stats: Arcia, Orlando
Skipping player with insufficient stats: Albies, Ozzie
Skipping player with insufficient stats: Harris II,
Data scraped for pitcher 642207 vs. Team 144
Skipping player with insufficient stats: Rojas, Josh
Skipping player with insufficient stats: Vaughn, Andrew
Skipping player with insufficient stats: Vargas, Miguel
Skipping player with insufficient stats: Slater, Austin
Skipping player with insufficient stats: Robert Jr.,
Data scraped for pitcher 642207 vs. Team 145
Skipping player with insufficient stats:

# To retrieve pitch-by-pitch data from BaseballSavant, we need the following:
* Relief pitcher IDs
* ID of their debut game
* IDs of all games they appeared in during the 2024 regular season
* IDs of all plate appearances across all games during the 2024 regular season

# 210 total relievers 
* 7 from each team chosen randomly
* All faced a minimum of 150 batters
* All debuted before 2024
* Mix of pitching styles & handedness

In [ ]:
bullpens_2024 = {143: [621237, 621381, 608032, 605513, 614179, 592836, 656546],
144: [621345, 628452, 641729, 572955, 605154, 669276, 592426],
146: [675540, 670124, 677053, 676534, 622786, 664350, 670766],
120: [592866, 606930, 640448, 663432, 676395, 678606, 674285],
121: [621242, 623211, 592773, 493603, 664208, 676130, 657585],
158: [605452, 571948, 657649, 683232, 606303, 663542, 665625],
142: [543859, 663485, 643377, 686973, 660896, 661395, 681892],
145: [660853, 681402, 605121, 663436, 694363, 676760, 621051],
112: [669713, 592767, 668338, 663878, 696136, 681799, 605347],
138: [668941, 664854, 552640, 669461, 667463, 681517, 571927],
118: [519293, 640451, 670167, 672582, 663704, 668674, 608717],
119: [595014, 623465, 676272, 641703, 543339, 607455, 621111],
141: [650893, 647315, 643338, 663893, 657024, 622253, 621114],
147: [605280, 643410, 641835, 592454, 657612, 596133, 664776],
111: [677161, 657514, 445276, 608371, 669711, 472610, 455119],
113: [641941, 594580, 458677, 664747, 596001, 608718, 668933],
139: [656876, 605488, 663992, 655889, 670955, 664126, 664076],
108: [660761, 519043, 656271, 519326, 690829, 669721, 672710],
133: [674370, 695243, 519008, 671212, 622072, 613534, 666205],
137: [676254, 643511, 663855, 666808, 573124, 676775, 663546],
135: [663158, 670970, 592094, 656945, 593974, 669093, 605397],
109: [674072, 640462, 656464, 657044, 686826, 573009, 679885],
115: [680767, 641755, 656234, 607536, 664875, 615698, 663567],
136: [660825, 608638, 554340, 642048, 676092, 662253, 608723],
114: [668909, 571510, 669212, 680704, 682120, 661403, 683769],
140: [681982, 502085, 656641, 489446, 605177, 600917, 670036],
117: [681869, 661527, 519151, 657571, 593576, 623352, 605463],
110: [669211, 657097, 672335, 572143, 642397, 666974, 622554],
116: [676684, 671345, 669724, 666159, 656412, 689225, 663947],
134: [656222, 666214, 642701, 547973, 670280, 693312, 670059]}

## Get IDs of the relievers' debut games
* This is needed in order to properly load the 3D pitch tracker pages for all games that relievers appeared in

In [22]:
# Store the ID for pitchers' debut games: { pitcher_id : debut game ID }
first_games = {}

# For all teams and relievers on those teams:
for team_id, relievers in bullpens_2024.items():
    print(f"Team ID: {team_id}")
    
    # Iterate over all relievers on the current team:
    for reliever_id in relievers:
        print(f"Reliever: {reliever_id}")
        
        # Use BaseballSavant's 3D pitch tracker to obtain debut game ID
        #  - We will use this page to obtain all pitch-by-pitch data later
        url = f'https://baseballsavant.mlb.com/visuals/pitch3d?player_id={reliever_id}#v=1&mainView=tracking&pov=umpire&hitterSide=all&marks=none&plays1=all&plays2=all&pitchColors=pitch_type&dualActive=false&summaryViewActive=false'
        driver.get(url)

        # Wait for the date selection drop down to appear
        try:
            select_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.NAME, 'date_sel'))
            )

            # The debut game will be under the first optgroup
            optgroups = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, "//select[@name='date_sel']/optgroup"))
            )

            # Get the first optgroup & look for all options
            optgroup = optgroups[0]
            options = optgroup.find_elements(By.TAG_NAME, 'option')

            # The debut game is the very first value available
            first_value = options[0].get_attribute('value')
            print(f"The first value under the first optgroup is: {first_value}")
            first_games[reliever_id] = first_value

        except Exception as e:
            print(f"Error finding debut game for {reliever_id}: {e}")
            continue
            
# Close the browser
driver.quit()

Team ID: 143
Reliever: 621237
The first value under the first optgroup is: 490507
Reliever: 621381
The first value under the first optgroup is: 490103
Reliever: 608032
The first value under the first optgroup is: 490101
Reliever: 605513
The first value under the first optgroup is: 531608
Reliever: 614179
The first value under the first optgroup is: 491586
Reliever: 592836
The first value under the first optgroup is: 490132
Reliever: 656546
The first value under the first optgroup is: 490585
Team ID: 144
Reliever: 621345
The first value under the first optgroup is: 491992
Reliever: 628452
The first value under the first optgroup is: 490107
Reliever: 641729
The first value under the first optgroup is: 490235
Reliever: 572955
The first value under the first optgroup is: 490726
Reliever: 605154
The first value under the first optgroup is: 490856
Reliever: 669276
The first value under the first optgroup is: 632277
Reliever: 592426
The first value under the first optgroup is: 490292
Team ID:

The first value under the first optgroup is: 634290
Reliever: 664126
The first value under the first optgroup is: 566798
Reliever: 664076
The first value under the first optgroup is: 631117
Team ID: 108
Reliever: 660761
The first value under the first optgroup is: 566402
Reliever: 519043
The first value under the first optgroup is: 490132
Reliever: 656271
The first value under the first optgroup is: 566831
Reliever: 519326
The first value under the first optgroup is: 490110
Reliever: 690829
The first value under the first optgroup is: 717971
Reliever: 669721
The first value under the first optgroup is: 716684
Reliever: 672710
The first value under the first optgroup is: 632287
Team ID: 133
Reliever: 674370
The first value under the first optgroup is: 717763
Reliever: 695243
The first value under the first optgroup is: 718506
Reliever: 519008
The first value under the first optgroup is: 490435
Reliever: 671212
The first value under the first optgroup is: 716538
Reliever: 622072
The firs

# Retrieve IDs of all 2024 regular season games that relievers appeared in

In [31]:
# Store IDs of all games for all relievers in 2024 : { pitcher_id : [list of game_ids] }
games = {}

# The following loop iterates the same as the cell above, but retrieving both IDs
# at once was not possible due to slow loading times

# For all teams and relievers on those teams:
for team_id, relievers in bullpens_2024.items():
    print(f"Team ID: {team_id}")  
    # Iterate over all relievers on the current team:
    for reliever_id in relievers:
        print(f"Reliever: {reliever_id}")
        
        # Use the same link as the previous cell
        url = f'https://baseballsavant.mlb.com/visuals/pitch3d?player_id={reliever_id}#v=1&mainView=tracking&pov=umpire&hitterSide=all&marks=none&plays1=all&plays2=all&pitchColors=pitch_type&dualActive=false&summaryViewActive=false'
        driver.get(url)

        # Wait for the optgroup with label "2024"
        try:
            optgroup = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//optgroup[@label='2024']"))
            )
        except:
            print(f"Game data not found for reliever {reliever_id}. Skipping...")
            continue
        
        # Find all options inside the optgroup
        options = optgroup.find_elements(By.TAG_NAME, 'option')

        # Extract game IDs from 2024
        for option in options:
            # Look for the ID number
            value = option.get_attribute('value')
            match = re.search(r'\d+', value)
            
            # If we find one, store it for the current reliever
            if match:
                game_id = match.group()
                if reliever_id not in games:
                    games[reliever_id] = []  
                games[reliever_id].append(game_id)

# Close the browser
driver.quit()

# Print the final dictionary
print("Collected game data:", games)

Team ID: 143
Reliever: 621237
Reliever: 621381
Reliever: 608032
Reliever: 605513
Reliever: 614179
Reliever: 592836
Reliever: 656546
Team ID: 144
Reliever: 621345
Reliever: 628452
Reliever: 641729
Reliever: 572955
Reliever: 605154
Reliever: 669276
Reliever: 592426
Team ID: 146
Reliever: 675540
Reliever: 670124
Reliever: 677053
Reliever: 676534
Reliever: 622786
Reliever: 664350
Reliever: 670766
Team ID: 120
Reliever: 592866
Reliever: 606930
Reliever: 640448
Reliever: 663432
Reliever: 676395
Reliever: 678606
Reliever: 674285
Team ID: 121
Reliever: 621242
Reliever: 623211
Reliever: 592773
Reliever: 493603
Reliever: 664208
Reliever: 676130
Reliever: 657585
Team ID: 158
Reliever: 605452
Reliever: 571948
Reliever: 657649
Reliever: 683232
Reliever: 606303
Reliever: 663542
Reliever: 665625
Team ID: 142
Reliever: 543859
Reliever: 663485
Reliever: 643377
Reliever: 686973
Reliever: 660896
Reliever: 661395
Reliever: 681892
Team ID: 145
Reliever: 660853
Reliever: 681402
Reliever: 605121
Reliever: 66

## Get IDs of all at-bats in all games relievers have appeared in

* Now that we have all the games the relievers appeared in and their debut game, we can obtain the at-bat IDs from all their appearances
* The URLs for the 3D pitch tracker pages require the previously obtained IDs

In [34]:
# Store IDs of all at bats in all games relievers appeared in : { (pitcher_id, game_id) : [ all at bat IDs ] }
at_bats = {}

# Reload the page in case of an error
def restart_driver():
    global driver  # Ensure we are using the same driver instance
    driver.quit()
    time.sleep(3)  # Let ChromeDriver fully close
    driver = webdriver.Chrome(service=service, options=options)
    
# For all relievers and games they appeared in:
for reliever, appearances in games.items():
    # Iterate over every game: 
    for game_id in appearances:
        try:
            print(reliever, game_id, first_games[reliever])
            url = f'https://baseballsavant.mlb.com/visuals/pitch3d?player_id={reliever}#v=1&mainView=tracking&pov=umpire&g1={game_id}&g2={first_games[reliever]}&hitterSide=all&marks=none&plays1=all&plays2=all&pitchColors=pitch_type&dualActive=false&summaryViewActive=false&chartYear=2024&summaryYear=2024'
            
            driver.get(url)
            driver.refresh()

            # Find the 'Pitches by PA' section of the dropdown menu
            #  - This contains the PA ID numbers we need for pitch sequencing data
            optgroup = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//optgroup[@label='Pitches By PA']"))
            )
            
            # Iterate over all PA IDs and store them for the current reliever
            options2 = optgroup.find_elements(By.TAG_NAME, 'option')
            for option in options2:
                pa = option.get_attribute('value')
                at_bats.setdefault((reliever, game_id), []).append(pa)

        except Exception as e:
            print(f"Error processing reliever {reliever} in game {game_id}: {e}")
            restart_driver()  # Restart ChromeDriver if a crash occurs

driver.quit()

621237 745604 490507
621237 745602 490507
621237 745600 490507
621237 744874 490507
621237 744873 490507
621237 745194 490507
621237 745594 490507
621237 745597 490507
621237 745591 490507
621237 745590 490507
621237 746725 490507
621237 745428 490507
621237 746239 490507
621237 746238 490507
621237 745589 490507
621237 745587 490507
621237 746071 490507
621237 746068 490507
621237 745826 490507
621237 745580 490507
621237 745579 490507
621237 745578 490507
621237 745573 490507
621237 746558 490507
621237 745337 490507
621237 745574 490507
621237 745572 490507
621237 745569 490507
621237 745570 490507
621237 745571 490507
621237 746951 490507
621237 747025 490507
621237 745567 490507
621237 745566 490507
621237 746456 490507
621237 745559 490507
621237 746858 490507
621237 747097 490507
621237 745556 490507
621237 745558 490507
621237 745549 490507
621237 745481 490507
621237 745470 490507
621237 745554 490507
621237 745550 490507
621237 745546 490507
621237 745224 490507
621237 745225

628452 747131 490107
628452 747125 490107
628452 745261 490107
628452 746152 490107
628452 747126 490107
628452 745827 490107
628452 745828 490107
628452 747122 490107
628452 747121 490107
628452 746874 490107
628452 747115 490107
628452 747116 490107
628452 747110 490107
628452 744844 490107
628452 744849 490107
628452 747026 490107
628452 747112 490107
628452 747109 490107
628452 747107 490107
628452 745727 490107
628452 745157 490107
628452 747104 490107
628452 747100 490107
628452 747099 490107
628452 747181 490107
628452 745385 490107
628452 747093 490107
628452 745794 490107
628452 745793 490107
628452 745958 490107
628452 745960 490107
628452 747092 490107
628452 747081 490107
628452 746520 490107
628452 745300 490107
628452 745302 490107
628452 746186 490107
628452 747083 490107
628452 747082 490107
628452 747080 490107
628452 745864 490107
628452 745866 490107
628452 745536 490107
628452 747079 490107
628452 747075 490107
628452 747074 490107
628452 747072 490107
628452 747069

677053 746067 661357
677053 746064 661357
677053 746066 661357
677053 747199 661357
677053 745412 661357
677053 746062 661357
677053 746056 661357
677053 746053 661357
677053 746054 661357
677053 745815 661357
677053 744846 661357
677053 746055 661357
677053 746049 661357
677053 746052 661357
677053 746051 661357
677053 746291 661357
677053 745559 661357
677053 745555 661357
677053 746045 661357
677053 746043 661357
677053 746039 661357
677053 746367 661357
677053 746687 661357
677053 746040 661357
677053 746041 661357
677053 746036 661357
677053 746035 661357
677053 745961 661357
677053 745061 661357
677053 747089 661357
677053 747087 661357
677053 746031 661357
677053 746030 661357
677053 746025 661357
677053 745548 661357
677053 745786 661357
677053 746029 661357
677053 746028 661357
676534 746082 663438
676534 746081 663438
676534 746893 663438
676534 746894 663438
676534 747132 663438
676534 746078 663438
676534 746074 663438
676534 746070 663438
676534 745667 663438
676534 746071

676395 744819 717417
676395 744816 717417
676395 747004 717417
676395 747000 717417
676395 745542 717417
676395 744814 717417
676395 744815 717417
676395 747080 717417
676395 747077 717417
676395 744810 717417
676395 744808 717417
676395 746020 717417
676395 745455 717417
676395 745456 717417
676395 744805 717417
676395 744803 717417
676395 744801 717417
676395 746823 717417
676395 746824 717417
676395 744795 717417
676395 744797 717417
678606 744829 717543
678606 744828 717543
678606 744827 717543
678606 745150 717543
678606 747172 717543
678606 744825 717543
678606 744821 717543
678606 744822 717543
678606 744820 717543
678606 744816 717543
678606 747004 717543
678606 745539 717543
678606 744814 717543
678606 747080 717543
678606 747077 717543
678606 744813 717543
678606 744811 717543
678606 744809 717543
678606 744806 717543
678606 746020 717543
678606 746019 717543
678606 745456 717543
678606 744804 717543
678606 744800 717543
678606 744801 717543
678606 745773 717543
678606 746823

676130 744888 661602
676130 745773 661602
676130 745767 661602
676130 745930 661602
657585 745843 566865
657585 747142 566865
657585 745838 566865
657585 745837 566865
657585 746160 566865
657585 746156 566865
657585 745346 566865
657585 745834 566865
657585 745831 566865
657585 745830 566865
657585 745100 566865
657585 745103 566865
657585 745186 566865
657585 745825 566865
657585 745580 566865
657585 746067 566865
657585 746064 566865
657585 746634 566865
657585 745822 566865
657585 745818 566865
657585 745819 566865
657585 745820 566865
657585 745813 566865
657585 744850 566865
657585 745571 566865
657585 745811 566865
657585 745805 566865
657585 745004 566865
657585 746862 566865
657585 745806 566865
657585 745804 566865
657585 745801 566865
657585 744840 566865
657585 745476 566865
657585 745477 566865
657585 745803 566865
657585 745219 566865
657585 745783 566865
657585 745786 566865
657585 745379 566865
657585 747162 566865
657585 747161 566865
657585 746749 566865
657585 746750

...

502085 745109 490186
502085 745114 490186
502085 745031 490186
502085 745030 490186
502085 745024 490186
502085 746408 490186
502085 746488 490186
502085 746487 490186
502085 747133 490186
502085 745028 490186
502085 745022 490186
502085 745021 490186
502085 746314 490186
502085 745669 490186
502085 745659 490186
502085 746554 490186
502085 745015 490186
502085 745013 490186
502085 745901 490186
502085 745017 490186
502085 745011 490186
502085 746061 490186
502085 745007 490186
502085 745008 490186
502085 746137 490186
502085 746136 490186
502085 745004 490186
502085 745006 490186
502085 744997 490186
502085 745975 490186
502085 747018 490186
502085 744992 490186
502085 744995 490186
502085 746208 490186
502085 746206 490186
502085 746369 490186
502085 744991 490186
502085 744988 490186
502085 744984 490186
502085 745147 490186
502085 744987 490186
502085 744982 490186
502085 744981 490186
502085 745708 490186
502085 746921 490186
502085 746919 490186
502085 744977 490186
502085 744975

661527 746338 632424
661527 746334 632424
519151 746418 490125
519151 746410 490125
519151 746411 490125
519151 745031 490125
519151 745032 490125
519151 746326 490125
519151 746408 490125
519151 746401 490125
519151 744872 490125
519151 744865 490125
519151 746889 490125
519151 746561 490125
519151 746398 490125
519151 746399 490125
519151 746402 490125
519151 745748 490125
519151 746393 490125
519151 746396 490125
519151 746390 490125
519151 746385 490125
519151 745663 490125
519151 745258 490125
519151 745249 490125
519151 746383 490125
519151 746380 490125
519151 746223 490125
519151 746216 490125
519151 745327 490125
519151 746377 490125
519151 746781 490125
519151 746782 490125
519151 746376 490125
519151 746372 490125
519151 745801 490125
519151 745802 490125
519151 744912 490125
519151 745883 490125
519151 746367 490125
519151 746371 490125
519151 745228 490125
519151 745229 490125
519151 746361 490125
519151 746360 490125
519151 746363 490125
519151 746354 490125
519151 744982

657097 747056 567167
657097 747049 567167
657097 746316 567167
657097 746242 567167
657097 746244 567167
657097 747050 567167
657097 747045 567167
657097 747047 567167
657097 747044 567167
657097 744859 567167
657097 744858 567167
657097 747043 567167
657097 747040 567167
657097 747038 567167
657097 747037 567167
657097 747035 567167
657097 745180 567167
657097 746792 567167
657097 747034 567167
657097 747032 567167
657097 747029 567167
657097 744931 567167
657097 744927 567167
657097 745083 567167
657097 747027 567167
657097 747026 567167
657097 747025 567167
657097 747023 567167
657097 745732 567167
657097 746375 567167
657097 747021 567167
657097 747017 567167
657097 747018 567167
657097 745234 567167
657097 745639 567167
657097 745637 567167
657097 747015 567167
657097 747013 567167
657097 747009 567167
657097 744994 567167
657097 746035 567167
657097 746037 567167
657097 747008 567167
657097 747039 567167
657097 746607 567167
657097 746605 567167
657097 746419 567167
657097 746988

676684 746462 634625
676684 746381 634625
676684 747108 634625
676684 747107 634625
676684 746459 634625
676684 746457 634625
676684 746456 634625
676684 746214 634625
676684 746211 634625
676684 745886 634625
676684 746693 634625
676684 746691 634625
676684 746451 634625
676684 746450 634625
676684 746447 634625
676684 744911 634625
676684 744909 634625
676684 746614 634625
676684 746448 634625
676684 746444 634625
676684 746441 634625
676684 746440 634625
676684 746439 634625
676684 745221 634625
676684 745226 634625
676684 745303 634625
676684 746436 634625
676684 746431 634625
676684 746758 634625
676684 746754 634625
676684 746432 634625
676684 746426 634625
676684 745370 634625
676684 745611 634625
676684 746425 634625
676684 746419 634625
676684 746260 634625
676684 746254 634625
676684 746983 634625
676684 746982 634625
676684 746420 634625
676684 746417 634625
671345 746821 633774
671345 746815 633774
671345 745842 633774
671345 745844 633774
671345 746497 633774
671345 745517

666214 746837 662873
666214 746836 662873
666214 746756 662873
666214 746432 662873
666214 745451 662873
666214 745446 662873
666214 745447 662873
666214 745121 662873
666214 746661 662873
666214 745441 662873
666214 745692 662873
666214 745686 662873
642701 745764 530259
642701 745765 530259
642701 745763 530259
642701 746658 530259
642701 744951 530259
642701 745758 530259
642701 745759 530259
642701 745754 530259
642701 745755 530259
642701 746000 530259
642701 747046 530259
642701 745756 530259
642701 745751 530259
642701 745748 530259
642701 745096 530259
642701 745744 530259
642701 745742 530259
642701 745415 530259
642701 745413 530259
642701 745330 530259
642701 745740 530259
642701 745737 530259
642701 745735 530259
642701 745168 530259
642701 746547 530259
642701 745488 530259
642701 746696 530259
642701 745482 530259
642701 745484 530259
642701 745476 530259
642701 745968 530259
642701 746766 530259
642701 746769 530259
642701 745481 530259
642701 745470 530259
642701 747171

In [37]:
at_bats_copy = {(621237, '745604'): ['58', '59', '60', '61', '62', '63', '64'], (621237, '745602'): ['67', '68', '69'], (621237, '745600'): ['63', '64', '65', '66'], (621237, '744874'): ['73', '74', '75', '76'], (621237, '744873'): ['68', '69', '70', '71'], (621237, '745194'): ['58', '59', '60', '61'], (621237, '745594'): ['64', '65', '66'], (621237, '745597'): ['56'], (621237, '745591'): ['77', '78', '79', '80'], (621237, '745590'): ['83'], (621237, '746725'): ['75', '76', '77', '78', '79'], (621237, '745428'): ['76', '77', '78'], (621237, '746239'): ['63', '64', '65', '66'], (621237, '746238'): ['62', '63', '64', '65', '66'], (621237, '745589'): ['71', '72', '73', '74'], (621237, '745587'): ['71', '72', '73', '74'], (621237, '746071'): ['79', '80', '81'], (621237, '746068'): ['59', '60', '61'], (621237, '745826'): ['85', '86', '87'], (621237, '745580'): ['81', '82', '83', '84', '85'], (621237, '745579'): ['58', '59'], (621237, '745578'): ['67', '68', '69', '70', '71'], (621237, '745573'): ['62', '63', '64', '65'], (621237, '746558'): ['67', '68', '69', '70', '71'], (621237, '745337'): ['56', '57', '58', '59', '60'], (621237, '745574'): ['55', '56', '57'], (621237, '745572'): ['71', '72', '73'], (621237, '745569'): ['62', '63', '64'], (621237, '745570'): ['59', '60', '61', '62'], (621237, '745571'): ['69', '70', '71', '72', '73', '74', '75'], (621237, '746951'): ['64', '65', '66', '67'], (621237, '747025'): ['66', '67', '68'], (621237, '745567'): ['65', '66', '67'], (621237, '745566'): ['58', '59', '60'], (621237, '746456'): ['70', '71', '72', '73', '74'], (621237, '745559'): ['59', '60', '61'], (621237, '746858'): ['73', '74', '75', '76'], (621237, '747097'): ['69', '70', '71', '72', '73', '74'], (621237, '745556'): ['59', '60', '61', '62'], (621237, '745558'): ['73', '74', '75'], (621237, '745549'): ['77', '78', '79', '80', '81', '82', '83'], (621237, '745481'): ['81', '82', '83', '84', '85'], (621237, '745470'): ['65', '66', '67', '68'], (621237, '745554'): ['63', '64', '65'], (621237, '745550'): ['53', '54', '55', '56'], (621237, '745546'): ['50'], (621237, '745224'): ['58', '59', '60', '61'], (621237, '745225'): ['68', '69', '70', '71'], (621237, '746115'): ['59', '60', '61'], (621237, '747169'): ['70', '71', '72', '73', '74'], (621237, '747166'): ['66', '67', '68', '69', '70', '71'], (621237, '745544'): ['69', '70', '71', '72', '73'], (621237, '745539'): ['55', '56', '57', '58'], (621237, '747083'): ['60', '61', '62', '63', '64', '65'], (621237, '746268'): ['60', '61', '62', '63', '64'], (621237, '746263'): ['74', '75', '76', '77'], (621237, '744891'): ['34', '35', '36', '37'], (621237, '746016'): ['65', '66', '67', '68'], (621237, '746017'): ['47', '48', '49'], (621237, '745535'): ['66', '67', '68'], (621237, '745525'): ['58', '59', '60'], (621237, '745939'): ['47', '48', '49', '50', '51', '52'], (621237, '745769'): ['65', '66', '67', '68', '69'], (621237, '745768'): ['58'], (621237, '744797'): ['45', '46', '47'], (621237, '744798'): ['51', '52', '53'], (621381, '745604'): ['48', '49', '50', '51'], (621381, '745602'): ['46', '47', '48', '49'], (621381, '745600'): ['50', '51', '52'], (621381, '744874'): ['50', '57', '58', '59'], (621381, '744871'): ['55', '56', '57', '58'], (621381, '745196'): ['57', '58', '59'], (621381, '745594'): ['41', '42', '43', '47', '48', '49'], (621381, '745592'): ['54', '55', '56', '57', '58'], (621381, '746722'): ['61', '62', '63'], (621381, '746725'): ['57', '58', '59'], (621381, '746239'): ['55', '56', '57'], (621381, '746238'): ['47', '48', '49'], (621381, '745589'): ['42', '43', '44'], (621381, '745585'): ['61', '62', '63', '64'], (621381, '745584'): ['45', '46'], (621381, '746069'): ['57', '58', '59', '60'], (621381, '746068'): ['67', '68', '69'], (621381, '745580'): ['26', '27', '32', '33', '34'], (621381, '745581'): ['53', '54', '55', '56'], (621381, '745578'): ['60'], (621381, '745576'): ['44', '48', '49', '50'], (621381, '745338'): ['70', '71', '72', '73'], (621381, '745337'): ['74', '75'], (621381, '745574'): ['48', '49'], (621381, '745572'): ['47', '48', '49'], (621381, '745568'): ['64', '65', '66', '67', '68'], (621381, '745571'): ['48', '52', '53', '54', '55'], (621381, '746951'): ['57', '58', '59'], (621381, '747025'): ['58', '59', '60', '61'], (621381, '745563'): ['60', '61', '62'], (621381, '745562'): ['67', '68', '73', '74', '75'], (621381, '746456'): ['47', '48', '49', '50'], (621381, '745565'): ['56', '57', '58', '59'], (621381, '745559'): ['53', '54'], (621381, '746858'): ['55', '56', '57', '58'], (621381, '747098'): ['35', '36', '37'], (621381, '745556'): ['54', '55'], (621381, '745558'): ['54', '55', '56', '57'], (621381, '745551'): ['55', '56'], (621381, '745478'): ['58', '59', '60', '61', '62', '63'], (621381, '745878'): ['68', '69', '70'], (621381, '745877'): ['65', '66', '67'], (621381, '745550'): ['69', '70', '71'], (621381, '745546'): ['54', '55', '56', '57', '58', '59'], (621381, '745225'): ['61', '62', '63', '64'], (621381, '746120'): ['59', '60', '61'], (621381, '746115'): ['66', '67', '68', '69'], (621381, '747168'): ['57', '58', '59'], (621381, '745544'): ['52', '53', '54'], (621381, '745542'): ['56', '60', '61'], (621381, '745539'): ['63', '64', '65', '66', '67', '68'], (621381, '747084'): ['52', '53', '54'], (621381, '746263'): ['63', '64', '65'], (621381, '745538'): ['74', '75', '76'], (621381, '745531'): ['65', '66', '67', '68'], (621381, '745536'): ['60', '61', '62', '63', '64', '65'], (621381, '744891'): ['91', '92', '93', '94'], (621381, '746021'): ['72', '73', '74'], (621381, '745535'): ['58', '59', '60'], (621381, '745525'): ['52', '53', '54'], (621381, '745935'): ['60', '61', '62', '63'], (621381, '745770'): ['65', '66', '67'], (621381, '745767'): ['60', '61', '62'], (621381, '745527'): ['57', '58', '59'], (621381, '744797'): ['51', '52', '53', '54'], (621381, '744798'): ['65', '66', '67', '68'], (608032, '747062'): ['67', '68', '69'], (608032, '746085'): ['77', '78', '79'], (608032, '746248'): ['65', '66', '67'], (608032, '746247'): ['67', '68', '69'], (608032, '745106'): ['75', '76', '77'], (608032, '745108'): ['62', '63', '64', '65', '66', '67'], (608032, '746246'): ['72', '73', '74'], (608032, '746236'): ['73', '74', '75', '76', '77'], (608032, '746239'): ['62', '67', '68', '69', '70'], (608032, '745508'): ['75', '76', '77'], (608032, '745507'): ['70', '71', '72'], (608032, '746240'): ['63', '64', '65', '66', '67', '68'], (608032, '746231'): ['77', '78', '79', '80'], (608032, '745013'): ['74', '75', '76', '81', '82', '83'], (608032, '746390'): ['76', '77', '78', '79', '80', '81'], (608032, '746385'): ['84', '85'], (608032, '746229'): ['68', '69', '70'], (608032, '746224'): ['61', '62', '63'], (608032, '746222'): ['66', '67', '68'], (608032, '746216'): ['80', '81', '82'], (608032, '745326'): ['85', '86', '87'], (608032, '745329'): ['73', '74', '75'], (608032, '746215'): ['66', '67', '68'], (608032, '746131'): ['74', '75', '76'], (608032, '746213'): ['77', '78', '79', '80'], (608032, '746214'): ['64', '65', '66'], (608032, '746211'): ['72', '73', '74', '75'], (608032, '746855'): ['58', '59', '60', '61'], (608032, '746205'): ['73', '74', '75'], (608032, '746207'): ['69', '70', '71', '72'], (608032, '746202'): ['65', '66', '67'], (608032, '745633'): ['83', '84', '85', '86'], (608032, '745232'): ['68', '69', '70'], (608032, '745231'): ['69', '70', '71'], (608032, '745546'): ['71', '72', '73'], (608032, '745543'): ['71', '72', '73'], (608032, '745225'): ['76', '77', '78', '84', '85', '86', '87'], (608032, '746120'): ['76', '77', '78', '79'], (608032, '747169'): ['80', '81', '82', '83', '84'], (608032, '745548'): ['66', '67', '68', '69'], (608032, '745544'): ['78', '79', '80'], (608032, '745542'): ['68', '69', '70', '71', '72', '73'], (608032, '747084'): ['65', '66', '67', '68', '69'], (608032, '746263'): ['82', '83', '84'], (608032, '745532'): ['62', '63', '64', '65'], (608032, '745536'): ['70', '71', '72', '76', '77', '78'], (608032, '744887'): ['68', '69', '70', '71'], (608032, '745534'): ['60', '61', '62', '63', '64'], (608032, '745525'): ['65', '66', '67'], (608032, '745528'): ['70', '71', '72', '73'], (608032, '745933'): ['63', '65', '66', '67'], (608032, '745527'): ['73', '74', '75', '76', '77'], (608032, '745521'): ['74', '75', '76', '77', '78'], (605513, '745598'): ['1', '2', '3', '4', '9', '10', '11', '16', '17', '18', '19', '20', '24', '25', '26', '31', '32', '33', '34'], (605513, '745194'): ['6', '7', '8', '12', '13', '14', '15', '16', '21', '22', '23', '24', '28', '29', '30', '38', '39', '40', '41', '45', '46', '47'], (605513, '745594'): ['1', '2', '3', '4', '5', '12', '13', '14', '20', '21', '22', '23', '24', '29', '30', '31', '32', '38', '39', '40'], (605513, '745592'): ['1', '2', '3', '10', '11', '12', '13', '17', '18', '19', '26', '27', '28', '33', '34', '35', '40', '41', '42', '46', '47', '48', '49', '50'], (605513, '746725'): ['5', '6', '7', '8', '14', '15', '16', '17', '23', '24', '25', '26', '27', '32', '33', '34', '35', '36', '42', '43', '44'], (605513, '746239'): ['6', '7', '8', '15', '16', '17', '18', '19', '23', '24', '25', '26', '31', '32', '33', '37', '38', '39', '40', '44', '45'], (605513, '745586'): ['61', '62', '63', '70', '71', '72'], (605513, '746068'): ['51', '52', '53', '54', '55'], (605513, '745582'): ['47', '48', '49', '56', '57', '58', '59', '68', '69', '70', '71', '72', '73', '74'], (605513, '745577'): ['74', '75', '76', '77', '78', '79', '80'], (605513, '746556'): ['76', '77', '78', '79', '80'], (605513, '745333'): ['75', '76', '77', '78', '79'], (605513, '745575'): ['29', '30', '31', '32', '36', '37', '38', '42', '43', '44'], (605513, '746948'): ['53', '54', '55', '56', '57', '64', '65', '66', '67'], (605513, '747023'): ['50', '51', '56', '57', '58', '64', '65', '66', '67', '68'], (605513, '745562'): ['36', '37', '38', '39', '43', '44', '45', '46', '47', '48', '53', '54', '55', '56'], (605513, '746456'): ['7', '8', '9', '10', '16', '17', '18', '22', '23', '24'], (614179, '745583'): ['67', '68', '69', '70', '77', '78', '79', '80', '81'], (614179, '745584'): ['69', '70', '71'], (614179, '746069'): ['76', '77', '78', '79', '80'], (614179, '745826'): ['67', '68', '69'], (614179, '745580'): ['67', '68', '69', '75', '76', '77'], (614179, '745576'): ['76', '77', '78'], (614179, '746558'): ['78', '79', '80'], (614179, '746556'): ['62', '63', '64', '65', '66'], (614179, '745338'): ['60', '61', '62', '63', '64', '65', '66'], (614179, '745575'): ['18', '19', '20', '21', '22'], (614179, '745568'): ['48', '49', '50', '51'], (614179, '745814'): ['74', '75', '76', '77', '78'], (614179, '745571'): ['76'], (614179, '746948'): ['45', '46', '47', '48', '49'], (614179, '746944'): ['72', '73', '74', '75'], (614179, '747024'): ['58', '62', '63', '64', '65', '66', '67', '68', '69'], (614179, '745567'): ['71', '72', '73'], (614179, '745561'): ['70', '71', '72', '73'], (614179, '745564'): ['72', '73', '74'], (614179, '746456'): ['41', '42', '43'], (614179, '745555'): ['48', '49', '55', '56', '57'], (614179, '746860'): ['67', '68', '69'], (614179, '747099'): ['49', '50', '51'], (614179, '747098'): ['22', '26', '27', '28', '29'], (614179, '745557'): ['76', '77', '78', '79', '80'], (614179, '745552'): ['60', '61', '62', '63', '64'], (614179, '745551'): ['60', '61', '62', '63', '64', '65'], (614179, '745478'): ['47', '48'], (614179, '745470'): ['74', '75', '76'], (614179, '745554'): ['54', '55', '56', '57'], (614179, '745546'): ['64', '65', '66', '67'], (614179, '745543'): ['46', '47'], (614179, '745224'): ['22', '23', '27', '28', '29'], (614179, '745222'): ['67', '68', '69', '70'], (614179, '747164'): ['53', '54', '55'], (614179, '745548'): ['36', '37', '38', '39'], (614179, '745544'): ['46', '47'], (614179, '747082'): ['61', '62', '63'], (614179, '746271'): ['55', '56', '57', '58', '59'], (614179, '746263'): ['53', '54', '55', '56'], (614179, '745540'): ['59', '60', '61', '62', '63'], (614179, '745533'): ['36', '37', '38', '39'], (614179, '744891'): ['71', '72', '73', '74'], (614179, '746021'): ['49', '50', '51'], (614179, '746018'): ['47'], (614179, '745529'): ['35', '36', '37'], (614179, '745530'): ['61', '62', '63'], (614179, '745769'): ['38', '39', '40', '41', '42'], (614179, '745768'): ['44', '45', '46', '47', '48'], (614179, '745521'): ['43', '51', '52', '53', '54'], (614179, '744797'): ['58', '59', '60', '61', '62'], (614179, '744798'): ['72', '73', '74', '75', '76', '77'], (592836, '745428'): ['6', '7', '8', '9', '15', '16', '17', '21', '22', '23', '24', '25', '26', '32', '33', '34', '38', '39', '40', '47', '48', '49', '50', '56', '57', '58', '59'], (592836, '745587'): ['1', '2', '3', '4', '5', '9', '10', '11', '18', '19', '20', '21', '30', '31', '32', '38', '39', '40', '41', '46', '47', '48', '53', '54', '55'], (592836, '746069'): ['4', '5', '6', '7', '11', '12', '13', '14', '19', '20', '21', '25', '26', '27', '28', '29', '34', '35', '36', '37', '38', '39', '48', '49', '50', '51'], (592836, '745580'): ['1', '2', '3', '4', '8', '9', '10', '11', '12', '17', '18', '19', '23', '24', '25'], (592836, '745576'): ['1', '2', '3', '4', '5', '6', '10', '11', '12', '13', '20', '21', '22', '23', '24', '29', '30', '31', '39', '40', '41', '42', '43'], (592836, '745338'): ['5', '6', '7', '11', '12', '13', '14', '15', '16', '17', '18', '25', '26', '27', '28', '36', '37', '38', '42', '43', '44', '45', '46', '47', '51', '52', '53', '54', '55'], (592836, '745572'): ['1', '2', '3', '4', '5', '10', '11', '12', '13', '20', '21', '22', '23', '24', '25', '29', '30', '31', '32', '36', '37', '38', '39'], (592836, '745571'): ['1', '2', '3', '9', '10', '11', '15', '16', '17', '18', '23', '24', '25', '34', '35', '36', '41', '42', '43', '44'], (592836, '747024'): ['6', '7', '8', '13', '14', '15', '16', '17', '21', '22', '23', '24', '25', '29', '30', '31', '32', '37', '38', '39', '45', '46', '47', '48', '49'], (592836, '745562'): ['1', '2', '3', '7', '8', '9', '10', '11', '16', '17', '18', '19', '20', '21', '22', '28', '29', '30'], (592836, '745548'): ['1', '2', '3', '4', '5', '6', '7', '12', '13', '14', '18', '19', '20', '21', '22', '27', '28', '29', '30'], (592836, '745539'): ['1', '2', '3', '4', '11', '12', '13', '18', '19', '20', '21', '22', '28', '29', '30', '31', '32', '33', '37', '38', '39'], (592836, '746268'): ['4', '5', '6', '7', '8', '12', '13', '14', '15', '21', '22', '23', '24', '25', '26', '27', '28', '29'], (592836, '745541'): ['1', '2', '3', '4', '5', '9', '10', '11', '17', '18', '19', '20', '21', '25', '26', '27', '28', '29', '30', '31', '32', '33', '37', '38', '39', '43', '44', '45'], (592836, '744891'): ['42', '43', '44', '50', '51', '52', '53', '58', '59', '60', '61', '62'], (592836, '746018'): ['61', '62', '63', '64', '65', '66', '67', '74', '75', '76', '77', '78'], (592836, '745528'): ['28', '29', '30', '31', '36', '37', '38', '39', '43', '44', '45'], (592836, '745769'): ['5', '6', '7', '8', '9', '10', '14', '15', '16', '22', '23', '24', '25', '26', '33', '34', '35', '36', '37'], (592836, '745526'): ['13', '14', '15', '16', '17', '18', '24', '25', '26', '27', '31', '32'], (656546, '745604'): ['52', '53', '54'], (656546, '745602'): ['39', '40', '41'], (656546, '745600'): ['41', '42', '43', '44', '45'], (656546, '744873'): ['51', '52', '53', '58', '59', '60'], (656546, '745194'): ['68', '69', '70', '71', '72', '73'], (656546, '745196'): ['74', '75', '76'], (656546, '745594'): ['70', '71', '72'], (656546, '745597'): ['61', '62', '63', '64', '65'], (656546, '745591'): ['69', '70', '71', '72'], (656546, '746723'): ['66', '67', '68'], (656546, '746724'): ['65', '69', '70', '71', '72'], (656546, '745425'): ['65', '66', '67'], (656546, '745428'): ['60', '61', '62'], (656546, '746239'): ['75', '76', '77'], (656546, '745589'): ['64', '65', '66'], (656546, '745587'): ['64', '65', '66', '67'], (656546, '746069'): ['69', '70', '71'], (656546, '746068'): ['73', '74', '75', '76'], (656546, '745580'): ['57', '58', '59', '60', '61', '62'], (656546, '745579'): ['63', '64', '65', '66'], (656546, '745578'): ['55', '56', '57', '58', '59'], (656546, '745573'): ['70', '71', '72'], (656546, '746558'): ['61', '62', '63'], (656546, '745337'): ['64', '65', '66', '67', '68'], (656546, '745574'): ['61', '62', '63'], (656546, '745572'): ['61', '62', '63', '64', '65'], (656546, '745569'): ['55', '56', '57'], (656546, '745570'): ['52', '53', '54', '55'], (656546, '745571'): ['56', '63', '64', '65'], (656546, '746944'): ['65', '66', '67', '68'], (656546, '747025'): ['53', '54'], (656546, '745563'): ['67', '68', '69'], (656546, '745566'): ['64', '65', '66', '67', '68', '69'], (656546, '746456'): ['63', '64', '65'], (656546, '745565'): ['64', '65', '66', '67', '68', '69'], (656546, '745555'): ['84', '85', '86', '87'], (656546, '746860'): ['70', '71', '72'], (656546, '747097'): ['79', '80', '81'], (656546, '745556'): ['66', '67', '68'], (656546, '745558'): ['63', '64', '65', '66', '67'], (656546, '745549'): ['63', '64', '65', '66', '67'], (656546, '745481'): ['73', '74', '75'], (656546, '745878'): ['55', '56', '57', '58'], (656546, '745877'): ['71', '72', '73', '74'], (656546, '745550'): ['62', '63', '64', '65'], (656546, '745546'): ['81', '82', '83', '89', '90', '91', '92', '93'], (656546, '745225'): ['48', '49', '50', '51', '52', '53', '54', '55'], (656546, '746120'): ['65', '66', '67', '68'], (656546, '747169'): ['61', '62', '63', '64'], (656546, '747168'): ['64'], (656546, '745548'): ['60', '61', '62'], (656546, '745544'): ['58', '59', '60', '61'], (656546, '745542'): ['62', '63'], (656546, '745539'): ['72', '73', '74', '75', '76'], (656546, '747084'): ['48'], (656546, '746271'): ['69', '70', '71'], (656546, '745538'): ['61', '62', '63', '64'], (656546, '745531'): ['72', '73', '74'], (656546, '745532'): ['55', '56', '57', '58'], (656546, '745536'): ['53', '54', '55', '56'], (656546, '744887'): ['60', '61', '62'], (656546, '745534'): ['52', '53', '54', '55', '56'], (656546, '745528'): ['61', '62', '63', '64'], (656546, '745935'): ['69', '70', '71'], (656546, '745770'): ['82', '83', '84'], (656546, '745527'): ['64', '65', '66', '67'], (656546, '745521'): ['67', '68', '69'], (656546, '744799'): ['59', '60', '61', '62', '63', '64', '65'], (621345, '745604'): ['69', '70', '71'], (621345, '745602'): ['64', '65', '66'], (621345, '746814'): ['49', '50', '51', '52'], (621345, '747140'): ['72', '73', '74'], (621345, '747142'): ['56', '57', '58', '59'], (621345, '746084'): ['68', '69', '70', '71'], (621345, '746079'): ['68', '69', '70'], (621345, '746406'): ['61', '65', '66', '67'], (621345, '746401'): ['76', '77'], (621345, '747135'): ['60', '61', '62', '63'], (621345, '747134'): ['60', '61', '62'], (621345, '747132'): ['72', '73', '74', '75', '76'], (621345, '747129'): ['69', '70', '71'], (621345, '747125'): ['75', '76', '77'], (621345, '745263'): ['60', '61'], (621345, '745261'): ['64', '65'], (621345, '746152'): ['58', '59', '60', '61', '62'], (621345, '746153'): ['58', '59', '60'], (621345, '745827'): ['58', '59', '60'], (621345, '745825'): ['73', '74', '75'], (621345, '747128'): ['67', '68', '69', '70'], (621345, '747119'): ['58', '59', '60'], (621345, '746875'): ['70', '71', '72'], (621345, '746874'): ['60', '61', '62'], (621345, '747117'): ['53', '54', '55', '56', '57'], (621345, '747100'): ['52', '53', '54'], (621345, '747099'): ['52', '53', '54'], (621345, '747180'): ['74', '75', '76'], (621345, '747178'): ['61'], (621345, '745385'): ['54', '55', '56', '57', '58', '59'], (621345, '747094'): ['49', '50', '51'], (621345, '745792'): ['62', '63', '64'], (621345, '745795'): ['55', '56', '57', '58'], (621345, '745958'): ['61', '62', '63'], (621345, '747092'): ['59', '60', '61'], (621345, '747088'): ['70', '71', '72'], (621345, '747085'): ['54', '55', '56', '57', '58', '59'], (621345, '746523'): ['60', '61', '62', '63', '64'], (621345, '746517'): ['60', '61', '62', '63', '64'], (628452, '747132'): ['61', '62', '63', '64', '65', '66', '67'], (628452, '747131'): ['60', '61', '62'], (628452, '747125'): ['67', '68', '69', '70'], (628452, '745261'): ['75', '76', '77', '78', '79'], (628452, '746152'): ['73', '74', '75', '76'], (628452, '747126'): ['74', '75', '76'], (628452, '745827'): ['65', '66', '67', '68', '69'], (628452, '745828'): ['68', '69', '70', '71', '72', '73'], (628452, '747122'): ['71', '72', '73', '74', '75'], (628452, '747121'): ['66', '67', '68'], (628452, '746874'): ['68', '69', '70'], (628452, '747115'): ['58', '59', '60'], (628452, '747116'): ['66', '67', '68', '69'], (628452, '747110'): ['70', '71', '72'], (628452, '744844'): ['66', '67', '68'], (628452, '744849'): ['70', '71', '72'], (628452, '747026'): ['75', '76', '77', '78', '79'], (628452, '747112'): ['77'], (628452, '747109'): ['67', '68', '69', '70', '71'], (628452, '747107'): ['64', '65', '66'], (628452, '745727'): ['64', '65', '66'], (628452, '745157'): ['78', '79', '80'], (628452, '747104'): ['66', '67', '68', '70'], (628452, '747100'): ['65', '66', '67', '68'], (628452, '747099'): ['67', '68', '69'], (628452, '747181'): ['68', '69', '70', '74', '75', '76'], (628452, '745385'): ['72', '73', '74'], (628452, '747093'): ['62', '63', '64', '65'], (628452, '745794'): ['62', '63', '64'], (628452, '745793'): ['64', '65', '66', '67', '68'], (628452, '745958'): ['76', '77', '78'], (628452, '745960'): ['76', '77', '78'], (628452, '747092'): ['65', '66', '67'], (628452, '747081'): ['70', '71', '72'], (628452, '746520'): ['81', '82', '83'], (628452, '745300'): ['62', '63', '64', '69', '70', '71'], (628452, '745302'): ['80', '81', '82'], (628452, '746186'): ['69', '70', '71'], (628452, '747083'): ['67', '68', '69'], (628452, '747082'): ['64', '65', '66'], (628452, '747080'): ['64', '65', '66', '70', '71', '72'], (628452, '745864'): ['73', '74', '78', '79', '80'], (628452, '745866'): ['70', '71', '72', '73'], (628452, '745536'): ['66', '67', '68', '69', '73', '74', '75'], (628452, '747079'): ['67', '68', '69'], (628452, '747075'): ['73', '74', '75', '76'], (628452, '747074'): ['66', '67', '68', '69', '70'], (628452, '747072'): ['72', '73', '74', '75', '81', '82', '83', '84'], (628452, '747069'): ['68', '69', '70'], (628452, '747071'): ['68', '69', '70', '72', '73', '74'], (628452, '746012'): ['75', '76', '77', '78'], (628452, '746010'): ['67', '68', '69', '70', '71', '75', '76', '77'], (628452, '747068'): ['66', '67', '68'], (628452, '747065'): ['65'], (628452, '747066'): ['61', '62', '63'], (628452, '747064'): ['63', '64', '65'], (628452, '747139'): ['60', '61', '62'], (641729, '745604'): ['44', '45', '46', '47'], (641729, '745602'): ['44', '45', '50'], (641729, '747143'): ['59', '60', '61', '62'], (641729, '747141'): ['42', '43', '47', '48', '49'], (641729, '747138'): ['67', '68'], (641729, '746079'): ['61', '62', '63', '64'], (641729, '746400'): ['57', '58', '59'], (641729, '746401'): ['69', '70', '71', '75'], (641729, '747135'): ['54', '55', '56'], (641729, '747132'): ['54', '55', '56'], (641729, '747131'): ['48', '49', '50', '51', '52', '53'], (641729, '745263'): ['50', '51', '52', '53', '54', '55'], (641729, '745261'): ['69', '70', '71'], (641729, '746152'): ['66', '67', '68', '69'], (641729, '747126'): ['63', '64', '65', '67'], (641729, '745828'): ['56', '57', '58', '59', '60'], (641729, '747122'): ['55', '56', '57'], (641729, '747119'): ['65', '66', '67', '68', '69', '70', '71', '72'], (641729, '746874'): ['51', '52', '53'], (641729, '745498'): ['63', '64', '65'], (641729, '747114'): ['53', '54', '55', '56'], (641729, '747116'): ['58', '59', '60', '61', '62'], (641729, '747110'): ['59', '60', '61', '62'], (641729, '744844'): ['60', '61', '62'], (641729, '744849'): ['63', '64', '65', '66'], (641729, '747028'): ['62', '63', '64', '65', '66', '67'], (641729, '747026'): ['64', '65', '66', '67'], (641729, '747108'): ['65', '66', '67'], (641729, '747107'): ['57', '58', '59', '60'], (641729, '745727'): ['58', '59', '60'], (641729, '745157'): ['71', '72', '73'], (641729, '747104'): ['58', '59', '60', '61', '62'], (641729, '747100'): ['58', '59', '60'], (641729, '747099'): ['59', '60', '61', '62'], (641729, '747181'): ['81', '82', '83', '84'], (641729, '747179'): ['62', '63', '64', '65', '66', '67', '68'], (641729, '745390'): ['57', '58', '59', '60'], (641729, '745385'): ['60', '65', '66', '67', '68'], (641729, '747093'): ['55', '56', '57'], (641729, '745794'): ['57', '58'], (641729, '745793'): ['57', '58', '59'], (641729, '745958'): ['69', '70', '71', '72'], (641729, '745960'): ['67', '68', '69', '70'], (641729, '747089'): ['65', '66', '67', '68', '69'], (641729, '747086'): ['63', '64', '65', '66', '67'], (641729, '746520'): ['75', '76', '77'], (641729, '746517'): ['75', '76', '77', '78', '79'], (641729, '745300'): ['54', '55', '56', '57'], (641729, '746186'): ['62', '63', '64', '65'], (641729, '747083'): ['57', '58', '59'], (641729, '747084'): ['55', '56', '57', '58'], (641729, '747080'): ['58', '59', '60'], (641729, '745864'): ['63', '64', '65', '66', '70', '71', '72'], (641729, '745866'): ['62', '63', '64', '65'], (641729, '745533'): ['65', '66', '67', '68', '69'], (641729, '745536'): ['57', '58', '59'], (641729, '747079'): ['58', '59', '60', '61'], (641729, '747075'): ['67', '68', '69'], (641729, '747074'): ['59', '60', '61'], (641729, '747072'): ['53', '54', '55', '56', '57', '58'], (641729, '747069'): ['60', '61', '62'], (641729, '747071'): ['55', '59', '60', '61', '62'], (641729, '746662'): ['62', '63', '64'], (641729, '746013'): ['68', '69', '70'], (641729, '746012'): ['68', '69', '70', '71'], (641729, '747068'): ['59', '60', '61', '62'], (641729, '747066'): ['48', '52', '53', '54', '55', '56'], (641729, '747064'): ['57', '58', '59'], (641729, '747139'): ['57', '58', '59'], (572955, '745604'): ['55', '56', '57'], (572955, '745599'): ['57', '58', '59', '60'], (572955, '746814'): ['58', '59', '60', '61', '62', '63'], (572955, '747143'): ['86', '87', '88', '89'], (572955, '747141'): ['62', '63', '64', '65'], (572955, '747142'): ['63', '64', '65', '66', '67', '68'], (572955, '746084'): ['61', '62', '63'], (572955, '746406'): ['50', '51', '57', '58', '59', '60'], (572955, '746401'): ['58', '59', '60', '61'], (572955, '747136'): ['67', '68', '69'], (572955, '747134'): ['54'], (572955, '747129'): ['59', '60', '61'], (572955, '745263'): ['44', '45', '46'], (572955, '747121'): ['56', '57', '58', '59', '60'], (572955, '746874'): ['44', '45', '46', '47'], (572955, '745493'): ['54', '55', '56', '57', '58', '59'], (572955, '747117'): ['70', '71', '72', '73', '74', '75'], (572955, '747116'): ['51', '52', '53', '54'], (572955, '747111'): ['55', '56', '57', '63', '64', '65'], (572955, '746949'): ['61', '62', '63', '64'], (572955, '744845'): ['60', '61', '62'], (572955, '747027'): ['49', '50', '51'], (572955, '747026'): ['55', '56', '57', '58', '59', '60'], (572955, '747113'): ['63', '64', '65', '66'], (572955, '747107'): ['50', '51', '52', '53'], (572955, '745727'): ['50', '51', '52', '53'], (572955, '745157'): ['63', '64', '65', '66'], (572955, '747104'): ['52', '53', '54'], (572955, '747102'): ['46', '47', '48', '52', '53', '54', '55'], (572955, '747098'): ['60', '61', '62', '63'], (572955, '747179'): ['54', '55', '56'], (572955, '745385'): ['46', '47', '48'], (572955, '747093'): ['71', '72', '73'], (572955, '745794'): ['67', '69', '70'], (572955, '745795'): ['46', '47', '48'], (572955, '745960'): ['49', '50', '58', '59', '60', '61'], (572955, '747087'): ['54', '55', '56', '57', '58'], (572955, '747085'): ['42', '43', '51', '52', '53'], (572955, '746523'): ['69', '70', '71', '72'], (572955, '746520'): ['61', '67', '68', '69', '70'], (572955, '745302'): ['64', '65', '66', '67', '68'], (572955, '746189'): ['50', '51', '52', '53'], (572955, '746190'): ['61', '62', '63', '64'], (572955, '747082'): ['52', '57', '58', '59', '60'], (572955, '747077'): ['72', '73', '74'], (572955, '745865'): ['71', '72', '73', '74'], (572955, '745864'): ['89', '90', '91', '92', '93', '94'], (572955, '745533'): ['56', '57', '58', '59', '60'], (572955, '745536'): ['50', '51', '52'], (572955, '747075'): ['59', '60', '61'], (572955, '747072'): ['49'], (572955, '747069'): ['54', '55', '56'], (572955, '746659'): ['61', '62', '68', '69', '70', '71', '72'], (572955, '746013'): ['64'], (572955, '746012'): ['61', '62', '63'], (572955, '746010'): ['49', '50', '51'], (572955, '747066'): ['45', '46', '47'], (572955, '747139'): ['74', '75', '76', '77', '78'], (605154, '746818'): ['75', '76', '77'], (605154, '746815'): ['45', '49', '50'], (605154, '746814'): ['53', '54', '55'], (605154, '746328'): ['43', '44'], (605154, '745919'): ['67', '68', '69', '70', '71'], (605154, '745918'): ['57', '58', '59', '60', '61'], (605154, '746803'): ['61', '62', '63'], (605154, '746804'): ['62', '63', '64', '65', '66'], (605154, '746800'): ['71', '72', '73', '74', '75'], (605154, '745183'): ['81', '82', '83', '84', '85'], (605154, '745185'): ['65', '66', '67'], (605154, '745097'): ['59', '63', '64', '65'], (605154, '746802'): ['59', '60', '61', '62'], (605154, '746797'): ['64', '65', '66'], (605154, '746796'): ['51', '52', '53', '58', '59', '60', '61', '62'], (605154, '744936'): ['52', '53', '54'], (605154, '746792'): ['77', '78', '79'], (605154, '746793'): ['61', '62', '63', '64'], (605154, '746788'): ['69', '70', '71', '72', '73'], (605154, '746791'): ['77', '78', '79', '80'], (605154, '745983'): ['68', '69', '70', '71', '72'], (605154, '745984'): ['64', '65', '66', '67'], (605154, '746872'): ['57', '58', '59'], (605154, '746873'): ['66', '72', '73', '74'], (605154, '746786'): ['55', '56', '57', '58'], (605154, '746787'): ['62', '63', '64', '65'], (605154, '745244'): ['60', '61', '62'], (605154, '745245'): ['58', '59', '60'], (605154, '747187'): ['62', '63', '64'], (605154, '746783'): ['65'], (605154, '746460'): ['59', '60', '61'], (605154, '746459'): ['64', '65', '66', '67', '68'], (605154, '746775'): ['73', '74', '75'], (605154, '746816'): ['50', '51', '52', '53', '54'], (605154, '746776'): ['58', '59', '60', '61'], (605154, '746779'): ['57', '58', '59'], (605154, '746623'): ['63', '64', '65'], (605154, '746046'): ['59', '60', '61', '62', '63'], (605154, '746039'): ['68', '69', '70', '71'], (605154, '746771'): ['70', '71', '72', '73'], (605154, '746772'): ['54', '55', '56', '57'], (605154, '746769'): ['69', '70', '71', '72'], (605154, '746287'): ['56', '57', '58', '59', '60', '61', '62'], (605154, '744988'): ['67', '68', '69', '70', '71', '72'], (605154, '746770'): ['68', '69', '70', '71'], (605154, '746765'): ['68', '69', '70', '71'], (605154, '745876'): ['60', '61', '62', '63', '64', '65'], (605154, '745629'): ['68', '69', '70'], (605154, '745630'): ['61', '62', '63', '64'], (605154, '746760'): ['74', '75', '76'], (605154, '746759'): ['75', '76', '77'], (605154, '746359'): ['56', '57', '58', '59', '60'], (605154, '745297'): ['65', '66', '67', '68'], (605154, '746756'): ['75', '76', '77', '78', '79', '80', '81'], (605154, '747073'): ['62', '63', '64'], (605154, '747076'): ['51', '52', '53', '54'], (605154, '744807'): ['45', '46', '47', '48', '54', '55', '56', '57', '62', '63', '64'], (605154, '747071'): ['75', '76', '77'], (605154, '746663'): ['67', '74', '75', '76', '77'], (669276, '745599'): ['65', '66', '67', '72', '73', '74'], (669276, '746817'): ['54', '58', '59', '60'], (669276, '747140'): ['44', '45', '47', '54', '55', '56', '57', '62', '63', '64'], (669276, '747137'): ['47', '48', '49', '54', '55', '56', '57', '58', '59', '60'], (669276, '746079'): ['52', '53', '54'], (669276, '747133'): ['43', '44', '49', '50', '51', '52', '53'], (669276, '747131'): ['66', '67', '68', '74', '76', '77', '79'], (669276, '745261'): ['52', '53', '54', '55', '60', '61', '62', '63'], (669276, '746155'): ['35', '36', '37', '38', '43', '44', '45', '51', '52', '53', '54'], (669276, '747127'): ['54', '55', '56', '57', '61', '62', '63', '64'], (669276, '745825'): ['68', '69'], (669276, '747128'): ['59', '60', '61', '62', '63'], (669276, '747120'): ['64', '65', '66'], (669276, '746874'): ['38', '39', '43'], (669276, '745498'): ['72', '73', '74', '75', '76'], (669276, '747117'): ['62', '63', '64', '65'], (669276, '747111'): ['71', '72', '73', '74', '75'], (669276, '747110'): ['47', '48', '49', '50', '51'], (669276, '744848'): ['50', '51', '52', '53', '54'], (669276, '747027'): ['57', '58', '59'], (669276, '747112'): ['72', '73', '74', '75', '76'], (669276, '747109'): ['32', '37', '38', '39'], (669276, '745729'): ['68', '69', '70'], (669276, '745157'): ['54', '55', '56', '57'], (669276, '747105'): ['64'], (669276, '747102'): ['37', '38', '44', '45'], (669276, '747097'): ['75', '76', '77', '78'], (669276, '747098'): ['52', '53', '54', '55'], (669276, '745390'): ['65', '66', '67', '68'], (669276, '747093'): ['45', '46', '47', '48', '49'], (669276, '747095'): ['66', '67', '68', '69'], (669276, '747091'): ['63', '64', '65', '70', '71', '72', '73', '74'], (669276, '745795'): ['32', '33', '34', '35', '39', '40', '41', '42'], (669276, '747092'): ['50', '51', '52', '53'], (669276, '747087'): ['43', '47', '48', '49'], (669276, '745302'): ['73', '74', '75'], (669276, '746189'): ['59', '60', '61', '62'], (669276, '746186'): ['47', '48', '49', '53', '54', '55'], (669276, '747077'): ['55', '56', '57'], (669276, '745865'): ['78', '79', '80', '81'], (669276, '745864'): ['45'], (669276, '745533'): ['73', '74', '75'], (669276, '745536'): ['46'], (669276, '747075'): ['49', '50', '51', '52'], (669276, '747072'): ['45', '46', '47', '48'], (669276, '747071'): ['50', '51', '52', '53', '54'], (669276, '746659'): ['77', '78', '79'], (669276, '746662'): ['53', '54', '55'], (669276, '746013'): ['60', '61', '62'], (669276, '746010'): ['40', '41', '42', '47', '48'], (669276, '747061'): ['60', '66', '67'], (669276, '747064'): ['42', '43'], (592426, '745445'): ['54', '55', '56'], (592426, '745346'): ['55', '56', '57'], (592426, '745350'): ['45', '46', '47', '48', '49'], (592426, '746966'): ['44', '45', '46', '51', '52', '53'], (592426, '745589'): ['60', '61', '62', '63'], (592426, '745583'): ['64', '65', '66'], (592426, '745585'): ['52', '56', '57', '58', '59', '60'], (592426, '746563'): ['59', '60', '61'], (592426, '745345'): ['84', '85', '86', '87'], (592426, '745343'): ['41', '42', '43', '44', '45'], (592426, '745335'): ['53', '54', '55', '56'], (592426, '745336'): ['51', '52', '53'], (592426, '745503'): ['74', '75', '76'], (592426, '745500'): ['56', '57', '58', '59', '60'], (592426, '745823'): ['69', '70', '71', '72', '73'], (592426, '745333'): ['49', '50', '51', '55', '56', '57'], (592426, '745331'): ['49', '50', '51'], (592426, '747196'): ['57', '58', '59', '60', '61', '62', '63', '64'], (592426, '745003'): ['63', '64', '65'], (592426, '745327'): ['61', '62', '63'], (592426, '745326'): ['53', '54', '55'], (592426, '745329'): ['63', '64', '65', '66'], (592426, '745321'): ['49', '50', '51'], (592426, '746867'): ['66', '67'], (592426, '745163'): ['61', '62', '63', '64', '65', '66'], (592426, '745323'): ['33', '34', '35', '36', '37', '38'], (592426, '745325'): ['38', '43', '44', '45', '46'], (592426, '745324'): ['75', '76', '77', '78'], (592426, '747100'): ['48', '49', '50', '51'], (592426, '746615'): ['53', '54', '55', '56'], (592426, '745316'): ['54', '55', '56'], (592426, '745318'): ['62', '63', '64', '65', '69', '70', '71', '72', '73', '77', '78', '79'], (592426, '746531'): ['72', '73', '74', '75'], (592426, '746118'): ['56', '57', '58'], (592426, '746117'): ['64', '65', '66'], (592426, '745315'): ['55'], (592426, '747092'): ['54', '55'], (592426, '747088'): ['32', '33', '34', '38', '39', '40', '41', '42', '47', '48', '49', '53', '54', '55'], (592426, '747081'): ['56', '57', '58', '59', '60', '66', '67', '68', '69'], (592426, '746517'): ['69', '70', '71', '72', '73', '74'], (592426, '745301'): ['68', '69', '70', '71', '72'], (592426, '746189'): ['67', '68', '69'], (592426, '747084'): ['62', '63', '64'], (592426, '747078'): ['48', '49', '50', '51'], (592426, '745866'): ['53', '54', '55', '56'], (592426, '747073'): ['46', '47', '48', '49'], (592426, '747076'): ['72'], (592426, '744807'): ['72', '73', '74', '75', '80', '81', '82', '83'], (592426, '747070'): ['72', '73', '74', '75'], (592426, '747067'): ['60', '61', '62', '66', '67', '68', '69'], (592426, '746663'): ['53', '54', '55', '56', '62', '63', '64', '65', '66'], (592426, '747061'): ['68'], (675540, '746974'): ['4', '5', '6', '7', '11', '12', '13', '18', '19', '20', '21', '25', '26', '27', '31', '32', '33', '34'], (675540, '746633'): ['1', '2', '3', '7', '8', '9', '13', '14', '15', '23', '24', '25', '26', '27', '32', '33', '34', '35', '36'], (675540, '746549'): ['5', '6', '7', '8', '16', '17', '18', '19', '20', '25', '26', '27', '32', '33', '34', '35', '36', '37', '38'], (675540, '747019'): ['46', '47', '48', '49', '53', '54', '55', '56', '60', '61', '62', '63'], (675540, '745064'): ['59', '60', '64', '65', '66', '67'], (675540, '746614'): ['1', '2', '3', '4', '9', '10', '11', '12', '18', '19', '20', '26', '27', '28', '29', '30', '37', '38'], (675540, '746608'): ['51', '52', '53', '58', '59', '60', '61', '62', '63', '64', '65', '71', '72', '73', '74', '75'], (675540, '745296'): ['48', '49'], (675540, '745292'): ['44', '45', '52', '53', '54'], (675540, '746021'): ['61', '62', '63'], (675540, '746016'): ['50', '51', '52', '53', '54', '55', '56', '57'], (675540, '745450'): ['23', '24', '25', '29', '30', '31', '32', '37', '38', '39', '43', '44', '45', '50', '51', '52'], (675540, '746014'): ['37', '38', '44', '45', '46', '50', '51', '52'], (675540, '746012'): ['60'], (675540, '746010'): ['43', '44', '45', '46'], (675540, '744878'): ['8', '9', '10', '11', '15', '16', '17', '21', '22', '23', '24', '28', '29', '30', '35', '36', '37'], (670124, '745299'): ['5', '6', '7', '12', '13', '14', '18', '19', '20', '21', '22', '27', '28', '29', '33', '34', '35', '39', '40', '41', '42'], (670124, '746021'): ['1', '2', '3', '4', '5', '6', '10', '11', '12', '17', '18', '19', '20', '26', '27', '28', '29', '30', '34', '35', '36', '41', '42', '43', '44'], (670124, '745446'): ['4', '5', '6', '7', '8', '9', '13', '14', '15', '16', '21', '22', '23', '24', '28', '29', '30', '31', '38', '39', '40', '41', '42', '43', '44'], (670124, '744801'): ['6', '7', '8', '9', '13', '14', '15', '21', '22', '23', '28', '29', '30', '31', '32', '33', '34', '38', '39', '40', '41'], (670124, '746012'): ['1', '2', '3', '4', '10', '11', '12', '13', '14', '18', '19', '20', '21', '22', '23', '27', '28', '29', '33', '34', '35', '36', '37', '38', '44', '45', '46'], (670124, '744879'): ['7', '8', '9', '10', '11', '18', '19', '20', '27', '28', '29', '30', '31', '32', '33', '42', '43', '44', '45', '46', '50', '51', '52', '53', '54'], (677053, '746089'): ['48', '49', '54', '55'], (677053, '746087'): ['57', '58', '59', '60', '61', '62', '63', '64'], (677053, '746085'): ['55', '56', '57', '58'], (677053, '745200'): ['64', '65', '66', '67', '68'], (677053, '745198'): ['56', '57', '58'], (677053, '745762'): ['59', '60', '61', '62'], (677053, '745763'): ['54', '55', '56'], (677053, '746084'): ['52', '53', '54'], (677053, '746079'): ['65', '66', '67'], (677053, '746080'): ['55', '56', '57', '58'], (677053, '746892'): ['63', '64', '65', '66', '67'], (677053, '746894'): ['63', '64', '65'], (677053, '747132'): ['50', '51', '52', '53'], (677053, '746078'): ['39', '40', '41', '42'], (677053, '746074'): ['53', '54', '55'], (677053, '746070'): ['53', '54', '55'], (677053, '745667'): ['52', '53', '54'], (677053, '746151'): ['56', '57', '58', '59'], (677053, '746068'): ['48', '49', '50'], (677053, '746471'): ['46', '51', '52', '53'], (677053, '746472'): ['52'], (677053, '746067'): ['79', '80'], (677053, '746064'): ['38', '39', '40'], (677053, '746066'): ['66', '67'], (677053, '747199'): ['56', '57', '58'], (677053, '745412'): ['54', '55', '56'], (677053, '746062'): ['51', '52', '53'], (677053, '746056'): ['57', '58', '59', '60'], (677053, '746053'): ['42', '43', '44'], (677053, '746054'): ['40', '41', '42', '43', '44'], (677053, '745815'): ['56', '57'], (677053, '744846'): ['56', '57', '58', '59', '60'], (677053, '746055'): ['81', '82', '83'], (677053, '746049'): ['71', '72', '73'], (677053, '746052'): ['51', '52', '53', '54'], (677053, '746051'): ['54', '55'], (677053, '746291'): ['49', '50', '51', '52'], (677053, '745559'): ['42', '46', '47', '48'], (677053, '745555'): ['70', '71', '72', '73'], (677053, '746045'): ['57', '58'], (677053, '746043'): ['69', '70', '72'], (677053, '746039'): ['64', '65', '66', '67'], (677053, '746367'): ['63'], (677053, '746687'): ['57', '58', '59', '60', '61', '62'], (677053, '746040'): ['50', '51', '52'], (677053, '746041'): ['51', '52', '53', '54'], (677053, '746036'): ['49', '50', '51'], (677053, '746035'): ['51', '52', '53'], (677053, '745961'): ['69', '70', '71', '72'], (677053, '745061'): ['44', '45', '46', '47'], (677053, '747089'): ['49', '50', '51', '52'], (677053, '747087'): ['66', '67', '68'], (677053, '746031'): ['56', '57', '58', '59', '60'], (677053, '746030'): ['54', '55', '56'], (677053, '746025'): ['62', '63', '64', '65', '66', '70', '71', '72'], (677053, '745548'): ['70', '71', '72'], (677053, '745786'): ['58', '65', '66'], (677053, '746029'): ['57', '58', '59', '60'], (677053, '746028'): ['62', '63', '64', '65'], (676534, '746082'): ['51', '52', '53'], (676534, '746081'): ['58', '65', '66', '67', '68'], (676534, '746893'): ['55', '56', '57', '58', '59', '66', '67', '68'], (676534, '746894'): ['69', '70', '71'], (676534, '747132'): ['57', '58', '59', '60', '68', '69', '70', '71'], (676534, '746078'): ['53', '54', '55', '56', '57', '58', '59'], (676534, '746074'): ['49', '50', '51', '52'], (676534, '746070'): ['39', '40', '45', '46', '47'], (676534, '745667'): ['65', '66', '67', '68', '69'], (676534, '746071'): ['63', '64', '65', '66', '67', '68', '69'], (676534, '746068'): ['56', '57', '58'], (676534, '746471'): ['57', '58', '59'], (676534, '746472'): ['57', '58', '59', '60'], (676534, '746067'): ['73', '74', '75', '76', '77', '78'], (676534, '746058'): ['49', '50', '51', '52', '53'], (676534, '747199'): ['63', '64', '65'], (676534, '747197'): ['64', '65', '66', '67'], (676534, '745414'): ['71', '72', '73', '74'], (676534, '746059'): ['58', '59', '60', '61'], (676534, '746056'): ['66', '67', '68', '69', '70'], (676534, '746053'): ['55', '56', '57'], (676534, '745809'): ['55', '59', '60', '61'], (676534, '746055'): ['52', '53', '54', '55', '56', '57', '58'], (676534, '746049'): ['83', '84', '85'], (676534, '746052'): ['55'], (676534, '746051'): ['61', '62', '63', '64'], (676534, '746293'): ['60', '61', '62'], (676534, '745560'): ['54', '55', '56', '57', '58'], (676534, '745559'): ['62', '63', '64'], (676534, '746045'): ['53', '54', '55', '56'], (676534, '746043'): ['77', '78', '80', '81', '82', '83'], (676534, '746039'): ['49', '50', '51', '56', '57', '58'], (676534, '746685'): ['68', '69', '70', '71', '72'], (676534, '746040'): ['64', '65', '66', '67'], (676534, '746042'): ['65', '66', '67', '68'], (676534, '746035'): ['61', '62', '63', '64'], (676534, '746037'): ['79', '80', '81'], (676534, '745966'): ['72', '73', '74', '75'], (676534, '745061'): ['70', '71', '72'], (676534, '747089'): ['57', '58', '59', '60', '61', '63', '64'], (676534, '747087'): ['74', '75', '76', '77'], (676534, '746031'): ['70', '71', '72', '73', '74', '75'], (676534, '746030'): ['70', '71', '72'], (676534, '746026'): ['80', '81', '82', '83'], (676534, '745785'): ['61', '62', '63'], (676534, '745786'): ['71', '72', '73', '74', '75'], (676534, '746028'): ['78', '79', '80', '81', '82', '83', '84'], (676534, '746023'): ['70', '71', '72', '73', '74'], (676534, '746510'): ['80', '81', '82'], (676534, '746514'): ['89', '90', '91', '92'], (676534, '745296'): ['67', '68', '69'], (676534, '745292'): ['73', '74', '75'], (676534, '746019'): ['54', '55', '56', '57', '58'], (622786, '745573'): ['48', '49', '50', '58', '59', '60', '61'], (622786, '745901'): ['53', '54', '55'], (622786, '745011'): ['51', '52', '53', '54', '58', '59', '60'], (622786, '746059'): ['52', '53', '54', '55', '56', '57', '62'], (622786, '745010'): ['38', '39', '40', '46', '47', '48', '49'], (622786, '745008'): ['64', '65', '66'], (622786, '745001'): ['63', '64', '65', '66'], (622786, '746139'): ['58', '64', '65', '66', '67'], (622786, '745241'): ['62', '63', '64', '65', '66', '67', '68'], (622786, '746854'): ['63', '64', '65', '73', '74', '75'], (622786, '746277'): ['52', '53', '54', '59', '60', '61', '62', '63'], (622786, '745061'): ['55', '56', '57', '58'], (622786, '747087'): ['29', '33', '34', '35'], (622786, '747088'): ['67', '68', '69'], (622786, '746031'): ['64', '65', '66'], (622786, '746030'): ['40', '41', '46', '47', '48'], (622786, '746026'): ['51', '52', '53'], (622786, '745548'): ['63', '64', '65'], (622786, '745786'): ['46', '47', '48', '55', '56', '57'], (622786, '746029'): ['65', '66', '67'], (622786, '746028'): ['51', '52', '58', '59', '60', '61'], (622786, '746023'): ['51', '52', '53', '58', '59', '60', '61'], (622786, '746514'): ['68', '69', '70', '74', '75', '76', '80', '81', '82'], (622786, '745296'): ['60', '61', '62', '63'], (622786, '746019'): ['43', '44', '45'], (622786, '746018'): ['79', '80', '81'], (622786, '744805'): ['63', '64', '65', '71', '72', '73'], (622786, '746015'): ['80', '81', '82', '83', '84', '85'], (622786, '746013'): ['71', '72', '73'], (622786, '746010'): ['72', '73', '74'], (622786, '745851'): ['71', '72', '78', '79', '80'], (622786, '745853'): ['67', '68', '69', '70', '75', '76', '78', '79'], (664350, '746089'): ['66', '67', '68', '69'], (664350, '746087'): ['39', '40', '41', '42', '43', '44', '48', '49', '50'], (664350, '745200'): ['60', '61', '62', '63'], (664350, '745762'): ['42', '46', '47', '48', '49', '50'], (664350, '746079'): ['44', '45', '46', '47'], (664350, '746076'): ['63', '64', '65', '66'], (664350, '746891'): ['61', '62', '63'], (664350, '747132'): ['4', '5', '6', '7', '8', '9', '10', '15', '16', '17', '22', '23', '24'], (664350, '746073'): ['1', '2', '3', '4', '5', '6', '7', '8', '9', '13', '14', '15', '16', '20', '21', '22', '26', '27', '28'], (664350, '745667'): ['8', '9', '10', '11', '12', '19', '20', '21', '22', '26', '27', '28', '32', '33', '34', '35', '36', '37', '38'], (664350, '746471'): ['5', '6', '7', '12', '13', '14', '15', '16', '17', '18', '19', '24', '25', '26', '31', '32', '33', '34', '42', '43', '44', '45'], (664350, '746064'): ['1', '2', '3', '4', '5', '6', '7', '8', '9', '14', '15', '16', '17', '23', '24', '25', '29', '30', '31', '32'], (664350, '747199'): ['7', '8', '9', '10', '11', '12', '16', '17', '18', '19', '20', '24', '25', '26', '31', '32', '33', '34', '40', '41', '42'], (664350, '746059'): ['1', '2', '3', '7', '8', '9', '16', '17', '18', '26', '27', '28', '29', '34', '35'], (670766, '745666'): ['28', '29', '30', '31', '32', '33', '34', '35', '41', '42', '43', '44', '48', '49', '50', '51', '56', '57', '58', '59', '60', '61', '62', '67', '68', '69', '70', '71'], (670766, '746621'): ['43', '44', '45', '46', '47', '48', '49', '50', '55', '56', '57', '58'], (670766, '746292'): ['41', '42', '43', '44', '45', '46', '47', '51', '52', '53', '59', '60', '61', '67', '68', '69', '70', '71', '72'], (670766, '746033'): ['49', '50', '51', '56', '57', '58', '59', '63', '64', '65', '66', '67', '68', '74', '75', '76', '77'], (670766, '746030'): ['84', '85', '86', '87', '88'], (670766, '745292'): ['4', '5', '6', '7', '8', '12', '13', '14', '15', '16', '21', '22', '23', '24', '25', '29', '30', '31', '39', '40'], (670766, '746018'): ['1', '2', '3', '4', '5', '6', '12', '13', '14', '19', '20', '21', '22', '28', '29', '30', '38', '39', '40', '41'], (670766, '744805'): ['5', '6', '7', '8', '9', '10', '11', '15', '16', '17', '18', '19', '26', '27', '28', '29', '33', '34', '35', '36', '40', '41', '42'], (670766, '746015'): ['1', '2', '3', '12', '13', '14', '15', '22', '23', '24', '25', '26', '27', '28', '29', '35', '36', '37'], (670766, '746010'): ['1', '2', '3', '4', '5', '6', '10', '11', '12', '13', '18', '19', '20', '24', '25', '26', '27'], (670766, '745853'): ['112', '113', '114'], (670766, '744878'): ['45', '46', '47', '51', '52', '53', '60', '61', '62', '63', '67', '68', '69', '70', '71'], (592866, '744869'): ['1', '2', '3', '7', '8', '9', '10', '11', '12', '13', '23', '24', '25', '30', '31', '32', '33', '38', '39', '40', '45'], (592866, '745363'): ['5', '6', '7', '8', '9', '10', '18', '19', '20', '24', '25', '26', '27', '32', '33', '34', '35', '44', '45', '46'], (592866, '745681'): ['6', '7', '8', '12', '13', '14', '15', '24', '25', '26', '33', '34', '35', '36', '41', '42', '43', '44', '52', '53', '54'], (592866, '744865'): ['1', '2', '3', '4', '9', '10', '11', '18', '19', '20', '24', '25', '26', '33', '34', '35', '39', '40', '41'], (592866, '746078'): ['4', '5', '6', '7', '12', '13', '14', '19', '20', '21', '22', '23', '24', '28', '29', '30', '31', '35', '36', '37'], (592866, '745023'): ['4', '5', '6', '11', '12', '13', '14', '15', '16', '20', '21', '22', '23', '24', '28', '29', '30', '34', '35', '36', '37', '38', '39'], (592866, '744859'): ['1', '2', '3', '4', '9', '10', '11', '12', '18', '19', '20', '25', '26', '27', '31', '32', '33', '34'], (592866, '746796'): ['6', '7', '8', '13', '14', '15', '16', '17', '21', '22', '23', '28', '29', '30', '38', '39', '40', '41', '42', '43'], (592866, '745577'): ['4', '5', '6', '11', '12', '13', '14', '15', '16', '20', '21', '22', '23', '24', '31', '32', '33', '38', '39'], (592866, '744851'): ['1', '2', '3', '7', '8', '9', '10', '16', '17', '18', '19', '23', '24', '25', '29', '30', '31', '32', '33', '38'], (592866, '747114'): ['5', '6', '7', '12', '13', '14', '15', '16', '25', '26', '27', '28', '29', '30', '36', '37', '38', '42', '43', '44', '49', '50', '51'], (592866, '746824'): ['5', '6', '7', '8', '13', '14', '15', '22', '23', '24', '25', '29', '30', '31', '35', '36', '37', '38'], (592866, '744797'): ['1', '2', '3', '14', '15', '16', '17', '18', '25', '26', '27', '28', '29', '34', '35', '36', '42', '43', '44'], (606930, '744858'): ['57', '58', '59'], (606930, '746964'): ['72', '73', '74', '75', '76'], (606930, '746799'): ['48', '49', '50', '51'], (606930, '745579'): ['48', '49', '50', '54', '55', '56'], (606930, '745577'): ['40', '41', '42'], (606930, '744860'): ['50', '51', '52', '53', '54', '60', '61', '62'], (606930, '744855'): ['48', '49', '50'], (606930, '747115'): ['44', '45', '46', '47', '48', '49'], (606930, '746635'): ['53', '54', '55', '56'], (606930, '744854'): ['55', '56', '57', '58', '59'], (606930, '744850'): ['62', '63', '64', '65'], (606930, '744844'): ['63', '64', '65'], (606930, '744849'): ['42', '43', '44', '52'], (606930, '746465'): ['61'], (606930, '744847'): ['64', '65', '66', '67'], (606930, '744843'): ['39', '40', '41', '42', '43', '44', '49', '50', '51', '52'], (606930, '744841'): ['45', '46', '47', '48'], (606930, '746541'): ['57', '58', '59', '60'], (606930, '745400'): ['34', '39', '40', '41', '45', '46', '47'], (606930, '745070'): ['47', '48', '49', '50', '51'], (606930, '744837'): ['49', '50', '51'], (606930, '744835'): ['45', '46', '54', '55', '56', '57', '61', '62', '63'], (606930, '744833'): ['56', '57', '58', '59', '60', '61'], (606930, '745796'): ['65', '66', '67', '68', '69'], (606930, '745963'): ['28', '29', '37', '38', '39', '44', '45', '46'], (606930, '744829'): ['49', '50', '51'], (606930, '744828'): ['54', '55', '56', '57'], (606930, '744824'): ['45', '46', '47', '48', '49', '53', '54', '55'], (606930, '745146'): ['77', '78', '79'], (606930, '747172'): ['56', '57', '58'], (606930, '747170'): ['50', '51', '52', '53'], (606930, '744825'): ['56', '57', '58', '59', '60'], (606930, '744826'): ['60'], (606930, '744821'): ['51', '52', '53'], (606930, '744819'): ['38', '39', '40', '41', '46', '47', '48'], (606930, '744818'): ['52', '53', '57', '58', '59', '60', '61'], (606930, '744816'): ['37', '38', '39'], (606930, '747000'): ['55', '56', '57'], (606930, '745542'): ['43', '44', '48', '49', '50', '51'], (606930, '745539'): ['62', '69', '70', '71'], (606930, '744815'): ['59', '60'], (606930, '747080'): ['54', '55', '56', '57', '61', '62'], (606930, '747078'): ['44', '45', '46', '47'], (606930, '744811'): ['58', '59', '60', '61'], (606930, '744809'): ['59', '60', '61'], (606930, '744808'): ['37', '38'], (606930, '744806'): ['49', '50', '51', '52', '53'], (606930, '746019'): ['46', '47', '48', '49', '50'], (606930, '745455'): ['54', '55'], (606930, '745456'): ['69', '70', '71', '72'], (606930, '744807'): ['35', '40', '41', '42'], (606930, '745773'): ['70', '71'], (606930, '745771'): ['34', '35', '36', '37', '42', '43', '44', '45'], (606930, '746825'): ['38', '39', '40', '41', '42'], (606930, '744795'): ['59', '60', '61'], (606930, '744796'): ['72'], (640448, '746736'): ['78', '79', '80', '81'], (640448, '746739'): ['77', '78', '79', '80', '81'], (640448, '744869'): ['68', '69', '70', '71'], (640448, '744871'): ['66', '67', '68', '69'], (640448, '745357'): ['67', '68', '74', '75', '76', '77', '78'], (640448, '745679'): ['73', '74'], (640448, '745676'): ['72', '73', '74'], (640448, '746158'): ['72', '73', '74'], (640448, '746159'): ['65', '66', '67'], (640448, '744865'): ['79', '80'], (640448, '744867'): ['65', '66', '67'], (640448, '746078'): ['67', '68', '69'], (640448, '746077'): ['90', '91', '92'], (640448, '745023'): ['65', '66', '67'], (640448, '744861'): ['84', '85', '86', '87'], (640448, '744859'): ['65', '66', '67'], (640448, '744858'): ['73', '74', '75', '76'], (640448, '746796'): ['72', '73', '74', '75', '76'], (640448, '745581'): ['68', '69', '70', '71', '72', '77', '78'], (640448, '744851'): ['61', '62', '63'], (640448, '747118'): ['73', '74', '75', '76', '77'], (640448, '747114'): ['71', '72', '73', '74'], (640448, '746632'): ['67', '68', '69', '70'], (640448, '744850'): ['69', '70', '71', '72'], (640448, '744845'): ['63', '64', '65'], (640448, '744849'): ['77', '78', '79'], (640448, '746464'): ['74', '78', '79', '80'], (640448, '746462'): ['81', '82', '83', '84'], (640448, '744846'): ['62', '63', '64', '65'], (640448, '744839'): ['65', '66', '67', '68'], (640448, '744842'): ['63', '64', '65'], (640448, '746541'): ['74', '75', '76', '77', '78'], (640448, '746536'): ['62', '63', '64', '65', '66'], (640448, '745071'): ['71', '72', '73', '74'], (640448, '744840'): ['67', '68', '69'], (640448, '744838'): ['64', '65', '66'], (640448, '744837'): ['66', '67', '68'], (640448, '744836'): ['74', '75', '76', '77', '78'], (640448, '745799'): ['63', '64', '65', '66', '67', '68', '69'], (640448, '745965'): ['72', '73', '74'], (640448, '745963'): ['74', '75', '76', '77'], (640448, '744829'): ['71', '72'], (640448, '744830'): ['75', '76', '77'], (640448, '744831'): ['66', '67', '68'], (640448, '745146'): ['87', '88', '89', '90'], (640448, '747172'): ['81', '82', '83', '84', '85', '86'], (640448, '744823'): ['62', '63', '67', '68', '69', '70'], (640448, '744826'): ['65', '66', '67'], (640448, '744819'): ['66', '67', '68', '69', '70', '71'], (640448, '744818'): ['67', '68', '69', '74', '75', '76', '77'], (640448, '745542'): ['74', '75', '76', '77'], (640448, '745539'): ['77', '78', '79'], (640448, '747078'): ['64', '70', '71', '72', '73', '74'], (640448, '744811'): ['62', '66', '67', '68', '69', '70', '71'], (640448, '744809'): ['66', '67', '68', '69', '70'], (640448, '746019'): ['63', '64', '68', '69', '70', '71'], (640448, '745455'): ['69', '70', '71', '72', '73', '74'], (640448, '745456'): ['86', '87', '88'], (640448, '744804'): ['67', '68', '69'], (640448, '744800'): ['64', '65', '66', '67'], (640448, '744803'): ['65', '66', '67'], (640448, '746826'): ['68', '69', '70', '71'], (640448, '744802'): ['70', '71', '72', '73', '74'], (640448, '744796'): ['65', '66', '67', '68', '69', '70', '71'], (640448, '744799'): ['66', '67', '68', '69'], (663432, '746737'): ['62', '63', '64', '65', '66'], (663432, '744875'): ['75', '76', '77', '78', '79', '80', '81', '82'], (663432, '744870'): ['63', '64', '65', '66', '68'], (663432, '744874'): ['60', '61', '62', '63', '64'], (663432, '745355'): ['61', '62', '63'], (663432, '746157'): ['65', '66', '67', '68', '69', '70'], (663432, '744872'): ['67', '68', '69'], (663432, '744865'): ['68', '69', '70', '71', '72'], (663432, '744864'): ['63', '64', '65', '66', '67', '68', '69', '70'], (663432, '746075'): ['80', '81', '82', '83', '84', '85', '86'], (663432, '745018'): ['60', '61', '62', '63', '64', '65', '66'], (663432, '744862'): ['72', '73', '74', '75', '76'], (663432, '745577'): ['68', '69', '70', '71', '72', '73'], (663432, '744860'): ['66', '67', '68', '69'], (663432, '744854'): ['80', '81', '82', '83'], (663432, '744852'): ['64', '65', '66'], (663432, '746465'): ['66', '67', '68'], (663432, '744843'): ['62', '63', '64', '65', '66'], (663432, '746540'): ['76', '77', '78', '85', '86', '87'], (663432, '745399'): ['68', '69', '70'], (663432, '745400'): ['57', '58', '59', '60', '61', '62', '63', '64'], (663432, '745072'): ['62', '63', '64'], (663432, '744835'): ['89', '90', '91'], (663432, '744832'): ['74', '75', '76', '77', '78'], (663432, '745962'): ['58', '59', '60', '61', '62', '63'], (663432, '744828'): ['64', '65', '66'], (663432, '744824'): ['59', '60', '61', '62', '63'], (663432, '745150'): ['87', '88', '89', '90', '91'], (663432, '747167'): ['71', '72', '73', '74', '75', '76', '77', '78'], (663432, '744825'): ['77', '78', '79'], (663432, '744817'): ['81', '82', '83', '84'], (663432, '744822'): ['63', '64', '65'], (663432, '744820'): ['50', '51', '52'], (663432, '744816'): ['74', '75', '76', '77', '78', '79'], (663432, '745545'): ['53', '54', '55', '61', '62', '63', '64', '65'], (663432, '744814'): ['51', '52', '53'], (663432, '744812'): ['67', '68', '69', '70'], (663432, '747077'): ['52', '53', '54'], (663432, '744813'): ['33', '34', '35', '40', '41'], (663432, '744810'): ['53', '54', '55'], (663432, '744808'): ['67', '68', '69'], (663432, '745451'): ['72', '73', '74', '75'], (663432, '745456'): ['48', '54', '55', '56', '57'], (663432, '744807'): ['65', '66', '67', '68', '69', '70', '71'], (663432, '744805'): ['69', '70'], (663432, '745772'): ['61', '62', '63'], (663432, '746824'): ['57', '62', '63', '64'], (663432, '746825'): ['47', '48', '49', '50'], (663432, '744795'): ['51', '52', '53'], (663432, '744797'): ['63', '64', '65', '66'], (676395, '746737'): ['41', '42', '43'], (676395, '746739'): ['48', '49', '50', '51', '52'], (676395, '744875'): ['57', '64', '65'], (676395, '744869'): ['46', '47', '48'], (676395, '744873'): ['54', '55', '56', '57'], (676395, '745357'): ['39', '40', '41', '42'], (676395, '745679'): ['54', '58', '59', '60'], (676395, '745676'): ['49', '53', '54', '55', '56'], (676395, '746159'): ['50', '51', '52'], (676395, '744872'): ['35', '36', '37', '38'], (676395, '744865'): ['46', '47', '48'], (676395, '744859'): ['39', '40', '41'], (676395, '744858'): ['50', '51', '52'], (676395, '746964'): ['53'], (676395, '746963'): ['56', '57', '58', '59', '60', '61'], (676395, '746796'): ['48', '49', '50'], (676395, '746795'): ['63'], (676395, '745581'): ['58', '59', '60'], (676395, '745577'): ['43', '44', '45', '46'], (676395, '744856'): ['49', '50', '56', '57'], (676395, '744851'): ['39', '40', '41'], (676395, '747117'): ['58', '59', '60'], (676395, '747114'): ['59'], (676395, '746635'): ['57', '58', '59'], (676395, '746632'): ['53', '54', '55'], (676395, '744848'): ['71', '72', '73'], (676395, '744849'): ['53', '54', '55', '60', '61', '62'], (676395, '746462'): ['57', '58', '59', '60', '67'], (676395, '746465'): ['58', '59', '60'], (676395, '744839'): ['61'], (676395, '744841'): ['53', '54', '55'], (676395, '746540'): ['67', '68', '69', '70'], (676395, '746536'): ['46', '47', '48'], (676395, '745397'): ['70'], (676395, '744840'): ['50', '51', '52'], (676395, '744838'): ['71', '72', '73', '74', '75', '76', '77'], (676395, '744837'): ['58', '59', '60'], (676395, '744836'): ['46', '47', '48', '49'], (676395, '744832'): ['41', '46', '47', '48'], (676395, '745799'): ['54', '55', '56'], (676395, '745965'): ['44', '45', '49', '50', '51', '52'], (676395, '744830'): ['32', '33', '34'], (676395, '744831'): ['56', '57', '58', '59'], (676395, '744828'): ['48', '49', '50'], (676395, '745146'): ['71', '72', '73'], (676395, '745145'): ['54', '55', '56', '57'], (676395, '747172'): ['74', '75', '76'], (676395, '747170'): ['54'], (676395, '744825'): ['47', '48', '49', '50'], (676395, '744826'): ['55', '56', '57', '58', '59'], (676395, '744817'): ['73', '74', '75'], (676395, '744819'): ['72', '79', '80', '81', '82', '83', '84'], (676395, '744816'): ['58', '62', '63', '64', '65'], (676395, '747004'): ['59', '60', '61'], (676395, '747000'): ['51', '52', '53', '54'], (676395, '745542'): ['57', '58', '59'], (676395, '744814'): ['66', '67', '68', '69'], (676395, '744815'): ['51', '52', '53', '58'], (676395, '747080'): ['63'], (676395, '747077'): ['49', '50', '51'], (676395, '744810'): ['42'], (676395, '744808'): ['44', '45', '46', '50', '51', '52', '53'], (676395, '746020'): ['75', '76', '77'], (676395, '745455'): ['45', '46', '47', '51', '52', '53'], (676395, '745456'): ['73', '74', '75'], (676395, '744805'): ['61', '62'], (676395, '744803'): ['50', '51', '52', '53'], (676395, '744801'): ['52', '53'], (676395, '746823'): ['61', '62', '63', '64', '65', '66'], (676395, '746824'): ['42', '43', '44'], (676395, '744795'): ['45', '46', '47'], (676395, '744797'): ['48', '49', '50'], (678606, '744829'): ['64', '65', '66', '67'], (678606, '744828'): ['58', '59'], (678606, '744827'): ['56', '57', '58', '59'], (678606, '745150'): ['57', '58', '65', '66', '67', '68'], (678606, '747172'): ['63', '64', '65', '66', '67'], (678606, '744825'): ['65', '66', '67', '68'], (678606, '744821'): ['68', '69', '70'], (678606, '744822'): ['44', '45', '46', '47', '48'], (678606, '744820'): ['58', '59', '60', '61'], (678606, '744816'): ['43', '44', '45', '46', '47'], (678606, '747004'): ['67', '68', '69', '70'], (678606, '745539'): ['59', '60', '61'], (678606, '744814'): ['54', '55', '59', '60', '61'], (678606, '747080'): ['75'], (678606, '747077'): ['58', '59', '60', '61'], (678606, '744813'): ['65', '71', '72', '73'], (678606, '744811'): ['54'], (678606, '744809'): ['53', '54', '55'], (678606, '744806'): ['54', '59', '60', '61'], (678606, '746020'): ['55', '59', '60', '61'], (678606, '746019'): ['59', '60', '61', '62'], (678606, '745456'): ['37', '38', '39', '40', '45', '46', '47'], (678606, '744804'): ['57', '58', '59', '60', '61'], (678606, '744800'): ['47', '48', '49', '50', '56', '57', '58', '59'], (678606, '744801'): ['66', '67', '68', '69', '70'], (678606, '745773'): ['63', '64', '65'], (678606, '746823'): ['70', '71', '72', '73'], (678606, '746826'): ['59', '60', '61'], (678606, '744802'): ['58', '59', '60', '64', '65', '66'], (678606, '744796'): ['58', '59', '60'], (678606, '744799'): ['47', '54', '55', '56', '57', '58'], (674285, '745341'): ['54', '55', '56', '57', '58', '64', '65', '66', '67', '68'], (674285, '745962'): ['44', '45', '46', '51', '52', '53', '54'], (674285, '745150'): ['72', '73', '74', '75', '76', '77'], (674285, '747167'): ['37', '38', '39', '40', '45', '46', '47', '48', '52', '53', '54', '55', '56', '57'], (674285, '744825'): ['69', '70', '71', '72'], (674285, '744821'): ['58', '59', '60', '61', '62'], (674285, '744822'): ['49', '53', '54', '55'], (674285, '744819'): ['25', '30', '31', '32', '33'], (674285, '744820'): ['37', '38', '39', '40', '44', '45', '46'], (674285, '747000'): ['61', '62', '63', '64', '65'], (674285, '745545'): ['35', '36', '37', '38', '39', '40', '45', '46', '47'], (674285, '744814'): ['45', '46'], (674285, '747080'): ['67', '68', '69', '73', '74'], (674285, '747078'): ['55', '56', '57', '61', '62', '63'], (674285, '744811'): ['51', '52', '53'], (674285, '744810'): ['59', '60', '61'], (674285, '744808'): ['58', '59', '60', '61'], (674285, '745451'): ['57', '58', '59'], (674285, '745453'): ['63', '64', '65', '66'], (674285, '744800'): ['41', '42', '43'], (674285, '744801'): ['49', '50', '51'], (674285, '745772'): ['37', '41', '42', '43', '44', '45'], (674285, '746824'): ['51', '52', '53', '54', '55', '56'], (674285, '746825'): ['54', '55', '56', '57'], (674285, '744795'): ['42', '43', '44'], (674285, '744797'): ['55', '56', '57'], (621242, '745850'): ['79', '80', '81', '82'], (621242, '745842'): ['59', '60', '61'], (621242, '745844'): ['70', '71', '72'], (621242, '746733'): ['75', '76', '77', '78', '79'], (621242, '746734'): ['72', '73', '74'], (621242, '745841'): ['66', '67', '68', '69'], (621242, '745840'): ['73', '74', '75', '76'], (621242, '746156'): ['79', '80', '81', '82'], (621242, '745346'): ['78', '79', '80', '81', '82'], (621242, '745834'): ['63', '64', '65'], (621242, '745832'): ['57', '58', '59', '60', '61'], (621242, '745830'): ['74', '75', '76', '80', '81'], (621242, '745103'): ['81', '82', '83', '84', '85'], (621242, '745182'): ['71', '72', '73'], (621242, '745825'): ['70', '71', '72'], (621242, '745826'): ['70', '71', '72', '73', '74', '75', '76', '77'], (621242, '745580'): ['70', '71', '72', '73', '74'], (621242, '746067'): ['81', '82', '83', '84', '85'], (621242, '745822'): ['50', '51', '52', '53'], (621242, '745823'): ['65', '66', '67', '68'], (621242, '745809'): ['62', '63', '64'], (621242, '745810'): ['61', '62', '63', '64'], (621242, '745004'): ['76', '77', '78'], (621242, '745476'): ['72', '73', '74'], (621242, '745477'): ['61', '62', '63', '72', '73', '74'], (621242, '745803'): ['75'], (621242, '745800'): ['72', '73', '74', '75', '76'], (621242, '746041'): ['73', '74', '75'], (621242, '746034'): ['77', '78', '79', '80', '81', '82'], (621242, '745794'): ['59', '60', '61'], (621242, '745787'): ['55', '60', '61', '62'], (621242, '745184'): ['69', '70', '71'], (621242, '746521'): ['75', '76', '77', '78'], (621242, '745788'): ['78', '79', '80', '81'], (621242, '745783'): ['85', '86', '87', '88'], (621242, '745780'): ['64', '65', '66'], (621242, '745781'): ['66', '67', '68'], (621242, '745376'): ['70', '71'], (621242, '747161'): ['66', '67', '68', '69'], (621242, '747159'): ['68', '69', '70'], (621242, '746749'): ['72', '73', '74'], (621242, '746750'): ['65', '66', '67'], (621242, '745776'): ['71', '72', '73', '74'], (621242, '745777'): ['63', '64', '65'], (621242, '745775'): ['65', '66', '67', '68', '69'], (621242, '744888'): ['67', '68', '69', '70'], (621242, '744881'): ['76'], (621242, '745530'): ['64', '65', '66', '67'], (621242, '745773'): ['59', '60', '61', '62'], (621242, '745769'): ['78', '79', '80'], (621242, '745768'): ['63', '70', '71', '72'], (621242, '745767'): ['55', '56', '57', '58', '63', '64', '65', '66', '67'], (621242, '745932'): ['73', '74', '75', '76'], (621242, '747139'): ['69', '70', '71', '72', '73', '79', '80', '81', '82'], (623211, '746062'): ['59', '60', '61', '62', '66', '67', '68', '69', '70', '71', '72'], (623211, '746060'): ['49', '50', '51', '52', '56', '57', '58', '59'], (623211, '746057'): ['40', '41', '42', '47', '48', '49', '54', '55', '56', '57', '63', '64', '65', '66'], (623211, '744847'): ['30', '31', '32', '33', '34', '38', '39', '40', '44', '45', '46', '52', '53', '54'], (623211, '746055'): ['45', '46', '47'], (623211, '746050'): ['55', '56', '60', '61', '62'], (623211, '746052'): ['46', '47'], (623211, '746051'): ['50', '51', '52', '53'], (623211, '746293'): ['54', '55'], (623211, '746291'): ['56', '57', '58', '59', '60', '61', '62'], (623211, '745555'): ['53', '54', '58', '59', '60', '61'], (623211, '746045'): ['30', '31', '32', '33', '39', '40', '41', '42', '46', '47', '48', '49'], (623211, '746044'): ['48', '49', '50', '54', '55', '56', '57'], (623211, '746368'): ['46', '51', '52', '53', '54'], (623211, '746687'): ['41', '42', '43', '49', '50', '51'], (623211, '746041'): ['59', '60', '61', '62'], (623211, '746034'): ['51', '52', '53', '58', '59', '60'], (623211, '746035'): ['45', '46', '47'], (623211, '745964'): ['66', '67', '68'], (623211, '745966'): ['65', '66', '67'], (623211, '745789'): ['68', '69', '70', '71'], (623211, '746194'): ['57', '58', '59', '60', '61', '62'], (623211, '746519'): ['50', '51', '52', '58', '59', '60'], (623211, '745219'): ['32', '33', '34', '38', '39', '40'], (623211, '745790'): ['51'], (623211, '745783'): ['46', '47', '48', '49', '50', '51'], (623211, '745786'): ['49', '50', '51', '52', '53'], (623211, '745782'): ['58', '59', '60', '61'], (623211, '745379'): ['76', '77', '78', '79', '80', '81', '82'], (623211, '747161'): ['44', '49', '50', '51'], (623211, '746751'): ['51', '52', '53', '54'], (623211, '745776'): ['44', '45', '46', '50', '51'], (623211, '744886'): ['47', '48', '49', '54', '55', '56', '62', '63'], (623211, '745772'): ['64', '65', '66'], (623211, '745771'): ['68', '69', '70'], (623211, '745770'): ['40', '41', '42', '43', '50', '51', '52', '53'], (623211, '747068'): ['47', '48', '49', '50'], (623211, '747064'): ['51', '52', '53', '54', '55', '56'], (623211, '747139'): ['48', '49'], (592773, '745276'): ['51', '52', '53', '54'], (592773, '745277'): ['70', '71', '72'], (592773, '746007'): ['62', '63', '64', '65', '66', '67'], (592773, '744957'): ['57', '58', '59', '60'], (592773, '744950'): ['77', '78', '79', '80'], (592773, '745274'): ['67', '68', '69', '70'], (592773, '745270'): ['68', '69', '70', '71', '72'], (592773, '746568'): ['64', '65', '66', '70', '71', '72'], (592773, '745026'): ['57'], (592773, '745027'): ['50', '51', '52', '53', '54'], (592773, '745266'): ['52', '53', '54', '55', '56'], (592773, '745260'): ['56', '57', '58', '59'], (592773, '746395'): ['68', '69', '70', '71'], (592773, '745915'): ['68', '69'], (592773, '745262'): ['47', '48', '49', '53'], (592773, '745256'): ['59', '60', '61', '62', '63'], (592773, '745259'): ['48', '49', '50'], (592773, '747036'): ['57', '58'], (592773, '745742'): ['62', '63', '64', '65'], (592773, '744851'): ['58', '59', '60'], (592773, '744855'): ['53', '54', '55', '56', '57'], (592773, '745258'): ['64', '65', '66'], (592773, '745253'): ['59', '60', '61'], (592773, '745252'): ['54', '55', '56', '57'], (592773, '745656'): ['57', '58', '59'], (592773, '745657'): ['65', '66', '67'], (592773, '746302'): ['81', '82', '83', '84', '87'], (592773, '745248'): ['70', '71', '72', '73', '74'], (592773, '745244'): ['70', '71', '72'], (592773, '745247'): ['65', '66', '67', '68'], (592773, '745241'): ['76', '77'], (592773, '746627'): ['69'], (592773, '746626'): ['56', '57', '58', '59'], (592773, '746052'): ['56', '57', '58'], (592773, '745078'): ['58', '59', '60', '61'], (592773, '745073'): ['58', '59', '60', '61'], (592773, '745238'): ['71', '72', '73'], (592773, '745243'): ['64', '65', '66', '67', '68'], (592773, '745236'): ['49', '50', '51'], (592773, '745237'): ['57', '58', '59'], (592773, '745233'): ['53', '54', '55', '56', '57', '58'], (592773, '745394'): ['57', '58', '59', '60'], (592773, '746207'): ['66', '67', '68'], (592773, '746201'): ['58'], (592773, '745229'): ['49', '50', '51', '52', '53'], (592773, '745232'): ['53', '54', '55'], (592773, '745795'): ['49', '50', '51', '52', '53', '54'], (592773, '745787'): ['51', '52', '53', '54'], (592773, '745184'): ['61', '62', '63'], (592773, '746522'): ['62', '63', '64'], (592773, '745216'): ['48', '49', '50', '51', '52'], (592773, '745788'): ['64', '65', '66'], (592773, '745783'): ['78', '79', '80', '81'], (592773, '745782'): ['51', '52', '53', '54'], (592773, '745378'): ['26', '27', '28', '29', '33', '34', '35'], (592773, '746751'): ['60', '61', '62'], (592773, '745778'): ['66', '67', '68'], (592773, '744888'): ['61', '62', '63'], (592773, '744881'): ['71', '72', '73', '74', '75'], (592773, '745528'): ['65', '66', '67', '68', '69'], (592773, '745769'): ['70', '71', '72', '73'], (592773, '747068'): ['40', '41', '42', '43'], (592773, '745931'): ['50', '51', '52'], (493603, '745850'): ['67', '68', '69', '70', '71', '72'], (493603, '745844'): ['63', '64', '65', '66'], (493603, '746733'): ['69', '70', '71'], (493603, '746734'): ['66', '67', '68'], (493603, '745838'): ['58', '59', '60'], (493603, '745841'): ['46', '47', '48'], (493603, '745840'): ['63', '64', '65'], (493603, '746160'): ['64', '65', '72', '73', '74'], (493603, '745346'): ['63', '68', '69', '70', '71'], (493603, '745831'): ['59', '60', '61'], (493603, '745829'): ['52', '53', '54', '55'], (493603, '745100'): ['59', '60', '61', '62', '63', '64'], (493603, '745182'): ['63', '64', '65', '66'], (493603, '745186'): ['80', '81', '82', '83', '84', '85'], (493603, '745825'): ['61', '62', '63'], (493603, '745826'): ['56', '57', '64', '65', '66'], (493603, '745580'): ['63', '64', '65', '66'], (493603, '746067'): ['70', '71', '72'], (493603, '746634'): ['62', '63', '64', '65', '66', '67'], (493603, '745823'): ['56', '57', '58', '59', '60'], (493603, '745818'): ['60', '61', '62', '63', '64'], (493603, '745817'): ['64', '65', '66', '67', '68'], (493603, '745813'): ['60', '61', '62'], (493603, '744854'): ['84', '85', '86', '87', '88'], (493603, '745814'): ['65', '66'], (493603, '745815'): ['64', '65', '66', '67'], (493603, '745810'): ['39', '40', '44', '45'], (493603, '745005'): ['65', '66', '67', '74', '75', '76', '77'], (493603, '746861'): ['63', '64', '65', '70', '71', '72', '73', '74'], (493603, '745806'): ['68', '69', '70'], (493603, '745808'): ['52', '53', '54'], (493603, '745804'): ['39', '40', '41', '42'], (493603, '745802'): ['73', '74', '75'], (493603, '744837'): ['48', '52', '53'], (493603, '745803'): ['57', '58', '59', '60'], (493603, '745799'): ['70', '71', '72', '73', '74', '75'], (493603, '746040'): ['53', '54', '55'], (493603, '745715'): ['49', '50', '51'], (493603, '745716'): ['45'], (493603, '745792'): ['65', '66', '67', '68', '69'], (493603, '745795'): ['69', '70', '71'], (493603, '745789'): ['52', '53', '54'], (493603, '746194'): ['68', '69', '70', '71'], (493603, '746191'): ['49', '50', '51'], (493603, '746522'): ['52', '53', '54', '55', '56'], (493603, '745220'): ['54', '55', '56', '57'], (493603, '745216'): ['53', '54', '55', '56'], (493603, '745790'): ['69', '70', '71', '72', '73'], (493603, '745782'): ['45', '46', '47'], (493603, '745378'): ['53', '54', '55', '56', '57', '61', '62', '63', '64', '65', '66'], (493603, '747162'): ['78', '79', '80'], (493603, '746749'): ['51', '52'], (493603, '745775'): ['51'], (493603, '744886'): ['70', '71', '72', '73'], (493603, '745772'): ['55', '56', '57', '58', '59', '60'], (493603, '745770'): ['33', '34', '35', '36'], (493603, '747068'): ['55', '56', '57', '58'], (493603, '745930'): ['54', '59', '60', '61', '62'], (493603, '747064'): ['60', '61', '62'], (493603, '747139'): ['53', '54', '55'], (664208, '745116'): ['70', '71', '72'], (664208, '745119'): ['57'], (664208, '745109'): ['68', '69', '71', '72', '73', '74', '75'], (664208, '746574'): ['54', '59', '60', '61', '62'], (664208, '746250'): ['50', '51', '52', '53'], (664208, '746247'): ['57', '58', '59'], (664208, '745112'): ['49'], (664208, '745106'): ['55', '56', '57', '58', '59', '60', '61', '62', '63'], (664208, '745108'): ['83', '84', '85'], (664208, '745104'): ['53', '54', '55', '56'], (664208, '745760'): ['54', '55', '56', '57'], (664208, '745759'): ['47'], (664208, '745102'): ['48', '49', '50', '51'], (664208, '746803'): ['88', '89'], (664208, '745101'): ['62', '63', '64'], (664208, '745100'): ['67', '68', '69', '70'], (664208, '745095'): ['61', '62', '63'], (664208, '745096'): ['62', '63', '64'], (664208, '746962'): ['51', '52', '53'], (664208, '746960'): ['50', '51', '52', '53', '54', '55'], (664208, '744937'): ['60', '61', '62'], (664208, '745089'): ['57', '58', '59'], (664208, '745087'): ['63', '64', '65', '66', '67'], (664208, '745086'): ['43', '44', '45'], (664208, '745081'): ['66', '67', '68'], (664208, '747032'): ['51', '52', '53'], (664208, '746060'): ['80', '81', '82'], (664208, '745082'): ['60', '61', '62', '63', '64', '65'], (664208, '745085'): ['59', '60', '61', '62', '63', '64', '65'], (664208, '745079'): ['62', '63', '64', '65'], (664208, '747112'): ['38', '39', '40', '44', '45', '46', '47'], (664208, '745893'): ['78'], (664208, '745887'): ['83', '84', '85'], (664208, '745483'): ['56', '57', '58', '62', '63', '64'], (664208, '745078'): ['57', '62', '63', '64'], (664208, '745076'): ['51', '52', '53', '54'], (664208, '745072'): ['65', '66', '67'], (664208, '746283'): ['68', '69', '70'], (664208, '744992'): ['56', '57', '58', '59'], (664208, '744995'): ['64', '65', '66'], (664208, '745799'): ['51', '52', '53'], (664208, '745800'): ['66', '67', '68'], (664208, '745797'): ['49'], (664208, '746040'): ['60', '61', '62', '63'], (664208, '746034'): ['68', '69', '70'], (664208, '745715'): ['68', '69', '70', '71'], (664208, '745794'): ['65', '66'], (664208, '745789'): ['58', '59', '60', '61', '62'], (664208, '746193'): ['75', '76', '77'], (664208, '746191'): ['62', '63', '64', '65'], (664208, '745219'): ['52', '53', '54', '55', '56', '57'], (664208, '745216'): ['66', '67', '73', '74', '75', '76'], (664208, '745788'): ['72', '73', '74'], (664208, '745783'): ['67', '68', '69', '70', '71', '72'], (664208, '745786'): ['63', '64', '68', '69', '70'], (664208, '745379'): ['64', '65', '66'], (664208, '745376'): ['66'], (664208, '747162'): ['69', '70', '71'], (664208, '747161'): ['63', '64', '65'], (664208, '746749'): ['64', '65', '66'], (664208, '746751'): ['66', '67', '68'], (664208, '745779'): ['65', '66', '67'], (664208, '745776'): ['59', '60', '61'], (664208, '745774'): ['62', '63', '64', '65', '66'], (664208, '745529'): ['61', '62', '63'], (664208, '745530'): ['60'], (664208, '745771'): ['59', '60', '61', '62', '63'], (664208, '745767'): ['35', '36', '37', '41', '42', '43'], (664208, '745931'): ['39', '40', '44', '45', '46'], (664208, '745932'): ['64', '65', '66', '67', '68'], (664208, '747139'): ['66', '67', '68'], (676130, '745843'): ['1', '2', '3', '4', '5', '10', '11', '12', '13', '14', '15', '20', '21', '22', '27', '28', '29', '33', '34', '35', '39', '40', '41'], (676130, '745841'): ['1', '2', '3', '8', '9', '10', '11', '12', '16', '17', '18', '22', '23', '24', '29', '30', '31', '37', '38', '39', '40'], (676130, '746156'): ['8', '9', '10', '11', '12', '13', '14', '19', '20', '21', '22', '23', '24', '30', '31', '32', '39', '40', '41', '45', '46', '47'], (676130, '745835'): ['1', '2', '3', '8', '9', '10', '11', '12', '13', '14', '19', '20', '21', '22', '27', '28', '29', '33', '34', '35', '40', '41', '42', '43'], (676130, '745829'): ['1', '2', '3', '4', '5', '9', '10', '11', '12', '17', '18', '19', '24', '25', '26', '30', '31', '32', '33', '34', '39', '40', '41'], (676130, '745186'): ['5', '6', '7', '8', '9', '10', '11', '17', '18', '19', '20', '21', '26', '27', '28', '32', '33', '34', '46', '47', '48', '49'], (676130, '745821'): ['1', '2', '3', '4', '8', '9', '10', '14', '15', '16', '17', '18', '19', '20', '21', '25', '26', '27', '31', '32', '33'], (676130, '744838'): ['61', '62', '63', '67', '68', '69', '70'], (676130, '745476'): ['43', '44', '51', '52', '53'], (676130, '745796'): ['64', '70', '71', '72'], (676130, '745798'): ['43', '48', '49', '50', '55', '56', '57', '61', '62'], (676130, '746041'): ['55', '56', '57', '58'], (676130, '746034'): ['54', '55', '56', '57', '61', '62', '63', '64'], (676130, '745794'): ['41', '42', '43', '47', '48', '49', '53', '54', '55'], (676130, '745791'): ['62', '63', '64', '65', '66', '75', '76', '77', '78', '83', '84', '85', '86', '87'], (676130, '746193'): ['62', '63', '64', '68', '69', '70'], (676130, '746521'): ['56', '57', '58', '59', '64', '65', '66'], (676130, '745788'): ['40', '41', '42', '43', '44', '53', '54', '55', '56', '57', '58', '59'], (676130, '745784'): ['65', '66', '67', '71', '72', '73', '74'], (676130, '745780'): ['57', '58', '59', '60'], (676130, '745781'): ['57', '58', '59', '60', '61'], (676130, '745376'): ['56', '57', '61', '62', '63', '64', '65'], (676130, '747159'): ['59', '60', '61', '62'], (676130, '746751'): ['74', '75', '76', '77', '78', '79'], (676130, '745778'): ['46', '47', '48', '54', '55'], (676130, '745777'): ['70', '71', '72'], (676130, '744888'): ['51', '52', '53', '54'], (676130, '745773'): ['50', '51', '52'], (676130, '745767'): ['48', '49', '50'], (676130, '745930'): ['39', '44', '45', '46'], (657585, '745843'): ['46', '47', '48', '52', '53', '54', '59', '60', '61', '62', '63'], (657585, '747142'): ['28', '34', '35', '36', '37', '44', '45', '46'], (657585, '745838'): ['48', '52', '53', '54'], (657585, '745837'): ['38', '39', '40', '44', '45', '46', '47', '48'], (657585, '746160'): ['49', '50', '51', '52', '53', '54', '55'], (657585, '746156'): ['86', '87', '88'], (657585, '745346'): ['48', '52', '53', '54', '58', '59', '60', '61', '62'], (657585, '745834'): ['70', '71', '73', '77', '78', '79'], (657585, '745831'): ['52', '53', '54', '55'], (657585, '745830'): ['66', '67', '68', '69', '73'], (657585, '745100'): ['53'], (657585, '745103'): ['63', '64', '65', '66'], (657585, '745186'): ['71', '72', '73', '74'], (657585, '745825'): ['43', '44', '45', '46', '53', '54', '55'], (657585, '745580'): ['46', '47', '48', '52', '53', '54', '55', '56'], (657585, '746067'): ['64', '65', '66'], (657585, '746064'): ['66', '67', '68', '77', '78', '79', '80'], (657585, '746634'): ['52', '53', '54', '55', '56', '57'], (657585, '745822'): ['58', '59', '60', '61', '62', '63', '64', '65'], (657585, '745818'): ['51', '52', '53'], (657585, '745819'): ['65', '66', '67', '68'], (657585, '745820'): ['80', '81', '82', '83'], (657585, '745813'): ['54', '55', '56'], (657585, '744850'): ['73', '74', '75', '76', '77', '78'], (657585, '745571'): ['61', '62', '66', '67', '68', '77', '78', '79'], (657585, '745811'): ['77', '78', '79'], (657585, '745805'): ['69', '70', '71'], (657585, '745004'): ['67', '68', '69'], (657585, '746862'): ['64', '65', '66', '67'], (657585, '745806'): ['74', '75', '80', '81', '82'], (657585, '745804'): ['78', '79', '80', '81', '82'], (657585, '745801'): ['70', '71', '72'], (657585, '744840'): ['88', '89'], (657585, '745476'): ['57', '58', '59', '60', '61', '62'], (657585, '745477'): ['48', '49', '50'], (657585, '745803'): ['70', '71', '72', '73', '74'], (657585, '745219'): ['45', '46', '47'], (657585, '745783'): ['56', '57', '58', '59', '60', '61', '62'], (657585, '745786'): ['54', '59', '60', '61', '62'], (657585, '745379'): ['57', '58', '59'], (657585, '747162'): ['61', '62', '63'], (657585, '747161'): ['58'], (657585, '746749'): ['56', '57', '58'], (657585, '746750'): ['56', '57', '58'], (657585, '745778'): ['56', '57', '58'], (657585, '745777'): ['51', '57', '58', '59'], (657585, '745774'): ['52', '56', '57', '58'], (657585, '744881'): ['52', '56', '57', '58'], (657585, '745528'): ['57', '58', '59', '60'], (657585, '745773'): ['66', '67', '68'], (657585, '745769'): ['62', '63', '64'], (657585, '745768'): ['60', '61', '62'], (657585, '745931'): ['56', '57', '58', '59', '60', '61', '62'], (605452, '746890'): ['5', '6', '7', '8', '14', '15', '16', '17', '21', '22', '23', '29', '30', '31', '32', '33', '37', '38', '39', '40', '44', '45', '46', '47'], (605452, '746312'): ['7', '8', '9', '10', '11', '15', '16', '17', '21', '22', '23', '24', '30', '31', '32', '36', '37', '38', '39', '44'], (605452, '745989'): ['1', '2', '3', '4', '5', '10', '11', '12', '22', '23', '24', '28', '29', '30', '31', '32', '36', '37', '38'], (605452, '746066'): ['4', '5', '6', '7'], (605452, '745958'): ['1', '2', '3', '4', '8', '9', '10', '14', '15', '16', '17', '18', '19', '20', '21', '22', '27', '28', '29', '30', '35', '36', '37', '38'], (605452, '747086'): ['68', '69', '70', '71', '72', '79', '80', '81'], (605452, '745956'): ['52', '53', '55', '56', '57', '61', '62', '63', '67', '68', '69'], (605452, '745954'): ['70', '71', '72', '73'], (605452, '745138'): ['41', '42', '43', '44', '50', '51', '52', '53'], (605452, '745625'): ['77', '78', '79', '80'], (605452, '745946'): ['63', '64', '65', '70', '71', '72', '73'], (605452, '746665'): ['61', '62', '63', '77', '78', '79', '80'], (605452, '745947'): ['67', '68', '69', '70', '74', '75', '76'], (605452, '745940'): ['45', '46', '47', '48', '52', '53', '54', '55', '59', '60', '61', '62'], (605452, '747151'): ['36', '37', '38', '39', '46', '47', '48', '49', '58', '59'], (605452, '745933'): ['39', '40', '41', '42'], (605452, '745937'): ['53', '54', '55', '56', '57'], (605452, '745936'): ['53', '54', '55', '56', '57', '61', '62', '63'], (605452, '745441'): ['74', '75', '76', '77', '78'], (605452, '745930'): ['40', '41', '42', '43', '47', '48', '49', '55', '56', '57', '58'], (571948, '746006'): ['68', '69', '70'], (571948, '746728'): ['62', '63', '64'], (571948, '747054'): ['70', '78', '79', '80'], (571948, '747056'): ['66', '67', '68', '69', '70', '71'], (571948, '746004'): ['40', '46', '47', '48'], (571948, '745189'): ['78', '79', '81'], (571948, '745190'): ['58', '59', '60', '66'], (571948, '745519'): ['47', '48', '49', '50', '51'], (571948, '745512'): ['49'], (571948, '746000'): ['53', '54', '55', '60'], (571948, '745997'): ['64', '65'], (571948, '746884'): ['40', '41', '42', '43'], (571948, '746313'): ['55', '60', '61', '62', '63'], (571948, '746315'): ['44', '48', '49', '50', '51'], (571948, '745995'): ['54', '55', '56'], (571948, '745994'): ['55', '56', '57', '58'], (571948, '745990'): ['66', '67', '68'], (571948, '746388'): ['65', '66', '67', '68', '69'], (571948, '746389'): ['42', '46', '47', '48', '49'], (571948, '746066'): ['41', '42', '43', '48', '49'], (571948, '746958'): ['66', '67', '68', '72', '73', '74', '75'], (571948, '745986'): ['63', '64', '65', '66', '67'], (571948, '745987'): ['69', '70', '71', '72', '73', '74'], (571948, '745983'): ['44', '45', '46', '52', '53', '54'], (571948, '745980'): ['74', '75', '76'], (571948, '745569'): ['58', '59', '60', '61'], (571948, '745570'): ['49', '50', '51'], (571948, '746463'): ['44', '45', '46'], (571948, '745978'): ['60', '61', '62', '67', '68', '69'], (571948, '745979'): ['53', '54'], (571948, '746215'): ['55', '56', '57', '58'], (571948, '745401'): ['60', '61', '62', '63', '65'], (571948, '745403'): ['63', '64', '65'], (571948, '745973'): ['1', '2', '3', '4', '10', '11', '12'], (571948, '746535'): ['48', '49', '50', '51', '52', '57', '58', '59'], (571948, '746537'): ['52'], (571948, '746127'): ['69', '70', '71'], (571948, '745968'): ['56', '57', '58', '59'], (571948, '745965'): ['53', '54', '55', '56', '57', '61', '62', '63'], (571948, '745962'): ['64', '65', '66', '67'], (571948, '746850'): ['52', '56', '57', '58', '59', '60', '65', '66', '67'], (571948, '745964'): ['46', '47', '52', '53', '54', '55', '56'], (571948, '745961'): ['66', '67', '68'], (571948, '745959'): ['40', '41', '48'], (571948, '745960'): ['75'], (571948, '744825'): ['61', '62', '63', '64'], (571948, '744826'): ['41', '42', '43', '44', '45'], (571948, '747081'): ['83', '84', '85', '86', '87', '95', '96', '97'], (571948, '745938'): ['76', '77'], (571948, '745940'): ['40', '41'], (571948, '745285'): ['61', '67', '68', '69'], (571948, '747148'): ['69', '70', '71', '75', '76', '77', '78'], (571948, '745939'): ['44', '45', '46', '53', '54', '55'], (571948, '745937'): ['40', '41', '42', '46', '47', '48'], (571948, '745936'): ['34', '35', '39', '40', '41'], (571948, '745441'): ['42', '43', '44'], (571948, '745930'): ['63', '64', '65', '66', '67', '68'], (571948, '745932'): ['50', '54', '55', '56'], (657649, '746002'): ['66', '67', '68', '69', '73', '74', '75', '76', '77'], (657649, '746004'): ['58', '59', '67', '68', '69'], (657649, '745516'): ['42', '43', '44', '45', '46', '47', '51', '52', '53'], (657649, '745512'): ['62', '63', '64', '65', '66'], (657649, '746000'): ['74', '75', '76', '78', '83', '84', '85'], (657649, '746884'): ['33', '34', '35'], (657649, '746313'): ['48', '49', '50', '51'], (657649, '746315'): ['62', '63', '69', '70', '71', '72'], (657649, '745995'): ['39', '40', '41', '47', '48', '49', '50'], (657649, '745990'): ['40', '41', '42', '48', '49', '50', '51', '52', '53', '54'], (657649, '745992'): ['59', '60', '61', '62'], (657649, '746388'): ['49', '50', '51', '52'], (657649, '746066'): ['11', '12', '13', '17', '18', '19', '24', '25', '26'], (657649, '746956'): ['5', '6', '7', '8'], (657649, '746958'): ['4', '5', '6', '13'], (657649, '745987'): ['40'], (657649, '745988'): ['41', '42', '43', '44', '45', '46', '50', '51', '52', '53'], (657649, '745980'): ['44', '45', '46', '47'], (657649, '745569'): ['4', '5', '6'], (657649, '745568'): ['4', '5', '6', '7', '11'], (657649, '746461'): ['37', '38', '44', '45', '46', '47'], (657649, '745981'): ['47', '48', '49', '54', '55', '56'], (657649, '745976'): ['1', '2', '3'], (657649, '745977'): ['54', '55', '61'], (657649, '745401'): ['47', '51', '52', '53'], (657649, '745403'): ['52', '53', '54', '55', '56'], (657649, '745974'): ['55', '56', '57', '58', '59', '60'], (657649, '745975'): ['79', '80', '81', '82'], (657649, '745970'): ['53', '58', '59', '60'], (657649, '745882'): ['57', '58', '59', '60', '61'], (657649, '746848'): ['58', '59', '60', '61', '62', '63'], (657649, '745966'): ['42', '43', '49', '50', '51', '52'], (657649, '745960'): ['51', '52', '53', '54', '55'], (657649, '744825'): ['73', '74', '75', '76'], (657649, '747085'): ['76', '77', '78', '79'], (657649, '745957'): ['62', '63', '64', '65', '66'], (657649, '745951'): ['54', '55', '56', '57', '58'], (657649, '745952'): ['52', '53', '54'], (657649, '745948'): ['59', '60', '61'], (657649, '745136'): ['62'], (657649, '745137'): ['44', '45', '46', '47'], (657649, '745617'): ['58', '60'], (657649, '745946'): ['57'], (657649, '745943'): ['55', '56', '57', '58', '59'], (657649, '746730'): ['62', '63', '64', '65'], (657649, '746669'): ['53', '58', '59', '60'], (657649, '745944'): ['78', '79', '80', '81'], (657649, '745942'): ['49', '50', '51', '52', '58', '59', '60', '61'], (657649, '747153'): ['55', '56', '57'], (657649, '747151'): ['85', '86', '87', '93', '94', '95', '96', '97'], (657649, '745933'): ['43', '47', '48', '49'], (657649, '745934'): ['51', '52', '54', '55'], (657649, '745929'): ['70', '71', '72', '73', '74'], (657649, '745443'): ['67', '68', '69'], (657649, '745931'): ['1', '2', '3', '4'], (683232, '746900'): ['75', '76', '77', '78', '79', '80'], (683232, '746576'): ['61', '62', '63', '64', '65'], (683232, '746569'): ['74', '75', '76', '77', '78', '79'], (683232, '746573'): ['63', '64', '65', '66', '67'], (683232, '745597'): ['60', '66', '67', '68'], (683232, '746568'): ['67', '68', '69'], (683232, '746565'): ['53', '54', '55', '56', '57', '58'], (683232, '746564'): ['57', '58', '59', '60', '61'], (683232, '746561'): ['58', '59', '64', '65', '66'], (683232, '746072'): ['62', '63', '64', '65', '66', '67'], (683232, '745511'): ['59', '60', '61', '62', '64'], (683232, '746559'): ['57', '62', '63', '64', '65'], (683232, '746554'): ['76', '77', '78', '79'], (683232, '745418'): ['66', '67', '68', '69', '70'], (683232, '745417'): ['64', '65', '66', '67'], (683232, '745339'): ['73', '74', '75', '76', '77', '78', '79', '80', '81'], (683232, '745661'): ['84', '85', '86', '92', '93', '94'], (683232, '746558'): ['64', '65', '66', '72', '73', '74', '75', '76'], (683232, '746557'): ['57', '58', '59', '60'], (683232, '746138'): ['44', '45', '51', '52', '53', '54'], (683232, '746550'): ['65', '66', '67'], (683232, '746544'): ['54', '55', '56', '57', '58'], (683232, '745897'): ['51', '55', '56', '57', '58', '59', '60'], (683232, '745896'): ['74', '75', '76', '77', '78', '79'], (683232, '746545'): ['66', '67', '68'], (683232, '746546'): ['52', '53', '54', '55', '56', '57'], (683232, '746538'): ['46', '47', '48', '49', '53', '54', '55'], (683232, '746540'): ['79', '80', '81', '82', '83', '84'], (683232, '746536'): ['43', '44', '45', '50', '51', '52'], (683232, '746778'): ['47', '48', '52', '53', '54'], (683232, '746779'): ['74', '75', '76'], (683232, '746537'): ['73', '74', '75', '76', '77', '78'], (683232, '746530'): ['48', '49'], (683232, '746690'): ['77', '78', '79', '80', '81'], (683232, '745800'): ['69', '70', '71'], (683232, '745797'): ['60', '61', '62', '63', '64'], (683232, '746527'): ['53', '54', '55', '59', '60', '61'], (683232, '746526'): ['57', '58', '59'], (683232, '745959'): ['57', '58', '59'], (683232, '745960'): ['62', '63', '64', '65', '66', '71', '72', '73', '74'], (683232, '744823'): ['64', '65', '66'], (683232, '747085'): ['60', '61', '62'], (683232, '745957'): ['55', '56', '57', '58'], (683232, '745956'): ['39', '43', '44', '45'], (683232, '745954'): ['62', '63', '64', '65'], (683232, '745951'): ['48', '49', '50'], (683232, '745949'): ['63', '64'], (683232, '745136'): ['57', '58', '59', '60', '61'], (683232, '745137'): ['51', '52', '53', '54', '55', '56'], (683232, '745443'): ['57', '58', '59', '60'], (683232, '745931'): ['53', '54', '55'], (606303, '746735'): ['57', '58', '59'], (606303, '747054'): ['85', '86', '87', '88'], (606303, '746004'): ['73', '74', '75'], (606303, '746005'): ['58', '59', '60'], (606303, '745189'): ['71', '72', '73'], (606303, '745192'): ['69', '70', '71'], (606303, '745513'): ['73', '74', '75', '76', '77'], (606303, '745512'): ['72', '73', '74', '75', '76'], (606303, '746890'): ['61', '62', '63', '64'], (606303, '746312'): ['62', '63', '64', '65', '66', '67', '68'], (606303, '745991'): ['77', '78', '79', '80', '81', '82'], (606303, '745992'): ['47', '48', '49'], (606303, '746066'): ['27', '33', '34', '35'], (606303, '746058'): ['75', '76', '77'], (606303, '746956'): ['78', '79', '80'], (606303, '746957'): ['41', '42'], (606303, '745987'): ['50', '51', '52', '53'], (606303, '745985'): ['43', '44', '48', '49'], (606303, '745980'): ['62', '63', '64'], (606303, '745568'): ['79'], (606303, '745570'): ['56', '57', '58'], (606303, '746461'): ['68', '69', '70'], (606303, '745982'): ['52', '53', '54'], (606303, '745977'): ['62', '63', '64', '65', '66'], (606303, '746217'): ['73', '74', '75', '76'], (606303, '745401'): ['82', '83', '84'], (606303, '745396'): ['64', '65', '66', '67', '68'], (606303, '745974'): ['65', '66', '67', '68'], (606303, '745975'): ['64', '65', '66'], (606303, '745972'): ['60', '61', '62', '63'], (606303, '746535'): ['85', '88'], (606303, '746532'): ['67', '68', '69'], (606303, '746127'): ['48', '49', '50'], (606303, '745968'): ['63', '64', '65', '66', '67', '68'], (606303, '745965'): ['46', '47', '48'], (606303, '745962'): ['73', '74', '75'], (606303, '745880'): ['67', '68', '69', '70'], (606303, '746850'): ['68'], (606303, '746851'): ['72', '73', '74'], (606303, '745961'): ['57', '58', '59', '60', '61'], (606303, '744825'): ['53', '54', '55'], (606303, '744826'): ['46', '47', '48'], (606303, '747085'): ['67', '68', '69', '70'], (606303, '745957'): ['70', '71', '72'], (606303, '745955'): ['49', '50'], (606303, '745951'): ['59', '65', '66', '67'], (606303, '745949'): ['69', '70', '71'], (606303, '745952'): ['45', '46', '47'], (606303, '745138'): ['72', '73', '74', '75', '76'], (606303, '745625'): ['60', '61', '62'], (606303, '745617'): ['67', '68', '69'], (606303, '745946'): ['52', '53', '54', '55', '56'], (606303, '745943'): ['60'], (606303, '746730'): ['55', '56', '57', '58'], (606303, '745944'): ['69', '70', '71', '72'], (606303, '745938'): ['73', '74'], (606303, '747153'): ['49', '50', '51'], (606303, '747151'): ['73'], (606303, '745935'): ['57', '58', '59'], (606303, '745934'): ['61', '62', '63', '64'], (606303, '745929'): ['61', '62', '63', '64'], (606303, '745441'): ['67', '68', '69', '70'], (606303, '745443'): ['61'], (606303, '745931'): ['41', '42', '43'], (663542, '745845'): ['53', '54', '55', '56', '60', '61', '62', '68', '69', '70', '71'], (663542, '746007'): ['44', '45', '51', '52', '53'], (663542, '746735'): ['66', '67', '68', '69', '70', '71'], (663542, '746732'): ['76'], (663542, '747054'): ['59', '60', '61', '62'], (663542, '747056'): ['52'], (663542, '746005'): ['25', '29', '30', '31', '35', '36', '37'], (663542, '745192'): ['45', '46', '47', '54', '55', '56', '61'], (663542, '745513'): ['45', '46', '52', '53', '54', '55'], (663542, '746000'): ['67', '68', '69', '70'], (663542, '745999'): ['44', '45', '46', '51', '52', '53', '54'], (663542, '745996'): ['51', '52', '53'], (663542, '746884'): ['60', '61', '62', '66', '67', '68', '69', '70'], (663542, '746312'): ['45', '46', '47', '48', '55', '56', '57', '58'], (663542, '745994'): ['65', '66', '67', '68'], (663542, '745989'): ['44', '45', '46', '51', '52', '53'], (663542, '746389'): ['60', '61', '62', '63', '64'], (663542, '746066'): ['57', '58', '59', '60', '61'], (663542, '746956'): ['61', '62', '63', '64', '69', '70', '71'], (663542, '745986'): ['43', '44', '45', '50', '51', '52', '53'], (663542, '745985'): ['50', '51', '52', '58', '59', '60', '61'], (663542, '745568'): ['46', '47', '52', '53', '54', '55'], (663542, '746461'): ['57', '58', '62', '63', '64'], (663542, '745978'): ['54', '55', '56'], (663542, '745976'): ['48', '49', '50', '54', '55', '56'], (663542, '746219'): ['52', '53', '54', '55', '62', '63', '64'], (663542, '745396'): ['48', '49', '50', '51', '57', '58', '59'], (663542, '745975'): ['57', '58', '59'], (663542, '745972'): ['52', '53', '54'], (663542, '746537'): ['59', '60', '61', '66', '67', '68', '69'], (663542, '746127'): ['54', '55', '56', '57', '58', '59', '60'], (663542, '746129'): ['67', '68', '69'], (663542, '745967'): ['49', '53', '54', '55', '56'], (663542, '745880'): ['49', '50', '51', '52', '57', '58'], (663542, '746848'): ['43', '44', '45'], (663542, '745953'): ['51', '52', '53', '54', '55'], (663542, '745950'): ['53', '54', '60', '61', '62'], (663542, '745948'): ['53', '54', '55'], (663542, '745138'): ['61', '62', '63', '64', '65'], (663542, '745625'): ['46', '47', '48', '52', '53', '54'], (663542, '745946'): ['41', '42', '43'], (663542, '746730'): ['42', '43', '44', '45', '49', '50', '51'], (663542, '745947'): ['49', '50', '59', '60', '61', '62', '63'], (665625, '746007'): ['58', '59', '60', '61'], (665625, '746732'): ['63', '64', '65', '71', '72', '73', '74', '75'], (665625, '747054'): ['45', '46', '47', '51', '52', '53', '54'], (665625, '746004'): ['53', '54', '55', '56', '57'], (665625, '746005'): ['41', '42', '43', '44'], (665625, '745189'): ['53', '54', '55'], (665625, '745192'): ['62', '63', '64', '65'], (665625, '745513'): ['59', '60', '61'], (665625, '746000'): ['61', '62'], (665625, '746001'): ['55', '56', '57', '58'], (665625, '745997'): ['46', '47', '48', '52', '53', '54', '55', '56'], (665625, '746890'): ['51', '52', '53'], (665625, '746313'): ['52', '53', '54'], (665625, '745990'): ['58', '59', '60', '61'], (665625, '745989'): ['58', '59', '60'], (665625, '746388'): ['57', '58', '59'], (665625, '746389'): ['54', '55', '56'], (665625, '746066'): ['50', '51', '52'], (665625, '746063'): ['53', '54', '55'], (665625, '746957'): ['60', '61', '62', '63', '65'], (665625, '745987'): ['58', '59', '60', '62', '63'], (665625, '745985'): ['67', '68', '69', '70'], (665625, '745980'): ['68', '69', '70'], (665625, '745568'): ['59', '60', '61', '62', '63'], (665625, '745981'): ['36', '37', '41', '42', '43'], (665625, '746215'): ['43', '47', '48', '49'], (665625, '745403'): ['66', '67', '68', '69', '70', '71'], (665625, '745974'): ['41', '42', '43', '44', '45'], (665625, '745973'): ['66', '67', '68', '69'], (665625, '745970'): ['44', '48', '49', '50', '51', '52'], (665625, '746535'): ['68', '69', '70', '71', '72'], (665625, '746127'): ['64', '65', '66', '67', '68'], (665625, '746122'): ['69', '70', '71', '72'], (665625, '745967'): ['63', '64', '65'], (665625, '745963'): ['55', '56', '61', '62', '63', '64'], (665625, '745880'): ['59', '60', '61'], (665625, '745882'): ['67', '68', '69', '70', '71', '72'], (665625, '746848'): ['49', '50', '51'], (665625, '745966'): ['53', '54', '55', '56'], (665625, '745959'): ['49', '50'], (665625, '745960'): ['56', '57'], (665625, '744825'): ['80', '81', '82'], (665625, '744826'): ['52', '53', '54'], (665625, '747081'): ['52', '53', '54', '55'], (665625, '745954'): ['52', '53', '54', '55'], (665625, '745950'): ['47', '48', '52'], (665625, '745949'): ['52', '53', '54', '55'], (665625, '745944'): ['86', '87', '88', '89', '90'], (665625, '745940'): ['66', '67', '68', '69'], (665625, '745285'): ['45', '46', '47', '48', '49', '50', '57', '58', '59', '60'], (543859, '745848'): ['53', '54', '55', '56', '60', '61', '62'], (543859, '745842'): ['66', '67', '68', '69', '70', '71', '72'], (543859, '745844'): ['84', '85', '86', '87', '88', '89', '90', '91'], (543859, '746491'): ['48', '49', '50', '51', '52', '53', '54', '59', '60', '61', '62'], (543859, '746156'): ['48', '49', '56', '57', '58'], (543859, '746154'): ['59', '60', '61', '62', '69', '70', '71', '72'], (543859, '746000'): ['79', '80', '81', '82', '86', '87'], (543859, '746001'): ['73', '74', '75', '82', '83', '84'], (543859, '747044'): ['60', '61', '62', '66', '67', '68'], (543859, '745752'): ['75', '76', '77', '78', '79', '80'], (543859, '745096'): ['65', '66', '67', '68'], (543859, '745745'): ['55', '56', '57', '58', '62', '63', '64', '68', '69', '70'], (543859, '745743'): ['46', '47', '48', '49', '50', '57', '58', '59'], (543859, '745413'): ['62', '63', '69', '70', '71'], (543859, '746225'): ['71', '72', '73', '74'], (543859, '745330'): ['61', '62', '63', '64', '65', '69', '70', '71'], (543859, '745740'): ['69', '70', '71'], (543859, '745737'): ['41', '42', '47', '48', '49', '50'], (543859, '745735'): ['45', '46', '53', '54', '55'], (543859, '746295'): ['68', '69', '70', '71'], (543859, '746947'): ['69', '77', '78', '79', '80'], (543859, '746945'): ['71', '72', '73'], (543859, '745733'): ['56', '57', '58', '59'], (543859, '745734'): ['19', '20', '26', '27', '28', '29', '30', '31'], (543859, '745731'): ['56'], (543859, '745806'): ['57', '58', '59'], (543859, '744922'): ['40', '41', '51', '52', '53', '54'], (543859, '744916'): ['55', '56', '57', '61', '62', '63'], (543859, '745728'): ['43', '44', '45', '49', '50', '51'], (543859, '745723'): ['57', '58', '59', '60'], (543859, '745074'): ['58', '59', '60', '61', '65', '66', '67'], (543859, '745065'): ['47', '51', '52', '53', '54'], (543859, '747009'): ['51', '52', '53', '58'], (543859, '745720'): ['37', '38', '42', '43', '44', '45', '49'], (543859, '745715'): ['44', '45', '46', '47', '48', '54'], (543859, '746931'): ['67', '68', '69', '70', '71', '72'], (543859, '745547'): ['52', '58', '59', '60', '61', '62'], (543859, '745546'): ['94', '95', '96', '98', '102', '103', '104'], (543859, '745719'): ['27', '28', '33', '34', '35', '36', '40', '41', '42', '51', '52'], (543859, '745713'): ['45', '46', '51', '52', '53', '54', '60'], (543859, '745708'): ['50', '55', '56', '57', '58', '59', '60', '61', '62'], (543859, '746759'): ['59', '60', '61', '71', '72', '73', '74', '78', '79', '80', '81'], (543859, '745707'): ['101', '102', '103', '104', '105', '106', '107'], (543859, '745705'): ['69', '70', '71'], (543859, '745703'): ['51', '52', '53', '58', '59', '60', '61', '62'], (543859, '745866'): ['57', '58', '59', '60', '61'], (543859, '745859'): ['26', '27', '28', '29', '30', '31', '32', '33'], (543859, '745047'): ['49', '53', '54', '55', '56'], (543859, '745045'): ['63', '64', '65', '69', '70', '71', '72'], (543859, '745860'): ['52', '53', '54', '58', '59', '60', '61'], (543859, '745855'): ['59', '60', '61', '62', '66', '67', '68'], (543859, '746582'): ['77'], (543859, '746905'): ['51', '52'], (543859, '746906'): ['54', '55', '56', '60', '61', '62', '63', '64'], (543859, '745851'): ['63', '64', '65', '66', '67'], (543859, '745853'): ['41', '45', '46', '47'], (543859, '745847'): ['69', '70', '71'], (663485, '746490'): ['54', '60', '61', '62', '67', '68', '69'], (663485, '745921'): ['51', '52', '53'], (663485, '745920'): ['25', '29', '30', '31', '32'], (663485, '745917'): ['40', '41', '42', '43'], (663485, '745918'): ['43', '44', '45', '46'], (663485, '746237'): ['47', '48', '49', '59', '60', '61', '62', '63'], (663485, '746806'): ['70', '71', '72'], (663485, '745914'): ['57', '58', '59', '60'], (663485, '745911'): ['68', '69', '70'], (663485, '745915'): ['43', '44', '45'], (663485, '744943'): ['50', '51', '52', '56', '57', '58', '59'], (663485, '744941'): ['47', '48', '49'], (663485, '745907'): ['50', '51', '52', '53', '54'], (663485, '745901'): ['50', '51', '52', '59', '60', '61'], (663485, '745904'): ['63', '64', '65', '66', '67'], (663485, '745900'): ['69', '70', '71', '72'], (663485, '746387'): ['69', '70', '71'], (663485, '745739'): ['44', '45', '46', '51', '52', '53'], (663485, '745738'): ['50', '51', '52', '53', '54'], (663485, '745491'): ['54', '55', '56', '57', '58'], (663485, '745899'): ['43', '44', '45', '49', '50', '51', '57', '58', '59', '60'], (663485, '745892'): ['36', '37', '38', '42', '43', '44', '45'], (663485, '745895'): ['72', '73', '74', '75', '76'], (663485, '745887'): ['51', '52', '53', '54'], (663485, '745889'): ['49', '50', '51', '52'], (663485, '747188'): ['62', '63', '64'], (663485, '747185'): ['69', '70'], (663485, '745238'): ['74', '75'], (663485, '745243'): ['61', '62', '63'], (663485, '745890'): ['63', '64', '65', '66'], (663485, '745885'): ['58', '59', '60', '61', '67', '68', '69', '70'], (663485, '746772'): ['51', '52', '53', '58', '59', '60'], (663485, '745311'): ['44', '50', '51', '52'], (663485, '745880'): ['78', '79', '80'], (663485, '745881'): ['59', '60', '61', '71', '72', '73', '74'], (663485, '745877'): ['68', '69', '70'], (663485, '745791'): ['37', '38', '42', '43', '44', '45'], (663485, '745789'): ['55', '56', '57'], (663485, '745879'): ['46', '47', '48', '49', '55', '56', '57', '58', '59'], (663485, '746842'): ['30', '35', '36', '37', '38', '39'], (663485, '745871'): ['41', '48', '49', '50'], (663485, '745872'): ['47', '48', '49', '50'], (663485, '744974'): ['49', '50', '51', '55', '56'], (663485, '744977'): ['68', '69', '70'], (663485, '745375'): ['57', '58', '59', '60'], (663485, '745863'): ['39', '40', '41', '46', '47', '48', '49'], (663485, '745864'): ['67', '68', '69'], (663485, '745861'): ['47', '48', '49', '50'], (663485, '745046'): ['70', '71', '72', '73'], (663485, '746258'): ['37', '38', '39', '43', '44', '45'], (663485, '745857'): ['29', '30', '31'], (663485, '745852'): ['44', '45', '46', '47', '55', '56', '57'], (663485, '746587'): ['47', '48', '49'], (663485, '746579'): ['57', '58', '59', '60'], (663485, '746905'): ['58', '59', '60', '61'], (663485, '746906'): ['50', '51', '52', '53'], (663485, '745854'): ['48', '49', '50'], (663485, '745853'): ['80', '81', '82', '83'], (663485, '745846'): ['46', '47', '48', '49'], (643377, '746489'): ['61', '62', '63'], (643377, '746492'): ['62', '63', '64', '65'], (643377, '747049'): ['66', '67', '68'], (643377, '745922'): ['57', '58', '59', '60', '61'], (643377, '745921'): ['69', '70', '71'], (643377, '745919'): ['72', '73', '74'], (643377, '745918'): ['76', '77', '78'], (643377, '746237'): ['64'], (643377, '746804'): ['57', '58', '59', '60', '61'], (643377, '746800'): ['65', '66', '67'], (643377, '745914'): ['45', '46', '47'], (643377, '745912'): ['42', '43', '44', '45', '46', '47'], (643377, '745906'): ['53', '54', '55', '56', '60', '61', '62'], (643377, '744940'): ['65', '66', '67', '68', '69', '70'], (643377, '746641'): ['50', '51', '52'], (643377, '744860'): ['63', '64', '65'], (643377, '744856'): ['54', '55'], (643377, '745905'): ['65', '66', '67'], (643377, '745904'): ['53', '54', '55', '56'], (643377, '745903'): ['58', '59', '60'], (643377, '745898'): ['64', '65', '66', '67'], (643377, '746383'): ['63', '64', '65'], (643377, '745496'): ['58', '59', '60', '61', '62'], (643377, '745491'): ['62', '63', '64', '65'], (643377, '745897'): ['52', '53', '54'], (643377, '745894'): ['59', '60', '61', '62'], (643377, '745892'): ['53', '54', '55', '56', '57'], (643377, '745891'): ['78', '79', '80', '81'], (643377, '745893'): ['64', '65', '66'], (643377, '745889'): ['80', '81', '82', '83'], (643377, '745648'): ['62', '63', '64', '65'], (643377, '745238'): ['56', '57', '58', '59', '61', '62'], (643377, '745239'): ['68', '69', '70'], (643377, '745886'): ['56', '57', '58', '59'], (643377, '745883'): ['58', '59', '60', '61'], (643377, '746771'): ['65', '66', '67', '68', '69'], (643377, '746773'): ['62', '63', '64', '65'], (643377, '745311'): ['64', '65', '66', '67'], (643377, '745314'): ['55', '56', '57'], (643377, '745880'): ['63', '64', '65', '66'], (643377, '745882'): ['62', '63', '64', '65', '66'], (643377, '745878'): ['51', '52', '53', '54'], (643377, '745787'): ['48', '49', '50'], (643377, '745875'): ['58', '59', '60'], (643377, '745876'): ['56', '57', '58', '59'], (643377, '746847'): ['64', '65', '66', '67'], (643377, '745927'): ['58', '59', '60'], (643377, '745874'): ['55', '56', '57', '58'], (643377, '744980'): ['60', '61', '62', '63'], (643377, '744974'): ['63', '64', '65'], (643377, '744975'): ['74', '75', '76'], (643377, '745374'): ['52', '53', '54', '55'], (643377, '745868'): ['58', '59', '60', '61'], (643377, '745863'): ['54', '55', '56'], (643377, '745864'): ['75', '76', '77'], (643377, '745862'): ['56'], (643377, '745861'): ['57', '58', '59', '60', '61', '62', '63'], (643377, '745046'): ['62', '63', '64', '65'], (643377, '745042'): ['56', '61', '62', '63', '64'], (643377, '746259'): ['59', '60', '61', '62'], (643377, '745857'): ['56', '57', '58', '66', '67', '68'], (643377, '746588'): ['63', '64', '68', '69', '70', '71'], (643377, '746587'): ['71', '72', '73'], (643377, '746579'): ['71', '72', '73'], (643377, '746905'): ['103', '104', '105'], (643377, '745854'): ['54', '55', '56', '57', '66'], (643377, '745853'): ['52', '53', '54', '55'], (686973, '746009'): ['4', '5', '6', '7', '12', '13', '14', '15', '16', '17', '18', '22', '23', '24', '25', '26', '27', '28', '35', '36', '37'], (686973, '745924'): ['1', '2', '3', '7', '8', '9', '10', '11', '15', '16', '17', '18', '23', '24', '25', '26', '27', '28', '32', '33', '34', '35', '36', '37'], (686973, '747053'): ['5', '6', '7', '8', '9', '10', '11', '16', '17', '18', '19', '23', '24', '25', '26', '27', '28', '34', '35', '36', '37', '38', '44', '45', '46', '47', '48'], (686973, '745920'): ['1', '2', '3', '4', '5', '6', '7', '14', '15', '16', '17', '21', '22', '23', '24'], (686973, '745899'): ['1', '2', '3', '9', '10', '11', '17', '18', '19', '20', '24', '25', '26', '31', '32', '33', '34', '35'], (686973, '745891'): ['28', '29', '35', '36', '37', '42', '43', '44', '45', '46', '51', '52', '53', '60', '61', '62', '63', '64', '65'], (686973, '745871'): ['1', '2', '3', '4', '10', '11', '12', '13', '19', '20', '21', '22', '23', '27', '28', '29', '30', '34', '35', '36', '37', '38', '39', '40'], (686973, '745869'): ['1', '2', '3', '4', '5', '9', '10', '11', '12', '13', '17', '18', '19', '20', '24', '25', '26', '32', '33', '34', '35', '36', '43', '44', '45', '46'], (686973, '745045'): ['12', '13', '14', '19', '20', '21', '27', '28', '29', '30', '31', '32', '33', '34', '35'], (686973, '745856'): ['65', '66', '67', '72', '73', '74', '75', '76'], (686973, '745858'): ['28', '29', '30', '31', '32', '33', '34', '35', '36'], (686973, '746582'): ['59', '60', '61'], (686973, '746579'): ['64', '65', '66', '67'], (686973, '746905'): ['79', '84', '86', '87'], (686973, '745854'): ['14', '15', '16', '20', '21', '22', '23', '24', '32', '33'], (686973, '745849'): ['32', '33', '34', '35', '36', '41', '42', '43', '44', '45'], (660896, '746333'): ['54', '55'], (660896, '746009'): ['41', '42', '43', '44', '48', '49', '50', '51', '52'], (660896, '745926'): ['52', '53', '54', '55', '60', '61', '62'], (660896, '745924'): ['67', '68', '69'], (660896, '745923'): ['48'], (660896, '746489'): ['75', '76', '77', '82', '83', '84', '85', '86'], (660896, '745911'): ['50', '51', '52', '53'], (660896, '745915'): ['61', '62', '63', '64', '70', '71', '72', '73', '74', '75', '76', '77', '78'], (660896, '745902'): ['54', '55', '56', '57', '58', '59', '63', '64', '65'], (660896, '745903'): ['45', '46', '47'], (660896, '745898'): ['58', '59', '60'], (660896, '746383'): ['45', '46', '47', '52', '53', '54'], (660896, '745738'): ['60', '61', '62', '63', '64'], (660896, '745492'): ['53', '54', '55'], (660896, '745491'): ['69', '70', '71', '72', '73'], (660896, '745896'): ['47', '48', '49', '50'], (660896, '745894'): ['68', '69', '70'], (660896, '745895'): ['57', '58', '65', '66', '67', '68'], (660896, '745893'): ['58', '59', '60'], (660896, '745887'): ['79', '80', '81', '82'], (660896, '745649'): ['61', '62', '63'], (660896, '747188'): ['53', '54', '55', '56', '57', '58'], (660896, '745238'): ['50', '51', '52'], (660896, '745239'): ['53', '54', '55', '56'], (660896, '745886'): ['44', '48', '49', '50'], (660896, '745884'): ['51', '52', '53', '54', '55', '56'], (660896, '745885'): ['54'], (660896, '746771'): ['57', '58', '59', '60'], (660896, '746773'): ['53', '54', '55'], (660896, '745311'): ['56', '57', '58'], (660896, '745314'): ['44'], (660896, '745880'): ['53', '54', '55', '56'], (660896, '745882'): ['53', '54', '55', '56'], (660896, '745878'): ['43', '44', '45', '46'], (660896, '745787'): ['41', '42', '43', '44'], (660896, '745875'): ['53', '54'], (660896, '745879'): ['63', '64', '65', '66', '67', '68'], (660896, '746847'): ['50', '51', '52'], (660896, '745927'): ['48', '49', '50', '51', '52'], (660896, '745872'): ['29', '30', '34', '35', '36'], (660896, '744980'): ['67', '68', '69'], (660896, '744977'): ['60', '61', '62'], (660896, '744975'): ['55', '56', '57', '58', '59', '60', '61'], (660896, '745870'): ['43', '44', '45', '46', '47', '48'], (660896, '745864'): ['39', '40'], (660896, '745866'): ['49', '50', '51', '52'], (660896, '745046'): ['42', '46', '47', '48', '49', '50'], (660896, '746258'): ['49', '50', '51'], (660896, '745857'): ['43', '48', '49', '50', '51'], (660896, '745855'): ['49', '50', '51'], (660896, '745852'): ['67', '68', '69', '73', '74', '75', '76'], (660896, '745854'): ['34', '35', '40', '41', '42'], (660896, '745853'): ['89', '90', '91', '92'], (660896, '745846'): ['40', '41', '42'], (661395, '746806'): ['80', '81', '82', '83'], (661395, '745914'): ['67', '68', '69'], (661395, '745911'): ['61', '62', '63', '64'], (661395, '745912'): ['55', '56', '57'], (661395, '745906'): ['69', '70', '71'], (661395, '744940'): ['56', '57', '58'], (661395, '744941'): ['64', '65', '66', '67'], (661395, '746641'): ['59', '60', '61', '62'], (661395, '746636'): ['63', '64', '65', '66', '67'], (661395, '744856'): ['67', '68', '69', '70', '71'], (661395, '745905'): ['71', '72', '73', '74'], (661395, '745901'): ['69', '70', '71'], (661395, '745904'): ['68', '69', '70', '71'], (661395, '745903'): ['67', '68', '69', '70'], (661395, '745898'): ['72', '73', '74', '75', '76', '77'], (661395, '746383'): ['69', '70', '71'], (661395, '745492'): ['59', '60', '61', '62', '63'], (661395, '745491'): ['78', '79', '80'], (661395, '745896'): ['71', '72', '73'], (661395, '745892'): ['72', '73', '74', '78', '79', '80'], (661395, '745895'): ['77'], (661395, '745893'): ['71', '72', '73'], (661395, '745887'): ['73', '74', '75'], (661395, '745649'): ['69', '70', '71', '72', '73', '74'], (661395, '747184'): ['69', '70'], (661395, '745238'): ['66', '67', '68', '69', '70'], (661395, '745243'): ['75', '76', '77'], (661395, '745886'): ['67', '68', '69'], (661395, '745883'): ['66', '67', '68', '69', '70'], (661395, '746771'): ['80', '81', '83'], (661395, '746773'): ['69', '70', '71'], (661395, '745311'): ['74', '75', '76'], (661395, '745314'): ['64'], (661395, '745880'): ['71', '72', '73'], (661395, '745878'): ['61', '62', '63', '64', '65', '66'], (661395, '745877'): ['75', '76', '77'], (661395, '745789'): ['72', '73', '74', '75', '76'], (661395, '745876'): ['68', '69', '70'], (661395, '746847'): ['56', '57', '58', '59'], (661395, '745927'): ['64', '65', '66', '67'], (661395, '745869'): ['50', '51', '52', '53', '54'], (661395, '744980'): ['75', '76', '77'], (661395, '744974'): ['69', '70', '71', '72'], (661395, '744975'): ['80', '81'], (661395, '745868'): ['66', '67', '68', '69'], (661395, '745863'): ['62', '63', '64', '65', '66', '67', '68'], (661395, '745864'): ['81', '82', '83', '84', '85', '86'], (661395, '745862'): ['60', '61', '62'], (661395, '745861'): ['70', '71', '72', '73'], (661395, '745046'): ['77', '78', '79'], (661395, '745042'): ['69', '70', '71'], (661395, '746259'): ['55', '56', '57', '58'], (661395, '745857'): ['73', '74', '75', '76', '77', '78'], (661395, '746587'): ['61', '62', '63', '68', '69', '70'], (661395, '746582'): ['65', '66', '67'], (661395, '746905'): ['66', '67', '68', '69', '70', '71'], (661395, '745854'): ['67', '68', '69'], (661395, '745853'): ['62', '63', '64', '71', '72', '73', '74'], (681892, '745925'): ['64', '65', '66', '67'], (681892, '745923'): ['41', '46', '47'], (681892, '746491'): ['41', '42', '43', '44'], (681892, '746490'): ['50', '51', '52', '53'], (681892, '745922'): ['42'], (681892, '745921'): ['59', '60', '61', '62'], (681892, '745917'): ['33', '34', '35', '36'], (681892, '746245'): ['59', '60', '61', '70', '71', '72', '73', '74'], (681892, '746806'): ['34', '35', '42', '43', '44', '45', '46', '52', '53'], (681892, '745913'): ['49', '50', '51', '52', '57', '58', '59', '60', '61', '62'], (681892, '745910'): ['56', '63', '64', '65', '66', '67'], (681892, '744941'): ['50', '51', '58', '59', '60'], (681892, '745908'): ['53', '54', '55', '56', '61', '62', '63', '64'], (681892, '746642'): ['39', '40', '41', '42', '43', '44', '45', '50', '51', '52', '53'], (681892, '744856'): ['51', '52', '53'], (681892, '745901'): ['41', '42', '43', '44', '45', '46'], (681892, '745902'): ['41', '42', '48', '49', '50'], (681892, '745649'): ['43', '44', '45', '50', '51', '52'], (681892, '747185'): ['74', '75', '76', '77', '78', '79', '83', '84', '85', '86'], (681892, '745243'): ['69', '70', '71'], (681892, '745888'): ['62', '63', '64', '69', '70', '71', '72', '73'], (681892, '745884'): ['83', '84', '85', '86', '87', '88', '89'], (681892, '746771'): ['89', '90', '91'], (681892, '745312'): ['69', '70', '71'], (681892, '745847'): ['57', '58', '59', '60', '61', '62', '63', '64'], (660853, '745361'): ['65', '66'], (660853, '745432'): ['62', '63', '64'], (660853, '745429'): ['70', '71', '72', '73'], (660853, '746162'): ['81', '83'], (660853, '746164'): ['49', '50', '51'], (660853, '746002'): ['62', '63', '64', '65'], (660853, '746565'): ['59', '60', '61', '62'], (660853, '746564'): ['54', '55', '56', '62'], (660853, '745428'): ['52', '53', '54', '55'], (660853, '745426'): ['49', '56', '57', '58'], (660853, '745423'): ['52'], (660853, '747209'): ['61', '62', '63'], (660853, '746886'): ['53', '54', '58', '59', '60'], (660853, '746885'): ['65', '71'], (660853, '745422'): ['42', '43'], (660853, '745418'): ['63', '64', '65', '71'], (660853, '747122'): ['60', '61'], (660853, '747121'): ['53', '54', '55', '61'], (660853, '746714'): ['52', '53', '54', '55'], (660853, '745411'): ['65', '66', '67', '68', '69'], (660853, '745414'): ['43', '44', '45', '46', '47', '51', '52', '53'], (660853, '746301'): ['60', '61', '62', '63'], (660853, '746222'): ['53', '54', '55', '56', '63', '64', '65'], (660853, '745408'): ['63', '64', '65', '66'], (660853, '745406'): ['46', '47', '48', '49'], (660853, '745409'): ['69', '70', '71', '72'], (660853, '745404'): ['60', '61', '62', '63'], (660853, '745805'): ['54', '55', '56', '57', '58'], (660853, '745807'): ['79'], (660853, '745401'): ['71'], (660853, '745403'): ['75', '82', '83', '84', '85'], (660853, '745397'): ['71', '72', '73', '74'], (660853, '746940'): ['78', '79', '80', '81'], (660853, '744999'): ['60', '61', '62', '63'], (660853, '745398'): ['64', '68', '69', '70', '71'], (660853, '745391'): ['61', '62', '63', '64', '65', '66'], (660853, '745393'): ['70', '71'], (660853, '745390'): ['61', '62', '63', '64'], (660853, '744824'): ['81', '82', '83'], (660853, '747008'): ['26', '27', '28', '29', '30', '34', '35', '36', '37'], (660853, '745719'): ['62', '63', '64', '65', '70', '71', '72'], (660853, '745718'): ['75', '76', '77'], (660853, '745709'): ['51', '52', '53', '54', '55', '61'], (660853, '745710'): ['38', '39', '40', '41', '46', '47', '48', '49'], (660853, '746762'): ['66', '67', '68', '69', '70', '71', '72', '73', '79', '80', '81', '82', '83'], (660853, '746758'): ['75', '76', '77', '78'], (660853, '746754'): ['56', '57', '58', '59', '60'], (660853, '746748'): ['56', '62', '63', '64', '65'], (660853, '746751'): ['55', '56', '57', '58', '59'], (660853, '746992'): ['82', '83'], (660853, '746991'): ['69', '70', '71', '72'], (660853, '746911'): ['37', '38', '39', '40', '44', '45', '46', '47'], (660853, '746746'): ['20', '24', '25', '26', '27'], (660853, '746740'): ['53', '57', '58', '59', '66', '67', '68'], (660853, '746176'): ['44', '45', '46', '47', '48'], (660853, '746172'): ['85', '86', '91', '92', '93', '94', '95'], (660853, '745360'): ['67', '68', '69', '70', '71'], (660853, '746742'): ['55', '56', '57', '58'], (660853, '746417'): ['57'], (660853, '746416'): ['78', '82', '83', '84'], (681402, '746175'): ['100'], (681402, '746895'): ['33', '34', '35', '36'], (681402, '747207'): ['71', '72', '76', '77', '78', '79', '80'], (681402, '746155'): ['67', '68', '69', '70', '71'], (681402, '745422'): ['54', '55', '56'], (681402, '745419'): ['39', '40', '41', '42', '43'], (681402, '745342'): ['64', '65', '66', '67', '68'], (681402, '746355'): ['63', '64', '65', '67', '68'], (681402, '745294'): ['55', '56', '57', '58', '59'], (681402, '745295'): ['64', '65', '66'], (681402, '746756'): ['40', '44', '45', '46', '51', '52', '53'], (681402, '746752'): ['41', '42', '49', '50', '51'], (681402, '746751'): ['45', '46', '47', '48', '49', '50'], (681402, '746750'): ['23', '24'], (681402, '746990'): ['64', '65', '66', '67'], (681402, '746911'): ['27', '28', '29', '30', '31'], (681402, '746747'): ['42', '43', '44'], (681402, '746744'): ['31', '32', '33', '34', '35', '36'], (681402, '746740'): ['79', '80', '81', '82'], (681402, '746172'): ['27', '28', '29', '33', '34', '35', '36'], (681402, '745364'): ['42', '43', '44', '45', '46', '47', '48'], (681402, '745358'): ['59'], (681402, '746743'): ['42', '43', '44', '45', '46'], (681402, '746738'): ['56', '57'], (681402, '746414'): ['65', '66', '67', '68'], (681402, '746416'): ['64', '65', '66'], (605121, '746812'): ['58', '59', '60', '61', '62', '63', '64'], (605121, '746810'): ['64', '65', '73', '74', '75'], (605121, '746796'): ['44', '45', '46', '47'], (605121, '746793'): ['38', '39', '47', '48', '49'], (605121, '746791'): ['36', '37', '38'], (605121, '746790'): ['42', '43', '44', '45', '46'], (605121, '745980'): ['55', '56', '57', '58'], (605121, '745984'): ['53', '54', '55', '56'], (605121, '746872'): ['46', '47', '48', '49'], (605121, '746784'): ['67', '68', '73', '74', '75', '76', '77'], (605121, '746787'): ['43', '44', '45', '52', '53', '54', '55'], (605121, '745244'): ['50', '51', '52'], (605121, '747189'): ['65', '66', '67'], (605121, '746781'): ['52', '53', '54', '55', '56'], (605121, '746782'): ['60', '61', '62'], (605121, '746459'): ['57', '58', '59', '60'], (605121, '746780'): ['54', '55', '59', '60', '61', '62'], (605121, '746775'): ['69'], (605121, '746816'): ['23', '24', '25', '26'], (605121, '746778'): ['69', '70', '71'], (605121, '746617'): ['59', '60', '61'], (605121, '746619'): ['25', '26', '27', '28', '29', '30', '31'], (605121, '746044'): ['67', '68', '69'], (605121, '746771'): ['56', '61', '62', '63', '64'], (605121, '746772'): ['39', '40', '41'], (605121, '746766'): ['58', '59', '60', '65', '66', '67', '68', '69', '70', '71'], (605121, '746287'): ['48', '49', '50', '51', '52'], (605121, '744985'): ['38', '39', '40'], (605121, '744989'): ['52', '57', '58', '59'], (605121, '746768'): ['39', '40', '41', '42', '43', '44', '45'], (605121, '746763'): ['56', '57', '58'], (605121, '745875'): ['36', '40', '41', '42', '43', '44'], (605121, '745628'): ['40', '41', '42', '43', '44'], (605121, '746760'): ['56', '60', '61', '62'], (605121, '746761'): ['56', '57', '58', '59'], (605121, '746762'): ['62', '63', '64'], (605121, '746759'): ['47', '51', '52'], (605121, '746357'): ['55', '61', '62', '63'], (605121, '746359'): ['49', '50', '51'], (605121, '745297'): ['48', '49', '50'], (605121, '746758'): ['57', '58', '59', '60'], (605121, '746753'): ['58', '59', '60', '61'], (605121, '746752'): ['66', '70', '71', '72', '73', '74', '75'], (605121, '746751'): ['69', '70', '71', '72', '73'], (605121, '746750'): ['59', '60', '61', '62', '64'], (605121, '746990'): ['75', '76', '77', '78', '79'], (605121, '746911'): ['69', '70', '71', '72'], (605121, '746910'): ['70', '71', '72', '73'], (605121, '746745'): ['73', '74', '75', '76'], (605121, '746744'): ['62', '63', '64'], (605121, '746740'): ['75', '76', '77', '78'], (605121, '746179'): ['73', '74', '75'], (605121, '746172'): ['56', '57', '58', '59'], (605121, '745364'): ['76'], (605121, '746743'): ['62', '63', '64', '65'], (605121, '746742'): ['63', '64', '65', '66', '67', '68'], (663436, '746768'): ['51', '52', '53', '54', '60', '61', '62', '63', '67', '68', '69', '70', '71', '72'], (663436, '745875'): ['4', '5', '6', '7', '8', '12', '13', '14', '15', '21', '22', '23', '24', '25', '26', '32', '33', '34', '35'], (663436, '745630'): ['4', '5', '6', '13', '14', '15', '16', '17', '21', '22', '23', '29', '30', '31', '36', '37', '38', '39', '43', '44', '45'], (663436, '746759'): ['1', '2', '3', '4', '9', '10', '11', '16', '17', '18', '23', '24', '25', '26', '33', '34', '35', '36', '37', '42', '43', '44', '45', '46'], (663436, '745295'): ['4', '5', '6', '10', '11', '12', '13', '14', '20', '21', '22', '23', '24', '25', '29', '30', '31', '36', '37'], (663436, '746753'): ['1', '2', '3', '4', '10', '11', '12', '13', '14', '18', '19', '20', '27', '28', '29', '30', '31', '36', '37', '38', '39', '40', '41'], (663436, '746751'): ['1', '2', '3', '4', '5', '6', '12', '13', '14', '21', '22', '23', '24', '25', '26', '32', '33', '34', '38', '39', '40'], (663436, '746912'): ['6', '7', '8', '9', '10', '16', '17', '18', '19', '24', '25', '26', '31', '32', '33', '34', '35', '40', '41', '42', '46', '47', '48', '49'], (663436, '746745'): ['1', '2', '3', '4', '5', '6', '11', '12', '13', '19', '20', '21', '22', '23', '24', '25'], (663436, '746176'): ['4', '5', '6', '10', '11', '12', '17', '18', '19', '20', '21', '25', '26', '27', '28', '29', '30', '35', '36', '37', '38', '39'], (663436, '746742'): ['1', '2', '3', '4', '5', '6', '11', '12', '13', '19', '20', '21', '22', '23', '27', '28', '29', '30', '31'], (694363, '746784'): ['53', '54', '55', '61', '62', '63', '64', '65', '66'], (694363, '747191'): ['65', '66', '67'], (694363, '747189'): ['71', '72', '73'], (694363, '746457'): ['19', '20', '21', '22', '26', '27', '28', '32', '33', '34', '35', '36'], (694363, '746816'): ['30', '31', '32', '36', '37', '38', '43', '44', '45', '46'], (694363, '746779'): ['73'], (694363, '746619'): ['5', '6', '7', '8', '9', '10', '16', '17', '18', '24'], (694363, '746771'): ['84', '85', '86', '87', '88'], (694363, '746774'): ['63', '64', '65', '66', '67'], (694363, '746769'): ['1', '2', '3', '4', '10', '11', '12'], (694363, '744989'): ['63', '64', '65', '66', '67', '68', '69', '70', '71'], (694363, '746767'): ['11', '16', '17', '18', '23', '24', '25', '26', '31', '32', '33', '34', '35', '39', '40', '41', '42'], (694363, '746763'): ['64', '65', '66', '67'], (694363, '745876'): ['66', '67'], (694363, '745628'): ['55', '56', '57', '58', '63', '64', '65', '66'], (694363, '746761'): ['40', '41', '42', '43', '44', '48', '49', '50', '51'], (694363, '746757'): ['57', '58', '59', '66', '67', '68', '69', '70'], (694363, '746355'): ['45', '46', '47', '48', '49', '53', '54', '55', '56', '57', '58'], (694363, '746758'): ['65', '66', '67', '72', '73', '74'], (694363, '746753'): ['46', '47', '48', '49', '53', '54', '55', '56', '57'], (694363, '746752'): ['55', '56', '57', '63', '64', '65'], (694363, '746750'): ['29', '30', '31', '32', '37', '38', '39', '40'], (694363, '746991'): ['24', '28', '29', '30', '35', '36', '37', '38', '39', '40'], (694363, '746746'): ['1', '2', '3', '4', '5', '6', '10', '11', '12', '16', '17', '18', '19'], (694363, '746741'): ['44', '45', '46', '52', '53', '54'], (694363, '746172'): ['4', '5', '6', '13', '14', '15', '19', '20', '21', '22'], (694363, '746742'): ['79', '80', '81'], (694363, '746417'): ['34', '35', '36', '37', '38', '39', '44', '45', '46', '47'], (676760, '745754'): ['45', '46', '50', '51'], (676760, '746000'): ['50', '51', '52', '56'], (676760, '746001'): ['44', '45', '46', '47', '59', '60'], (676760, '747044'): ['46', '47', '48', '53', '54', '55'], (676760, '745752'): ['55', '56', '57', '66', '67', '68', '69'], (676760, '745748'): ['68', '69', '70', '71', '72'], (676760, '746296'): ['53', '61', '62', '63', '64', '68', '69', '70', '71', '72'], (676760, '746946'): ['52', '53', '54', '55', '56', '62', '63', '64', '65', '66'], (676760, '745732'): ['34', '35', '36', '37', '38', '39', '40'], (676760, '745734'): ['58', '59', '60', '61', '62', '63'], (676760, '745712'): ['58', '62', '63', '64', '69', '70', '71', '72'], (676760, '745701'): ['80', '81', '82', '83', '84', '85'], (676760, '744964'): ['62', '63', '64', '65', '66', '67'], (621051, '746330'): ['48', '49', '50'], (621051, '746653'): ['60', '61', '62', '63', '64'], (621051, '746655'): ['76', '77', '78'], (621051, '746813'): ['53', '54', '55', '56', '57'], (621051, '746807'): ['48', '49', '50'], (621051, '745916'): ['65', '66', '67'], (621051, '745917'): ['70', '71', '72', '73', '74', '75'], (621051, '745918'): ['67', '68', '69', '70', '71'], (621051, '746803'): ['67', '68', '69', '70', '71', '72'], (621051, '746806'): ['55', '56', '57', '58'], (621051, '746800'): ['41', '42', '43', '44', '45'], (621051, '745183'): ['58', '59'], (621051, '745185'): ['51', '52', '53'], (621051, '745097'): ['56', '57', '58'], (621051, '746801'): ['48', '49', '50'], (621051, '746796'): ['63', '64', '68', '69', '70', '71'], (621051, '745747'): ['57', '58', '65'], (621051, '745745'): ['51', '52', '53', '54'], (621051, '745246'): ['69', '71'], (621051, '747187'): ['68', '69', '70'], (621051, '746781'): ['60', '61', '62', '63'], (621051, '746782'): ['53', '54', '55', '56'], (621051, '746780'): ['43', '48', '49', '50', '51'], (621051, '746778'): ['56', '57', '58'], (621051, '746779'): ['69', '70', '71'], (621051, '746617'): ['77', '78', '79', '80'], (621051, '746619'): ['72', '73', '74', '75'], (621051, '746286'): ['52', '53', '54'], (621051, '744988'): ['76', '77', '80'], (621051, '744984'): ['52', '53', '54'], (621051, '746768'): ['73', '80', '81', '82', '83'], (621051, '746765'): ['63', '64', '65', '66', '67'], (621051, '745875'): ['61', '62', '63', '64', '65', '66', '67'], (621051, '745879'): ['82', '83', '84', '85', '86', '87', '88', '89'], (621051, '746761'): ['75', '76', '77'], (621051, '746762'): ['84', '85', '86', '87', '88'], (621051, '746759'): ['68', '69', '70'], (669713, '746890'): ['1', '2', '3', '4', '9', '10', '11', '12', '13', '18', '19', '20', '24', '25', '26', '27', '28', '34', '35', '36', '41', '42', '43', '48'], (669713, '746879'): ['1', '2', '3', '4', '5', '9', '10', '11', '15', '16', '17', '23', '24', '25', '26', '30', '31', '32', '33', '34', '35', '36', '40', '41', '42', '43', '47'], (669713, '747128'): ['46', '47', '48', '49', '50', '51', '55', '56', '57', '58', '64', '65', '66'], (669713, '746881'): ['48', '52', '53', '54', '55'], (669713, '746880'): ['38', '39', '40', '44', '45', '46', '47'], (669713, '746875'): ['67', '68', '69'], (669713, '746874'): ['32', '33', '34', '35', '40', '41'], (669713, '745986'): ['58', '59', '60', '61', '62'], (669713, '745987'): ['54', '55', '56'], (669713, '745985'): ['57'], (669713, '746870'): ['51', '52', '53', '54'], (669713, '746872'): ['53', '54', '55', '56'], (669713, '746873'): ['67', '68', '69', '70'], (669713, '745077'): ['58', '59', '60', '65', '66', '67'], (669713, '746869'): ['51', '52', '53', '54'], (669713, '746868'): ['47', '48', '49', '55', '56', '57', '58', '59'], (669713, '746861'): ['35', '36', '37', '38', '39', '40', '44', '45', '46'], (669713, '745325'): ['4', '5', '6', '13', '14', '15', '16', '17', '18', '24', '25', '26', '32', '33', '34'], (669713, '746860'): ['1', '2', '3', '4', '10', '11', '12', '13', '14', '15', '19', '20', '21', '22', '23', '29', '30', '31', '35', '36', '37', '38', '39'], (669713, '746855'): ['1', '2', '3', '4', '8', '9', '10', '14', '15', '16', '22', '23', '24', '31', '32', '33', '38', '39', '40', '45', '46', '47'], (669713, '745152'): ['5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '21', '22', '23', '24', '25', '30', '31', '32', '33', '40', '41', '42', '43', '44'], (669713, '746854'): ['41', '45', '46', '47', '48'], (669713, '746826'): ['49', '50', '51', '56', '57', '58'], (669713, '745526'): ['62', '63', '64', '70', '71', '72', '73', '74'], (669713, '746820'): ['41', '42', '43', '48', '49', '50'], (592767, '745039'): ['78', '79', '80', '81', '82'], (592767, '746899'): ['57', '58', '59', '65', '66', '67'], (592767, '746897'): ['44', '45', '46', '50', '51', '52', '59', '60', '61', '62'], (592767, '745433'): ['42', '47', '48', '49', '50'], (592767, '745274'): ['43', '44', '45', '49', '50', '51', '52'], (592767, '747213'): ['57', '58', '59', '61', '62'], (592767, '747214'): ['96'], (592767, '747212'): ['72', '73', '74'], (592767, '746894'): ['41', '42', '43', '44', '45', '49', '50', '51', '56', '57'], (592767, '745172'): ['45', '46', '47', '48', '49', '50', '56', '57', '58', '59', '60'], (592767, '746877'): ['47', '48', '49', '53', '54', '55', '56', '57'], (592767, '746871'): ['45', '46', '47'], (592767, '746873'): ['54', '55', '56', '57', '58'], (592767, '746704'): ['61', '62', '63'], (592767, '745079'): ['59', '60', '61', '66', '67', '68'], (592767, '746866'): ['48', '49', '50', '51', '58', '59', '60', '61'], (592767, '746864'): ['44', '45', '50', '51', '52', '53', '54'], (592767, '745323'): ['79', '80', '81', '83'], (592767, '745325'): ['39', '40', '41', '42', '47', '48', '49'], (592767, '745971'): ['46', '47', '48', '49', '53', '54', '55', '56'], (592767, '746857'): ['60', '61', '62', '63', '64'], (592767, '746856'): ['40', '41', '42', '43', '48', '49', '50', '51'], (592767, '747011'): ['63', '64', '65', '66'], (592767, '745175'): ['34', '35', '41', '42', '43'], (592767, '745151'): ['50', '55', '56', '57'], (592767, '746852'): ['53', '54', '55', '59', '60', '61', '62'], (592767, '746850'): ['30', '31', '32', '39', '40', '41'], (592767, '746684'): ['61', '62', '63'], (592767, '746678'): ['51', '52', '53', '59', '60', '61'], (592767, '746845'): ['45', '46', '51', '52', '53'], (592767, '746846'): ['49', '50', '51'], (592767, '746840'): ['55', '56', '57', '58'], (592767, '746842'): ['40', '41', '42', '43'], (592767, '746760'): ['63', '64', '65'], (592767, '746601'): ['40', '46', '47', '48', '49'], (592767, '746843'): ['43', '44', '45'], (592767, '746841'): ['50', '51', '52', '53'], (592767, '746839'): ['58', '59', '60', '61'], (592767, '746024'): ['59', '60', '61', '62'], (592767, '745452'): ['42', '43', '44', '50', '51', '52', '53', '54', '55'], (592767, '744810'): ['73', '74', '75'], (592767, '746834'): ['45', '46', '47'], (592767, '746829'): ['44', '49', '50', '51', '52'], (592767, '746101'): ['59', '60', '61', '62'], (592767, '746509'): ['67', '68', '69', '70'], (592767, '746508'): ['80', '82'], (592767, '746831'): ['51', '52', '53', '54'], (592767, '746823'): ['52', '53', '54'], (592767, '745526'): ['43', '44', '45', '46', '54'], (592767, '746820'): ['55', '56', '57'], (668338, '745268'): ['63', '64', '65', '66'], (668338, '746570'): ['67', '68', '69'], (668338, '745028'): ['52', '53', '54', '55'], (668338, '745261'): ['49', '50', '51', '56', '57', '58', '59'], (668338, '746402'): ['64', '65', '66', '67'], (668338, '745915'): ['57', '58', '59', '60'], (668338, '745910'): ['57', '58', '59', '60', '61', '62', '68', '69', '70'], (668338, '747124'): ['57', '58', '59', '60', '65', '66', '67'], (668338, '746881'): ['59', '60', '61', '68', '69', '70'], (668338, '746875'): ['45', '50', '51', '52', '59', '60'], (668338, '745176'): ['58', '59'], (668338, '745987'): ['79', '80'], (668338, '745985'): ['65', '66'], (668338, '746877'): ['58', '62', '63', '64', '65'], (668338, '746872'): ['34', '35', '41', '42', '43', '44', '45'], (668338, '745079'): ['44', '49', '50', '51'], (668338, '745075'): ['56'], (668338, '746865'): ['60', '61', '62', '66', '67', '68'], (668338, '746867'): ['52', '56', '57', '58'], (668338, '746864'): ['68', '69'], (668338, '746862'): ['54', '60', '61', '62', '63'], (668338, '745323'): ['70', '71'], (668338, '745324'): ['63', '64', '65', '66'], (668338, '746858'): ['60', '61', '62', '63', '64', '65'], (668338, '746856'): ['71', '72', '73'], (668338, '747014'): ['59', '60', '61', '62'], (668338, '745175'): ['47', '48', '49'], (668338, '746852'): ['47', '48', '49'], (668338, '746850'): ['45', '46', '47', '48'], (668338, '746851'): ['45', '46', '47', '48'], (668338, '746274'): ['57', '58', '59'], (668338, '746678'): ['68', '69', '70', '71', '72', '73'], (668338, '746845'): ['37', '38', '39', '43', '44'], (668338, '746844'): ['60', '61', '62', '63'], (668338, '746840'): ['65', '66', '67'], (668338, '746760'): ['57', '58', '59'], (668338, '746598'): ['51', '52', '53', '54'], (668338, '746843'): ['76', '77', '78'], (668338, '746841'): ['33', '34', '35', '36'], (668338, '746839'): ['46', '47', '52', '53', '54'], (668338, '746024'): ['46', '47', '48', '52', '53', '54', '55'], (668338, '745458'): ['79', '80'], (668338, '745452'): ['60', '61', '62'], (668338, '744810'): ['69', '70', '71', '72'], (668338, '744808'): ['54', '55', '56', '57'], (668338, '746834'): ['51', '52', '53', '54', '55', '56'], (668338, '746829'): ['53', '57', '58', '59'], (668338, '746101'): ['71', '72', '73', '74'], (668338, '746100'): ['59', '60', '61'], (668338, '746508'): ['54', '55', '56'], (668338, '746506'): ['84', '85', '86'], (668338, '746831'): ['45', '46', '47'], (668338, '746824'): ['50', '58', '59', '60', '61'], (668338, '745526'): ['55', '56'], (668338, '746822'): ['50', '51', '52', '53'], (668338, '746819'): ['56', '57'], (663878, '745116'): ['66', '67', '68', '69'], (663878, '745115'): ['43', '44'], (663878, '746407'): ['45', '46', '47', '48'], (663878, '745766'): ['67', '68', '69', '70', '75', '76', '77', '78', '79'], (663878, '744950'): ['43', '44', '45', '46'], (663878, '744952'): ['68', '69', '70'], (663878, '745434'): ['58', '62', '63', '64', '65', '66', '67', '68'], (663878, '746321'): ['61', '62'], (663878, '744949'): ['54', '55', '56'], (663878, '744946'): ['70'], (663878, '744945'): ['77', '78', '79', '80', '81'], (663878, '744939'): ['53', '54', '55', '56', '57', '58'], (663878, '744862'): ['58', '59', '60', '61'], (663878, '744861'): ['53', '54', '55', '60'], (663878, '744940'): ['59', '60', '61'], (663878, '744941'): ['61', '62', '63'], (663878, '744934'): ['63', '64', '65'], (663878, '744938'): ['60', '61', '62', '63'], (663878, '744933'): ['76', '77', '78', '79'], (663878, '746475'): ['42', '43', '44', '49', '50', '51'], (663878, '746788'): ['49', '50', '51', '52', '57', '58', '59'], (663878, '744928'): ['69', '70', '71', '75', '76', '77'], (663878, '744929'): ['68', '69', '70', '71'], (663878, '744931'): ['59', '60', '61', '62', '63', '64'], (663878, '744923'): ['51', '52', '53'], (663878, '745651'): ['42', '43', '44', '45', '46'], (663878, '745981'): ['50', '51', '52', '53'], (663878, '744925'): ['66', '67', '68', '69'], (663878, '744926'): ['79'], (663878, '744917'): ['54', '55', '56', '57'], (663878, '746622'): ['63', '64', '65'], (663878, '746941'): ['62', '63', '64', '65'], (663878, '744922'): ['45', '46', '47', '48', '49', '50'], (663878, '744916'): ['50', '51', '52', '53', '54'], (663878, '744913'): ['48', '49', '50', '54', '55', '56', '57'], (663878, '745237'): ['52', '53', '54', '55'], (663878, '745233'): ['61', '62', '63'], (663878, '745316'): ['59', '60', '61', '62'], (663878, '747177'): ['71', '72', '73', '74', '75', '76', '77'], (663878, '744904'): ['52', '53', '54', '55'], (663878, '744907'): ['59', '60', '61', '62', '63', '68', '69', '70', '71'], (663878, '746684'): ['52', '53', '54', '58', '59', '60'], (663878, '746849'): ['59', '60', '61', '65', '66', '67', '68'], (663878, '746847'): ['53', '54', '55', '60', '61', '62', '63'], (663878, '746603'): ['50', '51', '52', '53'], (663878, '746601'): ['60', '61', '62'], (663878, '746841'): ['18', '19', '20', '24', '25', '26', '27', '28', '29'], (663878, '746837'): ['43', '44', '45', '46', '52', '53', '54'], (663878, '745461'): ['55', '56', '57', '58', '64', '65', '66'], (663878, '744810'): ['56', '57', '58', '62', '63', '64'], (663878, '746835'): ['63', '64', '65'], (663878, '746827'): ['49', '50', '51', '56', '57', '58'], (663878, '746097'): ['63', '64', '65', '66'], (663878, '746509'): ['71', '72', '73', '74'], (663878, '746508'): ['62', '63', '64', '65'], (663878, '746831'): ['55', '56', '57', '61', '62', '63', '64', '65', '66'], (663878, '746824'): ['45', '46', '47', '48', '49'], (663878, '745527'): ['5', '6', '7', '8'], (663878, '745521'): ['70', '71', '72', '73'], (663878, '746820'): ['59', '65', '66', '67', '68'], (696136, '745274'): ['5', '6', '7', '12', '13', '14', '15', '16', '17', '18', '23', '24', '25', '26', '31', '32', '33', '34', '35', '36', '37', '38'], (696136, '747212'): ['5', '6', '7', '8', '9', '14', '15', '16', '17', '21', '22', '23', '24', '28', '29', '30', '36', '37', '38', '39'], (696136, '746888'): ['1', '2', '3', '4', '13', '14', '15', '21', '22', '23', '28', '29', '30', '31', '32', '37', '38', '39', '40', '41', '45', '46', '47'], (696136, '746703'): ['41', '42', '43', '44', '45', '51', '52', '53', '57', '58', '59', '65', '66'], (696136, '746869'): ['1', '2', '3', '4', '9', '10', '11'], (696136, '744806'): ['5', '6', '7', '11', '12', '13', '14', '15', '24', '25', '26', '32', '33', '34', '39', '40', '41', '42'], (696136, '746832'): ['1', '2', '3', '4', '5', '6', '11', '12', '13', '18', '19', '20', '21', '22', '23', '24', '28', '29', '30', '31', '36', '37', '38'], (696136, '746100'): ['8', '9', '10', '11', '12', '13', '14', '15', '16', '20', '21', '22', '23', '24', '30', '31', '32', '33', '34', '35', '36'], (696136, '746830'): ['1', '2', '3', '4', '5', '11', '12', '13', '14', '19', '20', '21', '22', '23', '24', '32', '33', '34', '35', '39', '40', '41', '42'], (681799, '746860'): ['57', '58', '59', '60'], (681799, '746856'): ['55', '56', '57', '58', '59', '63', '64', '65', '66'], (681799, '745152'): ['48', '49', '50', '54', '55', '56', '57'], (681799, '746683'): ['63', '64', '65', '66', '67', '71', '72', '73', '74', '75'], (681799, '746838'): ['45', '46', '47', '48', '49', '54', '55', '56', '57', '58'], (681799, '746836'): ['61', '62', '63', '64'], (681799, '746022'): ['76', '77', '78'], (681799, '745458'): ['68', '73', '74', '75', '76', '77', '78'], (681799, '745452'): ['27', '31', '32', '33', '34', '35', '39', '40', '41', '42'], (681799, '744806'): ['78', '79', '80'], (681799, '746834'): ['61', '62', '63', '67', '68', '69', '70'], (681799, '746101'): ['81', '82', '83'], (681799, '746100'): ['40', '41', '42'], (681799, '746509'): ['57', '58', '59', '60'], (681799, '746506'): ['59', '60', '61', '62', '63'], (681799, '746831'): ['21', '26', '27', '28', '29'], (681799, '746823'): ['58', '59', '60'], (681799, '746825'): ['58', '59', '60', '61', '62'], (681799, '745521'): ['62', '63', '64', '65', '66'], (681799, '746820'): ['72', '73', '74', '75', '76', '77'], (605347, '745835'): ['53', '54', '58', '59', '60'], (605347, '745831'): ['66', '67', '68', '69'], (605347, '745829'): ['45', '46', '47'], (605347, '745101'): ['56', '57', '58', '59', '60', '61'], (605347, '745103'): ['56', '57', '58', '59'], (605347, '745186'): ['65', '66', '67'], (605347, '745828'): ['61', '62', '63', '64', '65', '66', '67'], (605347, '745826'): ['53', '54', '55'], (605347, '745821'): ['50', '54', '55', '56', '57'], (605347, '745580'): ['79', '80'], (605347, '746067'): ['91', '92'], (605347, '746637'): ['58', '59', '60'], (605347, '746633'): ['55', '59', '60', '61'], (605347, '745822'): ['69', '70', '71', '72'], (605347, '745818'): ['71', '72', '73', '74', '75', '76'], (605347, '745817'): ['69', '70', '71', '72'], (605347, '745970'): ['54', '55', '56', '57'], (605347, '745971'): ['36', '40', '41', '42'], (605347, '746860'): ['64', '65', '66'], (605347, '746856'): ['28', '32', '33', '34', '35', '36'], (605347, '747011'): ['71', '72', '73', '74', '75'], (605347, '745151'): ['64', '65', '66', '70', '71', '72', '73', '74'], (605347, '746854'): ['55', '56', '57', '58', '59'], (605347, '746848'): ['64', '65', '66', '67', '68'], (605347, '746277'): ['44', '45', '46', '47'], (605347, '746684'): ['39', '40', '45', '46', '47', '48'], (605347, '746683'): ['52', '53', '54', '55', '56'], (605347, '746849'): ['54', '55'], (605347, '746847'): ['46', '47', '48', '49'], (605347, '746842'): ['47', '48', '49', '50', '54', '55', '56', '57'], (605347, '746761'): ['70', '71', '72', '73', '74'], (605347, '746603'): ['74', '75'], (605347, '746601'): ['54', '55', '56'], (605347, '746843'): ['52', '53', '54'], (605347, '746841'): ['67', '68', '69', '70'], (605347, '746024'): ['78', '79', '80'], (605347, '744808'): ['63', '64', '65', '66'], (605347, '746833'): ['54', '55', '56', '57', '58', '59', '60'], (605347, '746831'): ['70', '71', '72'], (605347, '746823'): ['67', '68', '69'], (668941, '745200'): ['69', '70', '71'], (668941, '745197'): ['52', '53', '54', '55'], (668941, '745195'): ['49', '50', '51', '57', '58', '59', '60'], (668941, '747217'): ['60', '61', '62', '63'], (668941, '745677'): ['58', '59', '60', '61'], (668941, '745674'): ['62', '63', '64'], (668941, '745192'): ['52', '53', '57'], (668941, '745191'): ['63', '64'], (668941, '745188'): ['49', '50', '51', '52'], (668941, '745835'): ['50', '55', '56', '57'], (668941, '745833'): ['59', '60', '61'], (668941, '746478'): ['58', '59', '60', '61', '62'], (668941, '745187'): ['56', '57', '58'], (668941, '745183'): ['60', '61', '62'], (668941, '745994'): ['51', '52', '53', '54', '59'], (668941, '745990'): ['62', '63', '64', '65', '69'], (668941, '746235'): ['73', '74', '75', '76', '77'], (668941, '745178'): ['56', '57', '58', '59'], (668941, '745177'): ['57', '58', '59', '60', '61'], (668941, '745181'): ['50', '54', '55', '56'], (668941, '745180'): ['57', '58'], (668941, '745176'): ['55', '60', '61', '62', '63', '64'], (668941, '745172'): ['61', '62', '63', '64', '65', '66'], (668941, '745572'): ['54', '58', '59', '60'], (668941, '746382'): ['60', '66', '67', '68', '69', '70', '71'], (668941, '746379'): ['51', '52', '53'], (668941, '745169'): ['66', '67', '68', '69'], (668941, '745170'): ['49', '50', '51'], (668941, '746869'): ['58'], (668941, '746055'): ['63', '64', '65', '66'], (668941, '745163'): ['57', '58', '59'], (668941, '745165'): ['54', '55', '56'], (668941, '745158'): ['55', '56'], (668941, '745161'): ['56', '57', '58'], (668941, '745154'): ['54', '58', '59', '60'], (668941, '745482'): ['66', '67', '68'], (668941, '745487'): ['79', '80'], (668941, '744836'): ['80', '81', '82'], (668941, '744833'): ['64', '71', '72', '73', '74'], (668941, '745175'): ['63', '64', '65', '66'], (668941, '745475'): ['55', '56', '57'], (668941, '745146'): ['62', '66', '67', '68', '69', '70'], (668941, '745148'): ['65', '66', '67'], (668941, '746849'): ['62', '63', '64'], (668941, '746846'): ['62', '63', '64'], (668941, '745142'): ['58', '59', '60', '64'], (668941, '745143'): ['58', '59', '60', '65', '66'], (668941, '745144'): ['72', '73', '74'], (668941, '745140'): ['58', '59', '60'], (668941, '745138'): ['57', '58', '59', '60', '66'], (668941, '745137'): ['60', '61', '62', '63'], (668941, '745863'): ['57', '58', '59'], (668941, '745134'): ['63', '64', '65', '66'], (668941, '745701'): ['58', '68', '69', '70', '71', '72'], (668941, '745944'): ['53', '54', '55', '56'], (668941, '745938'): ['61', '62', '63'], (668941, '745125'): ['46', '47'], (668941, '745127'): ['55', '56'], (668941, '745129'): ['59', '60', '69', '70', '71', '72'], (668941, '744883'): ['61'], (668941, '745120'): ['56', '57', '58'], (668941, '745124'): ['62', '63', '64', '65', '66'], (664854, '745197'): ['67', '68', '69', '70'], (664854, '745194'): ['74', '76', '77', '78'], (664854, '745195'): ['66', '67', '68'], (664854, '747217'): ['78', '79', '80'], (664854, '745677'): ['66', '67', '68'], (664854, '745674'): ['68', '69', '70'], (664854, '745189'): ['74', '75', '76', '77'], (664854, '745191'): ['68', '69', '70', '71'], (664854, '745188'): ['68', '69', '70'], (664854, '745835'): ['68', '69', '70'], (664854, '745833'): ['74', '75', '76', '77', '78', '79'], (664854, '746477'): ['66', '67', '68'], (664854, '745187'): ['63', '64', '65'], (664854, '745183'): ['67', '68', '69', '70'], (664854, '745990'): ['77', '78', '79'], (664854, '746235'): ['82', '83', '84', '85'], (664854, '745181'): ['71', '72', '73', '74'], (664854, '745180'): ['62', '63', '64'], (664854, '745176'): ['73', '74', '75', '76', '77', '78', '79'], (664854, '745172'): ['70', '71', '72', '73', '74', '75'], (664854, '746706'): ['75', '76', '77', '78', '79', '80'], (664854, '745572'): ['82', '83', '84'], (664854, '746379'): ['64', '65', '66'], (664854, '745169'): ['77', '78', '79'], (664854, '745170'): ['57', '58', '59', '60', '61', '62', '63'], (664854, '745171'): ['66', '67', '68', '69'], (664854, '746869'): ['65', '66', '67', '68', '69'], (664854, '746866'): ['70', '71', '72', '73', '74'], (664854, '745164'): ['70', '71', '72', '73', '74'], (664854, '745165'): ['66', '67', '68'], (664854, '745160'): ['64', '65', '66', '67', '68', '69', '70'], (664854, '745158'): ['63', '64', '65', '66', '67'], (664854, '745161'): ['65', '66', '67'], (664854, '745154'): ['64', '65', '66'], (664854, '745482'): ['74', '75', '76'], (664854, '744836'): ['87', '88', '89', '90', '94', '95', '97'], (664854, '745175'): ['73', '74', '75', '76'], (664854, '747094'): ['69', '70', '71', '72', '73'], (664854, '745475'): ['64', '65', '66'], (664854, '745146'): ['74', '75', '76'], (664854, '745145'): ['65', '66', '67'], (664854, '746849'): ['69', '70', '71', '72', '73', '74'], (664854, '746846'): ['69', '70', '71'], (664854, '745141'): ['65', '66', '67', '68', '69', '70'], (664854, '745142'): ['71', '72', '73', '74', '75'], (664854, '746275'): ['81', '82', '83', '84'], (664854, '745140'): ['66', '67', '68'], (664854, '745138'): ['90', '91', '92'], (664854, '745137'): ['69', '70', '71', '72'], (664854, '745863'): ['69', '70', '71', '72'], (664854, '745134'): ['71', '72', '73', '74'], (664854, '745135'): ['69', '70', '71'], (664854, '745699'): ['72', '73', '74', '76'], (664854, '745944'): ['82', '83', '84', '85', '91', '92', '93'], (664854, '745125'): ['61', '62', '63', '64'], (664854, '745127'): ['64', '65', '66'], (664854, '745122'): ['65', '66', '67', '68'], (664854, '745124'): ['73', '74', '75'], (664854, '745118'): ['61', '62', '63'], (664854, '746501'): ['71'], (664854, '745280'): ['79', '80', '81', '82'], (552640, '745197'): ['59', '60', '61', '62'], (552640, '745195'): ['42', '46', '47', '48'], (552640, '745196'): ['72', '73'], (552640, '747217'): ['69', '70', '71'], (552640, '745677'): ['52', '53', '54'], (552640, '745192'): ['58', '59', '60'], (552640, '745191'): ['52', '53', '54', '55', '62'], (552640, '745188'): ['58', '59', '60'], (552640, '745835'): ['61', '62', '63', '64'], (552640, '745833'): ['67', '68', '69', '70'], (552640, '745183'): ['47', '48', '49', '50', '54', '55', '56'], (552640, '745182'): ['51', '52', '53', '54'], (552640, '745994'): ['60', '61', '62', '63', '64'], (552640, '745990'): ['70', '71'], (552640, '746233'): ['71'], (552640, '745178'): ['67', '68', '69', '70', '71'], (552640, '745177'): ['48', '49', '50'], (552640, '745181'): ['67'], (552640, '745180'): ['49', '50', '51', '52', '56'], (552640, '745176'): ['50', '51', '52', '53', '54'], (552640, '745172'): ['51', '52', '53', '54', '55'], (552640, '746706'): ['63', '64', '65', '66', '67'], (552640, '745572'): ['66', '67', '68', '69', '70'], (552640, '746382'): ['53', '57', '58', '59'], (552640, '746379'): ['57', '58', '59', '60'], (552640, '745169'): ['59', '60', '61', '62'], (552640, '745166'): ['51', '52', '53', '54', '55', '56'], (552640, '745171'): ['59', '60', '61', '62'], (552640, '745168'): ['69', '70', '71', '72'], (552640, '746866'): ['62', '63', '64', '65'], (552640, '746055'): ['84', '85', '86', '92', '93', '94'], (552640, '745164'): ['50', '51', '52', '53', '57'], (552640, '745165'): ['57', '58', '59', '60'], (552640, '745158'): ['51', '52', '53', '54'], (552640, '745161'): ['59'], (552640, '745154'): ['51', '52', '53'], (552640, '745487'): ['69', '70', '71', '77', '78'], (552640, '745484'): ['54', '55', '56', '57', '58'], (552640, '744833'): ['54', '55', '62', '63'], (552640, '745156'): ['70', '71', '72', '73'], (552640, '745175'): ['56', '57', '58', '62'], (552640, '745151'): ['58', '59', '60', '61', '62', '63'], (552640, '747093'): ['66', '67', '68', '69', '70'], (552640, '747094'): ['59', '60', '61', '62'], (552640, '745475'): ['50', '51'], (552640, '745146'): ['59', '60', '61'], (552640, '745145'): ['59', '60', '61'], (552640, '745141'): ['59', '60', '61'], (552640, '745142'): ['65', '66', '67'], (552640, '746275'): ['68', '69', '70', '71', '72'], (552640, '745144'): ['45', '46', '47'], (552640, '745138'): ['67', '68', '69', '70'], (552640, '745137'): ['48', '49', '50'], (552640, '745863'): ['50', '51', '52', '53'], (552640, '745132'): ['63', '67', '68', '69'], (552640, '745134'): ['56', '57'], (552640, '745699'): ['67', '68'], (552640, '745701'): ['77', '78', '79'], (552640, '745944'): ['60', '61', '62'], (552640, '745938'): ['64', '66', '70', '71', '72'], (552640, '745125'): ['51', '52', '53', '54', '55'], (552640, '745127'): ['48', '49', '54'], (552640, '745129'): ['47', '48', '49', '56', '57', '58'], (552640, '744883'): ['56', '57', '58', '59'], (552640, '745130'): ['56', '57', '58'], (552640, '745120'): ['48', '49', '50', '51'], (552640, '745124'): ['50', '51', '52', '53', '54'], (552640, '745118'): ['53', '54', '55', '56'], (552640, '746504'): ['55', '56', '57'], (552640, '746501'): ['60', '61', '62', '63', '64'], (552640, '745280'): ['64', '65', '66'], (669461, '745195'): ['39', '40', '41'], (669461, '745671'): ['61', '62', '63', '67', '68', '69'], (669461, '745189'): ['52', '56', '57', '58'], (669461, '745192'): ['66', '67', '68'], (669461, '745834'): ['62', '66', '67', '68', '69', '74', '76', '80', '81', '82', '83'], (669461, '746477'): ['57', '58', '59'], (669461, '746478'): ['50', '51', '52'], (669461, '745185'): ['1', '2', '3', '4', '8', '9', '10', '11', '15', '16', '17', '18', '23', '24'], (669461, '745993'): ['38', '39', '40', '41'], (669461, '746233'): ['4', '5', '6', '7', '11', '12', '13', '18', '19', '20', '21', '22', '23', '24', '28'], (669461, '745179'): ['1', '2', '3', '4', '8', '9', '10', '11', '12', '16', '17', '18', '19', '20', '24', '25'], (669461, '746708'): ['52', '53', '54', '55'], (669461, '745574'): ['54', '58', '59', '60'], (669461, '745572'): ['44', '45', '46'], (669461, '746380'): ['60'], (669461, '745174'): ['37', '43', '44', '45', '46', '47'], (669461, '745167'): ['44', '45', '46'], (669461, '745168'): ['57', '58'], (669461, '746865'): ['63', '64', '65'], (669461, '746049'): ['59', '60', '61', '62'], (669461, '746050'): ['4', '5', '6', '7', '8', '9', '17', '18', '19', '24', '25', '26', '32', '33'], (669461, '745158'): ['1', '2', '3', '7', '8', '9', '10', '14', '15', '16', '17', '24', '25', '26', '30', '31', '32', '37', '38', '39'], (669461, '745487'): ['54', '55', '62', '63', '64', '65'], (669461, '744836'): ['62', '69', '70', '71'], (669461, '744835'): ['40', '47', '48', '49', '50', '51', '52', '53', '58', '59', '60'], (669461, '745156'): ['26', '27', '28', '34', '35', '36', '37', '38', '45', '46'], (669461, '745152'): ['58', '59', '60'], (669461, '745151'): ['54'], (669461, '745472'): ['5', '6', '7', '8', '14', '15', '16', '17', '18', '19', '20', '21', '22', '26', '27', '28', '33'], (669461, '745150'): ['48', '49', '50', '51', '52', '53'], (669461, '746845'): ['57'], (669461, '746844'): ['56', '57', '58', '59', '64'], (669461, '745184'): ['58', '59', '60', '64', '65', '66', '67', '68'], (669461, '746275'): ['43', '44', '45', '51', '52', '53', '54', '59', '60', '61'], (669461, '746673'): ['53', '54', '55', '56'], (669461, '746677'): ['62', '63', '67', '68', '69', '70', '71'], (669461, '745144'): ['48', '56', '57', '58', '59'], (669461, '745139'): ['49', '50', '51'], (669461, '745138'): ['71', '77', '78', '80'], (669461, '745868'): ['54', '55', '56', '57'], (669461, '745133'): ['43', '44', '45', '49', '50', '51', '55'], (669461, '745134'): ['49', '54', '55'], (669461, '745135'): ['60', '61', '62'], (669461, '745699'): ['61', '62', '63', '64', '65', '66'], (669461, '745944'): ['66', '67', '68', '73', '74', '75', '77'], (669461, '745131'): ['38', '43', '44', '45', '46'], (669461, '745127'): ['41', '42', '43', '47'], (669461, '744882'): ['56', '57', '58', '59', '65', '66', '67', '68', '73', '74', '75'], (669461, '745122'): ['55', '56', '57', '58', '59'], (669461, '745121'): ['50', '51', '52'], (669461, '745124'): ['67', '68', '69'], (669461, '746504'): ['66', '67', '68', '69'], (669461, '746501'): ['68', '69', '70'], (669461, '745280'): ['71', '72', '73', '74'], (669461, '745281'): ['74', '75', '76', '77', '78'], (667463, '746166'): ['70', '71', '72'], (667463, '745198'): ['62', '63', '64', '65', '70', '71', '72', '73'], (667463, '745185'): ['46', '47', '48', '49', '50', '54', '55', '56'], (667463, '745182'): ['67', '68', '69'], (667463, '745995'): ['59', '60', '61', '62'], (667463, '746233'): ['59', '60', '61', '66', '67'], (667463, '746235'): ['65', '66', '67'], (667463, '745178'): ['78', '79', '80', '81', '82'], (667463, '745177'): ['40', '41', '42'], (667463, '745181'): ['64', '65', '66'], (667463, '745173'): ['35', '36', '37', '38', '44', '45', '46'], (667463, '745172'): ['42', '43', '44'], (667463, '746709'): ['56', '57', '58', '64', '65', '66', '67', '68'], (667463, '746706'): ['57', '58'], (667463, '745575'): ['54', '55', '56', '57'], (667463, '745572'): ['74', '75', '76'], (667463, '745174'): ['55', '56', '57', '63'], (667463, '745169'): ['48', '49', '50', '51', '52'], (667463, '745167'): ['62', '66', '67', '68', '69', '70'], (667463, '745168'): ['39', '40', '46', '47', '48'], (667463, '746865'): ['56', '57', '58', '59'], (667463, '746049'): ['76', '80', '81', '82'], (667463, '746050'): ['46', '47', '48', '49'], (667463, '745164'): ['58', '59', '60'], (667463, '745157'): ['74', '75', '76', '77'], (667463, '745162'): ['46', '47', '52', '53', '54', '58', '59', '60', '61'], (667463, '745159'): ['40', '41', '42', '43'], (667463, '745482'): ['50', '51', '52'], (667463, '745484'): ['64', '65', '66', '72', '73', '74', '75', '76'], (667463, '744832'): ['64', '65', '71'], (667463, '745156'): ['56', '57', '58', '64', '65', '66'], (667463, '745153'): ['58', '59'], (667463, '745175'): ['44', '45', '46', '50', '51', '52'], (667463, '745473'): ['60', '61', '62', '63', '64'], (667463, '745475'): ['43', '44', '45', '49'], (667463, '745146'): ['51', '52', '53', '58'], (667463, '745145'): ['49', '50', '51', '52'], (667463, '745147'): ['47', '48'], (667463, '746844'): ['41', '42', '46', '47', '48'], (667463, '745143'): ['74', '75'], (667463, '746269'): ['58', '59', '60', '61'], (667463, '746677'): ['42', '43', '44', '45', '53', '54'], (667463, '745139'): ['65', '66', '67', '68'], (667463, '745136'): ['48', '52', '53', '54', '55'], (667463, '745870'): ['66', '67', '68'], (667463, '745133'): ['67'], (667463, '745132'): ['52', '53', '58', '59', '60', '61', '62'], (667463, '745700'): ['46', '47', '51', '52', '53', '54'], (667463, '745701'): ['46', '47', '48', '52', '53', '54'], (667463, '745131'): ['52', '53', '54'], (667463, '745126'): ['48', '49', '55', '56', '57', '61'], (667463, '744882'): ['48', '49', '50', '51'], (667463, '745120'): ['65', '66', '67', '68'], (667463, '746504'): ['45', '46', '47', '48'], (667463, '746495'): ['61', '62', '63'], (667463, '745280'): ['57', '58', '59', '60'], (681517, '745185'): ['25', '30', '31', '32', '36', '37', '38'], (681517, '745186'): ['41', '42', '43', '44', '45', '50', '51', '52', '53'], (681517, '745993'): ['45', '46', '47', '48', '54', '55', '56', '57', '58'], (681517, '746233'): ['29', '30', '34', '35', '36', '42', '43', '44'], (681517, '745177'): ['71', '72', '73'], (681517, '745173'): ['53', '54', '55', '56', '61', '62', '63', '64'], (681517, '746708'): ['56', '57', '61', '62', '63'], (681517, '746706'): ['54', '55', '56'], (681517, '745574'): ['50', '51', '52', '53'], (681517, '746380'): ['40', '41', '42', '43', '44', '49', '50', '51', '58', '59'], (681517, '745169'): ['40', '41', '42', '43', '44'], (681517, '746865'): ['41', '46', '47', '48', '54', '55'], (681517, '746049'): ['68', '69', '70', '74', '75'], (681517, '746050'): ['50', '51', '57', '58', '59', '63', '64', '65'], (681517, '745163'): ['75', '76', '77', '78'], (681517, '745157'): ['38', '39', '40', '41', '49', '50', '51', '52', '53', '58', '59', '60', '61', '62'], (681517, '745152'): ['64', '65', '66', '72', '73', '74', '75', '76'], (681517, '747096'): ['64', '65', '66', '67', '75', '76', '77'], (681517, '745147'): ['66', '67', '71', '72', '73'], (681517, '745149'): ['67', '68', '69', '70'], (681517, '746845'): ['66', '67', '68', '69'], (681517, '746844'): ['65', '66', '67', '68'], (681517, '745143'): ['55', '56', '57'], (681517, '746682'): ['47', '48', '49', '55', '56'], (681517, '746677'): ['55', '60', '61'], (681517, '745136'): ['56', '63', '64', '65'], (681517, '745868'): ['62', '63', '64', '65'], (681517, '745133'): ['72', '73', '74'], (681517, '745947'): ['58', '64', '65', '66', '71', '72', '73'], (681517, '745131'): ['59', '60', '61', '62', '67', '68', '69', '70', '71'], (681517, '746495'): ['76', '77', '78'], (681517, '745281'): ['56', '66', '67', '68'], (571927, '746166'): ['4', '5', '6', '7', '12', '13', '14', '21', '22', '23', '24', '28', '29', '30', '31', '32', '39', '40', '41', '50', '51', '52'], (571927, '745197'): ['1', '2', '3', '7', '8', '9', '14', '15', '16', '17', '23', '24', '25', '26', '27', '34', '35', '36', '37'], (571927, '747217'): ['8', '9', '10', '11', '17', '18', '19', '20', '26', '27', '28', '29', '33', '34', '35', '39', '40', '41', '42', '43', '44'], (571927, '745671'): ['6', '7', '8', '9', '13', '14', '15', '16', '20', '21', '22', '23', '24', '25', '33', '34', '35', '36', '37', '42', '43', '44', '45', '46', '47'], (571927, '745193'): ['1', '2', '3', '8', '9', '10', '11', '16', '17', '18', '19', '20', '21', '22', '23', '27', '28', '29', '35', '36', '37', '38'], (571927, '746481'): ['4', '5', '6', '7', '8', '9', '15', '16', '17', '18', '23', '24', '25', '26', '27', '33', '34', '35'], (571927, '745944'): ['7', '8', '9', '10', '14', '15', '16', '20', '21', '22', '23', '24', '31', '32', '33', '37', '38', '39'], (571927, '745128'): ['25', '26', '27', '33', '34', '35', '40', '41', '42', '43', '44', '49', '50', '51'], (571927, '744883'): ['40', '41', '42', '44', '50', '51', '52'], (571927, '745123'): ['50', '51', '52', '53', '59', '60', '61', '62', '68', '69', '70'], (571927, '746495'): ['46', '47', '48', '49', '54', '55', '56', '57'], (571927, '745282'): ['70', '71', '72', '73'], (519293, '745838'): ['61', '62', '63', '64', '65', '66', '67'], (519293, '746807'): ['45', '46', '47'], (519293, '746325'): ['51', '52', '53', '54', '55', '56', '57'], (519293, '746316'): ['60', '61', '62'], (519293, '746482'): ['71', '72', '73', '74'], (519293, '746483'): ['48'], (519293, '746311'): ['46', '47', '48'], (519293, '746313'): ['59'], (519293, '746312'): ['42', '43'], (519293, '746240'): ['54', '55', '56'], (519293, '745259'): ['44', '45', '46', '47'], (519293, '746306'): ['56', '57', '58', '59'], (519293, '746304'): ['49', '50', '51', '52', '53'], (519293, '745088'): ['61', '62', '63', '64', '65'], (519293, '745900'): ['65', '66', '67', '68'], (519293, '746305'): ['45', '46', '47', '48'], (519293, '746631'): ['40', '45', '46', '47'], (519293, '746300'): ['55', '56', '57'], (519293, '746297'): ['76'], (519293, '746134'): ['56', '57', '58', '59'], (519293, '745650'): ['70', '71', '72'], (519293, '744997'): ['40', '41', '42', '43'], (519293, '746293'): ['51', '52', '56', '57'], (519293, '746289'): ['49', '50', '51', '52', '53'], (519293, '746290'): ['52', '53', '54'], (519293, '746285'): ['43', '44', '50', '51', '52', '53', '54'], (519293, '745155'): ['50', '51', '52'], (519293, '746932'): ['59', '60', '61', '62', '63'], (519293, '746280'): ['77', '78', '79', '80', '81'], (519293, '746277'): ['55', '56', '57', '58'], (519293, '746765'): ['72', '73', '74'], (519293, '746442'): ['68', '69', '70', '71'], (519293, '746439'): ['56', '57', '58', '59'], (519293, '746273'): ['73', '74', '75'], (519293, '746272'): ['70', '71', '72'], (519293, '746275'): ['62', '63', '64'], (519293, '745867'): ['57', '58', '62', '63', '64', '65', '66'], (519293, '746266'): ['77', '78', '79', '80', '81', '82'], (519293, '746271'): ['63', '64', '65', '66', '67', '68'], (519293, '746263'): ['66', '67', '68', '69', '70', '71', '72', '73'], (640451, '744869'): ['57', '58', '62', '63', '64'], (640451, '744871'): ['59', '60', '61'], (640451, '745357'): ['64', '65', '66'], (640451, '745679'): ['66', '67', '68', '69'], (640451, '745676'): ['61', '62', '63', '64'], (640451, '746158'): ['59', '60', '64', '65', '66', '67', '68'], (640451, '746159'): ['53', '57', '58', '59', '60'], (640451, '744865'): ['49', '50', '51', '52'], (640451, '744863'): ['56', '57', '61', '62', '63', '64'], (640451, '746078'): ['61', '62', '63'], (640451, '746077'): ['75', '76', '83', '84', '85', '86'], (640451, '745023'): ['53', '54', '58', '59', '60', '61'], (640451, '744868'): ['63', '64', '65', '66'], (640451, '744861'): ['68', '75', '76', '77'], (640451, '744859'): ['55', '56', '57', '58'], (640451, '744858'): ['80', '81', '82', '83'], (640451, '746964'): ['57', '58', '59'], (640451, '746796'): ['65', '66', '67'], (640451, '745581'): ['61'], (640451, '744856'): ['58', '62', '63', '64', '65', '66'], (640451, '744853'): ['56', '57', '58'], (640451, '744851'): ['54', '55', '56', '57'], (640451, '747118'): ['66', '67', '68'], (640451, '747117'): ['61', '66', '67', '68', '69'], (640451, '747114'): ['63', '64', '65', '66'], (640451, '746632'): ['60', '61', '62'], (640451, '744844'): ['53', '54', '55', '56', '57', '58', '59'], (640451, '744845'): ['55', '56', '57', '58', '59'], (640451, '746464'): ['63', '64', '65', '70', '71', '72', '73'], (640451, '746462'): ['71', '72', '73'], (640451, '744846'): ['50', '51', '52', '53', '54', '55'], (640451, '744842'): ['56', '57', '58', '59'], (640451, '746541'): ['66', '67', '68', '69', '70'], (640451, '745397'): ['76', '77', '78', '79', '80', '82'], (640451, '745071'): ['65', '66', '67'], (640451, '744840'): ['74', '75', '76', '77', '78', '79'], (640451, '744838'): ['56', '57', '58', '59', '60'], (640451, '744837'): ['61'], (640451, '744836'): ['57', '63', '64', '65', '66', '67', '68'], (640451, '745799'): ['48', '49', '50'], (640451, '745965'): ['64', '65', '66', '67'], (640451, '746286'): ['55', '56', '57', '58', '59', '60'], (640451, '746279'): ['59', '60', '61', '62', '63'], (640451, '746274'): ['52', '53', '54', '55', '56'], (640451, '746763'): ['59', '60', '61', '62', '63'], (640451, '746440'): ['68', '69', '70', '71', '72', '73'], (640451, '746439'): ['74', '75', '76'], (670167, '746333'): ['48', '49', '50', '51', '52'], (670167, '747058'): ['44', '45', '46'], (670167, '747057'): ['60', '61', '62', '63'], (670167, '746328'): ['48', '49', '50'], (670167, '746330'): ['51', '52', '53', '54', '55'], (670167, '746326'): ['67', '68', '69'], (670167, '745839'): ['76', '77', '78'], (670167, '745841'): ['49', '50', '51', '52', '53', '54'], (670167, '746807'): ['60', '61', '62'], (670167, '746325'): ['41'], (670167, '746324'): ['71', '72', '73', '74', '75'], (670167, '746321'): ['55', '56', '57'], (670167, '746317'): ['55', '56', '57'], (670167, '746482'): ['56', '57', '58', '59', '60'], (670167, '744942'): ['53', '61', '62', '63'], (670167, '746318'): ['47', '48', '49', '50', '51', '52'], (670167, '746314'): ['55', '56', '57', '58'], (670167, '746315'): ['64', '65', '66', '67', '68'], (670167, '746240'): ['69', '70', '71', '72'], (670167, '745257'): ['49', '50', '51', '57', '58', '59'], (670167, '746308'): ['47', '52', '53', '54', '55', '56', '57'], (670167, '746304'): ['54', '55'], (670167, '745086'): ['63', '64', '65', '66'], (670167, '745088'): ['50', '51', '52', '53', '54'], (670167, '745898'): ['56', '57'], (670167, '746303'): ['51', '55', '56', '57'], (670167, '746301'): ['45', '46', '47'], (670167, '746631'): ['72', '73'], (670167, '746300'): ['62', '63', '64', '65'], (670167, '746294'): ['77', '78', '79'], (670167, '746299'): ['51', '52', '53', '54', '55'], (670167, '746135'): ['59', '60', '61'], (670167, '745646'): ['55', '56', '57', '58', '59', '60', '61'], (670167, '745006'): ['45', '46', '47', '48'], (670167, '746298'): ['59', '60', '61', '62'], (670167, '746288'): ['59', '60', '61'], (670167, '746282'): ['40', '41', '42', '43', '44'], (670167, '746283'): ['51', '52', '53', '54'], (670167, '746533'): ['61', '62', '63', '64', '65'], (670167, '745156'): ['61', '62', '63'], (670167, '746932'): ['54'], (670167, '746280'): ['59', '60', '61', '62'], (670167, '746274'): ['48'], (670167, '746270'): ['70', '71', '72'], (670167, '746267'): ['58', '59', '60', '61'], (670167, '746628'): ['61', '62', '68', '69', '70'], (670167, '746594'): ['55', '56', '57'], (670167, '746352'): ['64', '65', '66'], (670167, '746347'): ['62', '63', '64', '65', '66'], (670167, '746257'): ['64', '65', '66'], (670167, '745694'): ['66', '67', '68'], (670167, '745448'): ['49', '50', '51', '52', '53'], (670167, '746260'): ['52', '53', '54'], (670167, '746256'): ['56', '57', '58'], (670167, '744795'): ['54', '55', '56', '57', '58'], (670167, '744796'): ['55', '56', '57'], (670167, '747066'): ['49', '50', '51'], (670167, '747061'): ['64', '65'], (672582, '746327'): ['50'], (672582, '746330'): ['37', '38', '45', '46', '47'], (672582, '746323'): ['59', '60', '61'], (672582, '746322'): ['58'], (672582, '745839'): ['60', '61', '62', '63'], (672582, '746809'): ['52', '53', '54', '55'], (672582, '746324'): ['45', '46', '47', '48', '49'], (672582, '746316'): ['47', '48', '49', '54', '55', '56'], (672582, '746317'): ['35', '36', '37', '38', '46', '47', '48', '49', '50'], (672582, '746484'): ['50', '51', '52'], (672582, '744945'): ['51'], (672582, '746318'): ['62', '63', '64', '65'], (672582, '746314'): ['40', '41', '42', '43', '48', '49', '50'], (672582, '746315'): ['55', '56', '57'], (672582, '746312'): ['54'], (672582, '745256'): ['56', '57', '58'], (672582, '745259'): ['54', '55'], (672582, '746308'): ['62', '63', '64'], (672582, '746309'): ['59', '60', '61'], (672582, '746304'): ['65', '66', '67', '68', '69', '70'], (672582, '745086'): ['57', '58', '59'], (672582, '745900'): ['58', '59', '60', '61'], (672582, '746303'): ['38', '47', '48', '49', '50'], (672582, '746301'): ['53', '54', '55', '56'], (672582, '746630'): ['62', '63', '64'], (672582, '746631'): ['54', '55', '56', '57'], (672582, '746297'): ['51'], (672582, '746294'): ['68', '69', '70', '71'], (672582, '746299'): ['56'], (672582, '746130'): ['53', '57', '58'], (672582, '745645'): ['64', '68', '69', '70'], (672582, '744997'): ['54', '55', '56', '57', '58', '59'], (672582, '746291'): ['54', '55'], (672582, '746289'): ['36', '37', '38', '39', '43', '44', '45'], (672582, '746283'): ['55'], (672582, '746285'): ['67', '68', '69', '73', '74', '75', '76', '77'], (672582, '746530'): ['62', '63', '64', '65'], (672582, '745156'): ['50', '51', '52', '59', '60'], (672582, '746932'): ['29', '35', '36', '37', '38', '39'], (672582, '746281'): ['48', '49', '50'], (672582, '746279'): ['40', '45', '46', '47'], (672582, '746274'): ['56', '61', '62', '63', '64'], (672582, '746765'): ['45', '49', '50', '51', '52'], (672582, '746443'): ['61', '62', '63'], (672582, '746276'): ['48', '49', '53', '54', '55', '56'], (672582, '746275'): ['73', '74', '75', '76', '77', '78'], (672582, '745874'): ['70', '71', '72'], (672582, '746675'): ['67', '68', '69', '70', '71'], (672582, '746266'): ['59', '60', '61', '68', '69', '70', '71', '72', '73'], (672582, '746271'): ['72', '73', '74', '75'], (672582, '746263'): ['57', '58', '59', '60', '61', '62'], (672582, '746260'): ['61', '62', '68', '69', '70'], (672582, '746261'): ['77'], (672582, '746256'): ['43', '48', '49', '50', '51'], (672582, '744802'): ['67', '68', '69'], (672582, '744795'): ['48', '49', '50'], (672582, '744796'): ['49', '50', '51'], (672582, '747061'): ['56', '57', '61', '62', '63'], (663704, '747059'): ['55', '56', '60', '61', '62'], (663704, '746328'): ['67'], (663704, '746330'): ['70', '71', '72', '73', '74'], (663704, '746326'): ['75', '76', '77', '81', '82', '83', '84'], (663704, '745839'): ['82', '83', '84', '85', '86'], (663704, '746813'): ['65', '66', '67', '68'], (663704, '746807'): ['67', '68', '69', '70'], (663704, '746321'): ['63', '64', '65', '69', '70', '71', '72'], (663704, '746317'): ['69', '70', '71', '72'], (663704, '744942'): ['67', '68', '69'], (663704, '746314'): ['63', '64', '65', '66', '67', '68', '72', '73', '74', '75'], (663704, '746315'): ['73', '74', '75', '76', '77', '78', '79'], (663704, '746312'): ['75', '76'], (663704, '746234'): ['69', '70', '71', '72'], (663704, '745257'): ['60', '61', '65', '66', '67', '68'], (663704, '746308'): ['69', '70', '71', '72', '73'], (663704, '746304'): ['74', '75', '76', '77'], (663704, '745086'): ['71', '72', '73', '79', '80', '81', '82'], (663704, '746303'): ['58', '59', '60', '61'], (663704, '746301'): ['57', '58', '59'], (663704, '746631'): ['79', '80', '81'], (663704, '746300'): ['71', '72', '73'], (663704, '746297'): ['70', '71', '72', '73', '74', '75'], (663704, '746299'): ['62', '63', '64', '65'], (663704, '746130'): ['59'], (663704, '745645'): ['55', '60', '61', '62', '63'], (663704, '746298'): ['66', '67', '68'], (663704, '746291'): ['66', '67', '68'], (663704, '746288'): ['65', '66', '67'], (663704, '746290'): ['70', '71', '72'], (663704, '746283'): ['66', '67', '71', '72', '73', '74'], (663704, '745155'): ['66', '67', '68', '69', '70'], (663704, '745156'): ['74', '75', '76', '77'], (663704, '746932'): ['67', '68', '69', '70'], (663704, '746281'): ['54', '55', '56', '57', '58'], (663704, '746279'): ['67', '68', '69', '70', '71', '72', '73', '74'], (663704, '746274'): ['68', '69', '70', '71', '72', '73', '74'], (663704, '746765'): ['79', '80', '81'], (663704, '746443'): ['73', '74', '75', '76'], (663704, '746440'): ['86', '88', '89'], (663704, '746276'): ['76', '77', '78'], (663704, '746275'): ['79', '80'], (663704, '745867'): ['70', '71', '72', '73'], (663704, '746674'): ['56', '60', '61'], (663704, '746676'): ['75', '76', '77'], (663704, '746266'): ['44', '45', '46', '47', '48'], (663704, '746268'): ['56', '57', '58', '59'], (663704, '746628'): ['42', '43', '44', '45'], (663704, '746594'): ['38', '39', '40', '44', '45', '46'], (663704, '746353'): ['67', '68', '69', '70'], (663704, '746264'): ['64', '65', '66'], (663704, '746257'): ['51', '52', '53'], (663704, '745698'): ['57', '58', '59', '60'], (663704, '745448'): ['66', '67', '68', '69'], (663704, '746260'): ['58', '59', '60', '61'], (668674, '746493'): ['67', '68', '69'], (668674, '745029'): ['53', '54', '55'], (668674, '745679'): ['55', '56', '57'], (668674, '745681'): ['72', '73', '74'], (668674, '745671'): ['64', '65', '66'], (668674, '745757'): ['47', '48', '49', '56', '57', '58'], (668674, '745755'): ['54', '55', '56', '57'], (668674, '747045'): ['75', '76', '77'], (668674, '745673'): ['64', '65', '66'], (668674, '745665'): ['57', '58', '62', '63', '64', '65', '66'], (668674, '745669'): ['55', '56', '57', '58', '59', '60', '61'], (668674, '745262'): ['60', '61', '62'], (668674, '746396'): ['52', '53', '54'], (668674, '746308'): ['58', '59', '60', '61'], (668674, '745662'): ['60', '61', '62'], (668674, '745661'): ['66', '67', '68', '75', '76', '77'], (668674, '745658'): ['53', '54', '59', '60'], (668674, '745405'): ['66', '67', '68', '69'], (668674, '745892'): ['65', '66', '67', '68'], (668674, '745891'): ['41', '47', '48', '49', '50'], (668674, '745650'): ['64', '65', '66', '67', '68'], (668674, '745649'): ['58', '59', '60', '64', '65', '66', '67'], (668674, '745648'): ['52', '53', '54'], (668674, '746212'): ['56', '57', '58', '59'], (668674, '747183'): ['68', '69', '70', '71'], (668674, '745647'): ['62', '63', '64', '65'], (668674, '745641'): ['61', '62', '63', '64'], (668674, '745637'): ['60', '61', '62'], (668674, '746934'): ['58', '59', '60', '67', '68', '69'], (668674, '745552'): ['68', '69'], (668674, '745633'): ['65', '66', '67', '68'], (668674, '745635'): ['59', '60', '61'], (668674, '746199'): ['65', '66', '67'], (668674, '746200'): ['71', '72', '73', '74'], (668674, '746764'): ['65', '66', '67'], (668674, '746443'): ['48'], (668674, '746440'): ['62', '63', '64'], (668674, '746272'): ['58', '59', '64', '65', '66'], (668674, '746275'): ['55', '56', '57', '58'], (668674, '746269'): ['68', '75', '76', '77', '78', '79'], (668674, '745869'): ['68', '72', '73', '74'], (668674, '746676'): ['58', '59', '63', '64', '65', '66'], (668674, '746267'): ['65', '66', '67'], (668674, '746268'): ['74', '75', '76', '77'], (668674, '746628'): ['74', '75', '76', '77'], (668674, '746589'): ['63', '64', '65', '66', '67', '68'], (668674, '746352'): ['59', '60', '61', '62'], (668674, '746264'): ['59', '60', '61', '62', '63'], (668674, '746262'): ['65', '66', '67'], (668674, '746259'): ['63', '64', '65'], (668674, '746258'): ['60', '61', '62', '63'], (668674, '745697'): ['62', '63', '64', '65', '70', '71', '72'], (668674, '745448'): ['77', '78', '79', '80'], (668674, '746261'): ['65', '66', '67', '68', '72', '73', '74', '75', '76'], (668674, '744802'): ['75', '76', '77'], (668674, '744795'): ['68', '69', '70'], (668674, '744796'): ['73', '74', '75'], (608717, '746310'): ['59', '60', '61', '62'], (608717, '746307'): ['67', '73', '74', '75'], (608717, '745086'): ['46', '47', '48', '54', '55', '56'], (608717, '745903'): ['48', '49', '50', '51', '55', '56', '57'], (608717, '745898'): ['49', '50', '51', '52', '53', '54', '55'], (608717, '746305'): ['53', '54', '55', '56'], (608717, '746302'): ['42', '43', '44', '45', '49', '50', '51', '52', '53', '54'], (608717, '746297'): ['62', '63', '64'], (608717, '746296'): ['65', '66', '67'], (608717, '746134'): ['60'], (608717, '745650'): ['61', '62', '63'], (608717, '745646'): ['70', '71'], (608717, '745645'): ['71', '72'], (608717, '744997'): ['47', '48', '49', '50'], (608717, '746298'): ['53', '54', '55'], (608717, '746292'): ['54', '55', '56', '57', '58'], (608717, '746282'): ['49', '50', '51', '52', '53', '54'], (608717, '746283'): ['62', '63', '64', '65'], (608717, '746529'): ['60', '61', '62', '63', '64'], (608717, '746936'): ['62', '63', '64', '65', '66'], (608717, '746284'): ['59', '60', '61', '62'], (608717, '746280'): ['68', '69', '70', '71', '72'], (608717, '746277'): ['64', '65', '66', '67'], (608717, '746278'): ['72'], (608717, '746765'): ['53', '54', '59', '60', '61', '62'], (608717, '746443'): ['67', '68', '69'], (608717, '746439'): ['64', '65', '66'], (608717, '746276'): ['57', '58', '59', '60', '67', '68', '70'], (608717, '746275'): ['65', '66', '67'], (608717, '745867'): ['45', '46', '47', '48', '53', '54', '55', '56'], (608717, '746675'): ['75', '76', '77', '78'], (608717, '746266'): ['54', '55', '56', '57', '58'], (608717, '746271'): ['52', '53', '54', '60', '61', '62'], (608717, '746594'): ['70', '71', '72'], (608717, '746589'): ['72', '73', '74', '75', '76', '77'], (608717, '746348'): ['59', '60', '61', '62'], (608717, '746347'): ['67', '68', '69'], (608717, '746264'): ['72', '73', '74'], (608717, '745698'): ['69', '70', '71', '72', '73', '74', '75'], (608717, '745447'): ['72', '73', '74', '75', '76', '77', '78'], (595014, '746153'): ['55', '56', '57'], (595014, '746149'): ['63', '64', '65'], (595014, '745421'): ['55', '56', '57'], (595014, '745343'): ['69', '70', '71', '72'], (595014, '746148'): ['59', '60', '61'], (595014, '746144'): ['66', '67', '68', '69', '70'], (595014, '746710'): ['62', '63'], (595014, '745818'): ['77', '78', '79'], (595014, '745817'): ['60', '61', '62', '63'], (595014, '746138'): ['64', '65', '66', '67', '68'], (595014, '745495'): ['71', '72', '73', '74'], (595014, '745737'): ['61', '62'], (595014, '746137'): ['52', '53', '54', '55'], (595014, '746136'): ['48', '49', '50', '51'], (595014, '746135'): ['41', '42', '43', '44', '45', '46'], (595014, '746538'): ['67', '68', '69', '70'], (595014, '746539'): ['54', '55', '56'], (595014, '746775'): ['58', '62', '63', '64', '65'], (595014, '745319'): ['69', '70'], (595014, '745317'): ['73', '74', '75', '76'], (595014, '746133'): ['64', '65', '66', '67', '68'], (595014, '746127'): ['51', '52', '53'], (595014, '746129'): ['55', '56', '57'], (595014, '745556'): ['48', '49', '50'], (595014, '746450'): ['55', '56', '57', '58'], (595014, '746447'): ['64', '65', '66', '67', '68'], (595014, '746121'): ['86', '87', '88', '89'], (595014, '746124'): ['55', '56', '57', '58'], (595014, '746118'): ['59', '60', '61', '62', '63'], (595014, '746117'): ['71', '72'], (595014, '746362'): ['76', '80'], (595014, '745389'): ['70', '71', '72', '73', '74'], (595014, '745636'): ['66', '67', '68'], (595014, '745627'): ['49', '50', '51', '52'], (595014, '746110'): ['65', '66', '67', '68', '69', '70'], (595014, '746108'): ['55', '56', '57', '58', '59'], (595014, '746106'): ['54', '55', '56', '57'], (595014, '746102'): ['60', '66', '67', '68'], (595014, '747156'): ['72', '73', '74', '75'], (595014, '746185'): ['58', '59', '60'], (595014, '746099'): ['42', '43', '44', '45', '46'], (595014, '746097'): ['39', '40', '41', '42'], (595014, '746100'): ['56', '57', '58'], (595014, '747071'): ['56', '57', '58'], (595014, '747067'): ['44', '45', '46'], (595014, '746014'): ['56', '57', '58', '59'], (595014, '746094'): ['60', '61', '62'], (595014, '746092'): ['74', '75', '76'], (595014, '746095'): ['63', '64', '65'], (595014, '746088'): ['61', '62', '63', '64'], (623465, '745444'): ['77', '78', '79'], (623465, '746167'): ['66', '67', '68'], (623465, '746168'): ['83'], (623465, '746169'): ['68', '72', '73', '74', '75'], (623465, '746896'): ['72', '73', '74', '75', '76', '77'], (623465, '745925'): ['68', '69', '70'], (623465, '746162'): ['70', '71', '72', '73'], (623465, '746161'): ['68', '69', '70', '71', '72'], (623465, '746156'): ['72', '73', '74', '75', '76'], (623465, '744863'): ['75', '76', '77', '78', '79', '80'], (623465, '744867'): ['68', '69', '70'], (623465, '744946'): ['71', '72', '73', '74', '75'], (623465, '747211'): ['81', '82', '83'], (623465, '746152'): ['63', '64', '65'], (623465, '746138'): ['72', '73', '74'], (623465, '745495'): ['79', '80', '81', '82', '83', '84', '85'], (623465, '746139'): ['78', '79', '80'], (623465, '746136'): ['64', '65', '66'], (623465, '746130'): ['60', '61', '62', '63'], (623465, '746542'): ['84'], (623465, '746539'): ['76', '77', '78'], (623465, '746131'): ['71', '72', '73'], (623465, '746780'): ['63', '64', '65', '66'], (623465, '746775'): ['76', '77', '78'], (623465, '745317'): ['82', '83', '84', '85'], (623465, '745320'): ['69', '70', '71', '72', '73'], (623465, '746133'): ['73', '74', '75', '76'], (623465, '746127'): ['72', '73', '74'], (623465, '746129'): ['63', '64', '65', '66'], (623465, '746450'): ['47', '48', '49', '50'], (623465, '746447'): ['84', '85', '86', '87', '88', '89'], (623465, '746121'): ['78', '79', '80', '81'], (623465, '746125'): ['71', '72', '73', '74', '75', '76'], (623465, '746118'): ['76', '77'], (623465, '746117'): ['61', '62', '63'], (623465, '746362'): ['53', '54', '55', '56'], (623465, '745631'): ['48', '49', '50', '51'], (623465, '745627'): ['44'], (623465, '746113'): ['66', '67', '68'], (623465, '746112'): ['45'], (623465, '746114'): ['80', '81', '82'], (623465, '745951'): ['62', '63', '64'], (623465, '745144'): ['60', '61', '62', '63'], (623465, '745139'): ['52', '53', '54', '55'], (623465, '746109'): ['62', '63', '64'], (623465, '746111'): ['57', '58', '59', '60'], (623465, '746108'): ['72', '73', '74', '75'], (623465, '746103'): ['61', '62', '63'], (623465, '746102'): ['75', '76', '77', '78'], (623465, '747158'): ['79', '80', '81'], (623465, '746185'): ['51', '52', '53'], (623465, '746098'): ['64', '65', '66', '67', '68'], (623465, '746097'): ['55', '56', '58', '59', '60', '61', '62'], (623465, '746100'): ['72', '73', '74'], (623465, '747070'): ['48', '49', '50', '51', '52', '53'], (623465, '747067'): ['38', '39', '40'], (623465, '746011'): ['64', '65', '66', '67', '68'], (623465, '746094'): ['51', '52', '53'], (623465, '746095'): ['56', '57', '58'], (623465, '746088'): ['39', '40', '41', '42'], (623465, '746496'): ['50', '51', '52'], (676272, '746775'): ['5', '6', '7', '8', '9', '10', '11', '17', '18', '19', '20', '21'], (676272, '746133'): ['1', '2', '3', '4', '10', '11', '12', '17', '18', '19', '20', '26', '27', '28', '29', '30', '31', '32', '33', '37', '38', '39'], (676272, '745557'): ['6', '7', '8', '9', '13', '14', '15', '16', '17', '18', '23', '24', '25', '26', '27', '31', '32', '33', '34', '35', '36', '37', '38', '39'], (676272, '745140'): ['6', '7', '8', '9', '10', '15', '16', '17', '22', '23', '24', '25', '26', '27', '31', '32', '33', '34', '39', '40', '41', '42'], (676272, '746111'): ['1', '2', '3', '4', '5', '9', '10', '11', '12', '16', '17', '18', '19', '20', '21', '25', '26', '27', '28', '34', '35', '36', '43', '44', '45'], (676272, '746102'): ['1', '2', '3', '9', '10', '11', '17', '18', '19', '20', '21', '27', '28', '29', '38', '39', '40', '41', '42', '43'], (676272, '746178'): ['4', '5', '6', '7', '8', '9', '10', '11', '16', '17', '18', '19', '23', '24', '25', '29', '30', '31', '36', '37', '38', '39', '40'], (676272, '746100'): ['1', '2', '3', '4', '5', '6', '7', '17', '18', '19', '25', '26', '27', '28', '29', '37', '38', '39', '43', '44', '45', '46'], (676272, '746015'): ['4', '5', '6', '7', '8', '9', '10', '11', '16', '17', '18', '19', '20', '21'], (641703, '746118'): ['46', '47', '48', '53', '54', '55'], (641703, '746117'): ['79', '80', '81'], (641703, '746361'): ['67'], (641703, '746364'): ['73', '74', '75'], (641703, '745386'): ['50', '51', '52', '56', '57', '58', '59', '60', '61', '65', '66', '67', '68', '69'], (641703, '746120'): ['43', '44', '45', '46', '47', '48', '53', '54', '55', '56', '62', '63', '64', '69', '70', '71', '72', '73', '74', '75'], (641703, '745951'): ['46', '47', '51', '52', '53', '60', '61'], (641703, '745140'): ['53', '54', '55', '56', '57', '61', '62', '63', '64', '65'], (641703, '747158'): ['51', '52', '53', '54', '61', '62', '63', '64'], (641703, '747155'): ['82', '83', '84', '85', '86', '87'], (641703, '746099'): ['64', '65', '66', '70', '71', '72', '73'], (641703, '747069'): ['57', '58', '59', '63', '64', '65', '66', '67'], (641703, '747067'): ['63', '64', '65', '70', '71', '72', '73'], (641703, '746014'): ['75', '76', '77', '78', '79'], (641703, '746096'): ['56', '57', '58', '59', '64', '65', '66'], (641703, '746093'): ['59', '60', '61', '62', '67', '68', '69'], (543339, '745925'): ['61', '62', '63'], (543339, '746162'): ['44', '45', '46', '47', '48'], (543339, '746161'): ['61', '62', '63'], (543339, '746157'): ['71', '72', '73', '74'], (543339, '746160'): ['56', '57', '58', '59', '60'], (543339, '744863'): ['67', '68', '69', '70'], (543339, '744867'): ['60', '61', '62', '63', '64'], (543339, '747211'): ['58', '59', '60'], (543339, '747210'): ['63', '64', '65', '66', '67', '68'], (543339, '746152'): ['54', '55', '56', '57'], (543339, '746151'): ['60', '61', '62'], (543339, '745343'): ['63', '64', '65'], (543339, '746148'): ['67', '68', '69'], (543339, '746142'): ['53', '54', '55'], (543339, '746146'): ['69', '70', '71'], (543339, '745818'): ['65', '66', '67', '68', '69', '70'], (543339, '746138'): ['55', '56', '57'], (543339, '746141'): ['70', '71', '72'], (543339, '745495'): ['63', '64', '65', '66', '67'], (543339, '745737'): ['66', '67', '68'], (543339, '746137'): ['60', '61', '62'], (543339, '746134'): ['53', '54', '55'], (543339, '746130'): ['54', '55', '56'], (543339, '746538'): ['58', '59', '60', '61'], (543339, '746539'): ['60', '61', '62'], (543339, '746780'): ['44', '45', '46', '47'], (543339, '746775'): ['70', '71', '72'], (543339, '745319'): ['45', '46', '47', '48', '49'], (543339, '745317'): ['90', '91', '92', '93', '94', '95'], (543339, '746133'): ['44', '45', '46', '47'], (543339, '746127'): ['61', '62', '63'], (543339, '746129'): ['47', '48', '49', '50'], (543339, '745556'): ['63', '64', '65'], (543339, '746450'): ['76', '77', '78', '79', '80'], (543339, '746123'): ['64', '65', '66'], (543339, '746125'): ['77', '78'], (543339, '746124'): ['65', '66', '67', '68'], (543339, '746117'): ['52', '53', '54', '55'], (543339, '746362'): ['69', '70', '71', '72', '73', '74', '75'], (543339, '745631'): ['58', '59', '60'], (543339, '745627'): ['63', '64', '65'], (543339, '746116'): ['64', '65', '66', '67'], (543339, '746113'): ['72', '73', '74', '75', '76'], (543339, '746112'): ['67', '68', '69', '70'], (543339, '745953'): ['69', '70', '71'], (543339, '745950'): ['63', '64', '65', '66', '67', '68'], (543339, '745139'): ['61', '62', '63', '64'], (543339, '746107'): ['73', '74', '75'], (543339, '746102'): ['55', '56', '57', '58', '59'], (543339, '747156'): ['65', '66', '67'], (543339, '746185'): ['45', '46', '47'], (543339, '746104'): ['45', '46', '47'], (543339, '746100'): ['47', '48', '49', '50', '51'], (543339, '747069'): ['50', '51', '52', '53'], (543339, '747067'): ['56', '57', '58', '59'], (543339, '746015'): ['75', '76', '77', '78', '79'], (543339, '746094'): ['31', '32', '33', '34', '35'], (543339, '746096'): ['70', '71', '72', '73', '74'], (543339, '746093'): ['43', '44', '45', '46'], (543339, '746494'): ['81', '82', '83', '84', '85'], (607455, '745817'): ['51', '52', '53', '54', '55'], (607455, '746140'): ['49', '50', '51'], (607455, '745494'): ['34', '35', '36', '44', '45', '46', '47', '48'], (607455, '745737'): ['57', '58', '59', '60'], (607455, '746139'): ['71', '72', '73', '74'], (607455, '746137'): ['66', '67', '68'], (607455, '746135'): ['47', '52', '53', '54', '55', '56', '57', '58', '64', '65', '66'], (607455, '746542'): ['40', '41', '42', '43', '44'], (607455, '746131'): ['64', '65', '66', '70'], (607455, '746775'): ['50', '55', '56', '57'], (607455, '745319'): ['41'], (607455, '745317'): ['65', '66', '67', '68'], (607455, '746128'): ['63', '65'], (607455, '746126'): ['38', '39', '40', '42', '48', '49', '50', '51'], (607455, '746122'): ['73', '74', '75', '76', '78', '79'], (607455, '745556'): ['43', '44'], (607455, '745558'): ['6', '7', '8', '9', '10', '11'], (607455, '746447'): ['54', '55', '56', '57'], (607455, '746449'): ['56', '57'], (607455, '746123'): ['40', '41', '42'], (607455, '746121'): ['52', '53', '54', '55', '56'], (607455, '746125'): ['61', '62', '63', '64'], (607455, '746119'): ['41', '42', '43', '44', '45', '49'], (607455, '746362'): ['57', '61', '62', '63'], (607455, '745389'): ['63', '64', '65'], (607455, '745631'): ['65', '66', '67', '68'], (607455, '745627'): ['69', '70', '71'], (607455, '746116'): ['55', '56', '57', '58', '59'], (607455, '746113'): ['54', '58', '59', '60', '61'], (607455, '746114'): ['62', '63', '64', '65', '66'], (607455, '745951'): ['38', '39', '45'], (607455, '745950'): ['69'], (607455, '745144'): ['52', '53', '54', '55'], (607455, '746107'): ['55', '56'], (607455, '746110'): ['52', '53', '58', '59', '60'], (607455, '746106'): ['65', '66', '67', '68'], (607455, '746103'): ['41', '47', '48', '49', '50'], (607455, '746102'): ['48', '49', '50', '51'], (607455, '747156'): ['80', '81', '82', '83', '84', '85', '86'], (607455, '747155'): ['61', '62', '63', '64'], (607455, '746104'): ['52', '53', '54', '55', '57'], (607455, '746101'): ['47', '48', '49', '50', '51'], (607455, '746088'): ['43', '48', '49', '50'], (607455, '746496'): ['4', '5', '6'], (621111, '746149'): ['1', '2', '3', '4', '5', '6', '14', '15', '16', '17', '18', '24', '25', '26', '27', '33', '34', '35'], (621111, '745419'): ['4', '5', '6', '7', '8', '9', '13', '14', '15', '16', '17', '21', '22', '23', '27', '28', '29', '30'], (621111, '746142'): ['1', '2', '3', '7', '8', '9', '13', '14', '15', '16', '21', '22', '23', '30', '31', '32', '33', '39', '40', '41', '42'], (621111, '746710'): ['5', '6', '7', '8', '9', '15', '16', '17', '18', '22', '23', '24', '29', '30', '31', '35', '36', '37', '38', '43', '44', '45'], (621111, '746140'): ['1', '2', '3', '4', '5', '6', '10', '11', '12', '16', '17', '18', '19', '20', '21', '22', '23', '28', '29', '30', '31', '36', '37', '38', '43', '44', '45'], (621111, '745495'): ['10', '11', '12', '13', '17', '18', '19', '20', '21', '22', '27', '28', '29', '30', '31', '32', '33', '34', '35'], (621111, '746137'): ['1', '2', '3', '4', '10', '11', '12', '13', '18', '19', '20', '26', '27', '28', '29', '30', '34', '35', '36', '37', '38', '39', '40'], (621111, '746542'): ['4', '5', '6', '7', '8', '9', '10', '15', '16', '17', '18', '19', '20', '25', '26', '27', '28', '33', '34', '35', '36'], (621111, '745951'): ['9', '10', '11', '12', '13', '18', '19', '20', '21', '22', '23', '24', '29', '30', '31', '35', '36', '37'], (621111, '746107'): ['1', '2', '3', '4', '5', '6', '7', '12', '13', '14', '15', '20', '21', '22', '23', '24', '25', '31', '32', '33', '34'], (621111, '746103'): ['1', '2', '3', '8', '9', '10', '11', '12', '13', '14', '15', '19', '20', '21', '22', '30', '31', '32', '33', '37', '38', '39', '40'], (621111, '746185'): ['5', '6', '7', '8', '9', '13', '14', '15', '16', '17', '25', '26', '27', '31', '32', '33', '37', '38', '39', '40', '41'], (621111, '746101'): ['1', '2', '3', '4', '5', '6', '7', '12', '13', '14', '18', '19', '20', '27', '28', '29', '30', '31', '35', '36', '37', '45', '46'], (621111, '747071'): ['5', '6', '7', '12', '13', '14', '15', '16', '20', '21', '22', '23', '24', '25', '26', '27', '31', '32', '33', '40', '41', '42', '43', '47', '48', '49'], (621111, '746096'): ['1', '2', '3', '4', '5', '6', '11', '12', '13', '14', '15', '16', '20', '21', '22', '29', '30', '31', '38', '39', '40', '41', '48', '49'], (621111, '746088'): ['1', '2', '3', '4', '5', '10', '11', '12', '13', '17', '18', '19', '23', '24', '25', '30', '31', '32', '33', '34'], (650893, '745119'): ['58', '59', '60', '61'], (650893, '746409'): ['46', '47', '51', '52', '53', '54'], (650893, '745764'): ['51', '52', '53'], (650893, '745765'): ['68', '72', '73', '74', '75', '76', '77'], (650893, '744954'): ['64', '65'], (650893, '744952'): ['59', '60', '61'], (650893, '744948'): ['49', '50', '51', '59', '60', '61', '62'], (650893, '745431'): ['54', '55', '56'], (650893, '746317'): ['42', '44', '45'], (650893, '744946'): ['50', '51', '52', '53'], (650893, '744945'): ['39', '43', '44', '45', '46'], (650893, '744942'): ['58', '60'], (650893, '744868'): ['67', '68', '70', '71', '72', '73', '74'], (650893, '744861'): ['61', '62', '63', '69'], (650893, '745586'): ['49', '50', '51', '52', '58'], (650893, '744940'): ['62', '63', '64'], (650893, '747038'): ['52', '57', '58'], (650893, '744934'): ['47', '48'], (650893, '744935'): ['62', '63', '64', '65', '66'], (650893, '744933'): ['67', '68', '69', '70', '71'], (650893, '746475'): ['56', '57', '58'], (650893, '746788'): ['74', '75', '76', '77'], (650893, '746791'): ['71', '72', '73', '74', '75', '76'], (650893, '744928'): ['91', '92', '98', '99', '100'], (650893, '744931'): ['15', '16', '17', '18', '19'], (650893, '745651'): ['77', '78', '79'], (650893, '744924'): ['41', '47', '48', '49'], (650893, '744925'): ['59', '60', '61'], (650893, '744917'): ['72', '73', '74'], (650893, '744920'): ['58', '59', '60'], (650893, '746622'): ['47', '48', '49', '50', '51'], (650893, '746621'): ['65', '66', '67', '68', '69'], (650893, '746942'): ['50', '51', '52'], (650893, '744922'): ['55', '56', '57'], (650893, '744916'): ['44', '45', '46'], (650893, '744912'): ['48', '49', '50', '51', '57'], (650893, '745237'): ['56'], (650893, '745233'): ['88', '89', '90', '91', '92'], (650893, '747175'): ['38', '39', '40', '41', '42'], (650893, '747176'): ['50', '51'], (650893, '744911'): ['52', '56', '57'], (650893, '744909'): ['56', '61', '62', '63'], (650893, '744904'): ['59', '60', '61'], (650893, '744905'): ['53', '54', '55', '56'], (650893, '744906'): ['44', '45', '50', '51', '52'], (650893, '744902'): ['61', '62', '63', '64', '65', '66'], (650893, '747039'): ['82', '83', '84', '85'], (650893, '745719'): ['50', '56', '57', '58'], (650893, '745718'): ['57', '59', '60', '61', '62'], (650893, '744901'): ['58', '59', '60', '61'], (650893, '744903'): ['44', '45', '46', '47'], (650893, '746192'): ['63'], (650893, '746843'): ['73', '74', '75'], (650893, '746838'): ['62'], (650893, '744893'): ['58', '59', '60', '61'], (650893, '744889'): ['57', '58', '59'], (650893, '746914'): ['55', '56', '57', '58'], (650893, '744891'): ['69', '70', '75', '76', '77', '78', '79'], (650893, '747076'): ['62', '63', '64'], (650893, '747072'): ['62'], (650893, '744886'): ['67', '68', '69'], (650893, '744881'): ['67', '68', '69', '70'], (650893, '744882'): ['60', '61', '62', '63', '64'], (650893, '744883'): ['49', '53', '54'], (650893, '744962'): ['53', '54', '55'], (650893, '745038'): ['68', '69'], (650893, '745036'): ['54', '55', '56', '57'], (650893, '744876'): ['58', '59', '60', '61', '62'], (650893, '744878'): ['54', '55', '56', '57', '58', '59'], (647315, '744861'): ['78', '79', '80', '82', '83'], (647315, '745586'): ['59', '60'], (647315, '745584'): ['57', '58', '65'], (647315, '744943'): ['35', '36', '37', '38', '39'], (647315, '747038'): ['45', '46', '50', '51'], (647315, '744934'): ['40', '44', '45', '46'], (647315, '744937'): ['63', '64', '65', '66', '67'], (647315, '744936'): ['49', '50', '51'], (647315, '746474'): ['49', '50', '51'], (647315, '746469'): ['51', '59', '60', '61', '62', '63'], (647315, '746788'): ['66', '67', '68'], (647315, '746790'): ['56', '57', '58', '59'], (647315, '744932'): ['59', '60', '61'], (647315, '744931'): ['50', '51', '52', '53', '54'], (647315, '744923'): ['58', '59', '60', '61', '62'], (647315, '745651'): ['36', '37', '38'], (647315, '745982'): ['43'], (647315, '745978'): ['45', '46', '47', '48', '49'], (647315, '744924'): ['61', '62'], (647315, '744925'): ['23'], (647315, '744920'): ['70', '71'], (647315, '746621'): ['32', '33', '34', '38', '39'], (647315, '746941'): ['69', '70', '74', '75'], (647315, '746942'): ['44'], (647315, '744919'): ['63', '64', '65', '66'], (647315, '744921'): ['66', '67'], (647315, '744914'): ['60', '61', '62', '63', '64'], (647315, '744915'): ['44', '49', '50', '51', '52'], (647315, '745237'): ['60', '61', '62', '63', '64', '65'], (647315, '745233'): ['68', '69', '70', '72'], (647315, '745318'): ['61'], (647315, '745313'): ['63', '64', '65'], (647315, '747177'): ['59', '60', '61', '62', '63', '64', '65'], (647315, '744906'): ['53', '58', '59'], (647315, '744902'): ['67', '71', '72', '73'], (647315, '747005'): ['67', '68', '69', '70', '71'], (647315, '747003'): ['68', '69', '70', '71', '72'], (647315, '745719'): ['73', '74', '75'], (647315, '745718'): ['44', '48', '49'], (647315, '744899'): ['68', '69', '70'], (647315, '744897'): ['61'], (647315, '746192'): ['60', '61', '62'], (647315, '746841'): ['47', '48', '49'], (647315, '744896'): ['65', '66', '67', '68', '69', '70'], (647315, '744893'): ['66', '67', '68', '74', '75', '76', '77'], (647315, '744892'): ['68', '69', '70'], (647315, '745862'): ['45', '49', '50'], (647315, '745861'): ['45', '46', '51'], (647315, '744891'): ['63', '64', '65'], (647315, '747072'): ['77', '78', '79', '80', '85', '86', '88'], (647315, '744886'): ['60', '61'], (647315, '744883'): ['55'], (647315, '744961'): ['55', '56', '57'], (647315, '745043'): ['60', '61', '62'], (647315, '745036'): ['45', '46', '47'], (647315, '744885'): ['37', '38'], (643338, '745981'): ['57', '58', '59'], (643338, '744926'): ['58', '59', '60'], (643338, '744917'): ['66', '67'], (643338, '746621'): ['73', '74', '75', '76', '77'], (643338, '746943'): ['77', '78', '79'], (643338, '746942'): ['67', '68', '69', '70'], (643338, '744916'): ['65', '66', '67', '68'], (643338, '744913'): ['68', '69', '70', '71'], (643338, '744912'): ['70', '71', '72', '73'], (643338, '745235'): ['72', '76', '77', '78', '79'], (643338, '745233'): ['77', '78', '79', '80', '82'], (643338, '745313'): ['69', '70', '71', '72'], (643338, '747175'): ['71', '72', '73', '74', '75'], (643338, '747176'): ['75', '79', '80', '81', '82'], (643338, '744911'): ['65', '66', '67'], (643338, '744909'): ['64', '65', '71', '72', '73'], (643338, '744905'): ['47', '48', '49'], (643338, '744906'): ['69', '74', '75', '76'], (643338, '747039'): ['67', '74', '75', '76'], (643338, '745719'): ['80', '81', '82'], (643338, '745718'): ['72', '74', '79', '80', '81'], (643338, '744901'): ['65', '66', '67'], (643338, '744899'): ['71', '72', '73', '74', '75'], (643338, '744900'): ['67', '68', '69'], (643338, '746192'): ['67', '68', '69', '70'], (643338, '746843'): ['79', '80'], (643338, '746838'): ['68', '69', '70', '71'], (643338, '744893'): ['81', '82', '83'], (643338, '744889'): ['64', '65', '66', '67'], (643338, '744890'): ['62', '63', '64'], (643338, '746914'): ['63', '64', '65', '66'], (643338, '745861'): ['64', '65', '66', '67', '68', '69'], (643338, '744891'): ['84', '85', '86', '87', '88'], (643338, '747072'): ['67', '68', '69', '70'], (643338, '744886'): ['74', '75', '76'], (643338, '744881'): ['61', '62', '63', '64', '65', '66'], (643338, '744883'): ['62', '63', '64', '65', '66'], (643338, '744962'): ['66', '67', '68', '69'], (643338, '744876'): ['63', '67', '68', '69'], (643338, '744877'): ['68', '69', '70', '71'], (663893, '744949'): ['42', '43', '44', '45', '46', '50', '51', '52', '53'], (663893, '744939'): ['63', '64', '65'], (663893, '744862'): ['49', '50', '56', '57'], (663893, '744932'): ['65', '66', '67', '68', '69'], (663893, '744930'): ['51', '52', '53', '59', '60'], (663893, '745982'): ['47', '48', '49', '51', '55', '56', '57'], (663893, '745978'): ['63', '64', '65', '66'], (663893, '744924'): ['54', '55', '56', '60'], (663893, '744926'): ['64', '65', '66'], (663893, '744917'): ['58', '62', '63', '64', '65'], (663893, '746622'): ['56', '57', '58'], (663893, '746621'): ['40', '41', '42'], (663893, '746943'): ['60', '61', '62', '63', '64'], (663893, '744921'): ['59', '60', '61', '65'], (663893, '744915'): ['57', '58'], (663893, '745235'): ['69', '70', '71'], (663893, '745233'): ['73'], (663893, '745318'): ['55', '56', '57', '58', '59', '60'], (663893, '747175'): ['54', '55', '56'], (663893, '747176'): ['68', '72', '73', '74'], (663893, '744910'): ['46', '47'], (663893, '744904'): ['40', '44', '45'], (663893, '744905'): ['40', '41', '42'], (663893, '744906'): ['60', '66', '67', '68'], (663893, '747039'): ['57', '62', '63', '64', '65', '66'], (663893, '747005'): ['40', '41', '42', '43'], (663893, '745719'): ['59', '60', '66', '67', '68', '69'], (663893, '745718'): ['41', '43'], (663893, '744897'): ['52', '53', '54'], (663893, '746188'): ['61', '62', '63', '64'], (663893, '746843'): ['55', '56', '57', '58'], (663893, '744898'): ['54', '55', '56', '57'], (663893, '744893'): ['41', '42', '43', '44'], (663893, '744894'): ['49', '50', '51', '52'], (663893, '746915'): ['68', '75', '76', '77'], (663893, '746913'): ['51', '55', '56', '57', '58', '59'], (663893, '745862'): ['51', '57', '58', '59'], (663893, '745861'): ['52', '53', '54', '55'], (663893, '744891'): ['55', '56', '57'], (663893, '744887'): ['48', '49', '50', '55'], (663893, '747076'): ['79', '80'], (663893, '747072'): ['50', '51', '52'], (663893, '744886'): ['57', '58', '59'], (663893, '744882'): ['76', '77', '78'], (663893, '744961'): ['71', '72', '73', '74'], (663893, '745038'): ['53', '54', '60', '61'], (663893, '744885'): ['52', '53', '54'], (663893, '744877'): ['51', '52', '53', '54'], (663893, '744878'): ['39', '40', '41', '42'], (657024, '744942'): ['54', '55', '56', '57'], (657024, '744868'): ['55', '56', '57'], (657024, '744861'): ['70', '71', '72', '73', '74'], (657024, '745586'): ['69'], (657024, '744943'): ['63', '64', '65'], (657024, '744941'): ['52', '53', '54', '55', '56', '57'], (657024, '744934'): ['52', '53', '54', '55'], (657024, '744935'): ['74', '75', '76', '77'], (657024, '744936'): ['64', '65', '66'], (657024, '746475'): ['64', '65', '66', '67', '68'], (657024, '746469'): ['68', '69', '70'], (657024, '746788'): ['63', '64', '65'], (657024, '746942'): ['60', '61', '62', '63'], (657024, '744910'): ['54', '55', '56'], (657024, '744904'): ['65', '66', '67', '68'], (657024, '747007'): ['73', '74', '75', '76', '77', '78', '79'], (657024, '747003'): ['77', '78', '79', '80', '81'], (657024, '745714'): ['51', '52', '53'], (657024, '744903'): ['64', '65', '66'], (657024, '744900'): ['58', '59', '60', '61'], (657024, '744895'): ['52', '53', '54', '55'], (657024, '746187'): ['70', '71', '72', '73', '74'], (657024, '746843'): ['63', '64', '65'], (657024, '744898'): ['63', '64', '65'], (657024, '744893'): ['45', '46', '47', '54', '55', '56', '57'], (657024, '744889'): ['49', '50', '51'], (657024, '746916'): ['67', '68', '69'], (657024, '745859'): ['73', '74', '75'], (657024, '744891'): ['66', '67', '68'], (657024, '747074'): ['62', '63', '64', '65'], (657024, '747072'): ['59', '60', '61'], (657024, '744888'): ['64', '65', '66'], (657024, '744882'): ['82', '83', '84'], (657024, '744883'): ['46', '47', '48'], (657024, '744961'): ['89'], (657024, '744962'): ['59', '60', '61'], (657024, '745036'): ['63', '64', '65', '66'], (657024, '744885'): ['67', '68', '69'], (657024, '744877'): ['62', '63', '64'], (657024, '744878'): ['43', '44', '48', '49', '50'], (622253, '747055'): ['57', '58', '59', '63', '64', '65', '66'], (622253, '747053'): ['42', '43', '49', '50', '51'], (622253, '746325'): ['58', '59', '60', '61', '62', '63', '64'], (622253, '746316'): ['74', '75', '76', '77'], (622253, '746246'): ['48', '53', '54', '55'], (622253, '747052'): ['64', '65', '66'], (622253, '747045'): ['72', '73', '74'], (622253, '746794'): ['49', '55', '56', '57', '66', '67', '68'], (622253, '747034'): ['78', '79', '80', '81', '82'], (622253, '747031'): ['66', '67', '68', '69', '70'], (622253, '747029'): ['56', '57', '58', '62', '63'], (622253, '747030'): ['58', '59', '60', '66', '67', '68', '69', '70'], (622253, '744923'): ['48', '49', '50'], (622253, '745085'): ['47', '48', '49'], (622253, '747027'): ['67', '68', '69', '70'], (622253, '747028'): ['48', '49', '50'], (622253, '745733'): ['55', '60', '61', '62'], (622253, '745732'): ['88', '89', '90', '91'], (622253, '746375'): ['58', '59', '60', '61', '62', '63', '64'], (622253, '746373'): ['53', '54', '55', '56', '57', '58'], (622253, '747022'): ['81', '82', '83'], (622253, '747016'): ['51', '52', '53', '54', '59', '60', '61', '62', '63'], (622253, '745642'): ['19', '20', '21', '22', '23', '24', '25', '32', '33', '34', '35', '36'], (622253, '747014'): ['72', '73', '74', '75', '76', '77'], (622253, '747015'): ['69', '70', '71', '72', '73'], (622253, '744962'): ['46', '47', '48'], (622253, '745038'): ['62', '63', '67'], (622253, '744885'): ['59', '60', '61', '62', '63'], (622253, '744880'): ['37', '41', '42', '43', '44', '45'], (621114, '746942'): ['13', '14', '15', '19', '20'], (621114, '745318'): ['66', '67', '68', '74', '75', '76'], (621114, '747175'): ['47', '48', '49'], (621114, '747176'): ['65', '66', '67'], (621114, '744911'): ['58'], (621114, '744910'): ['61', '62', '63'], (621114, '744904'): ['46'], (621114, '744907'): ['49', '50', '54', '55', '56', '57', '58'], (621114, '747005'): ['49', '50', '51'], (621114, '747003'): ['59', '60', '61', '62', '63'], (621114, '745714'): ['58', '59', '60', '61', '62', '63'], (621114, '745718'): ['69', '70', '71'], (621114, '744903'): ['51', '52', '53', '54'], (621114, '744897'): ['42', '46', '47', '48'], (621114, '746187'): ['64', '65', '66'], (621114, '746838'): ['59', '60', '61'], (621114, '744898'): ['44', '45', '46', '47', '48', '49'], (621114, '744894'): ['1', '2', '3', '4', '5', '9'], (621114, '746913'): ['60'], (621114, '745859'): ['82', '83', '84', '85'], (621114, '744891'): ['89', '90'], (621114, '744887'): ['63', '64', '65', '66', '67'], (621114, '747076'): ['55', '56', '57', '58'], (621114, '744888'): ['1', '2', '3', '4', '11', '12', '13'], (621114, '744884'): ['60', '61', '62'], (621114, '744961'): ['44', '52', '53', '54'], (621114, '745043'): ['53', '54', '55'], (621114, '745036'): ['4', '5', '6', '7', '12'], (621114, '744876'): ['52', '53', '54'], (621114, '744879'): ['78', '79', '80', '81'], (621114, '744880'): ['1', '2', '3', '4', '5', '6'], (605280, '746418'): ['78', '79', '80', '81', '82'], (605280, '746410'): ['70', '71', '72', '73'], (605280, '746413'): ['74', '75', '76', '77', '78'], (605280, '747219'): ['72', '73', '74', '75', '76', '82', '83'], (605280, '745766'): ['84', '85', '86'], (605280, '745762'): ['70', '71', '72'], (605280, '746656'): ['73', '74', '75', '76'], (605280, '746648'): ['75', '76', '77', '78'], (605280, '744948'): ['75', '76', '77', '78'], (605280, '745758'): ['71', '72', '73', '74'], (605280, '745760'): ['58', '59', '60', '61'], (605280, '745754'): ['65', '66', '67'], (605280, '746000'): ['71', '72', '73'], (605280, '747046'): ['64', '65'], (605280, '747048'): ['56', '57', '61', '62', '63', '64'], (605280, '745750'): ['63', '67', '68', '69'], (605280, '745094'): ['66', '67', '68', '69', '70', '71'], (605280, '745909'): ['75', '76', '77'], (605280, '745747'): ['69', '70', '71', '72'], (605280, '745745'): ['71', '72'], (605280, '745746'): ['65', '66', '67', '68', '69', '70', '71', '72'], (605280, '745741'): ['66', '70', '71', '72', '73', '74'], (605280, '745411'): ['70', '71', '72'], (605280, '746229'): ['65', '66', '67'], (605280, '746228'): ['69', '70', '71', '72'], (605280, '745332'): ['67', '68', '69', '70', '71'], (605280, '745330'): ['81', '82', '83'], (605280, '745738'): ['80', '81', '82', '83'], (605280, '745737'): ['63', '64', '65'], (605280, '745736'): ['67', '68', '69', '70', '71'], (605280, '746299'): ['66', '67', '68', '69', '70'], (605280, '745733'): ['69', '70', '71', '72', '73'], (605280, '745732'): ['83', '84', '85', '86', '87'], (605280, '745727'): ['61', '62', '63'], (605280, '744921'): ['68', '69', '70', '71', '72'], (605280, '745728'): ['66', '67', '68'], (605280, '745725'): ['65', '66', '67', '68', '69'], (605280, '745069'): ['68', '72', '73', '74'], (605280, '747012'): ['72', '73', '74', '75'], (605280, '747009'): ['74', '75', '76', '77', '78', '79', '80'], (605280, '745715'): ['72', '73', '74'], (605280, '746933'): ['79', '80', '81', '82'], (605280, '746931'): ['87', '88', '89', '96', '97', '98', '99'], (605280, '745546'): ['74', '75', '76', '77', '78', '79', '80'], (605280, '745543'): ['74', '75', '76'], (605280, '745714'): ['78'], (605280, '745718'): ['67', '68'], (605280, '745709'): ['77', '78', '79'], (605280, '745711'): ['73', '77', '78', '79', '80', '81', '82'], (605280, '746438'): ['67', '68', '69'], (605280, '746431'): ['65', '66', '67', '68', '69'], (605280, '745707'): ['75', '76', '77', '78'], (605280, '745702'): ['62', '63', '64', '65'], (605280, '744813'): ['74', '75', '76'], (605280, '745700'): ['64', '65', '66'], (605280, '744972'): ['74', '75', '76', '77'], (605280, '744968'): ['73', '74', '75', '76', '77'], (605280, '746827'): ['59', '60', '61'], (605280, '745697'): ['50', '51', '52', '53', '54'], (605280, '745695'): ['73', '74'], (605280, '745693'): ['65', '66', '67'], (605280, '745199'): ['54', '55', '60', '61'], (605280, '745204'): ['69', '70', '71', '72'], (605280, '745608'): ['68', '69', '70'], (605280, '745691'): ['59', '60', '61'], (605280, '745692'): ['52', '53', '54'], (605280, '745686'): ['71', '72', '73'], (643410, '745271'): ['53', '54'], (643410, '747214'): ['71', '72', '73', '78'], (643410, '746893'): ['60', '61', '62', '63'], (643410, '746888'): ['61', '62', '63', '64'], (643410, '746889'): ['62', '63', '64', '65', '66'], (643410, '746967'): ['72', '73', '74'], (643410, '745832'): ['55', '56'], (643410, '745829'): ['56', '57', '58'], (643410, '746884'): ['55', '56', '57', '58', '63', '64', '65'], (643410, '745499'): ['59', '60', '64', '65', '66'], (643410, '746878'): ['59', '60', '61', '62', '63'], (643410, '746882'): ['49', '50', '51'], (643410, '746875'): ['61', '62', '63'], (643410, '745176'): ['65', '66', '67', '68', '69', '70'], (643410, '745986'): ['54', '55', '56', '57'], (643410, '745987'): ['57'], (643410, '745985'): ['53', '54', '55', '56'], (643410, '746870'): ['67', '68', '69'], (643410, '746873'): ['71'], (643410, '746703'): ['67', '68'], (643410, '746705'): ['60', '64', '65', '66', '67'], (643410, '745077'): ['55', '56', '57'], (643410, '745075'): ['50', '51', '52', '53', '54', '55'], (643410, '746868'): ['60', '61', '62', '70', '71', '72', '73'], (643410, '746864'): ['63', '64', '65', '66', '67'], (643410, '747014'): ['69', '70', '71'], (643410, '745153'): ['60', '61', '62'], (643410, '745175'): ['53', '54', '55', '59'], (643410, '746853'): ['49', '50', '51'], (643410, '746850'): ['53', '54', '55'], (643410, '746851'): ['61', '62', '63'], (643410, '746274'): ['60', '65', '66', '67'], (643410, '745546'): ['84', '85', '87', '88'], (643410, '745543'): ['64', '65', '66', '67', '68', '69', '70'], (643410, '745714'): ['73', '74', '75', '76', '77'], (643410, '745718'): ['78', '82', '83', '84', '85'], (643410, '745713'): ['73', '74', '75', '76'], (643410, '745708'): ['66', '67', '68', '69', '70'], (643410, '745711'): ['67', '68', '69', '70', '71', '72'], (643410, '746757'): ['60', '61', '62', '63', '64'], (643410, '746434'): ['52', '53', '54', '55'], (643410, '746431'): ['73', '74'], (643410, '745707'): ['51', '52', '53', '54'], (643410, '745703'): ['34', '35', '36', '37', '42', '43'], (643410, '744813'): ['60', '66', '67'], (643410, '745699'): ['43', '44', '45', '46', '47', '52', '53', '54'], (643410, '744964'): ['55', '56', '57'], (643410, '745694'): ['41', '42', '43', '48', '49', '50', '51'], (643410, '745696'): ['43', '49', '50', '51', '52', '53', '54'], (643410, '745610'): ['62', '63', '64', '70', '71', '72'], (643410, '745688'): ['52', '53'], (643410, '745690'): ['76', '77', '78', '79'], (643410, '745686'): ['38', '39', '40', '41', '42', '43'], (641835, '745766'): ['57', '58', '59'], (641835, '745765'): ['55', '56', '57', '58'], (641835, '744950'): ['69', '70', '71', '72'], (641835, '744953'): ['54', '55', '56'], (641835, '744951'): ['51', '52'], (641835, '744948'): ['70', '71', '72', '73', '74'], (641835, '745430'): ['55', '56', '57'], (641835, '746321'): ['66', '67', '68'], (641835, '744949'): ['60', '61', '62', '63'], (641835, '744947'): ['62'], (641835, '744945'): ['62', '63', '64'], (641835, '744868'): ['58', '59', '60'], (641835, '744862'): ['62'], (641835, '745586'): ['64', '65', '66', '67', '68'], (641835, '745584'): ['66', '67', '68'], (641835, '747038'): ['70'], (641835, '744938'): ['73', '74', '75', '76'], (641835, '744936'): ['58', '59', '60'], (641835, '746474'): ['68', '69', '70', '71'], (641835, '746791'): ['81', '82', '83'], (641835, '746790'): ['47', '48', '49', '50'], (641835, '744928'): ['83', '84', '85', '90'], (641835, '744931'): ['68', '69', '70'], (641835, '744927'): ['52', '53', '54'], (641835, '745978'): ['50', '51', '52', '53'], (641835, '744925'): ['20', '21', '22'], (641835, '744918'): ['53', '54', '55', '56', '58', '59'], (641835, '744920'): ['53', '54'], (641835, '746624'): ['47', '48', '49', '53', '54', '55'], (641835, '746941'): ['66', '67'], (641835, '744922'): ['72', '73', '74', '75', '76'], (641835, '746434'): ['46', '47', '48'], (641835, '745707'): ['98', '99', '100'], (641835, '745704'): ['56', '57', '58', '59', '60', '65', '66', '67'], (641835, '745703'): ['44', '45', '46', '47'], (641835, '745706'): ['74', '75', '76', '77'], (641835, '744811'): ['46', '47', '48', '49', '50'], (641835, '745699'): ['58', '59', '60'], (641835, '745701'): ['40', '49', '50'], (641835, '744964'): ['51', '52', '53', '54'], (641835, '745694'): ['56', '57', '58', '59', '63', '64', '65'], (641835, '745689'): ['39', '44', '45', '46', '47', '53', '54', '55', '60', '61', '62', '63', '64'], (641835, '745205'): ['56', '57', '58', '59'], (641835, '745610'): ['77', '78', '79'], (641835, '745688'): ['41', '42', '43', '44', '49', '50', '51'], (641835, '745686'): ['54', '55', '56'], (592454, '745739'): ['47', '48', '49', '50'], (592454, '745738'): ['71', '72', '73', '74', '75'], (592454, '745735'): ['59', '60', '61', '62', '63'], (592454, '746294'): ['65', '66', '67'], (592454, '746947'): ['52', '53', '54', '55'], (592454, '746945'): ['46', '47', '48'], (592454, '745732'): ['57', '58'], (592454, '745734'): ['37', '38', '39', '40'], (592454, '745727'): ['54', '55', '56', '57'], (592454, '745806'): ['76', '77', '78', '79'], (592454, '744922'): ['69', '70', '71'], (592454, '745730'): ['56', '57', '58', '59'], (592454, '745728'): ['58', '59', '60'], (592454, '745725'): ['73', '74', '75', '76', '78'], (592454, '745074'): ['68', '69'], (592454, '745069'): ['64', '65', '66', '67'], (592454, '747012'): ['54', '55', '56', '57'], (592454, '747009'): ['42', '43', '44', '45'], (592454, '745721'): ['57', '58', '59'], (592454, '745717'): ['59', '60', '61', '62'], (592454, '746933'): ['48', '54', '55', '56'], (592454, '746931'): ['73', '79', '80', '81'], (592454, '745546'): ['60', '61', '62', '63'], (592454, '745543'): ['58', '59', '60'], (592454, '745714'): ['64', '65', '66'], (592454, '745718'): ['63', '64', '65', '66'], (592454, '745710'): ['73', '74', '75'], (592454, '745708'): ['77', '78', '79'], (592454, '745711'): ['54', '55', '56', '57', '58', '59'], (592454, '746757'): ['65', '71', '72'], (592454, '746438'): ['61', '62', '63'], (592454, '746431'): ['58', '59', '60'], (592454, '745707'): ['67', '68', '69', '70'], (592454, '745703'): ['76', '77', '78', '79'], (592454, '745706'): ['48', '49', '50', '54', '55'], (592454, '745700'): ['58'], (592454, '745701'): ['62', '63', '64', '65'], (592454, '744968'): ['50', '51', '52', '53', '61'], (592454, '746832'): ['59', '60', '61'], (592454, '745697'): ['59', '60', '61'], (592454, '745695'): ['64', '65', '66', '67', '68', '72'], (592454, '745693'): ['74'], (592454, '745199'): ['51', '52', '53'], (592454, '745688'): ['68', '69', '70'], (592454, '745691'): ['51', '57', '58'], (592454, '745692'): ['58', '59', '60', '61'], (657612, '746812'): ['68', '69', '70', '71', '72', '73', '74'], (657612, '746810'): ['80', '81', '82', '83'], (657612, '745592'): ['59', '60', '61'], (657612, '745590'): ['70', '71', '72'], (657612, '745916'): ['36', '42', '43', '44'], (657612, '745919'): ['54', '55', '60'], (657612, '746804'): ['50'], (657612, '746800'): ['54', '55', '56', '57'], (657612, '745183'): ['43', '44', '51', '52', '53', '57'], (657612, '745098'): ['51', '52', '53', '57', '58', '59', '60'], (657612, '746797'): ['38', '42', '43', '44'], (657612, '746798'): ['43', '45', '46', '47'], (657612, '746796'): ['37'], (657612, '745747'): ['66', '67', '68'], (657612, '745745'): ['43', '44', '45', '46'], (657612, '744935'): ['67', '68', '69', '70', '71', '72', '73'], (657612, '746792'): ['69', '70', '71', '72', '73'], (657612, '746794'): ['69', '70', '71', '72', '73'], (657612, '746791'): ['62', '63', '64', '65', '66', '67', '68', '69', '70'], (657612, '745983'): ['55', '56', '57'], (657612, '745984'): ['61', '62', '63'], (657612, '746784'): ['40', '41', '42', '43', '47', '48', '49', '50', '51', '52'], (657612, '745244'): ['67', '68', '69'], (657612, '745734'): ['44', '45', '46', '47', '48', '49', '50'], (657612, '745806'): ['65', '66', '67'], (657612, '745808'): ['67', '68', '73', '74', '75', '76', '77', '78'], (657612, '744922'): ['83', '84', '85', '86'], (657612, '744916'): ['64', '69', '70', '71'], (657612, '745726'): ['45', '46', '47', '52', '53', '54', '55'], (657612, '745724'): ['42', '43', '54', '55', '56', '60', '61', '62', '63'], (657612, '745069'): ['42', '43', '48', '49', '50'], (657612, '747013'): ['58', '59', '60', '61', '65', '66', '67'], (657612, '745722'): ['45', '46', '50'], (657612, '745716'): ['51', '57', '58', '59', '64', '65'], (657612, '746933'): ['57', '58', '67'], (657612, '746931'): ['57', '58', '59', '66'], (657612, '745719'): ['53', '54', '55', '61'], (657612, '745713'): ['61', '62', '66', '67', '68', '69'], (657612, '745710'): ['60', '61', '62', '63', '64', '65'], (657612, '745712'): ['77', '78', '79'], (657612, '746762'): ['48', '49', '50', '51', '52', '53', '58', '59', '60', '65'], (657612, '746759'): ['48', '49', '50'], (657612, '746434'): ['60', '61', '62', '63', '64'], (657612, '745707'): ['34', '35', '36', '42', '43', '44', '45', '46', '47'], (657612, '745705'): ['53', '54', '55', '56', '60'], (657612, '745703'): ['63', '64', '68', '69', '70', '71'], (657612, '744811'): ['55', '56', '57'], (657612, '745699'): ['36', '37', '38'], (657612, '744972'): ['65', '66', '67', '72', '73'], (657612, '744968'): ['66', '67'], (657612, '745697'): ['40', '41', '42', '43'], (657612, '745695'): ['53', '58', '59', '60'], (657612, '745689'): ['70', '71', '72', '73'], (657612, '745693'): ['55', '56', '64'], (657612, '745204'): ['56', '61', '62', '64', '65'], (657612, '745608'): ['61', '62', '63'], (657612, '745691'): ['67', '68', '69', '70', '71', '72'], (657612, '745685'): ['59', '60', '61'], (596133, '745092'): ['60', '61', '62', '71', '72', '73'], (596133, '745908'): ['57', '58', '59', '60', '65', '66', '67', '68'], (596133, '745746'): ['58', '59'], (596133, '745743'): ['60', '61', '62', '68', '69', '70'], (596133, '745411'): ['53', '54', '55', '56', '60', '61', '62', '63', '64'], (596133, '746229'): ['55', '56', '57', '61', '62', '63', '64'], (596133, '745331'): ['60', '61', '65', '66', '67', '68'], (596133, '745739'): ['62', '63', '64'], (596133, '745738'): ['58', '59', '65', '66', '67'], (596133, '745736'): ['54', '55', '59', '60', '61', '62'], (596133, '746294'): ['72', '73', '74', '75', '76'], (596133, '746299'): ['57', '58', '59', '60', '61'], (596133, '746945'): ['58', '59', '60', '61', '62'], (596133, '745733'): ['63', '64', '65'], (596133, '745731'): ['62', '63', '64', '65'], (596133, '744922'): ['58', '59', '60', '61', '62'], (596133, '745730'): ['64', '65', '66', '70', '71', '72'], (596133, '745725'): ['49', '50', '51', '56', '57', '58', '59', '60'], (596133, '745723'): ['43', '48', '49', '50', '51', '52'], (596133, '745069'): ['51', '55', '56', '57', '58', '59'], (596133, '747012'): ['63', '64', '65'], (596133, '747009'): ['59', '60', '64', '65'], (596133, '745722'): ['61', '62', '63', '67', '68', '69'], (596133, '745715'): ['62', '63', '64', '65', '66', '67'], (596133, '746933'): ['68', '69', '70', '71', '76', '77', '78'], (596133, '745546'): ['68', '69', '70'], (596133, '745543'): ['49', '50', '51', '56', '57'], (596133, '745714'): ['54', '55', '56', '57'], (596133, '745718'): ['47', '53', '54', '55', '56'], (596133, '745709'): ['70', '71', '72'], (596133, '745708'): ['44', '45', '46', '47', '48', '49'], (596133, '746438'): ['52', '53', '54', '55', '56'], (596133, '746431'): ['52', '53', '54'], (596133, '745707'): ['59', '60', '61', '62'], (596133, '745705'): ['61', '62', '63'], (596133, '745702'): ['48', '49', '50', '51'], (596133, '745706'): ['64', '65', '66', '67'], (596133, '744809'): ['62', '63', '64', '65'], (596133, '745699'): ['69', '70', '71'], (596133, '744972'): ['55', '56', '57', '58', '59'], (596133, '746832'): ['67', '68', '69', '70'], (596133, '745698'): ['64', '65', '66', '67', '68'], (596133, '745697'): ['75', '76', '80', '81', '82'], (596133, '745696'): ['64', '65', '66', '67', '71', '72', '73'], (596133, '745199'): ['62', '63', '67', '68', '69', '70'], (596133, '745606'): ['68', '69', '70', '71'], (596133, '745608'): ['71', '72', '80', '81', '82'], (596133, '745686'): ['60', '61', '62', '63'], (664776, '744919'): ['61', '62', '67', '68', '69', '70'], (664776, '744921'): ['44', '45'], (664776, '745730'): ['45', '46', '47', '48'], (664776, '745726'): ['56', '57', '58', '64', '65', '66'], (664776, '745074'): ['41', '42', '43', '44', '49', '50', '51'], (664776, '745065'): ['58', '59', '60', '61'], (664776, '747013'): ['72', '73', '74'], (664776, '747009'): ['66', '67'], (664776, '745721'): ['64', '65', '66'], (664776, '745722'): ['51', '52', '53', '54'], (664776, '745717'): ['70', '71', '72'], (664776, '746931'): ['48', '49', '50', '51', '56'], (664776, '746929'): ['58', '59', '60', '69', '70'], (664776, '745546'): ['52', '53'], (664776, '745719'): ['76', '77', '78', '79'], (664776, '745714'): ['48', '49', '50'], (664776, '745709'): ['62', '63', '64', '65', '66'], (664776, '745710'): ['53', '54', '55', '56'], (664776, '745711'): ['44', '45', '46', '47', '53'], (664776, '746757'): ['73', '74', '75', '76'], (664776, '746434'): ['34', '35', '39', '40', '41'], (664776, '745707'): ['82', '83', '85', '90', '91', '92', '93'], (664776, '745702'): ['56', '57', '58'], (664776, '745706'): ['56', '57'], (664776, '744813'): ['68', '69', '70'], (664776, '744809'): ['52', '56', '57', '58'], (664776, '745700'): ['55', '56', '57'], (664776, '745701'): ['51', '59', '60', '61'], (664776, '744968'): ['62', '63', '64', '65'], (664776, '746832'): ['52', '53', '54', '55'], (664776, '746827'): ['52', '53', '54', '55'], (664776, '745698'): ['53', '54', '55'], (664776, '745697'): ['66', '67', '68', '69', '73', '74'], (664776, '745693'): ['71', '72', '73'], (664776, '745204'): ['54', '55'], (677161, '746968'): ['60', '61', '62', '63'], (677161, '746969'): ['42', '43', '44', '49', '50', '51'], (677161, '745913'): ['44', '45', '46', '47', '48', '53', '54', '55'], (677161, '746965'): ['51', '52', '53', '54', '55'], (677161, '746962'): ['68', '69', '70', '73', '74', '75', '76', '77'], (677161, '746959'): ['50', '51', '52', '53', '54', '55'], (677161, '745177'): ['69', '70'], (677161, '745089'): ['69', '70', '71', '72', '73', '77', '78', '79', '80'], (677161, '746958'): ['46', '47', '48', '49', '53', '54', '55', '62', '63', '64', '65'], (677161, '746950'): ['60', '61', '62'], (677161, '746786'): ['46'], (677161, '746787'): ['4', '5', '6', '10', '11', '12', '13', '14'], (677161, '746948'): ['50', '51', '52', '58'], (677161, '746947'): ['49', '50', '51', '57', '58', '59'], (677161, '746945'): ['52', '53', '54'], (677161, '744917'): ['53', '59', '60', '61'], (677161, '746700'): ['57'], (677161, '746699'): ['6', '7', '8', '12', '13', '14', '15', '20', '21'], (677161, '746940'): ['64', '65', '66', '67', '68', '69'], (677161, '746047'): ['56', '57', '58', '59', '60', '61'], (677161, '746043'): ['73', '75', '76', '84', '85', '86', '87'], (677161, '746937'): ['74', '75', '76'], (677161, '746930'): ['59', '60', '65', '66', '67', '68', '69'], (677161, '746123'): ['50', '51', '52', '53', '57', '58'], (677161, '746526'): ['77', '78', '79', '84', '85', '86', '87'], (677161, '746933'): ['62', '63', '64', '65', '66'], (677161, '746931'): ['77', '78'], (677161, '746926'): ['41', '42', '43', '44'], (677161, '746925'): ['79', '80', '81', '82', '83'], (677161, '744979'): ['56', '57', '58', '59'], (677161, '746273'): ['55', '56', '57'], (677161, '746927'): ['56', '57', '58', '59'], (677161, '746922'): ['56', '57', '58', '59'], (677161, '746921'): ['56', '57', '58'], (677161, '746919'): ['81', '82', '83', '84', '85', '86'], (677161, '747001'): ['11', '12'], (677161, '746349'): ['61', '62', '63'], (677161, '746350'): ['40'], (677161, '746917'): ['41', '42', '43'], (677161, '746915'): ['1', '2', '3', '9', '10', '11', '15', '16', '17', '18', '19'], (677161, '746426'): ['38', '43', '44', '45'], (677161, '745776'): ['47', '48', '49'], (677161, '746912'): ['50', '51', '52', '53'], (677161, '746910'): ['61', '62', '63', '64', '66', '67'], (677161, '746907'): ['70', '71', '72', '73', '74'], (677161, '745695'): ['50'], (677161, '745696'): ['55', '56', '57'], (657514, '746974'): ['38', '43', '44', '45'], (657514, '746976'): ['57', '58', '59', '60', '61'], (657514, '746970'): ['1', '2', '3', '11', '12', '13'], (657514, '746647'): ['50', '51', '52', '53'], (657514, '746646'): ['23', '30', '31', '32', '33'], (657514, '746971'): ['28', '29', '30', '31', '32'], (657514, '746968'): ['45', '46', '47'], (657514, '746969'): ['36', '37', '41'], (657514, '745911'): ['7', '8', '9', '10', '11'], (657514, '745913'): ['38', '39'], (657514, '747126'): ['59', '60', '61'], (657514, '746964'): ['60', '61', '62'], (657514, '746963'): ['40', '41', '42', '46'], (657514, '746962'): ['83', '84', '85', '86'], (657514, '746959'): ['33', '39', '40', '41', '42'], (657514, '745177'): ['47', '51', '52', '53', '54', '55'], (657514, '746956'): ['41', '42'], (657514, '747033'): ['52', '53', '54', '55', '59', '60', '61'], (657514, '746952'): ['40', '41', '48', '49', '50', '51'], (657514, '746787'): ['46', '47', '48', '49', '51', '56', '57', '59'], (657514, '746948'): ['59', '60', '61', '62', '63'], (657514, '746946'): ['61'], (657514, '746945'): ['49', '50', '51'], (657514, '744917'): ['68', '69', '70'], (657514, '746700'): ['39', '40'], (657514, '746699'): ['22', '31', '32'], (657514, '746938'): ['42', '43', '44'], (657514, '746045'): ['63'], (657514, '746043'): ['58', '59', '60'], (657514, '745724'): ['39', '44', '45', '46'], (657514, '746932'): ['57', '58'], (657514, '746123'): ['59', '61'], (657514, '746524'): ['66', '67', '68', '69'], (657514, '746525'): ['69', '70', '71', '72', '73', '74', '75'], (657514, '746933'): ['59', '60', '61'], (657514, '746931'): ['46', '47', '52', '53', '54', '55'], (657514, '744987'): ['52', '53', '54', '55', '57'], (657514, '744979'): ['65', '66', '67'], (657514, '746273'): ['46'], (657514, '746272'): ['39', '40', '41', '45'], (657514, '746923'): ['19', '20', '25', '26', '27', '28', '32', '33', '34', '35'], (657514, '746919'): ['54'], (657514, '747001'): ['8', '9', '10'], (657514, '746997'): ['53', '54', '55'], (657514, '746999'): ['63', '64', '65', '66'], (657514, '746350'): ['59'], (657514, '746917'): ['50', '51', '52', '53'], (657514, '746918'): ['53', '54', '55'], (657514, '746916'): ['49', '50', '51', '52', '57', '58', '59', '60'], (657514, '746433'): ['53', '54', '55', '59', '60', '61', '62'], (657514, '746426'): ['59', '60'], (657514, '746912'): ['60', '61', '62'], (657514, '746911'): ['61', '62'], (657514, '746909'): ['49', '50', '51'], (657514, '746907'): ['52', '53', '55'], (445276, '746967'): ['69', '70', '71'], (445276, '745913'): ['76', '77', '78', '79', '80'], (445276, '746964'): ['66', '67', '68', '69', '70', '71'], (445276, '746963'): ['62', '63', '64'], (445276, '746965'): ['66', '67', '68', '69'], (445276, '746962'): ['61', '62', '63', '64'], (445276, '746959'): ['74', '75', '76', '77', '78', '79'], (445276, '745090'): ['67', '68', '69', '70'], (445276, '746957'): ['66', '67', '68'], (445276, '746953'): ['70', '71', '72', '73'], (445276, '746787'): ['66', '67', '68', '69', '74', '75', '76'], (445276, '746948'): ['79', '80', '81', '82', '83'], (445276, '746946'): ['72', '78', '79', '80'], (445276, '744918'): ['79'], (445276, '744917'): ['75', '76', '77'], (445276, '746700'): ['72', '73', '74', '75'], (445276, '746699'): ['77', '78', '79', '80'], (445276, '746941'): ['71', '72', '73'], (445276, '746938'): ['66', '67', '68'], (445276, '746043'): ['64', '65', '66', '68'], (445276, '745725'): ['79', '80', '81', '82'], (445276, '745723'): ['61', '62', '63'], (445276, '746932'): ['64', '65', '66', '71', '72', '73', '74', '75'], (445276, '746121'): ['72', '73', '74', '76', '77'], (445276, '746125'): ['65', '66', '67', '68', '69', '70'], (445276, '746933'): ['83', '84', '85', '86', '87'], (445276, '746931'): ['82', '83', '84', '85', '86'], (445276, '746925'): ['70', '71', '72', '73'], (445276, '746273'): ['72', '76', '77', '78'], (445276, '746921'): ['75', '76', '77'], (445276, '746924'): ['64', '71', '72', '73'], (445276, '747001'): ['94', '95', '96'], (445276, '746997'): ['72', '73', '74'], (445276, '746351'): ['75', '76'], (445276, '746349'): ['78', '79', '80'], (445276, '746916'): ['70', '71', '72', '73'], (445276, '746913'): ['61', '62', '63'], (445276, '746433'): ['68', '69', '70', '71', '72'], (445276, '745776'): ['62', '63', '64', '65', '66'], (445276, '746911'): ['73', '74', '75'], (445276, '746908'): ['65', '66', '67'], (445276, '745695'): ['69', '70', '71'], (445276, '745040'): ['64', '65', '66', '67', '68'], (445276, '746905'): ['72', '73', '74'], (445276, '746903'): ['71', '72', '73'], (608371, '746810'): ['56', '66', '67'], (608371, '745267'): ['49', '50', '51', '52', '53', '54', '55', '56'], (608371, '746729'): ['55', '56', '57', '58'], (608371, '746727'): ['59', '60', '61', '62'], (608371, '746725'): ['47', '48', '49'], (608371, '745025'): ['54', '55', '59', '60'], (608371, '745022'): ['46', '47'], (608371, '746720'): ['53', '54', '55'], (608371, '746717'): ['59', '60', '61', '62'], (608371, '745340'): ['65', '66'], (608371, '745345'): ['91'], (608371, '747204'): ['75'], (608371, '746150'): ['58', '59', '60'], (608371, '746148'): ['47', '48', '49'], (608371, '746714'): ['64', '65', '66'], (608371, '746713'): ['71', '72', '73', '74', '75', '76'], (608371, '746708'): ['64', '65', '66'], (608371, '746706'): ['53'], (608371, '746877'): ['66', '67', '68', '69'], (608371, '746870'): ['70', '71', '72', '73', '74'], (608371, '746544'): ['62', '63', '64', '65'], (608371, '746703'): ['69', '70', '71'], (608371, '746702'): ['36', '37', '38', '42'], (608371, '746701'): ['55', '56', '57', '58'], (608371, '745977'): ['45', '46', '47', '48', '49', '50'], (608371, '746700'): ['61', '62', '63'], (608371, '746696'): ['66', '67', '68', '69'], (608371, '746695'): ['57', '58', '59'], (608371, '745159'): ['71', '72', '73', '74', '75'], (608371, '745728'): ['61', '62', '63', '64'], (608371, '746694'): ['58', '59', '60'], (608371, '746690'): ['49', '50', '51', '52'], (608371, '746688'): ['70', '71', '72'], (608371, '746687'): ['70', '71', '72', '73', '74'], (608371, '746686'): ['47', '48', '49'], (608371, '744831'): ['55', '60', '61', '62', '63'], (608371, '747091'): ['60', '61', '62'], (608371, '745063'): ['64', '65'], (608371, '746684'): ['55', '56', '57'], (608371, '746925'): ['56', '57', '58'], (608371, '744986'): ['64', '69', '70', '71'], (608371, '746276'): ['66'], (608371, '746927'): ['51', '52', '53', '54', '55'], (608371, '746923'): ['4', '8', '9', '10', '15', '16', '17', '18'], (608371, '746924'): ['48', '49', '54', '55', '56'], (608371, '747001'): ['63', '64', '65'], (608371, '746997'): ['56', '57', '58'], (608371, '746351'): ['66', '67', '68', '69', '70', '71'], (608371, '746917'): ['47', '48', '49'], (608371, '746918'): ['67', '68', '69', '70'], (608371, '746905'): ['62', '63', '64', '65'], (608371, '746903'): ['59', '60', '61'], (608371, '744876'): ['35', '36', '37', '38', '39'], (608371, '746902'): ['61', '62', '63', '64', '65'], (669711, '746249'): ['45'], (669711, '746248'): ['39', '40', '44', '45', '46', '47'], (669711, '746977'): ['59', '60', '65'], (669711, '746979'): ['40', '41', '42', '43', '47', '48', '49'], (669711, '746974'): ['46', '47'], (669711, '746976'): ['82', '83', '85', '86', '87'], (669711, '745515'): ['69'], (669711, '745514'): ['46', '47'], (669711, '746646'): ['56', '57', '58'], (669711, '746968'): ['48', '53', '54', '55'], (669711, '746966'): ['68', '69', '70'], (669711, '745911'): ['15', '16', '17', '21', '22', '23', '24'], (669711, '747126'): ['62'], (669711, '746961'): ['50', '51', '52', '53', '58', '59', '60', '61', '62', '63'], (669711, '746960'): ['47', '48', '49'], (669711, '745179'): ['55', '56', '57', '58', '59', '60', '61'], (669711, '745090'): ['51', '52', '53'], (669711, '745089'): ['63', '64', '65'], (669711, '746958'): ['69', '70', '71'], (669711, '747033'): ['76', '77', '78', '79', '80'], (669711, '746954'): ['38'], (669711, '746955'): ['63', '64', '65', '66', '67'], (669711, '746953'): ['63', '64', '65'], (669711, '746785'): ['42', '46', '47', '48', '49', '50'], (669711, '746787'): ['18', '19', '20', '21'], (669711, '746951'): ['48', '49', '50'], (669711, '746946'): ['67', '68', '69', '70', '71'], (669711, '746945'): ['63', '64', '65', '66', '74', '75', '76', '77'], (669711, '746700'): ['46', '47', '48', '49', '50'], (669711, '746699'): ['33', '34', '35', '36'], (669711, '746943'): ['24', '25'], (669711, '746939'): ['38', '39', '40', '41', '42', '43', '44', '45'], (669711, '746045'): ['79', '80', '81', '82'], (669711, '746043'): ['94', '95', '96', '97', '98'], (669711, '745724'): ['47', '48', '49', '50', '51'], (669711, '746937'): ['57', '58', '59', '65', '66', '67', '68', '69'], (669711, '746936'): ['58', '59', '60', '61', '67', '68', '69'], (669711, '746123'): ['62', '63'], (669711, '746121'): ['82', '83', '84', '85', '90', '91', '93'], (669711, '746526'): ['91', '92', '94'], (669711, '746524'): ['74', '75', '76'], (669711, '746929'): ['53', '54', '55', '61', '62', '63', '64', '65', '66', '67', '68'], (669711, '746349'): ['55', '56', '57'], (669711, '746350'): ['54', '55', '56', '57', '58'], (669711, '746916'): ['33', '37', '38', '39', '40', '41'], (669711, '746914'): ['59', '60', '61', '62'], (669711, '746424'): ['50', '51', '52'], (669711, '745776'): ['70'], (669711, '746910'): ['42', '43', '48', '49', '50'], (669711, '746908'): ['72', '73', '74', '75'], (669711, '745696'): ['61', '62', '63'], (669711, '745689'): ['74', '75', '76'], (669711, '745693'): ['63', '68', '69', '70'], (669711, '745041'): ['55', '56', '57', '58'], (669711, '746905'): ['88', '89', '90', '91'], (669711, '746903'): ['65', '66', '67'], (669711, '744885'): ['48', '49', '50', '51'], (669711, '744877'): ['61'], (669711, '746902'): ['38', '39', '40', '44', '45', '46'], (472610, '746978'): ['58', '59'], (472610, '745106'): ['52', '53', '54'], (472610, '745108'): ['44', '45', '46'], (472610, '745104'): ['46', '50', '51', '52'], (472610, '746726'): ['70', '71', '72', '73'], (472610, '746246'): ['59', '60', '61', '66', '67', '68'], (472610, '746237'): ['50', '51', '52', '53', '54', '55', '56'], (472610, '746239'): ['52', '53', '54'], (472610, '746238'): ['67', '68', '69', '70'], (472610, '745508'): ['69', '70', '71'], (472610, '745507'): ['65', '66'], (472610, '746240'): ['57', '58', '59'], (472610, '746233'): ['53', '54', '55', '56', '57', '58'], (472610, '746231'): ['66', '67', '68', '69', '70', '71'], (472610, '745013'): ['59', '60', '61', '62', '63'], (472610, '745012'): ['63', '64', '68', '69', '70'], (472610, '746385'): ['63', '64', '65'], (472610, '746386'): ['65', '66', '67'], (472610, '746226'): ['63', '64', '65'], (472610, '746229'): ['52', '53', '54'], (472610, '746228'): ['66', '67', '68'], (472610, '745253'): ['55', '56', '57', '58'], (472610, '745252'): ['61', '62', '63', '64', '65'], (472610, '746222'): ['57', '58', '59', '60', '61'], (472610, '746216'): ['69', '70', '71', '72'], (472610, '747192'): ['57', '58', '59', '60', '61', '62'], (472610, '745329'): ['67', '68', '69'], (472610, '746215'): ['50', '51', '52', '53', '54'], (472610, '746219'): ['56', '57', '58', '59', '60', '61'], (472610, '746131'): ['60', '61', '62', '63', '67', '68', '69'], (472610, '746213'): ['57', '58', '59', '60'], (472610, '746942'): ['56', '57', '58', '59'], (472610, '746214'): ['55', '56', '57'], (472610, '746859'): ['56', '57', '58'], (472610, '746206'): ['49', '50', '51', '52', '53', '60', '61', '62'], (472610, '746207'): ['61', '62', '63', '64', '65'], (472610, '746202'): ['58', '59', '60'], (472610, '745633'): ['74', '75', '76'], (472610, '745232'): ['58', '59', '60', '61'], (472610, '745231'): ['61', '62', '63', '64'], (472610, '746195'): ['77', '78', '79'], (472610, '744986'): ['78', '79', '80'], (472610, '746276'): ['72', '73', '74', '75', '79'], (472610, '746273'): ['68', '69', '70', '71'], (472610, '746927'): ['66', '67', '68', '74'], (472610, '746923'): ['36', '37', '38', '39', '40', '44', '45', '46', '48', '49', '50'], (472610, '746919'): ['58', '59', '60', '61', '62', '63'], (472610, '747001'): ['72', '73', '74', '75', '76', '77', '78'], (472610, '746349'): ['68', '69', '70', '71'], (472610, '746350'): ['63', '64', '65'], (472610, '746917'): ['58', '59', '60', '61', '62'], (472610, '746903'): ['53', '54', '55'], (472610, '744876'): ['55', '56', '57'], (472610, '746904'): ['64', '65', '66'], (472610, '746902'): ['52', '53', '54', '55', '56', '57'], (455119, '746980'): ['57', '58', '59', '60', '61', '62', '63'], (455119, '746976'): ['66', '67', '68'], (455119, '746970'): ['70', '71', '72'], (455119, '745515'): ['59', '60', '61'], (455119, '746646'): ['62', '63', '64', '65'], (455119, '746967'): ['56', '60', '61', '62', '63', '64', '65'], (455119, '746966'): ['62', '63', '64'], (455119, '747127'): ['58', '59', '60'], (455119, '746963'): ['53', '54', '55'], (455119, '746965'): ['59', '60', '61'], (455119, '746962'): ['54', '55', '56'], (455119, '746959'): ['67', '68', '69', '70'], (455119, '745179'): ['75', '76', '77'], (455119, '745090'): ['60', '61', '62'], (455119, '746957'): ['50', '51', '52', '53'], (455119, '746954'): ['50', '51', '52', '53', '54', '55', '56'], (455119, '744920'): ['78', '79', '80', '81'], (455119, '746700'): ['64', '65', '66', '67'], (455119, '746699'): ['63', '64', '65'], (455119, '746938'): ['59', '60', '61', '62'], (455119, '746045'): ['67', '68', '69', '70'], (455119, '746272'): ['67', '68', '69'], (455119, '746922'): ['66', '67', '68', '69'], (455119, '746921'): ['65', '66', '67', '68'], (455119, '746919'): ['64'], (455119, '747001'): ['84', '85', '86', '87', '88', '89'], (455119, '746997'): ['64', '65', '66', '67'], (455119, '746351'): ['58', '59', '60', '61', '62'], (455119, '746350'): ['70', '71', '72', '73'], (455119, '746915'): ['69', '70', '71', '72', '73'], (455119, '746916'): ['64', '65', '66'], (455119, '746433'): ['79', '80', '81'], (455119, '745778'): ['59', '60', '61', '62', '63', '64', '65'], (455119, '746908'): ['49', '50', '51', '52'], (455119, '745695'): ['61', '62', '63'], (455119, '745040'): ['58', '59', '60'], (455119, '746905'): ['53', '54', '55', '56', '57'], (455119, '744885'): ['70', '71', '72', '73'], (455119, '744876'): ['70', '71', '72'], (455119, '746901'): ['58', '59', '60', '61', '62'], (641941, '746810'): ['76', '77', '78', '79'], (641941, '745270'): ['37', '38', '39', '40', '41'], (641941, '746727'): ['32', '33', '34', '39', '40', '41'], (641941, '746722'): ['64', '65', '66'], (641941, '746725'): ['60', '61', '62', '63'], (641941, '745022'): ['52', '53', '54', '55', '56'], (641941, '745426'): ['61', '62', '63', '64', '65'], (641941, '746721'): ['46', '47', '48', '49', '50', '51'], (641941, '746719'): ['65', '66', '67'], (641941, '746718'): ['64', '65', '66', '67'], (641941, '745340'): ['49', '50', '51'], (641941, '745345'): ['88', '89', '90'], (641941, '747204'): ['42', '43', '44'], (641941, '746150'): ['10', '15', '16', '17'], (641941, '746144'): ['65', '71', '72', '73'], (641941, '746703'): ['46', '47', '48', '49', '50'], (641941, '745956'): ['64', '65', '66'], (641941, '746677'): ['1', '2', '3', '7', '8', '9'], (641941, '746675'): ['55', '56', '57', '58'], (641941, '744893'): ['73', '78', '79', '80'], (641941, '745459'): ['65', '66', '67', '68'], (641941, '746672'): ['58', '59', '60', '61', '62', '63', '64'], (641941, '746670'): ['76', '77', '78'], (641941, '746730'): ['59', '60', '61'], (641941, '746668'): ['62', '63', '70', '71', '72', '78', '79', '80'], (641941, '745777'): ['52', '53', '54', '55', '56'], (641941, '745774'): ['47'], (641941, '745126'): ['70', '71', '72', '73', '74', '75'], (641941, '745855'): ['63', '64', '65'], (641941, '746659'): ['73', '74', '75', '76'], (641941, '746581'): ['55', '56', '57', '58'], (641941, '746583'): ['53', '54', '55', '56', '57', '58'], (594580, '746720'): ['59', '60', '61'], (594580, '746719'): ['54', '55', '56'], (594580, '746150'): ['69', '70', '71'], (594580, '746144'): ['62', '63', '64'], (594580, '746714'): ['77', '78', '80'], (594580, '746713'): ['55', '56'], (594580, '746710'): ['56', '57', '58'], (594580, '746708'): ['58', '59', '60'], (594580, '746877'): ['59', '60', '61'], (594580, '746544'): ['35', '36'], (594580, '746707'): ['67', '68', '69'], (594580, '746703'): ['54', '55', '56'], (594580, '746702'): ['24', '28', '29', '30', '31'], (594580, '746701'): ['62', '63', '64'], (594580, '745977'): ['56', '57', '58'], (594580, '745489'): ['58'], (594580, '746698'): ['48', '49', '50'], (594580, '746700'): ['51', '52', '53'], (594580, '745162'): ['79', '80', '81', '82'], (594580, '745154'): ['41', '45', '46', '47', '48'], (594580, '745730'): ['61', '62', '63'], (594580, '745726'): ['75', '76', '77'], (594580, '746694'): ['55', '56', '57'], (594580, '746691'): ['35'], (594580, '746688'): ['49', '50', '51', '52'], (594580, '746687'): ['48'], (594580, '746686'): ['50'], (594580, '744830'): ['65'], (594580, '747091'): ['55', '56'], (594580, '746684'): ['41', '42', '43', '44'], (594580, '746679'): ['55', '56', '57'], (594580, '746680'): ['48', '49', '54', '55'], (594580, '746033'): ['78', '79', '80'], (594580, '746030'): ['66', '67', '68'], (594580, '745956'): ['58', '59', '60'], (594580, '746677'): ['13', '14', '15'], (594580, '746675'): ['33', '34', '35', '36', '37', '38'], (594580, '744898'): ['52', '53', '58', '59'], (594580, '744893'): ['69', '70', '71', '72'], (594580, '745460'): ['18', '19', '20'], (594580, '746671'): ['39', '41'], (594580, '746672'): ['21', '22', '23', '24'], (594580, '746665'): ['33', '34', '35', '36', '37'], (594580, '746668'): ['25', '31', '32', '33'], (594580, '745775'): ['40', '41', '42'], (458677, '746810'): ['68', '69', '70', '71', '72'], (458677, '745270'): ['42', '43', '47', '48'], (458677, '746725'): ['50'], (458677, '745025'): ['61', '62'], (458677, '745022'): ['60', '61', '62'], (458677, '745426'): ['69'], (458677, '746721'): ['52'], (458677, '746718'): ['47', '48', '49', '50', '51'], (458677, '745340'): ['55', '56', '57', '58', '64'], (458677, '745345'): ['63', '64', '65'], (458677, '747204'): ['71', '72', '73', '74'], (458677, '746150'): ['18'], (458677, '746548'): ['78', '79', '80'], (458677, '746544'): ['59', '60', '61'], (458677, '746704'): ['57', '58', '59', '60'], (458677, '746703'): ['72', '73'], (458677, '746702'): ['43', '44', '45', '52', '53'], (458677, '745979'): ['63', '64', '65', '66'], (458677, '746700'): ['58', '59', '60'], (458677, '746696'): ['74', '75', '76', '77', '78', '79'], (458677, '746695'): ['44', '45', '46', '50', '51', '52'], (458677, '745159'): ['62', '63', '64', '65', '66', '67'], (458677, '745728'): ['65'], (458677, '745726'): ['67', '68', '69', '70'], (458677, '746694'): ['65', '66', '67', '68'], (458677, '746690'): ['58', '59', '60', '61'], (458677, '746688'): ['62', '63', '64'], (458677, '746687'): ['63', '64', '65', '66'], (458677, '746686'): ['60'], (458677, '744830'): ['70', '71', '72', '73', '74'], (458677, '744831'): ['64', '65'], (458677, '747091'): ['83', '84', '85'], (458677, '746684'): ['49', '50', '51'], (458677, '746683'): ['51', '57', '58', '59'], (458677, '746681'): ['48', '49', '50', '51', '52'], (458677, '746038'): ['84', '85', '86', '87'], (458677, '746031'): ['61', '62', '63', '67'], (458677, '746677'): ['52', '56', '57', '58', '59'], (458677, '746674'): ['49', '50', '51', '52'], (458677, '746676'): ['50', '51', '52'], (458677, '744898'): ['66', '67', '68'], (458677, '745465'): ['62', '63', '64'], (458677, '745460'): ['46', '47', '48', '49', '50'], (458677, '746671'): ['53', '54', '55', '56'], (458677, '746730'): ['66', '67', '68'], (458677, '746667'): ['72', '73', '74', '75', '76', '77'], (458677, '746668'): ['81'], (458677, '745777'): ['73'], (458677, '745774'): ['53', '54', '55'], (458677, '745126'): ['50', '51', '52', '53', '54'], (458677, '745127'): ['62', '63'], (458677, '745852'): ['58', '59', '60', '61', '62'], (458677, '746662'): ['56', '57', '58', '59'], (458677, '746660'): ['57', '58', '59', '60', '61'], (458677, '746581'): ['47', '48', '49', '50', '51'], (458677, '746819'): ['59', '61'], (664747, '746731'): ['77', '78', '79', '80', '81'], (664747, '746735'): ['78', '79', '80', '81', '82'], (664747, '746810'): ['84', '85', '86', '87'], (664747, '745267'): ['57', '61', '62', '63', '64'], (664747, '746726'): ['74', '75', '76'], (664747, '746727'): ['67', '68', '69', '70'], (664747, '746725'): ['80', '81', '82'], (664747, '745020'): ['73', '74'], (664747, '745427'): ['64', '65', '66', '67', '68', '69'], (664747, '746719'): ['60', '61', '62', '63', '64'], (664747, '745340'): ['70', '71', '72', '73'], (664747, '745345'): ['79', '80', '81', '83'], (664747, '747204'): ['80', '81', '82', '83', '84'], (664747, '746148'): ['62', '63', '64', '65', '66'], (664747, '746144'): ['73', '77', '78', '79', '80'], (664747, '746716'): ['62', '63', '64'], (664747, '746714'): ['71', '72', '73'], (664747, '746710'): ['64', '65', '66'], (664747, '746711'): ['68', '69'], (664747, '746877'): ['75', '76', '77', '78', '79'], (664747, '746871'): ['73', '74', '75', '76'], (664747, '746707'): ['73', '74', '75', '76'], (664747, '746704'): ['64', '65', '66', '67', '68'], (664747, '746701'): ['69', '70', '71'], (664747, '745979'): ['78', '79', '80', '81', '82', '83', '84'], (664747, '745488'): ['64', '65', '66'], (664747, '746698'): ['67', '68', '69', '70'], (664747, '746695'): ['63', '64', '65', '66'], (664747, '745154'): ['61', '62', '63'], (664747, '745730'): ['73', '74', '75'], (664747, '745728'): ['69', '70', '71'], (664747, '746691'): ['56', '57', '58', '59', '60', '61', '62'], (664747, '746685'): ['73', '74', '75', '76', '77', '78'], (664747, '746686'): ['65', '66', '67', '68', '69'], (664747, '747095'): ['70', '71', '72', '73', '74', '75'], (664747, '745068'): ['74', '75', '76', '77'], (664747, '746683'): ['76', '77', '78', '79'], (664747, '746681'): ['64', '65', '66', '67'], (664747, '746031'): ['68', '69'], (664747, '745956'): ['70', '71', '72'], (664747, '746673'): ['63', '64', '65', '66'], (664747, '746675'): ['63', '64', '65', '66'], (664747, '744898'): ['72', '73', '74', '75'], (664747, '745460'): ['65', '66', '67', '68'], (664747, '745459'): ['72', '73', '74'], (664747, '746672'): ['68', '69', '70'], (664747, '746670'): ['72', '73', '74', '75'], (664747, '746730'): ['73', '74', '75', '76'], (664747, '746669'): ['81', '82', '83'], (664747, '746664'): ['64', '65', '66'], (664747, '745777'): ['66', '67', '68', '69'], (664747, '745774'): ['68', '69', '70'], (664747, '747090'): ['66', '67', '68'], (664747, '745855'): ['69', '70', '71'], (664747, '746659'): ['80', '81', '82', '83'], (664747, '746657'): ['53', '54', '55', '56', '57'], (664747, '746819'): ['62', '63', '64', '65', '66'], (596001, '746031'): ['49', '50', '51', '52'], (596001, '745957'): ['49', '50', '51'], (596001, '746682'): ['54', '59', '60', '61', '66', '67', '68'], (596001, '746675'): ['72', '73', '74'], (596001, '744896'): ['48', '49', '54', '55', '56'], (596001, '745465'): ['38', '39', '40', '44', '45', '46', '47', '52', '53', '54', '55', '56'], (596001, '746671'): ['1', '2', '3', '7', '8', '9', '14', '15', '16', '20', '21', '22'], (596001, '746669'): ['30', '31', '35', '36', '37', '38', '39', '46', '47', '48', '54', '55', '56', '57'], (596001, '745775'): ['4', '5', '6', '12', '13', '14', '18', '19', '20', '25', '26', '27', '28', '32', '33', '34', '35'], (596001, '745129'): ['4', '5', '6', '10', '11', '12', '13', '17', '18', '19', '20', '24', '25', '26', '30', '31', '32', '33'], (596001, '746662'): ['1', '2', '3', '8', '9', '10', '14', '15', '16', '21', '22', '23', '24', '25', '30', '31', '32', '37', '38', '39'], (596001, '746583'): ['4', '5', '6', '7', '11', '12', '13', '17', '18', '19', '23', '24', '25', '26', '27', '31', '32', '33'], (608718, '745423'): ['58', '59', '64', '65', '66'], (608718, '746719'): ['39', '40', '41', '45', '46', '47', '48', '49', '50'], (608718, '746717'): ['66', '67', '68'], (608718, '745344'): ['55', '56', '57', '58', '59', '60'], (608718, '747204'): ['51', '52', '53', '54'], (608718, '747205'): ['65', '66', '67'], (608718, '746150'): ['7', '8', '9'], (608718, '746712'): ['36', '37', '41', '42', '43', '44', '45'], (608718, '746713'): ['81', '82', '83', '84'], (608718, '746711'): ['1', '2', '3'], (608718, '746709'): ['69', '70', '71', '72'], (608718, '746706'): ['59', '60', '61', '62'], (608718, '746548'): ['61', '62', '63', '64', '69', '70', '71', '72'], (608718, '746544'): ['84', '85', '86', '87'], (608718, '746705'): ['61', '62', '63'], (608718, '746702'): ['1', '2', '3', '4'], (608718, '745976'): ['57', '58', '59'], (608718, '745490'): ['54', '55', '56', '57', '62', '63', '64', '65'], (608718, '746699'): ['48', '49', '50', '51', '60', '61', '62', '66', '67', '68'], (608718, '746697'): ['75', '76', '77'], (608718, '745162'): ['65', '70', '71', '72', '73', '74', '75'], (608718, '745159'): ['81', '82', '83'], (608718, '746693'): ['52', '53', '54', '55', '56', '60', '61', '62', '63', '67', '68', '69'], (608718, '746690'): ['82', '83', '84', '85', '86'], (608718, '746689'): ['59', '60', '61', '62', '63'], (608718, '746687'): ['78', '79', '80', '81', '82'], (608718, '744829'): ['41', '46'], (608718, '746669'): ['63', '64', '65', '69', '70', '71', '75', '76', '77'], (608718, '746666'): ['60', '61', '66', '67', '68', '69', '73', '74', '75', '79', '80', '81', '82'], (608718, '745126'): ['64', '65', '66'], (608718, '745127'): ['53'], (608718, '745858'): ['59', '60', '61', '62', '69', '70', '71', '72', '76', '77', '78'], (608718, '746659'): ['66', '67'], (608718, '746661'): ['54', '55', '56', '57', '58', '59'], (608718, '746657'): ['61', '62', '63', '64', '65', '66'], (608718, '746820'): ['47', '51', '52', '53', '54'], (668933, '746544'): ['5', '6', '7', '8', '9', '10', '11', '16', '17', '18', '19', '25', '26', '27', '28', '32', '33', '34'], (668933, '746695'): ['1', '2', '3', '7', '8', '9', '10', '11', '12', '16', '17', '18', '24', '25', '26', '27', '28', '33', '34', '35', '36', '37', '41', '42', '43'], (668933, '745730'): ['5', '6', '7', '8', '12', '13', '14', '15', '16', '20', '21', '22', '23', '30', '31', '32', '41', '42', '43', '44', '49', '50', '51'], (668933, '746691'): ['1', '2', '3', '4', '5', '9', '10', '11', '15', '16', '17', '18', '19', '24', '25', '26', '27', '28', '32', '33', '34'], (656876, '745211'): ['5', '6', '7', '13', '14', '15', '16'], (656876, '745050'): ['51', '52', '53', '54', '59', '60', '61', '62', '63', '64'], (656876, '745047'): ['50', '51', '52', '57', '58', '59'], (656876, '746986'): ['53', '54', '62', '63', '64'], (656876, '745525'): ['47', '48', '49', '50', '51', '55', '56', '57'], (656876, '746584'): ['4', '5', '6', '11', '12', '13', '14'], (656876, '745040'): ['44', '45', '46', '47', '48', '54', '55', '56'], (656876, '745038'): ['64', '65', '66', '70', '71', '72'], (656876, '746420'): ['59', '60', '61', '66', '67', '68', '69', '70', '71'], (656876, '746902'): ['66', '67', '68', '69', '73', '74', '75', '76'], (605488, '745047'): ['1', '2', '3', '8', '9', '10', '11', '12', '13', '18', '19', '20', '21', '22', '26', '27', '28', '36', '37', '38', '39', '43', '44', '45'], (663992, '746879'): ['48', '49', '50'], (663992, '745505'): ['64', '65', '66', '67', '68'], (663992, '745499'): ['46', '47', '48'], (663992, '745093'): ['50', '51', '52'], (663992, '745089'): ['37', '38', '39', '40'], (663992, '745087'): ['71', '72', '73', '74'], (663992, '745086'): ['77', '78', '83', '84', '85', '86', '87', '88'], (663992, '745081'): ['90', '92', '93'], (663992, '747032'): ['48', '49', '50'], (663992, '746056'): ['48', '49', '50'], (663992, '745083'): ['45', '46', '47'], (663992, '745143'): ['50', '51', '52', '53', '54'], (663992, '745062'): ['43', '44', '45', '46', '47'], (663992, '745058'): ['52', '53', '54', '55', '60', '61', '62'], (663992, '745049'): ['66', '72', '73', '74'], (663992, '745620'): ['53', '54', '55', '56'], (663992, '745623'): ['65', '66', '67', '68', '69'], (663992, '746111'): ['41', '42', '46'], (663992, '746106'): ['58', '59', '60', '61', '62', '63', '64'], (663992, '745052'): ['55', '56', '57', '58', '63', '64', '65'], (663992, '745048'): ['47', '48', '55', '56', '57'], (663992, '745045'): ['73', '74', '75', '76'], (663992, '746993'): ['49', '50', '51', '52'], (663992, '746989'): ['47', '48', '49', '53'], (663992, '745535'): ['64', '65', '69', '70', '71'], (663992, '746591'): ['72', '73', '74'], (663992, '746585'): ['59', '60', '61', '62'], (663992, '745040'): ['57'], (663992, '745036'): ['58', '59', '60', '61'], (663992, '746420'): ['43', '49', '50'], (663992, '746901'): ['55', '56', '57'], (655889, '745095'): ['69', '70', '71', '72'], (655889, '745097'): ['66', '67', '68', '69'], (655889, '746962'): ['78', '79', '80', '81', '82', '88'], (655889, '746959'): ['71', '72', '73'], (655889, '744934'): ['60', '61', '62'], (655889, '744937'): ['48', '49', '50', '54', '55', '56'], (655889, '745069'): ['69', '70', '71'], (655889, '745065'): ['66', '67'], (655889, '745721'): ['67', '68', '69', '70'], (655889, '745722'): ['41', '42', '43'], (655889, '744904'): ['47', '48', '49', '50', '51'], (655889, '744905'): ['50', '51', '52'], (655889, '745068'): ['70', '71', '72', '73'], (655889, '745063'): ['28', '29', '30', '31'], (655889, '745060'): ['68', '69', '70', '71'], (655889, '746358'): ['55', '56', '57'], (655889, '746356'): ['60', '61', '62'], (655889, '745143'): ['61', '62', '63', '64'], (655889, '745056'): ['69', '70'], (655889, '745057'): ['57', '58', '59', '60', '61'], (655889, '745053'): ['51', '52', '53'], (655889, '745049'): ['61', '62', '63', '64', '65'], (655889, '745622'): ['69', '70', '71'], (655889, '746111'): ['67', '68', '69', '70'], (655889, '746108'): ['76', '77', '78', '79'], (655889, '745213'): ['67', '68', '69', '70'], (655889, '745052'): ['72', '73', '74'], (655889, '745048'): ['71', '72', '73', '74', '75'], (655889, '745046'): ['74', '75', '76'], (655889, '746993'): ['57', '58', '59', '60'], (655889, '745534'): ['57', '58', '59', '65'], (655889, '745535'): ['61', '62', '63'], (655889, '746591'): ['64', '65', '66', '67'], (655889, '745043'): ['48', '49', '50', '51', '52'], (655889, '745038'): ['50'], (655889, '746420'): ['51', '52', '53', '54', '55'], (655889, '746904'): ['61', '62', '63'], (670955, '745071'): ['56', '57', '62', '63', '64', '68', '69', '70'], (670955, '746283'): ['39', '40', '41', '42', '43', '48', '49', '50'], (670955, '744993'): ['52', '53', '54', '55'], (670955, '745067'): ['54', '55', '56', '57'], (670955, '745722'): ['64', '65', '66'], (670955, '744907'): ['64', '65', '66', '67', '72', '73', '74'], (670955, '745063'): ['16', '17', '18', '22', '23', '24'], (670955, '745061'): ['40', '41', '42', '43', '48', '49', '50', '51', '52', '53', '54'], (670955, '746358'): ['61', '62', '63'], (670955, '746356'): ['46', '47', '48', '54', '55', '56'], (670955, '745062'): ['66', '67', '68'], (670955, '745057'): ['42', '43', '44', '45', '49', '50', '51'], (670955, '745053'): ['35', '36', '37', '38'], (670955, '745049'): ['46', '47', '48'], (670955, '745051'): ['98', '99', '100', '101'], (670955, '745624'): ['56', '60', '61', '62'], (670955, '746111'): ['47', '48', '49', '53', '54', '55'], (670955, '745213'): ['40', '41', '42', '43', '44'], (670955, '745048'): ['58', '63', '64', '65'], (670955, '745047'): ['63', '64', '65', '66'], (670955, '746989'): ['62', '63', '68', '69', '70', '71', '72'], (670955, '745535'): ['72', '73', '74', '75', '76', '77'], (670955, '746586'): ['59', '60', '64', '65', '66'], (670955, '745041'): ['52', '53', '54'], (670955, '745043'): ['56', '57', '58'], (670955, '745036'): ['67', '68', '69', '70', '71', '72'], (670955, '746904'): ['67', '68', '69', '70'], (664126, '745112'): ['68', '69', '70', '71'], (664126, '745108'): ['68', '69', '70'], (664126, '745105'): ['68', '69', '70', '71', '73', '74'], (664126, '745096'): ['69', '70', '71'], (664126, '746962'): ['65', '66', '67'], (664126, '746960'): ['66', '67', '68'], (664126, '744934'): ['66', '67', '68'], (664126, '745089'): ['74', '75', '76'], (664126, '745086'): ['67', '68', '69', '70'], (664126, '745088'): ['66', '67', '68', '69', '70', '71'], (664126, '745084'): ['65', '66', '67', '68'], (664126, '745081'): ['73', '74', '75', '76', '77'], (664126, '747030'): ['81', '82', '83', '84'], (664126, '746060'): ['83', '84', '85'], (664126, '746056'): ['71', '72', '73'], (664126, '745085'): ['71', '72', '73', '74'], (664126, '745075'): ['65', '66', '67'], (664126, '747113'): ['67', '68', '69', '70', '71'], (664126, '747109'): ['72', '73', '74', '75'], (664126, '745893'): ['74', '75', '76', '77'], (664126, '745889'): ['76', '77', '78', '79', '84', '85', '86'], (664126, '745485'): ['68', '69', '70'], (664126, '745078'): ['73', '74', '75'], (664126, '745070'): ['67', '68', '69', '70'], (664126, '746282'): ['72', '73', '74'], (664126, '746285'): ['87', '88', '89', '90'], (664126, '745074'): ['70', '71', '72', '73'], (664126, '745065'): ['71', '72', '73', '74', '75', '76'], (664126, '745066'): ['60', '61', '62', '63', '64'], (664126, '745722'): ['71', '72', '73', '74', '75'], (664126, '744904'): ['69', '70', '71'], (664126, '745068'): ['64', '65', '66'], (664126, '745063'): ['66', '67', '68', '69'], (664126, '746358'): ['70', '71', '72'], (664126, '746356'): ['67', '68', '69'], (664126, '745143'): ['76', '77', '78'], (664126, '745056'): ['78', '79', '80'], (664126, '745057'): ['68', '69', '70', '71'], (664126, '745053'): ['58', '59', '60'], (664126, '745054'): ['68', '69', '70', '71', '72', '73'], (664126, '745051'): ['69', '70', '71', '72', '73', '74', '75'], (664076, '745116'): ['61', '62', '63', '64', '65'], (664076, '745115'): ['64', '65', '66', '67', '71', '72', '73'], (664076, '745114'): ['60', '61', '62', '63', '64', '65', '66', '67'], (664076, '746575'): ['62'], (664076, '746574'): ['50', '51', '52', '53'], (664076, '745112'): ['38', '39', '40', '41', '42', '47', '48'], (664076, '745108'): ['89', '90', '91', '92', '93'], (664076, '745104'): ['47', '48', '49'], (664076, '745760'): ['70', '71', '72'], (664076, '745099'): ['68', '69', '70', '71'], (664076, '746803'): ['60'], (664076, '745996'): ['49', '50'], (664076, '745100'): ['46', '47', '48', '49'], (664076, '745103'): ['52', '53', '54', '55'], (664076, '745094'): ['50'], (664076, '745096'): ['45', '50', '51', '52'], (664076, '746961'): ['64', '65', '66', '67'], (664076, '746962'): ['57', '58', '59', '60'], (664076, '746959'): ['63', '64', '65', '66'], (664076, '744938'): ['77', '78', '79'], (664076, '745090'): ['42', '47', '48', '49'], (664076, '745086'): ['49', '50', '51', '52', '53'], (664076, '745088'): ['44', '45', '46', '47'], (664076, '745084'): ['50', '51', '52'], (664076, '745081'): ['84', '85', '86'], (664076, '747030'): ['72', '73', '74', '75', '76'], (664076, '746056'): ['61', '62', '63', '64', '65'], (664076, '745085'): ['50', '51', '52', '53', '54', '55'], (664076, '745077'): ['45', '46'], (664076, '745079'): ['55', '56', '57', '58'], (664076, '747113'): ['56', '57', '58', '59', '60', '61', '62'], (664076, '745893'): ['61', '62', '63'], (664076, '745889'): ['61', '62', '63'], (664076, '745486'): ['61', '62', '63'], (664076, '745485'): ['43', '44', '45', '49'], (664076, '745076'): ['41', '42', '43'], (664076, '745072'): ['57', '58', '59', '60', '61'], (664076, '746285'): ['58', '59', '60'], (664076, '744993'): ['67', '68', '69', '70', '71', '72', '73'], (664076, '745069'): ['60', '61', '62', '63'], (664076, '745065'): ['48', '49', '50'], (664076, '745067'): ['61', '62', '63', '64', '65'], (664076, '745721'): ['60', '61', '62', '63'], (664076, '745722'): ['35', '36'], (664076, '744905'): ['46'], (664076, '745068'): ['48', '49', '50'], (664076, '745063'): ['54', '55', '56', '57'], (664076, '745060'): ['60'], (664076, '746354'): ['48', '49', '50'], (664076, '745143'): ['45', '46'], (664076, '745056'): ['50', '54', '55', '56', '57'], (664076, '745053'): ['64', '65', '66', '67', '68'], (664076, '745049'): ['54', '55', '56'], (664076, '745051'): ['75', '81', '82', '83', '84', '85'], (664076, '745622'): ['63', '64', '65'], (664076, '746111'): ['56', '61', '62', '63'], (664076, '746108'): ['85', '86', '87'], (664076, '745213'): ['58', '59', '60', '61', '62'], (664076, '745052'): ['80', '81', '82', '83', '84', '85', '86', '87'], (664076, '745046'): ['66', '67', '68', '69'], (664076, '745047'): ['67'], (664076, '746989'): ['59', '60', '61'], (664076, '745534'): ['66', '67', '68', '69', '70'], (664076, '746591'): ['55', '56', '57', '58'], (664076, '745044'): ['55', '56', '57', '58', '59'], (664076, '745041'): ['59', '60', '61'], (664076, '745038'): ['55', '56', '57', '58', '59'], (664076, '746415'): ['60', '61', '62', '63', '64', '65'], (660761, '746390'): ['47', '48', '49', '56', '57', '58'], (660761, '746227'): ['37', '38', '43', '44', '45', '46', '47', '48', '49', '54', '55', '56', '57', '62', '63', '64'], (660761, '746225'): ['48', '49', '50', '59', '60', '61', '62', '63', '68', '69', '70'], (660761, '745252'): ['66'], (660761, '747190'): ['6', '7', '8', '9', '13', '14', '15', '16', '17', '18', '25', '26', '27', '28'], (660761, '745321'): ['22', '23', '24', '25', '26', '30', '31', '32', '33', '34'], (660761, '745860'): ['55', '56', '57', '62', '63', '64', '70', '71', '72', '73'], (660761, '746181'): ['33', '34', '35', '36', '41', '42', '43', '47', '48', '49', '53', '54', '55', '56', '57'], (660761, '746341'): ['7', '8', '9', '10', '15', '16', '17', '18', '19', '23', '24', '25', '26', '31', '32', '33', '34', '35', '36', '40', '41', '42'], (660761, '746742'): ['7', '8', '9', '10', '14', '15', '16', '17', '18', '24', '25', '26', '34', '35', '36', '37', '38', '39', '40', '44', '45', '46'], (519043, '746246'): ['56', '57', '58'], (519043, '746243'): ['64', '65', '66', '67', '68', '69', '70'], (519043, '746237'): ['57', '58'], (519043, '746236'): ['65', '66', '67', '68'], (519043, '746239'): ['58', '59', '60', '61'], (519043, '746643'): ['58', '59', '60', '61', '62'], (519043, '745507'): ['58', '59', '60', '64'], (519043, '746240'): ['49', '50', '51', '52', '53'], (519043, '746233'): ['48', '49', '50', '51', '52'], (519043, '746231'): ['55', '56', '57', '58', '59'], (519043, '745016'): ['66', '67', '68', '69', '70'], (519043, '746385'): ['70', '71', '72'], (519043, '746226'): ['56', '57', '58', '59'], (519043, '746230'): ['49', '50', '51'], (519043, '746229'): ['58', '59', '60'], (519043, '745253'): ['62', '63', '64', '65', '66'], (519043, '745252'): ['58', '59', '60'], (519043, '746222'): ['46', '47', '48', '49'], (519043, '746221'): ['63', '64', '65'], (519043, '746216'): ['59', '60', '61', '62', '63'], (519043, '747193'): ['73', '74', '75', '76'], (519043, '745329'): ['50', '51', '52', '53'], (519043, '746215'): ['59', '60', '61', '62'], (519043, '746131'): ['35', '36', '37', '38', '39'], (519043, '746212'): ['40', '41', '42'], (519043, '746214'): ['35', '36', '37'], (519043, '746209'): ['64', '65', '66'], (519043, '745643'): ['46', '47', '48', '49'], (519043, '745641'): ['49', '50', '51', '52', '53'], (519043, '746855'): ['50', '51', '52', '53', '54'], (519043, '746206'): ['63', '67', '68', '69', '70'], (519043, '746201'): ['44', '45', '46', '47', '48'], (519043, '745640'): ['54', '55', '56', '57', '58', '59', '60'], (519043, '746199'): ['44', '45', '46', '47', '48'], (519043, '746200'): ['59', '60', '61'], (519043, '746203'): ['63', '64', '65', '66', '67'], (519043, '746198'): ['65', '66', '67'], (519043, '746194'): ['64', '65', '66'], (519043, '745709'): ['67', '68', '69', '73', '74', '76'], (519043, '744818'): ['70', '71', '72', '73', '78'], (519043, '746192'): ['52', '56', '57', '58', '59'], (519043, '746189'): ['63', '64', '65'], (519043, '746270'): ['67', '68', '69'], (519043, '744892'): ['52', '53', '54', '55'], (656271, '746408'): ['67', '68', '69', '70', '71', '72'], (656271, '745241'): ['33', '34', '35', '41', '42', '43', '47', '48', '49'], (656271, '744997'): ['62', '63', '64', '65'], (656271, '745973'): ['45', '46', '47', '48', '49'], (656271, '747020'): ['69', '70', '71', '72', '73', '74'], (656271, '747018'): ['51', '52', '53', '57', '58'], (656271, '744979'): ['53', '54', '55', '60', '61', '62'], (656271, '744981'): ['65', '66', '67', '68', '69', '70'], (656271, '745712'): ['37', '38', '39', '40', '41', '42', '43', '44', '48', '49', '50', '51'], (656271, '746189'): ['39', '40', '41', '45', '46', '47'], (656271, '746186'): ['66', '67', '68'], (656271, '746270'): ['60', '61', '62'], (656271, '744894'): ['6', '7', '8', '12', '13', '14'], (656271, '744890'): ['51', '52', '53'], (656271, '746429'): ['47', '48', '49'], (656271, '746430'): ['55', '56', '57'], (656271, '746182'): ['36', '37', '38', '39', '40'], (656271, '746183'): ['58', '59', '60', '61', '62'], (656271, '746185'): ['72', '73'], (656271, '744967'): ['61', '62', '63'], (656271, '744965'): ['49', '50', '55'], (656271, '745857'): ['55', '59', '60', '61', '63'], (656271, '746181'): ['67', '68', '69', '70', '71', '72'], (656271, '746176'): ['49', '50', '51'], (656271, '746172'): ['52', '53', '54', '55'], (656271, '746340'): ['77', '78', '79'], (656271, '746339'): ['67', '68'], (656271, '746743'): ['58', '59', '60', '61'], (656271, '746742'): ['69', '70', '71'], (656271, '746171'): ['67', '68', '69', '70', '71', '72'], (519326, '746238'): ['44', '45', '46'], (519326, '746644'): ['52'], (519326, '745509'): ['54', '58', '59', '60'], (519326, '746232'): ['62', '63', '64', '65'], (519326, '746234'): ['53', '54', '55'], (519326, '746235'): ['48', '49', '50', '51', '52', '59'], (519326, '745016'): ['71'], (519326, '745013'): ['68', '69', '70'], (519326, '746390'): ['69', '70', '71'], (519326, '746385'): ['76', '77', '78', '79'], (519326, '746226'): ['49', '50', '51', '52'], (519326, '746229'): ['45', '46', '47'], (519326, '746228'): ['49', '50', '51', '52', '53'], (519326, '745252'): ['53'], (519326, '746224'): ['47', '52', '53', '55', '56'], (519326, '746222'): ['62'], (519326, '746220'): ['49', '50', '51', '52', '53'], (519326, '746216'): ['50', '51', '52', '53'], (519326, '747190'): ['68', '69', '70', '77', '78', '79', '80'], (519326, '747192'): ['54', '55', '56'], (519326, '745329'): ['59', '60', '61', '62'], (519326, '745321'): ['35', '36', '37', '38', '39', '40', '41'], (519326, '746131'): ['22', '23', '28', '29', '30'], (519326, '746213'): ['49', '50', '51'], (519326, '746212'): ['52', '53', '54', '55'], (519326, '746214'): ['41', '42', '43', '48', '49', '50'], (519326, '745647'): ['66', '67', '68'], (519326, '745641'): ['27', '32', '33', '34', '35'], (519326, '746859'): ['42', '43', '44'], (519326, '746855'): ['41', '42', '43', '44'], (519326, '746206'): ['21', '22', '28', '29', '30', '31'], (519326, '746205'): ['54', '55', '56', '57', '58'], (519326, '746201'): ['31', '36', '37', '38', '39', '40'], (519326, '745633'): ['43', '44', '50', '51', '52'], (519326, '745232'): ['50', '51', '52'], (519326, '746199'): ['33', '34', '38', '39', '40'], (519326, '746203'): ['55', '56', '57', '58'], (519326, '746197'): ['78', '79', '80'], (519326, '746196'): ['65', '66', '67', '68', '69'], (519326, '746194'): ['53', '54', '55', '56'], (519326, '745713'): ['47', '48', '49', '50', '55', '56', '57', '58', '59'], (519326, '744818'): ['54', '55', '56'], (519326, '744820'): ['62', '63', '64', '65'], (519326, '746188'): ['65', '66', '67', '68', '69'], (519326, '746189'): ['54', '55', '56', '57', '58'], (519326, '746266'): ['66', '67'], (519326, '744889'): ['53', '54', '55'], (519326, '744892'): ['44', '45', '46', '47', '48'], (519326, '746430'): ['62', '63', '64'], (519326, '746184'): ['60', '61', '62', '63'], (519326, '746185'): ['48', '49', '50'], (519326, '746178'): ['54', '59', '60', '61', '62'], (519326, '744967'): ['55', '56', '57'], (519326, '744963'): ['45', '46', '47', '48'], (519326, '745857'): ['64', '65'], (519326, '746177'): ['47', '48', '49', '50'], (519326, '746180'): ['69', '70', '71'], (519326, '746176'): ['57', '58', '59', '60', '61'], (519326, '746341'): ['53', '54', '55', '56'], (519326, '746339'): ['40', '41', '47', '48', '49'], (519326, '746743'): ['54', '55', '56', '57'], (519326, '746171'): ['46', '47', '48', '49'], (690829, '746223'): ['62', '63', '64', '65', '66'], (690829, '747190'): ['58', '63', '64', '65', '66', '67'], (690829, '745326'): ['59', '60', '66', '67', '68', '69', '70'], (690829, '745321'): ['6', '7', '8', '9', '14', '15', '16', '17', '18'], (690829, '746131'): ['47', '48', '49', '53', '54', '55'], (690829, '746218'): ['58', '59', '60', '61', '62'], (690829, '746213'): ['70', '71'], (690829, '746210'): ['61', '62', '63'], (690829, '746211'): ['79', '80', '81'], (690829, '746856'): ['74', '75', '76', '77'], (690829, '746206'): ['36', '37', '38', '39', '43', '44', '45'], (690829, '746207'): ['53', '54', '55', '56'], (690829, '746202'): ['50', '51', '52', '53'], (690829, '745633'): ['60', '61', '62', '63', '64'], (690829, '745227'): ['62', '63'], (690829, '746199'): ['53', '54', '55', '59', '60', '61', '62', '63', '64'], (690829, '746195'): ['63', '64', '65', '71', '72', '73'], (690829, '746197'): ['68', '69', '70'], (690829, '746196'): ['58', '59', '60', '61'], (690829, '746194'): ['67', '73', '74', '75'], (690829, '745710'): ['76', '77', '78', '79', '80'], (690829, '744818'): ['79'], (690829, '744816'): ['80', '81', '82', '83', '84', '85', '86'], (690829, '746189'): ['66', '70', '71', '72', '73'], (690829, '746266'): ['83', '84', '85'], (690829, '744889'): ['60', '61', '62', '63'], (690829, '746432'): ['65', '69', '70', '71', '72'], (690829, '746182'): ['69', '70', '71', '72', '73'], (690829, '746183'): ['66', '67', '68'], (690829, '746185'): ['61', '62', '63'], (669721, '745643'): ['4', '5', '6', '7', '11', '12', '13', '14', '15', '16', '20', '21', '22', '23', '27', '28', '29', '35', '36', '37', '38', '42', '43', '44', '45'], (669721, '746208'): ['1', '2', '3', '4', '5', '9', '10', '11', '18', '19', '20', '24', '25', '26', '27', '28', '29', '30', '31'], (669721, '746198'): ['1', '2', '3', '4', '5', '6', '10', '11', '12', '13', '17', '18', '19', '23', '24', '25', '26', '31', '32', '33', '34'], (669721, '745709'): ['6', '7', '8', '9', '10', '11', '12', '18', '19', '20', '21', '26', '27', '28', '29', '30', '36', '37', '38'], (669721, '746192'): ['1', '2', '3', '4', '5', '9', '10', '11', '12', '13', '17', '18', '19', '20', '21', '22', '23', '28', '29', '30', '34', '35', '36', '37', '38'], (672710, '745840'): ['70', '71', '72'], (672710, '745518'): ['45', '46', '47', '48', '52', '53', '54', '55', '56', '57'], (672710, '745513'): ['47', '48', '49', '50', '51', '56', '57', '58'], (672710, '745512'): ['77', '78', '79', '80', '81', '82'], (672710, '745672'): ['44', '45', '46', '48', '49', '53', '54', '55', '56'], (672710, '745508'): ['39', '43', '44', '45', '46', '47', '48', '52', '53', '54', '55', '56'], (672710, '745506'): ['75', '76', '77'], (672710, '746227'): ['68', '69', '70', '71', '76', '77', '78'], (672710, '746230'): ['55', '56', '57'], (672710, '746228'): ['58', '59', '60', '61', '62'], (672710, '745251'): ['50', '51', '52', '53', '54', '58', '59', '60'], (672710, '746220'): ['58', '59', '60', '61', '65', '66', '67', '68'], (672710, '747192'): ['63', '67', '68', '69', '70', '71', '72'], (672710, '745326'): ['79', '80', '81'], (672710, '746217'): ['44', '45', '46', '47', '52', '53', '54', '59', '60', '61'], (672710, '746132'): ['63', '64', '65'], (672710, '746212'): ['1', '2', '3', '4', '11', '12', '13', '14', '15', '19', '20', '21', '25', '26'], (672710, '745641'): ['4', '5', '6', '7', '11', '12', '13', '20', '21', '22', '23', '24', '25', '26'], (672710, '746206'): ['1', '2', '3', '4', '9', '10', '11', '16', '17', '18', '20'], (672710, '746201'): ['64', '65', '66'], (672710, '745640'): ['35', '36', '37', '41', '42', '43', '51', '52', '53'], (672710, '745227'): ['67', '68', '69', '70'], (672710, '746199'): ['68', '69', '70', '71', '72'], (672710, '746200'): ['67', '68', '69', '70'], (672710, '746195'): ['37', '45', '46', '47'], (672710, '746193'): ['58', '59', '60', '61', '65', '66', '67'], (672710, '746191'): ['66', '67', '68'], (672710, '745713'): ['70', '71', '72'], (672710, '745710'): ['57', '58', '59'], (672710, '744820'): ['76'], (672710, '746187'): ['67', '68', '69'], (672710, '746189'): ['28', '29', '36', '37', '38'], (672710, '746186'): ['56', '58', '59', '60', '61'], (672710, '746267'): ['56', '57', '62', '63', '64'], (672710, '744889'): ['68', '69'], (672710, '746432'): ['51', '52', '53', '54', '55'], (672710, '746182'): ['44', '45', '46', '47', '52', '53', '54', '55'], (672710, '746185'): ['67', '68', '69', '71'], (672710, '744963'): ['62', '63', '64', '65', '66', '67'], (672710, '745856'): ['55', '61', '62', '63', '64'], (672710, '746180'): ['29', '31', '35', '36', '37', '43', '44', '45', '46'], (672710, '746340'): ['52', '53', '54', '55', '60', '61', '62', '63', '64', '71', '72', '73'], (672710, '746173'): ['45', '46', '47', '48', '53', '54', '55'], (672710, '746174'): ['65', '66', '67', '68', '69'], (674370, '745637'): ['67', '68', '69', '70', '71', '72'], (674370, '745552'): ['34', '35', '36', '44', '45', '46', '50', '51', '52'], (674370, '745640'): ['68', '69', '70'], (674370, '745632'): ['1', '2', '3', '4', '10', '11', '12', '17', '18', '19', '20', '26', '27', '28', '34', '35', '36', '37', '38'], (674370, '746195'): ['4', '5', '6', '7', '12', '13', '14', '25', '26', '27', '28', '29', '30', '31', '32', '38', '39', '40', '41', '42', '43'], (674370, '745627'): ['1', '2', '3', '4', '5', '6', '7', '14', '15', '16', '17', '22', '23', '24', '25', '29', '30', '31', '35', '36', '37', '38', '45', '46', '47', '48'], (674370, '744897'): ['4', '5', '6', '7', '13', '14', '15', '16', '20', '21', '22', '27', '28', '29', '34', '35', '36', '37', '43', '44', '45'], (674370, '745626'): ['1', '2', '3', '7', '8', '9', '14', '15', '16', '20', '21', '22', '27', '28', '29', '30', '37', '38', '39', '40', '41'], (674370, '745623'): ['1', '2', '3', '4', '5', '11', '12', '13', '20', '21', '22', '28', '29', '30', '31', '32', '37', '38', '39', '43'], (674370, '746672'): ['5', '6', '7', '11', '12', '13', '18', '19', '20', '25', '26', '27', '31', '32', '33', '34', '40', '41', '42', '49', '50'], (674370, '745619'): ['1', '2', '3', '4', '5', '10', '11', '12', '16', '17', '18', '19', '27', '28', '29', '33', '34', '35', '36', '40', '41', '42', '43'], (674370, '745613'): ['56', '57', '58', '59', '64', '65', '66', '67'], (695243, '745030'): ['68', '69', '70'], (695243, '745029'): ['61', '62', '63'], (695243, '745681'): ['78', '79', '80'], (695243, '745671'): ['71', '72', '73', '74'], (695243, '745757'): ['65', '66', '67'], (695243, '745755'): ['65', '69', '70', '71', '72'], (695243, '747050'): ['76', '77', '78'], (695243, '745673'): ['70', '71', '72'], (695243, '745670'): ['71', '72', '73'], (695243, '745664'): ['65', '66', '67', '71', '72', '73'], (695243, '746396'): ['60', '61', '62', '63', '67', '68', '69'], (695243, '746306'): ['68', '69', '70'], (695243, '745662'): ['69', '70', '71'], (695243, '745660'): ['74', '75', '76', '81', '82', '83', '85'], (695243, '745658'): ['67', '68', '69'], (695243, '745091'): ['63', '64', '65', '66', '67'], (695243, '745081'): ['69', '70', '71', '72', '78', '79', '80', '83'], (695243, '745654'): ['53', '54', '58', '59', '60', '61'], (695243, '745655'): ['59', '60', '61'], (695243, '745651'): ['63', '64', '65'], (695243, '745405'): ['74', '75'], (695243, '745892'): ['69', '70', '71', '75', '76', '77'], (695243, '745650'): ['73', '74', '75'], (695243, '745646'): ['72', '73', '74', '75'], (695243, '745649'): ['75', '76', '77'], (695243, '746212'): ['65', '66', '67', '68'], (695243, '747183'): ['79', '80', '81'], (695243, '745647'): ['69', '70', '71', '72', '73', '74'], (695243, '746934'): ['73', '74', '75', '76'], (695243, '745552'): ['73', '74', '75', '76'], (695243, '745635'): ['65', '66', '67'], (695243, '745630'): ['65', '66', '67'], (695243, '744897'): ['62', '63', '64', '65'], (695243, '744895'): ['73', '74', '75'], (695243, '745783'): ['82', '83', '84', '89', '90', '91', '92', '93'], (695243, '745621'): ['60', '61', '62'], (695243, '745620'): ['63', '64', '65', '66'], (695243, '745623'): ['70', '71', '72', '73'], (695243, '745618'): ['60', '64', '65', '66'], (695243, '746671'): ['78', '79', '80', '81', '82', '83', '84'], (695243, '746672'): ['71', '72', '73', '74', '75'], (695243, '744970'): ['75', '76', '77', '78', '84', '85', '86', '87', '88'], (695243, '745611'): ['69', '70', '71', '75', '76', '77', '78'], (695243, '746346'): ['79', '80', '81'], (695243, '746342'): ['75', '76', '77'], (695243, '746744'): ['65', '66', '67', '68'], (695243, '746830'): ['72', '73', '74', '75'], (695243, '746831'): ['73', '74', '75'], (695243, '745606'): ['55', '56', '57', '58'], (695243, '745605'): ['68', '69', '70', '71', '72'], (695243, '745609'): ['62', '68', '69', '70'], (695243, '745203'): ['79', '80', '81', '82', '83'], (519008, '745024'): ['58', '59', '60', '61'], (519008, '745679'): ['50'], (519008, '745676'): ['65', '66', '67'], (519008, '745674'): ['46'], (519008, '745671'): ['52'], (519008, '746651'): ['69', '70', '71', '72'], (519008, '746652'): ['59', '60', '61'], (519008, '745754'): ['54', '55', '56'], (519008, '745755'): ['62', '63', '64'], (519008, '747050'): ['47'], (519008, '747045'): ['67', '68', '69'], (519008, '745670'): ['52', '53', '54'], (519008, '745667'): ['70', '71', '72', '73', '74', '75', '76', '77', '78'], (519008, '745659'): ['35', '36', '37', '38', '44', '45', '46', '47'], (519008, '745255'): ['66', '67', '68'], (519008, '746393'): ['47', '48', '49'], (519008, '746397'): ['56', '57', '58'], (519008, '746310'): ['39'], (519008, '746308'): ['50', '51'], (519008, '745661'): ['45', '46', '50'], (519008, '745663'): ['74', '75', '76', '77'], (519008, '745091'): ['42', '43', '49'], (519008, '745084'): ['62', '63', '64'], (519008, '745081'): ['95'], (519008, '747111'): ['48', '49', '50', '51'], (519008, '745654'): ['46'], (519008, '745657'): ['57', '61', '62', '63', '64'], (519008, '745409'): ['73', '74', '75'], (519008, '745405'): ['57', '58', '59', '65'], (519008, '745891'): ['34', '38', '39', '40'], (519008, '745646'): ['47'], (519008, '745645'): ['65', '66', '67'], (519008, '745644'): ['62', '63'], (519008, '746213'): ['55', '56'], (519008, '746212'): ['50', '51'], (519008, '747183'): ['72'], (519008, '747182'): ['54', '55', '56'], (519008, '745639'): ['49', '50', '51', '52'], (519008, '745642'): ['84', '90', '91', '92', '93', '94', '95'], (519008, '746934'): ['54', '55', '56', '57'], (519008, '745552'): ['65', '66', '67'], (519008, '745549'): ['72', '74', '75', '76'], (519008, '745551'): ['57', '58', '59'], (519008, '745633'): ['49', '53', '54', '55', '56'], (519008, '745632'): ['69', '70', '71', '72', '73', '74'], (519008, '746200'): ['66'], (519008, '746195'): ['44'], (519008, '745306'): ['46'], (519008, '745631'): ['52', '53', '54'], (519008, '745630'): ['46', '47', '48', '49'], (519008, '744900'): ['48'], (519008, '745788'): ['75', '76', '77'], (519008, '745790'): ['48'], (519008, '745626'): ['59', '60'], (519008, '745623'): ['55'], (519008, '745617'): ['43', '44', '45'], (519008, '745618'): ['42'], (519008, '746672'): ['51', '52', '53'], (519008, '746670'): ['56', '57'], (519008, '744970'): ['61', '62', '63'], (519008, '745619'): ['44', '45', '46'], (519008, '745612'): ['39', '40', '42'], (519008, '745611'): ['36', '37'], (519008, '745613'): ['46', '47', '48', '49'], (519008, '746346'): ['57', '58'], (519008, '746342'): ['61'], (519008, '746337'): ['61', '62', '63'], (519008, '746741'): ['62', '63', '64', '65', '66'], (519008, '746830'): ['68'], (519008, '745606'): ['62', '63', '64', '65'], (519008, '745608'): ['60', '64', '65'], (519008, '745605'): ['41'], (519008, '745609'): ['46', '47'], (519008, '745203'): ['69', '70', '71', '72'], (671212, '745676'): ['1', '2', '3', '4', '10', '11', '12', '16', '17', '18', '19', '20', '26', '27', '28', '29', '33', '34', '35', '36'], (671212, '746650'): ['5', '6', '7', '14', '15', '16', '17', '18', '22', '23', '24', '29', '30', '31', '36', '37', '38', '39', '40', '41', '42', '43', '47', '48', '49'], (671212, '745753'): ['6', '7', '8', '9', '10', '14', '15', '16', '17', '18', '19', '23', '24', '25', '26', '27', '28'], (671212, '745672'): ['1', '2', '3', '4', '5', '6', '11', '12', '13', '18', '19', '20', '24', '25', '26', '27', '33', '34', '35'], (671212, '745667'): ['1', '2', '3', '4', '5', '6', '7'], (671212, '745788'): ['8', '9', '10', '14', '15', '16', '17', '18', '19', '28', '29', '30', '31', '32', '36', '37', '38', '39', '45', '46', '47', '48', '49'], (671212, '745620'): ['1', '2', '3', '4', '5', '11', '12', '13', '19', '20', '21', '22', '26', '27', '28', '29', '33', '34', '35', '36', '41', '42', '43', '44'], (671212, '745617'): ['1', '2', '3', '8', '9', '10', '11', '12', '13', '14', '15', '16', '20', '21', '22', '23', '27', '28', '29', '34', '35', '36', '37', '38'], (671212, '746670'): ['51', '52', '53', '54', '55'], (671212, '745607'): ['33', '34', '35', '36', '37', '44', '45', '46', '51', '52', '53', '54', '55'], (671212, '745201'): ['37', '38', '39', '44', '45', '46', '47', '48'], (613534, '746498'): ['51', '52', '53'], (613534, '745029'): ['48', '49'], (613534, '745679'): ['47', '48', '49'], (613534, '745677'): ['62', '63', '64', '65'], (613534, '745671'): ['58', '59', '60'], (613534, '746651'): ['58', '59', '60', '61', '62', '63'], (613534, '745754'): ['60', '61', '62', '63', '64'], (613534, '745755'): ['49', '50'], (613534, '747045'): ['60', '61', '62'], (613534, '745670'): ['55', '56', '57'], (613534, '745665'): ['48', '49'], (613534, '745669'): ['47', '48', '49'], (613534, '745664'): ['49', '55', '56', '57', '58'], (613534, '745262'): ['50', '51', '52'], (613534, '746396'): ['46', '47', '48'], (613534, '746310'): ['43', '44', '45', '46'], (613534, '746306'): ['41', '42', '43'], (613534, '745662'): ['53', '54'], (613534, '745661'): ['81', '82', '83'], (613534, '745658'): ['51', '52'], (613534, '745084'): ['56', '57'], (613534, '745081'): ['62', '63', '64', '65'], (613534, '747111'): ['79'], (613534, '747110'): ['52', '53', '54', '55', '56', '58'], (613534, '745654'): ['43', '44', '45'], (613534, '745655'): ['43', '44', '45', '46', '47', '48'], (613534, '745651'): ['69', '70', '71', '72', '73', '75', '76'], (613534, '745404'): ['65', '66', '67'], (613534, '745892'): ['52'], (613534, '745891'): ['58', '59', '67', '68', '69'], (613534, '745650'): ['57', '58', '59', '60'], (613534, '745646'): ['63', '64', '65', '66'], (613534, '745649'): ['53', '54', '55', '56', '57'], (613534, '745648'): ['45', '46', '47'], (613534, '746212'): ['47', '48', '49'], (613534, '747182'): ['58', '59', '60', '61'], (613534, '745641'): ['54', '55', '56', '57'], (613534, '745639'): ['57', '58', '59'], (613534, '746937'): ['77', '78', '79', '80', '81', '82'], (613534, '746934'): ['48'], (613534, '745549'): ['58', '59', '60', '61', '62'], (613534, '745640'): ['49', '50'], (613534, '745638'): ['53', '54', '55', '56'], (613534, '745635'): ['54', '55'], (613534, '746199'): ['56', '57', '58'], (613534, '746203'): ['51', '52', '53', '54'], (613534, '745309'): ['64'], (613534, '745631'): ['55', '56', '57'], (613534, '745627'): ['59', '60', '61', '62'], (613534, '745628'): ['59', '60', '61', '62'], (613534, '744900'): ['62', '63', '64', '65', '66'], (613534, '745788'): ['50', '51', '52'], (666205, '746650'): ['53', '54', '55', '56', '61', '62', '63', '64', '65', '66', '67'], (666205, '745753'): ['58', '59', '60', '61', '62', '63', '67', '68', '69'], (666205, '747045'): ['44', '45', '46', '47', '51', '52', '53', '54', '59'], (666205, '745666'): ['85', '86', '87', '88', '89'], (666205, '745659'): ['54', '55', '56', '58', '65', '66', '67', '68', '72', '73', '74', '75', '76', '77', '78'], (666205, '745255'): ['25', '26', '27', '28', '32', '33', '34', '39', '40', '41', '42', '43', '50', '51', '52', '53'], (666205, '746306'): ['60', '61', '62', '63'], (666205, '745661'): ['87', '88', '89', '90', '91'], (666205, '745663'): ['35', '40', '41', '42', '43', '44', '50', '51', '52', '53', '58', '59', '60', '66', '67', '68', '69'], (666205, '746935'): ['47', '48', '49', '50', '56', '57', '58'], (666205, '745551'): ['77', '78', '79'], (666205, '745634'): ['56', '57', '58', '64', '65', '66', '71', '72', '73', '74', '75'], (666205, '746195'): ['66', '67', '68', '69', '70', '74', '75', '76'], (666205, '745636'): ['69', '70', '71', '72', '73', '74', '75', '76', '77', '78'], (666205, '746740'): ['43', '44', '45', '54', '55', '56', '60', '61', '62'], (666205, '745608'): ['40', '42', '43', '47', '48', '49', '56'], (666205, '745201'): ['56', '57', '58', '62', '63', '64', '65'], (676254, '745362'): ['57', '58', '59'], (676254, '745357'): ['52', '53', '54', '55', '56'], (676254, '745355'): ['51', '52', '53', '54'], (676254, '745112'): ['64', '65', '66', '67'], (676254, '746080'): ['66', '67', '68'], (676254, '746081'): ['46', '47', '48', '49', '50', '51'], (676254, '745356'): ['63', '64', '65'], (676254, '745354'): ['41', '42', '43'], (676254, '745351'): ['50', '51', '52'], (676254, '745346'): ['1', '2', '3', '4', '5'], (676254, '745347'): ['46', '47', '48', '49', '50', '51'], (676254, '745349'): ['55', '56', '57', '58'], (676254, '746969'): ['45', '46', '47', '48'], (676254, '745587'): ['68', '69', '70'], (676254, '746559'): ['66', '67', '68'], (676254, '746562'): ['63', '64', '65'], (676254, '745344'): ['43', '44', '45', '46'], (676254, '745345'): ['60', '61', '62', '66', '67', '68', '69'], (676254, '745335'): ['62', '63', '64'], (676254, '745339'): ['47', '48', '49', '50', '51'], (676254, '745503'): ['59', '60', '61'], (676254, '745502'): ['76', '77', '78'], (676254, '745822'): ['66', '67', '68'], (676254, '745824'): ['59', '60', '61', '62'], (676254, '745338'): ['67', '68', '69'], (676254, '745332'): ['55', '56', '57', '58', '59', '60'], (676254, '745330'): ['57', '58', '59', '60'], (676254, '747195'): ['69', '70', '71', '72'], (676254, '745001'): ['48', '54', '55', '56'], (676254, '745334'): ['54', '55', '56', '57'], (676254, '745328'): ['55', '56', '57'], (676254, '745329'): ['54', '55', '56', '57', '58'], (676254, '745321'): ['85', '86'], (676254, '746864'): ['70', '71', '72'], (676254, '745164'): ['61', '62', '63'], (676254, '745322'): ['43', '44', '48', '49', '50'], (676254, '745324'): ['59', '60', '61', '62'], (676254, '745317'): ['63', '64', '69', '70', '71', '72'], (676254, '747102'): ['56', '57', '58'], (676254, '746620'): ['54', '55', '59', '60', '61', '62'], (676254, '746616'): ['54', '58', '59', '60', '61'], (676254, '745316'): ['45', '46', '47', '48', '49', '50'], (676254, '745312'): ['45', '46', '54', '55', '56', '57', '58'], (676254, '745314'): ['62', '63'], (676254, '746531'): ['58', '59', '60', '61', '63', '64'], (676254, '746528'): ['50', '51', '52', '53'], (676254, '746124'): ['52', '53', '54'], (676254, '746119'): ['46', '47', '48'], (676254, '745315'): ['66'], (676254, '745307'): ['49', '50', '51', '52'], (676254, '745309'): ['65', '66', '67'], (676254, '746680'): ['42', '43', '44', '45'], (676254, '744821'): ['50', '54', '55', '56', '57'], (676254, '744822'): ['61', '62'], (676254, '744819'): ['63', '64', '65'], (676254, '745304'): ['67', '68', '69', '70', '71'], (676254, '745300'): ['58', '59', '60', '61'], (676254, '745621'): ['63', '64', '65', '66', '72', '73', '74', '75', '76', '77'], (676254, '745295'): ['67', '68', '69'], (676254, '745217'): ['71', '72', '73', '77', '78', '79'], (676254, '745218'): ['79', '80', '81', '82'], (676254, '745946'): ['74', '75', '76', '77'], (676254, '745299'): ['63', '64', '65'], (676254, '745289'): ['61', '67', '68', '69', '70'], (676254, '745372'): ['71', '72', '73', '74'], (676254, '745368'): ['71', '75', '76', '77'], (676254, '745287'): ['60', '61', '62', '63'], (676254, '746988'): ['73', '74', '75'], (676254, '746985'): ['73', '74', '75', '76'], (676254, '746253'): ['63', '64', '65', '66'], (676254, '747150'): ['76', '77', '78'], (676254, '745281'): ['69', '70', '71', '72', '73'], (643511, '746076'): ['59', '60', '61', '62'], (643511, '745356'): ['50', '51', '52', '53'], (643511, '745354'): ['64', '65', '66'], (643511, '745351'): ['58', '59', '60'], (643511, '745348'): ['64', '65', '66', '67'], (643511, '745347'): ['55', '56', '57', '58'], (643511, '745350'): ['54', '55', '56', '57', '58'], (643511, '746969'): ['61', '62', '63'], (643511, '745585'): ['65', '66', '67', '68'], (643511, '746562'): ['70', '71', '72', '73'], (643511, '745340'): ['67', '68', '69'], (643511, '745344'): ['61', '62', '63', '64'], (643511, '745343'): ['59', '60', '61', '62'], (643511, '745341'): ['59', '60', '61', '62', '63'], (643511, '745335'): ['72', '73', '74', '75'], (643511, '745339'): ['68', '69', '70', '71', '72'], (643511, '745503'): ['67', '68', '69', '70'], (643511, '745502'): ['68', '69', '70'], (643511, '745500'): ['76', '77', '78', '79'], (643511, '745824'): ['66', '67', '68', '69', '70', '72'], (643511, '745337'): ['52', '53', '54', '55'], (643511, '745333'): ['72', '73', '74'], (643511, '745330'): ['66', '67', '68'], (643511, '747194'): ['57', '58', '59'], (643511, '747195'): ['79', '80', '81', '82'], (643511, '745008'): ['67', '68', '69'], (643511, '745001'): ['60', '61', '62'], (643511, '745328'): ['62', '63', '64'], (643511, '745321'): ['67', '68', '69', '70'], (643511, '746867'): ['59', '60', '61', '62', '63', '64', '65'], (643511, '745164'): ['67', '68', '69'], (643511, '745165'): ['65'], (643511, '745322'): ['55', '56', '57'], (643511, '745325'): ['59', '60', '61', '62', '63'], (643511, '745319'): ['56', '57', '58', '59'], (643511, '745320'): ['78', '79'], (643511, '747102'): ['64', '65', '66', '67', '68'], (643511, '747103'): ['62', '63', '64'], (643511, '746620'): ['66', '67', '68', '69', '70'], (643511, '745313'): ['58', '59', '60', '61', '62'], (643511, '745312'): ['65', '66', '67', '68'], (643511, '745314'): ['52', '53', '54'], (643511, '746531'): ['68', '69', '70', '71'], (643511, '746528'): ['58', '59', '60', '61'], (643511, '746119'): ['53', '54', '55', '56'], (643511, '746117'): ['73', '74', '75', '76', '77', '78'], (643511, '745307'): ['59', '60', '61'], (643511, '746680'): ['53', '64', '65', '66'], (643511, '744821'): ['67'], (643511, '744822'): ['66', '67', '68'], (643511, '745308'): ['61', '62', '63'], (643511, '745304'): ['59', '60', '61'], (643511, '745300'): ['51', '52', '53'], (643511, '745298'): ['61'], (643511, '745621'): ['57', '58', '59'], (643511, '745294'): ['63', '64', '65', '66'], (643511, '745295'): ['60', '61', '62', '63'], (643511, '745217'): ['65', '66', '67', '68', '69', '70'], (643511, '745212'): ['62', '63', '64', '65', '66'], (643511, '745946'): ['62', '66', '67', '68', '69'], (643511, '745299'): ['52', '53', '54'], (643511, '745289'): ['51', '52', '53', '54'], (643511, '745372'): ['57', '58', '59'], (643511, '745368'): ['61', '62', '63'], (643511, '745287'): ['53', '54', '55', '56'], (643511, '746987'): ['58', '59', '60', '61'], (643511, '746988'): ['66', '67', '68'], (643511, '746256'): ['59', '60', '61', '62'], (643511, '746253'): ['53', '54', '55', '56', '58'], (643511, '745281'): ['61', '62', '63', '64', '65'], (663855, '746562'): ['4', '5', '6', '18', '19', '20', '21', '22', '28', '29', '30', '34', '35', '36', '37', '38', '42', '43', '44', '45', '46', '47', '48'], (663855, '745343'): ['1', '2', '3', '4', '5', '10', '11', '12', '19', '20', '21', '22', '26', '27', '28', '29', '33', '34', '35', '36', '37'], (663855, '745336'): ['1', '2', '3', '4', '8', '9', '10', '16', '17', '18', '22', '23', '24', '30', '31', '32', '33', '34', '35'], (663855, '745823'): ['4', '5', '6', '7', '11', '12', '13', '14', '18', '19', '20', '21', '25', '26', '27', '28', '29', '34', '35', '36'], (663855, '745331'): ['1', '2', '3', '4', '9', '10', '11', '12', '17', '18', '19', '20', '21', '22', '26', '27', '28', '32', '33', '34', '38', '39', '40'], (663855, '747195'): ['6', '7', '8', '9', '10', '11', '16', '17', '18', '19', '32', '33', '34', '35', '36', '41', '42', '43', '44', '45', '46'], (663855, '745327'): ['1', '2', '3', '8', '9', '10', '11', '16', '17', '18', '19', '24', '25', '26', '27', '31', '32', '33', '34', '35'], (663855, '746868'): ['4', '5', '6', '11', '12', '13', '14', '20', '21', '22', '23', '29', '30', '31', '32', '33', '34', '40', '41', '42', '43'], (663855, '745163'): ['8', '9', '10', '11', '12', '13', '18', '19', '20', '26', '27', '28', '29', '30', '31', '35', '36', '37', '38', '39', '40'], (663855, '745324'): ['1', '2', '3', '7', '8', '9', '10', '15', '16', '17', '18', '19', '20', '21', '22', '28', '29', '30', '34', '35', '36', '37'], (663855, '747100'): ['4', '5', '6', '7', '12', '13', '14', '15', '16', '17', '21', '22', '23', '24', '29', '30', '31', '32', '37', '38', '39', '40', '41', '42'], (663855, '745313'): ['1', '2', '3', '4', '5', '6', '7', '13', '14', '15', '19', '20', '21', '26', '27', '28', '29', '30', '31', '36', '37', '38'], (663855, '746118'): ['5', '6', '7', '8', '9', '14', '15', '16', '17', '23', '24', '25', '26', '27', '32', '33', '34', '35', '36'], (663855, '745305'): ['43', '44', '45', '46', '47', '55', '56', '57', '58'], (663855, '746681'): ['44', '45', '46', '47', '53', '54', '55'], (663855, '744819'): ['49', '50', '51', '52'], (663855, '745303'): ['71', '72', '73', '74', '75'], (663855, '745302'): ['69', '70', '71', '72'], (663855, '745626'): ['52', '53', '54', '55', '56'], (663855, '745294'): ['71', '72', '73', '74', '75'], (663855, '745218'): ['61', '62', '63', '64', '65'], (663855, '745284'): ['55', '56', '57', '58', '62', '63', '64', '65', '66'], (666808, '745347'): ['64', '65', '66', '67', '68'], (666808, '745350'): ['62', '63', '64', '65'], (666808, '745349'): ['62', '63', '64', '65', '66', '67'], (666808, '746969'): ['67', '68', '69'], (666808, '746559'): ['72', '73', '74', '75', '76'], (666808, '746562'): ['78', '79', '80', '81', '82'], (666808, '745345'): ['70', '74', '75', '76', '77', '78'], (666808, '745343'): ['66', '67', '68'], (666808, '745341'): ['69', '70', '71', '72', '73'], (666808, '745336'): ['68', '69', '70'], (666808, '745503'): ['77', '78', '79', '80', '81', '82'], (666808, '745502'): ['88', '89', '90', '91'], (666808, '745822'): ['73', '74', '75', '76', '77', '78', '79'], (666808, '745823'): ['82', '83', '84'], (666808, '745338'): ['74', '75', '76', '77'], (666808, '745330'): ['72', '73', '74', '75', '76', '77', '78', '79'], (666808, '745008'): ['73', '74', '75'], (666808, '745001'): ['68', '69', '70', '71', '72'], (666808, '745334'): ['62', '63', '64', '65', '66'], (666808, '745328'): ['69', '70', '71'], (666808, '745321'): ['78', '79', '80', '81', '82', '83', '84'], (666808, '746868'): ['83', '84', '85'], (666808, '745322'): ['65', '66', '67'], (666808, '745325'): ['67', '68', '69', '70'], (666808, '745319'): ['65', '66', '67', '68'], (666808, '745317'): ['77', '78', '79', '80', '81'], (666808, '747102'): ['73', '74', '75'], (666808, '747103'): ['68', '69', '70', '71'], (666808, '746620'): ['74', '75', '76', '77'], (666808, '745312'): ['72', '73', '74', '75', '76'], (666808, '745314'): ['58', '59', '60', '61'], (666808, '746528'): ['65', '66', '67', '68'], (666808, '745315'): ['74', '75', '76', '77', '78'], (666808, '745307'): ['66', '67', '68', '69'], (666808, '745305'): ['74', '75', '76', '77', '78'], (666808, '744821'): ['71', '72', '73'], (666808, '744822'): ['72', '73', '74', '75', '76'], (666808, '744819'): ['73', '74', '75', '76', '77', '78'], (666808, '745218'): ['66', '67', '73', '74', '75'], (666808, '745946'): ['46', '47', '48', '49', '50', '51'], (666808, '745296'): ['64', '65', '66'], (666808, '745292'): ['55', '56', '57', '58', '59'], (666808, '745290'): ['59', '60', '61', '62', '63'], (666808, '745289'): ['58', '59', '60'], (666808, '745372'): ['63', '64', '65'], (666808, '745368'): ['67', '68', '69', '70'], (666808, '745288'): ['44', '45', '46', '47'], (666808, '745286'): ['67', '68', '69', '70', '71'], (666808, '745287'): ['68', '69', '70', '71', '72'], (666808, '746256'): ['67', '68', '69', '70', '71', '72'], (666808, '747150'): ['69', '70', '71', '72'], (666808, '747144'): ['59', '60', '61'], (666808, '745280'): ['61', '62', '63'], (666808, '745281'): ['46', '47', '48', '49'], (573124, '745589'): ['67', '68', '69', '70'], (573124, '745587'): ['60', '61', '62', '63'], (573124, '746562'): ['56', '57'], (573124, '746563'): ['67', '68', '69', '70', '71'], (573124, '745344'): ['37', '38', '39'], (573124, '745343'): ['73', '74', '75', '76', '77'], (573124, '745339'): ['83', '84', '85'], (573124, '745336'): ['58', '59', '60', '61', '62'], (573124, '745500'): ['41', '42', '47', '48', '49', '50'], (573124, '745823'): ['43', '44', '45'], (573124, '745337'): ['9', '10', '11', '15', '16', '17', '18'], (573124, '745331'): ['41', '42', '43'], (573124, '745330'): ['80'], (573124, '747196'): ['65', '72', '73', '74', '75'], (573124, '745003'): ['56', '57', '58', '59'], (573124, '745327'): ['36', '37', '41', '42', '43'], (573124, '745326'): ['61', '62', '63', '64', '65'], (573124, '745321'): ['56', '57', '58', '59', '60'], (573124, '746864'): ['46', '47', '48', '49'], (573124, '745164'): ['45', '46', '54', '55', '56'], (573124, '745165'): ['61', '62', '63', '64'], (573124, '745322'): ['35', '36', '41', '42'], (573124, '745317'): ['54', '55', '61', '62'], (573124, '745320'): ['48', '49', '50'], (573124, '746620'): ['42', '43', '44', '45'], (573124, '746615'): ['62', '63', '64'], (573124, '745313'): ['66', '67', '68'], (573124, '745311'): ['45', '46', '47'], (573124, '746118'): ['49', '50', '51', '52'], (573124, '746119'): ['76', '77', '78', '79'], (573124, '745310'): ['50', '51', '52'], (573124, '745305'): ['36', '37', '38'], (573124, '745309'): ['56', '57', '58', '59'], (573124, '746681'): ['60', '61', '62', '63'], (573124, '744821'): ['63', '64', '65', '66'], (573124, '744817'): ['57', '61', '62', '63'], (573124, '744819'): ['87', '88', '89'], (573124, '745303'): ['64', '65', '66'], (573124, '745300'): ['65', '66', '67', '68'], (573124, '745301'): ['63', '64', '65', '66', '67'], (573124, '745217'): ['48', '49', '50'], (573124, '745212'): ['47', '48', '53', '54', '55'], (573124, '745296'): ['39', '40', '41', '42'], (573124, '745290'): ['64', '68', '69', '70', '71'], (573124, '745368'): ['57', '58', '59', '60'], (573124, '745291'): ['60', '61', '62', '63', '64'], (573124, '745286'): ['56', '60', '61'], (573124, '745287'): ['46', '47', '48'], (573124, '746985'): ['46', '47', '48', '49'], (573124, '747144'): ['42', '43', '44'], (573124, '745280'): ['39', '40', '41', '46', '47', '48', '49'], (573124, '745282'): ['34', '35'], (676775, '745112'): ['5', '6', '7', '8', '14', '15', '16', '17', '18', '19', '20', '25', '26', '27', '28', '29', '34', '35', '36', '37', '43', '44', '45', '46'], (676775, '746076'): ['4', '5', '6', '7', '13', '14', '15', '16', '20', '21', '22', '26', '27', '28', '32', '33', '34', '41', '42', '43', '44', '45'], (676775, '745351'): ['1', '2', '3', '7', '8', '9', '17', '18', '19', '26', '27', '28', '32', '33', '34', '35', '40', '41', '42', '48', '49'], (676775, '745349'): ['1', '2', '3', '4', '8', '9', '10', '14', '15', '16', '25', '26', '27', '31', '32', '33', '34', '35', '39', '40', '41', '42'], (676775, '745583'): ['5', '6', '7', '8', '9', '10', '11', '12'], (676775, '746563'): ['6', '7', '8', '9', '13', '14', '15', '20', '21', '22', '29', '30', '31', '32', '33', '34', '35', '36', '37'], (676775, '745342'): ['1', '2', '3', '4', '9', '10', '11', '15', '16', '17', '23', '24', '25', '26', '27', '28', '29', '30', '34', '35'], (676775, '745003'): ['4', '5', '6', '7', '8', '9', '10', '14', '15', '16', '17', '18', '25', '26', '27', '28', '33', '34', '35', '39', '40', '41'], (676775, '745329'): ['1', '2', '3', '4', '12', '13', '14', '15', '20', '21', '22', '23', '27', '28', '29', '30', '34', '35', '36', '37', '45', '46', '47', '48', '49'], (676775, '745164'): ['24', '25', '26', '27', '28'], (663546, '745589'): ['45', '46', '47', '52', '53', '54'], (663546, '745587'): ['42', '43', '44', '45', '49', '50', '51', '52'], (663546, '746562'): ['52', '53', '54', '55'], (663546, '745344'): ['68', '69', '70', '71'], (663546, '745345'): ['50', '51', '52', '53', '57', '58', '59'], (663546, '745341'): ['50', '51', '52'], (663546, '745335'): ['30', '31', '32', '33', '39', '40', '41', '42', '43'], (663546, '745336'): ['44', '45', '46'], (663546, '745502'): ['33', '34', '35', '36', '37', '44', '45', '46'], (663546, '745823'): ['46', '51', '52', '53'], (663546, '745337'): ['61', '62', '63', '69', '70', '72'], (663546, '745331'): ['56', '57', '58', '62', '63', '64'], (663546, '747194'): ['44', '49', '50', '51', '52', '53'], (663546, '747195'): ['47', '51', '52', '53', '54'], (663546, '745334'): ['49', '50'], (663546, '745327'): ['48', '49', '50', '55', '56', '57'], (663546, '745326'): ['82', '83', '84'], (663546, '745329'): ['70', '71', '72'], (663546, '745321'): ['43', '44', '45'], (663546, '746864'): ['30', '31', '32', '33', '37', '38', '39'], (663546, '745163'): ['44', '45', '46', '47', '48', '52', '53', '54', '55', '56'], (663546, '745322'): ['21', '22', '26', '27', '28', '32', '33', '34'], (663546, '745324'): ['67', '68', '69', '70', '71'], (663546, '745317'): ['86', '87', '88', '89', '97', '98', '99', '100', '101', '102', '103', '104'], (663546, '747100'): ['55', '56', '57', '61', '62', '63', '64'], (663546, '746615'): ['68', '69', '70', '71'], (663546, '746616'): ['44', '48', '49', '50', '51', '52', '53'], (663546, '745318'): ['49', '50', '51', '52', '53', '54'], (663546, '745311'): ['68', '69', '70', '71', '72', '73'], (663546, '746527'): ['62', '63', '64'], (663546, '746118'): ['37', '41', '42', '43', '44', '45'], (663546, '746117'): ['56', '57', '58', '59', '60'], (663546, '745310'): ['44', '45', '46'], (663546, '746681'): ['34', '38', '39', '40'], (663546, '744817'): ['64', '65', '66', '70', '71', '72', '76', '77', '78', '79', '80'], (663546, '744819'): ['45'], (663546, '745303'): ['47', '48', '55', '56', '57', '58', '59'], (663546, '745302'): ['46', '47', '48', '49'], (663546, '745301'): ['9', '15', '16', '17', '21', '22', '23', '24', '31', '32', '33'], (663546, '745295'): ['54'], (663546, '745217'): ['38', '39', '40', '41', '42'], (663546, '745212'): ['28', '29', '30', '35', '36', '37', '38', '39', '43', '44', '45', '46'], (663546, '745943'): ['39'], (663546, '745945'): ['35', '36', '37', '38', '43', '44', '45'], (663546, '745293'): ['55', '56', '57', '62', '63', '64'], (663546, '745371'): ['42', '43', '49', '50', '51', '52'], (663546, '745291'): ['53', '54', '55', '56'], (663546, '745285'): ['70', '71', '72', '73', '80', '81', '82'], (663546, '745284'): ['35', '36', '37', '38', '39', '44', '45', '46', '47', '48', '49', '50', '51'], (663546, '746987'): ['51', '52', '53'], (663546, '746988'): ['57', '58', '59', '60', '61', '62'], (663546, '746256'): ['47'], (663546, '746253'): ['47', '48', '49'], (663546, '747144'): ['30', '34', '35', '36', '37', '38'], (663546, '745281'): ['37', '38', '39', '40', '41', '42'], (663158, '746164'): ['76', '77', '78'], (663158, '746002'): ['82', '83'], (663158, '745434'): ['69', '70', '71'], (663158, '746565'): ['66', '67', '68', '69'], (663158, '746567'): ['69', '70', '71'], (663158, '745426'): ['73', '74', '75'], (663158, '745423'): ['67', '68', '69'], (663158, '746886'): ['71', '72', '76', '77', '78'], (663158, '746879'): ['66', '67', '68'], (663158, '745421'): ['58', '59', '60'], (663158, '745419'): ['66', '67', '68', '69'], (663158, '747122'): ['66', '67', '68', '69', '70'], (663158, '747119'): ['80', '81', '82'], (663158, '746712'): ['70', '71', '72', '73'], (663158, '746714'): ['82', '83', '84', '85'], (663158, '745413'): ['68', '72', '73', '74', '75', '76'], (663158, '745416'): ['67', '68', '69'], (663158, '746303'): ['84', '85'], (663158, '746305'): ['76', '77', '78', '79'], (663158, '745409'): ['76', '77', '78'], (663158, '745404'): ['64', '71', '72', '73'], (663158, '745405'): ['70', '71', '72', '73'], (663158, '745563'): ['66', '70', '71', '72', '73'], (663158, '745561'): ['74', '75', '76'], (663158, '745402'): ['71'], (663158, '745397'): ['64', '65', '66', '67'], (663158, '745399'): ['71', '72', '73', '74', '75'], (663158, '744998'): ['74', '78', '79', '80'], (663158, '744996'): ['68', '69', '70', '71'], (663158, '745398'): ['72', '73', '74', '75', '76'], (663158, '745394'): ['62', '63', '64', '65'], (663158, '745392'): ['65', '66', '67', '68', '69'], (663158, '746610'): ['62', '66', '67', '68'], (663158, '744828'): ['67', '68', '69', '70'], (663158, '747010'): ['68', '69', '70', '76', '77', '78'], (663158, '745389'): ['75', '76', '77'], (663158, '745388'): ['60', '61', '62'], (663158, '745469'): ['86', '87', '88', '89', '90'], (663158, '745466'): ['77', '78', '79', '80', '81'], (663158, '746026'): ['84', '85', '86'], (663158, '745383'): ['62', '63', '64', '65', '66'], (663158, '745382'): ['64', '65', '66', '67'], (663158, '745380'): ['66', '67', '68', '69', '70'], (663158, '745374'): ['70', '71', '72'], (663158, '745376'): ['67', '68', '69'], (663158, '745133'): ['75', '76', '77', '78', '79'], (663158, '745134'): ['75', '76', '77', '78', '79'], (663158, '745048'): ['76', '77', '78', '79', '80'], (663158, '745373'): ['64', '65', '66'], (663158, '745370'): ['76', '77', '78', '79'], (663158, '745369'): ['61', '62', '63', '64', '65', '66'], (663158, '745210'): ['69', '75', '76', '77', '78'], (663158, '745287'): ['64', '65', '66', '67'], (663158, '745367'): ['69', '70', '71'], (663158, '745366'): ['66', '67', '68', '69'], (663158, '745364'): ['64', '65', '66', '67', '68'], (663158, '745358'): ['60', '61', '62', '63'], (663158, '746093'): ['70', '71', '72', '73'], (663158, '747145'): ['76', '77', '78', '79'], (670970, '745425'): ['47', '48', '49', '50', '54', '55', '56', '57', '62', '63', '64'], (670970, '745426'): ['44', '45', '46', '47', '48'], (670970, '747209'): ['68', '69', '70', '71'], (670970, '746885'): ['36', '37', '38', '44', '45', '46', '47', '48'], (670970, '745422'): ['39', '40', '41'], (670970, '745420'): ['33', '37', '38', '39', '44', '45', '46', '52', '53', '54'], (670970, '747120'): ['67', '68', '69'], (670970, '746716'): ['31', '32', '33', '34', '39', '40', '41'], (670970, '745416'): ['40', '41', '42', '49', '50', '51', '52'], (670970, '746305'): ['49', '50', '51', '52', '57', '58', '59'], (670970, '746224'): ['49', '50', '51', '57', '58'], (670970, '745408'): ['72', '73', '75', '76'], (670970, '745409'): ['57', '58', '59'], (670970, '745404'): ['54', '55', '56'], (670970, '745810'): ['56', '57', '58', '59', '60'], (670970, '745567'): ['43', '44', '48', '49', '50', '51'], (670970, '745563'): ['50', '51', '52'], (670970, '745401'): ['41', '48', '49', '50'], (670970, '745402'): ['58', '59', '60'], (670970, '745397'): ['49', '50', '51', '52', '53', '54'], (670970, '745400'): ['70', '71', '72'], (670970, '746938'): ['54', '55', '56', '57', '58'], (670970, '744998'): ['57', '58', '64', '65', '66'], (670970, '744996'): ['60', '61', '62', '63'], (670970, '745395'): ['70', '71', '72', '73'], (670970, '745394'): ['46', '47', '48', '49'], (670970, '745392'): ['56', '57', '58', '59'], (670970, '744828'): ['52', '53'], (670970, '747010'): ['57', '58', '64', '65', '66', '67'], (670970, '747008'): ['72', '73', '74', '75'], (670970, '745389'): ['51', '52', '53', '54', '55', '56'], (670970, '745386'): ['49'], (670970, '745384'): ['44', '51', '52', '53', '54', '55'], (670970, '745467'): ['39', '44', '45', '46'], (670970, '745469'): ['101', '102', '103'], (670970, '746026'): ['75', '76', '77', '78', '79'], (670970, '745381'): ['69', '70', '71', '72', '73', '74', '75'], (670970, '746515'): ['56'], (670970, '746516'): ['65', '66', '67', '68'], (670970, '745375'): ['68', '69', '70'], (670970, '745377'): ['44', '45', '46', '47'], (670970, '745133'): ['59', '60', '61', '62', '63'], (670970, '745135'): ['63', '64', '65', '66', '67', '68'], (670970, '745052'): ['88', '89', '90'], (670970, '745370'): ['50', '51', '52', '53'], (670970, '745371'): ['60', '61'], (670970, '745214'): ['41', '42', '43', '44', '45'], (670970, '745284'): ['59', '60', '61'], (670970, '745287'): ['73', '74', '75'], (670970, '745366'): ['73', '74', '75'], (670970, '745364'): ['73', '74', '75'], (670970, '745358'): ['44', '45', '46'], (670970, '746093'): ['47', '48', '49', '50'], (670970, '746095'): ['53', '54', '59'], (670970, '747145'): ['59', '60', '61'], (592094, '745108'): ['75', '76', '77', '78', '79'], (592094, '745760'): ['62', '63', '64'], (592094, '745099'): ['55', '56', '57', '62', '63', '64'], (592094, '746803'): ['79', '80', '81', '82'], (592094, '745999'): ['62', '63', '64', '65', '66', '67'], (592094, '745101'): ['76', '77', '78', '79', '80'], (592094, '745100'): ['54', '55', '56', '57', '58'], (592094, '745094'): ['55', '56', '57'], (592094, '745096'): ['53', '54'], (592094, '746961'): ['71', '72', '73', '74'], (592094, '746962'): ['71', '72'], (592094, '746959'): ['80', '81', '82', '83'], (592094, '744938'): ['68', '69', '70', '71', '72'], (592094, '745090'): ['54', '55', '56', '57', '59'], (592094, '745086'): ['60', '61', '62'], (592094, '745091'): ['60', '61', '62'], (592094, '745084'): ['58', '59', '60', '61'], (592094, '746056'): ['54', '55', '56'], (592094, '745082'): ['52', '53', '54', '55', '56'], (592094, '745077'): ['61', '62', '63', '64'], (592094, '747112'): ['68', '69', '70', '71'], (592094, '747109'): ['62', '63', '64', '65', '66'], (592094, '745893'): ['67', '68', '69', '70'], (592094, '745887'): ['76', '77', '78'], (592094, '745485'): ['60', '61', '62', '63', '64'], (592094, '745073'): ['54', '55', '56', '57'], (592094, '745070'): ['60', '61', '62'], (592094, '745072'): ['50', '51', '52'], (592094, '746282'): ['65', '66', '67'], (592094, '746285'): ['61', '62', '63', '70', '71', '72'], (592094, '745074'): ['62', '63', '64'], (592094, '745065'): ['55', '56', '57', '62'], (592094, '745064'): ['68', '69', '70'], (592094, '745066'): ['53', '54', '55'], (592094, '745722'): ['55', '56', '57', '58', '59', '60'], (592094, '744904'): ['62', '63', '64'], (592094, '744905'): ['57', '58', '59', '61', '62', '63', '64'], (592094, '745068'): ['56', '57', '58'], (592094, '745386'): ['62', '63', '64'], (592094, '745388'): ['42', '43', '44', '45'], (592094, '745467'): ['47', '53', '54', '55', '56', '57'], (592094, '745466'): ['65', '66', '67', '68'], (592094, '746032'): ['66', '67', '68'], (592094, '746025'): ['67', '68', '69'], (592094, '745382'): ['48', '49', '50', '51'], (592094, '746515'): ['61', '62', '63', '64'], (592094, '745380'): ['52', '53', '54'], (592094, '745378'): ['58', '59', '60'], (592094, '745376'): ['49', '50', '51', '52'], (592094, '745132'): ['70', '71', '72', '73'], (592094, '745134'): ['67', '68', '69', '70'], (592094, '745048'): ['52', '53', '54'], (592094, '745373'): ['46', '47', '48', '49'], (592094, '745370'): ['70', '71', '72'], (592094, '745369'): ['49', '50'], (592094, '745368'): ['64', '65', '66'], (592094, '745210'): ['61', '65', '66', '67', '68'], (592094, '745286'): ['64', '65', '66'], (592094, '745287'): ['57', '58', '59'], (592094, '745367'): ['49', '50', '51', '52'], (592094, '745366'): ['56', '57', '58', '59', '60', '61'], (592094, '745364'): ['49', '50', '51'], (592094, '745360'): ['61', '62', '63', '64', '65'], (592094, '746093'): ['63', '64', '65', '66'], (592094, '747145'): ['62', '68', '69', '70', '71'], (656945, '746073'): ['73', '74', '75', '77'], (656945, '746072'): ['60', '61'], (656945, '746147'): ['62', '63', '64'], (656945, '746068'): ['62', '63', '64', '65', '66', '70', '71', '72'], (656945, '746473'): ['56', '57', '58', '59', '60'], (656945, '746472'): ['66', '67', '68', '69', '70'], (656945, '746067'): ['88', '89', '90'], (656945, '746066'): ['73', '74', '75', '76'], (656945, '746063'): ['56', '57', '58', '59'], (656945, '747197'): ['73', '74', '75'], (656945, '745414'): ['79', '80', '81'], (656945, '746060'): ['75', '76', '77', '78', '79'], (656945, '746053'): ['58', '65', '66', '67'], (656945, '745815'): ['61', '62', '63', '68', '69', '70'], (656945, '745809'): ['65', '66', '67'], (656945, '746055'): ['67', '68', '69', '70'], (656945, '746049'): ['77', '78', '79'], (656945, '746050'): ['66', '67', '68', '69'], (656945, '746052'): ['59', '60', '61'], (656945, '746051'): ['68', '69', '70', '71', '72', '73'], (656945, '746293'): ['66', '67', '68', '69'], (656945, '745565'): ['79', '80', '81'], (656945, '745559'): ['68', '69', '70'], (656945, '746043'): ['61', '62', '63'], (656945, '746044'): ['70', '71', '72'], (656945, '746367'): ['68', '69', '70'], (656945, '746686'): ['64', '70', '71', '72', '74'], (656945, '746040'): ['68', '75', '76', '77'], (656945, '746042'): ['72', '73', '74'], (656945, '746036'): ['68', '69', '70', '71'], (656945, '746035'): ['69', '70', '71'], (656945, '745964'): ['73', '74', '75', '76', '77'], (656945, '745388'): ['52', '53', '54', '55'], (656945, '745467'): ['58', '63', '64', '65', '66'], (656945, '745469'): ['91', '98', '99', '100'], (656945, '746032'): ['76', '77', '78', '79'], (656945, '746026'): ['92', '93', '94', '95'], (656945, '745383'): ['53', '54', '55', '56'], (656945, '745382'): ['55', '56', '57', '58'], (656945, '746515'): ['69', '70', '71', '72'], (656945, '745380'): ['60', '61', '62'], (656945, '745374'): ['56', '57', '58', '59', '60', '61'], (656945, '745376'): ['58', '59', '60'], (656945, '745133'): ['68', '69', '70', '71'], (656945, '745132'): ['78', '79', '80'], (656945, '745052'): ['79'], (656945, '745048'): ['66', '67', '68', '69', '70'], (656945, '745373'): ['53', '54', '55', '56'], (656945, '745370'): ['60', '61', '62', '63'], (656945, '745369'): ['54', '55', '56', '57'], (656945, '745368'): ['72', '73', '74'], (656945, '745210'): ['56', '57', '58', '59', '60'], (656945, '745286'): ['57', '58', '59'], (656945, '745287'): ['49', '50', '51', '52'], (656945, '745367'): ['57', '58', '59', '60', '61', '62'], (656945, '745365'): ['64', '65'], (656945, '745364'): ['56', '57', '58', '59'], (656945, '745360'): ['66', '72', '73', '74', '75', '76'], (656945, '746093'): ['56', '57', '58'], (656945, '746088'): ['54', '55', '56', '57', '58', '60'], (593974, '746886'): ['61', '62', '63', '67'], (593974, '746879'): ['59', '60', '61'], (593974, '745419'): ['57', '58', '59', '60', '61'], (593974, '745418'): ['72', '73'], (593974, '747121'): ['62', '63', '64', '65'], (593974, '746712'): ['60', '64', '65', '66'], (593974, '745413'): ['64', '65', '66', '67'], (593974, '745412'): ['57', '58', '59'], (593974, '746303'): ['69', '70', '71', '72'], (593974, '746305'): ['70'], (593974, '746221'): ['52', '53', '54', '55', '60'], (593974, '745406'): ['57', '58', '59', '60'], (593974, '745405'): ['42', '43', '44', '45'], (593974, '745567'): ['68', '69', '70'], (593974, '745563'): ['63', '64', '65'], (593974, '745401'): ['59', '66', '67', '68', '69', '70'], (593974, '745402'): ['42', '43', '44', '45'], (593974, '745397'): ['75'], (593974, '745399'): ['45', '46', '47'], (593974, '746939'): ['47', '48', '49', '53'], (593974, '746938'): ['39', '40', '41'], (593974, '745398'): ['56', '57', '62', '63'], (593974, '745395'): ['77', '78', '79'], (593974, '745393'): ['67', '68', '69', '70'], (593974, '745370'): ['39', '40', '41', '42'], (593974, '745372'): ['53', '54', '55'], (593974, '745214'): ['61', '62', '63'], (593974, '746095'): ['68'], (593974, '747146'): ['58', '59', '60', '61'], (669093, '747208'): ['55', '61', '62', '63'], (669093, '746886'): ['68', '69', '70'], (669093, '745420'): ['60', '61', '62'], (669093, '745417'): ['44', '45', '46', '51', '52', '53'], (669093, '747122'): ['76', '77', '78'], (669093, '747119'): ['73', '74', '75'], (669093, '746714'): ['67', '68', '69', '70', '74', '75', '76'], (669093, '745413'): ['44', '45', '55', '56', '57'], (669093, '745412'): ['65', '66', '67'], (669093, '746303'): ['45', '46'], (669093, '746305'): ['60', '61', '62', '66', '67', '68', '69'], (669093, '746224'): ['59', '60'], (669093, '745408'): ['55', '56', '57', '58'], (669093, '745406'): ['61', '62', '63', '67', '68', '69'], (669093, '745409'): ['60'], (669093, '745405'): ['60', '61', '62', '63', '64'], (669093, '745807'): ['72', '73', '74', '75', '76', '77', '78'], (669093, '745563'): ['56', '57', '58', '59'], (669093, '745561'): ['66', '67', '68', '69'], (669093, '745401'): ['76', '77', '78', '79', '80', '81'], (669093, '745397'): ['58', '59', '60'], (669093, '745399'): ['64', '65', '66', '67'], (669093, '746938'): ['63', '64', '65'], (669093, '744998'): ['71', '72', '73'], (669093, '744996'): ['45', '46', '47', '52', '53', '54'], (669093, '745395'): ['62', '63', '64'], (669093, '745394'): ['50', '54', '55', '56'], (669093, '745392'): ['49', '50', '51', '52'], (669093, '745385'): ['61', '62', '63', '64'], (669093, '746613'): ['63', '64', '65'], (669093, '746610'): ['59', '60', '61'], (669093, '744828'): ['60', '61', '62', '63'], (669093, '747010'): ['54', '55', '56'], (669093, '747008'): ['62', '63', '64', '65', '66', '70', '71'], (669093, '745389'): ['66', '67', '68', '69'], (669093, '745387'): ['41', '42', '43', '44', '45', '46', '47'], (669093, '745384'): ['62', '63', '64'], (669093, '745467'): ['72', '73', '74'], (669093, '745469'): ['57', '58', '59', '60', '61'], (669093, '746026'): ['44', '45', '46', '47', '48'], (669093, '745383'): ['44', '45', '46', '47', '48'], (669093, '746515'): ['78', '79', '80'], (669093, '745375'): ['77', '78', '79', '80', '81'], (669093, '745376'): ['42', '43', '44'], (669093, '745132'): ['64', '65', '66'], (669093, '745052'): ['70', '71', '75', '76', '77', '78'], (669093, '745048'): ['59', '60', '61', '62'], (669093, '745370'): ['83', '84', '85', '86'], (669093, '745369'): ['67'], (669093, '745371'): ['66', '67', '68'], (669093, '745210'): ['46', '47', '48'], (669093, '745286'): ['72', '73', '74'], (669093, '745287'): ['42', '43', '44', '45'], (669093, '745366'): ['76', '77'], (669093, '745360'): ['53', '54', '55'], (669093, '745358'): ['51', '52', '53'], (669093, '746093'): ['51'], (669093, '746095'): ['49', '50', '51', '52'], (669093, '747145'): ['52', '53', '54', '55'], (674072, '747187'): ['48', '49', '50', '51', '52', '56', '57', '58', '59', '60', '61', '65', '66', '67', '71', '72', '73', '74', '75', '76'], (674072, '745564'): ['4', '5', '6', '7', '8', '9', '13', '14', '15', '19', '20', '21', '22', '23', '24', '28', '29', '30', '31'], (640462, '745414'): ['63', '64', '65', '66'], (640462, '746059'): ['44', '45', '49', '50', '51'], (640462, '746061'): ['47', '48', '49', '50'], (640462, '746056'): ['36', '37', '38', '45', '46', '47'], (640462, '746053'): ['49', '50', '51'], (640462, '746054'): ['48', '49', '50', '51'], (640462, '744847'): ['55', '56', '57'], (640462, '744839'): ['50', '54', '55', '56'], (640462, '746055'): ['87', '88', '89', '90', '91'], (640462, '746050'): ['43', '44', '45', '52', '53', '54'], (640462, '746051'): ['37', '38', '39', '44', '45', '46'], (640462, '746293'): ['43', '44', '45', '53'], (640462, '745565'): ['70', '71', '72', '73'], (640462, '745559'): ['55', '56', '57', '58'], (640462, '745555'): ['62', '63', '64'], (640462, '746043'): ['38', '39', '43', '44', '45'], (640462, '746044'): ['64', '65', '66'], (640462, '746368'): ['55'], (640462, '746687'): ['75', '76', '77'], (640462, '746686'): ['55', '61', '62', '63'], (640462, '746040'): ['56', '57', '58', '59'], (640462, '746042'): ['56', '57', '58', '59'], (640462, '746036'): ['56', '57', '58', '62', '63', '64'], (640462, '747171'): ['67', '68', '69', '70'], (640462, '747173'): ['46', '47', '48'], (640462, '747170'): ['55', '56', '57'], (640462, '745471'): ['56', '57', '58', '59'], (640462, '745468'): ['79', '80'], (640462, '746600'): ['71'], (640462, '746604'): ['53', '54'], (640462, '747168'): ['54', '55', '56'], (640462, '747166'): ['72', '73', '74', '75'], (640462, '747163'): ['81', '82', '83'], (640462, '745054'): ['53', '54', '60'], (640462, '745051'): ['76', '77', '78'], (640462, '746029'): ['55', '56', '61', '62', '63', '64'], (640462, '746917'): ['63', '64', '65', '66'], (640462, '746918'): ['64', '65', '66'], (640462, '747159'): ['50', '51'], (640462, '747158'): ['59', '60', '65', '66'], (640462, '745293'): ['61'], (640462, '745290'): ['65', '66', '67'], (640462, '746345'): ['57', '58', '59', '65', '66', '67', '68'], (640462, '747152'): ['59', '60', '61'], (640462, '747153'): ['65', '66', '67', '68', '70'], (640462, '747151'): ['74', '75', '76'], (640462, '746502'): ['53', '57', '58'], (640462, '745937'): ['60', '61', '62'], (640462, '745934'): ['81', '82', '83', '84'], (640462, '745936'): ['66', '67', '68'], (640462, '747144'): ['64', '65', '72'], (640462, '747145'): ['63', '64', '65', '66', '67'], (640462, '747146'): ['63', '64', '65', '66'], (656464, '747214'): ['93', '94', '95'], (656464, '745353'): ['70', '71', '72', '73', '74'], (656464, '745266'): ['58', '59'], (656464, '745265'): ['65', '66', '67'], (656464, '747210'): ['69', '70', '71', '72'], (656464, '747206'): ['74', '75', '76', '77'], (656464, '746717'): ['69', '70', '71', '72'], (656464, '746715'): ['70', '71', '72'], (656464, '747041'): ['61', '62', '63', '64'], (656464, '747204'): ['65', '66', '67', '68', '69', '70'], (656464, '747202'): ['62', '63', '64', '65', '66', '67', '68'], (656464, '746143'): ['66', '67', '68'], (656464, '747197'): ['49', '50', '51', '52', '53', '54', '55'], (656464, '745819'): ['63', '64'], (656464, '745812'): ['40', '46', '47'], (656464, '747196'): ['38', '39', '40', '45', '46', '47'], (656464, '745408'): ['52', '59', '60', '61', '62'], (656464, '745407'): ['43', '44', '45', '50', '51'], (656464, '747190'): ['50', '51'], (656464, '747192'): ['51', '52', '53'], (656464, '747189'): ['51', '52', '53'], (656464, '744842'): ['53', '54', '55'], (656464, '745562'): ['57', '58', '59'], (656464, '747188'): ['51', '52'], (656464, '747183'): ['37', '38', '39', '40'], (656464, '747186'): ['51', '52', '53', '54', '59', '60', '61'], (656464, '746133'): ['69', '70', '71', '72'], (656464, '746128'): ['68', '74', '75', '76'], (656464, '745395'): ['41', '42', '46', '47', '48'], (656464, '747181'): ['49', '50', '51', '52'], (656464, '747179'): ['42', '43', '44'], (656464, '747178'): ['46', '47', '48', '49'], (656464, '747176'): ['59', '60', '61', '62', '63', '64'], (656464, '746854'): ['53', '54', '60', '61', '62'], (656464, '746852'): ['50', '51', '52'], (656464, '746281'): ['67', '68', '69', '70', '71'], (656464, '747174'): ['54', '55', '56'], (656464, '747173'): ['42'], (656464, '747170'): ['62', '63', '64', '65', '66', '67'], (656464, '745471'): ['51', '52', '53', '54', '55'], (656464, '746600'): ['68', '69', '70'], (656464, '746602'): ['48', '55', '56', '57', '58'], (656464, '747164'): ['42', '43', '44', '50', '51', '52'], (656464, '747160'): ['51', '52', '53', '54'], (656464, '747163'): ['63', '64', '65', '66', '67'], (656464, '745051'): ['80', '86', '87', '88', '89'], (656464, '746027'): ['56', '64', '65', '66'], (656464, '746028'): ['66', '67', '68'], (656464, '746920'): ['62', '63', '64'], (656464, '747161'): ['47', '48', '53', '54', '55'], (656464, '747156'): ['76', '77', '78', '79'], (656464, '747158'): ['56', '57', '58'], (656464, '745290'): ['54', '55', '56', '57', '58'], (656464, '745289'): ['65', '66', '71', '72'], (656464, '746345'): ['53'], (656464, '747152'): ['43', '44', '45', '46', '47'], (656464, '747154'): ['46'], (656464, '747151'): ['50', '51', '52', '53', '54', '55', '56'], (656464, '746502'): ['50', '51', '52'], (656464, '745934'): ['56', '57', '58', '59', '60'], (656464, '745936'): ['58', '59', '60'], (656464, '747144'): ['73', '74', '75'], (656464, '747145'): ['72', '73', '74', '75'], (656464, '747146'): ['36', '40', '41'], (657044, '745353'): ['61', '62', '63'], (657044, '745191'): ['65', '66', '67', '72'], (657044, '745265'): ['58', '59', '60'], (657044, '747210'): ['53', '54', '55', '60', '61', '62'], (657044, '747206'): ['67', '68', '69', '70'], (657044, '746717'): ['63', '64', '65'], (657044, '746715'): ['62', '63'], (657044, '747041'): ['45', '49', '50', '51'], (657044, '747204'): ['49', '50'], (657044, '747203'): ['52', '53', '54', '55'], (657044, '747198'): ['54', '55', '56', '64', '65', '66', '67', '68'], (657044, '746145'): ['56', '57', '58', '59', '64'], (657044, '747199'): ['52', '53', '54', '55', '59', '60'], (657044, '745819'): ['54', '55', '56', '60', '61', '62'], (657044, '747194'): ['56', '60', '61', '62', '63', '64'], (657044, '745408'): ['67', '68', '69', '70', '71'], (657044, '745407'): ['72', '73', '74', '75', '76'], (657044, '747192'): ['64', '65', '66'], (657044, '747189'): ['68', '69', '70'], (657044, '744841'): ['62', '63', '64'], (657044, '745562'): ['69', '70', '71', '72'], (657044, '747188'): ['59', '60', '61'], (657044, '747183'): ['55', '61', '62', '63', '64', '65', '66', '67'], (657044, '747182'): ['62', '63', '64'], (657044, '746126'): ['66', '67', '68'], (657044, '745395'): ['68', '69', '74', '75', '76'], (657044, '747181'): ['56', '57', '58', '59'], (657044, '747179'): ['57', '58', '59', '60', '61'], (657044, '747178'): ['53', '54', '55', '56', '57'], (657044, '746854'): ['69', '70', '71', '72'], (657044, '746852'): ['63', '64', '65'], (657044, '746281'): ['62'], (657044, '747174'): ['62', '63', '64', '65', '66'], (657044, '747173'): ['52', '53', '54'], (657044, '747170'): ['76', '77', '78'], (657044, '745471'): ['81', '82', '83', '84', '85'], (657044, '745468'): ['75', '76', '77', '78'], (657044, '746600'): ['75', '76', '77', '78'], (657044, '746602'): ['62', '63', '64'], (657044, '747165'): ['60', '61', '62'], (657044, '747163'): ['73', '74', '75', '76', '77'], (657044, '745054'): ['61', '62', '63', '64', '65', '66', '67'], (657044, '746027'): ['72', '73', '74'], (657044, '746029'): ['48', '49', '53', '54'], (657044, '746917'): ['54', '55', '56', '57'], (657044, '747161'): ['59', '60', '61', '62'], (657044, '747159'): ['56', '57', '58'], (657044, '747158'): ['67', '68', '69', '73', '74', '75'], (657044, '745293'): ['65', '66', '67', '68', '69', '70', '71'], (657044, '746345'): ['74', '75', '76', '77', '78', '79'], (657044, '747152'): ['52', '53', '54'], (657044, '747153'): ['46', '47', '48'], (657044, '747151'): ['63', '64', '65', '66', '67'], (657044, '746502'): ['59', '64', '65', '66'], (657044, '746500'): ['60', '61'], (657044, '745934'): ['74', '75', '76'], (657044, '745929'): ['65', '66', '67', '68', '69'], (657044, '747145'): ['56', '57', '58'], (657044, '747146'): ['49', '50', '51'], (686826, '747216'): ['58', '59', '64', '65', '66'], (686826, '747213'): ['71', '73', '78', '79', '80', '81', '82', '83'], (686826, '745356'): ['49', '54', '55', '56', '57'], (686826, '745354'): ['55', '56', '60', '61', '62', '63'], (686826, '745353'): ['51', '52', '53', '54', '56'], (686826, '745188'): ['61', '62', '63', '64'], (686826, '745266'): ['47', '48', '49'], (686826, '747210'): ['8', '9', '10', '16', '17', '18', '23', '24', '25', '26', '27'], (686826, '747209'): ['57', '58', '59', '60', '64', '65', '66', '67'], (686826, '746718'): ['54', '55', '56', '57'], (686826, '747041'): ['78', '79', '80', '85'], (686826, '747205'): ['70', '71'], (686826, '747202'): ['74', '75', '76', '77', '78', '79'], (686826, '746146'): ['56', '57', '58', '64', '65', '66', '67', '68'], (686826, '746143'): ['72', '73', '74'], (686826, '747200'): ['53', '54', '55', '56'], (686826, '747197'): ['68', '69', '70', '71', '72'], (686826, '745011'): ['55', '56', '57'], (686826, '745819'): ['6', '7', '8', '9'], (686826, '745812'): ['48', '49', '57', '58', '59', '60'], (686826, '747196'): ['66', '67', '68', '69', '70', '71'], (686826, '747195'): ['58', '59', '60', '64', '65', '66', '67', '68'], (686826, '745407'): ['81', '82', '83'], (686826, '747190'): ['71', '72', '73', '74', '75', '76', '81', '82', '83', '84'], (686826, '747191'): ['62', '63', '64'], (686826, '747187'): ['38', '39', '40', '41', '42'], (686826, '744843'): ['59', '60', '61', '67', '68', '69'], (686826, '745566'): ['53', '54', '55', '56', '57', '61', '62', '63'], (686826, '747184'): ['71', '72', '73', '75'], (686826, '747185'): ['47', '48', '49', '50', '51', '52', '57', '58', '59', '60', '65', '66', '67'], (686826, '746128'): ['39', '40', '41', '42', '48', '49', '50', '56', '57'], (686826, '745395'): ['61', '65', '66', '67'], (686826, '745391'): ['67', '68', '69', '70'], (686826, '747175'): ['57', '58', '59', '60', '61'], (686826, '747177'): ['56', '57', '58'], (686826, '746853'): ['52', '53', '54', '55'], (686826, '746281'): ['44', '45', '46', '47', '51', '52', '53'], (686826, '747171'): ['75', '76', '77', '78'], (686826, '747173'): ['72', '73', '74', '75'], (686826, '747167'): ['66', '67', '68', '69', '70'], (573009, '747213'): ['49'], (573009, '747212'): ['57', '58', '59', '61'], (573009, '745354'): ['52', '53', '54'], (573009, '745191'): ['61'], (573009, '745266'): ['50', '51', '57'], (573009, '745265'): ['50', '51', '52'], (573009, '747210'): ['37', '38', '42', '43', '44', '45', '46'], (573009, '747206'): ['60', '61', '62', '63'], (573009, '746718'): ['61', '62', '63'], (573009, '746717'): ['73', '74'], (573009, '747041'): ['55', '56', '57'], (573009, '747204'): ['55', '56', '57'], (573009, '747202'): ['52', '54'], (573009, '746146'): ['5', '6', '7'], (573009, '746145'): ['65', '66'], (573009, '746143'): ['57'], (573009, '747199'): ['61', '62'], (573009, '745819'): ['47', '52', '53'], (573009, '747196'): ['48'], (573009, '745408'): ['44', '45', '46', '50', '51'], (573009, '745407'): ['52', '53', '57', '58'], (573009, '747193'): ['68', '69', '70', '71'], (573009, '747191'): ['68', '69', '70', '71'], (573009, '747189'): ['74', '75', '76', '77', '78', '79'], (573009, '744842'): ['60', '61', '62'], (573009, '745562'): ['60', '61', '62'], (573009, '745566'): ['46', '47', '48', '52'], (573009, '747185'): ['80', '81', '82'], (573009, '747182'): ['51', '52', '53'], (573009, '746133'): ['5', '6', '7'], (573009, '746128'): ['58', '59', '66', '67'], (573009, '745395'): ['55', '56', '57', '58', '59', '60'], (573009, '745391'): ['50'], (573009, '747180'): ['63', '65', '66'], (573009, '747179'): ['35', '36', '37', '38'], (573009, '747176'): ['52', '53', '54', '55'], (573009, '746853'): ['70', '71'], (573009, '746281'): ['59', '60', '61'], (573009, '746279'): ['64', '65', '66'], (573009, '747173'): ['39', '40', '41'], (573009, '747172'): ['77', '78', '79', '80'], (573009, '745474'): ['49', '54', '55', '56', '57'], (573009, '746602'): ['65'], (573009, '746604'): ['65'], (573009, '747164'): ['56', '57', '58'], (573009, '747166'): ['51', '52', '53'], (573009, '747160'): ['65', '66', '67'], (573009, '745049'): ['57', '58', '59', '60'], (573009, '745051'): ['46', '48', '50'], (573009, '746028'): ['73', '74', '75'], (573009, '746920'): ['70', '71', '72', '73'], (573009, '746918'): ['56', '57', '58'], (573009, '747156'): ['53', '59', '60'], (573009, '747158'): ['49', '50', '55'], (573009, '747157'): ['54'], (573009, '745289'): ['62', '63', '64'], (573009, '746343'): ['42'], (573009, '746344'): ['59', '60'], (573009, '747153'): ['52', '53', '54', '58'], (573009, '747151'): ['57'], (573009, '746503'): ['54', '55'], (573009, '746500'): ['76', '77', '78', '79', '80'], (573009, '745936'): ['42', '43', '44', '50', '51', '52'], (573009, '745929'): ['75', '76', '77', '78', '79'], (573009, '747145'): ['49'], (573009, '747146'): ['42', '43'], (573009, '747147'): ['47', '48'], (679885, '747212'): ['62', '63', '67', '68', '69', '70'], (679885, '745352'): ['74', '75', '76', '84', '85', '86', '87'], (679885, '745269'): ['49', '50', '51', '55', '56', '57'], (679885, '747211'): ['79', '80'], (679885, '747210'): ['33', '34', '35', '36'], (679885, '747208'): ['48', '49', '50', '51', '52', '56', '57', '58', '59', '60'], (679885, '747206'): ['53', '54', '55', '56'], (679885, '747043'): ['57', '59', '65', '66', '67', '68'], (679885, '747042'): ['73', '74', '75'], (679885, '747204'): ['76', '77', '78', '79'], (679885, '747202'): ['55', '56', '57'], (679885, '746143'): ['51', '55', '56'], (679885, '747197'): ['35', '41', '42', '43', '44'], (679885, '745011'): ['68', '69'], (679885, '745819'): ['26', '27', '28', '33', '34', '35', '39', '40', '41', '45', '46'], (679885, '745813'): ['57', '58', '59', '63', '64', '65'], (679885, '747196'): ['52', '53', '54', '55', '56'], (679885, '745407'): ['59', '64', '65', '66', '67'], (679885, '747190'): ['59', '60', '61', '62'], (679885, '747191'): ['54', '55', '56', '57'], (679885, '747189'): ['59', '60', '61', '62', '63', '64'], (679885, '744843'): ['53', '54', '55'], (679885, '745566'): ['36', '37', '38', '42', '43', '44', '45'], (679885, '747185'): ['71', '72', '73'], (679885, '747183'): ['44', '45', '46', '47', '48', '53', '54'], (679885, '746133'): ['52', '59', '60', '61', '62', '63'], (679885, '746126'): ['43', '44', '45', '46', '47', '52', '53', '54', '59', '60', '61'], (679885, '745391'): ['55', '56', '57'], (679885, '747181'): ['71', '72', '73', '77', '78', '79', '80'], (679885, '747179'): ['48', '49', '50', '51'], (679885, '747175'): ['68', '69', '70'], (679885, '747176'): ['69', '70', '71'], (679885, '746854'): ['66', '67', '68'], (679885, '746852'): ['56', '57', '58'], (679885, '746853'): ['59', '60', '61', '62', '63', '64'], (679885, '746279'): ['55', '56', '57', '58'], (679885, '747171'): ['61', '62', '63'], (679885, '747173'): ['67', '68', '69', '70', '71'], (679885, '747167'): ['79', '80', '81'], (679885, '745471'): ['72', '73', '74', '75'], (679885, '745468'): ['59', '60', '61', '62'], (679885, '746600'): ['82', '84', '85'], (679885, '746602'): ['70', '71', '72', '73', '74', '75'], (679885, '747168'): ['60', '61', '62', '63'], (679885, '747165'): ['66', '67', '68'], (679885, '745054'): ['74', '75', '76', '77'], (679885, '745051'): ['94', '95', '96', '97', '102', '103', '104'], (679885, '746029'): ['68', '69', '70'], (679885, '746028'): ['76', '77', '85', '86', '87', '88', '89', '90'], (679885, '746918'): ['71', '72', '73', '74'], (679885, '747161'): ['70', '71', '72'], (679885, '747159'): ['63', '64', '65', '66', '67'], (679885, '747158'): ['76', '77', '78'], (679885, '745293'): ['72', '73', '79', '80', '81', '82', '84'], (679885, '745290'): ['72', '73', '74', '75', '76', '77'], (679885, '747152'): ['68', '69', '70'], (679885, '747153'): ['59', '60', '61'], (679885, '747151'): ['81', '82', '83', '84', '88', '89', '91'], (679885, '746500'): ['65', '66', '67', '68'], (679885, '745937'): ['66', '67', '68'], (679885, '745934'): ['65', '66', '67', '68', '69'], (679885, '745929'): ['80', '81', '82', '83'], (679885, '747144'): ['54', '55', '56', '57', '58', '62', '63'], (679885, '747146'): ['55', '56', '57', '62'], (680767, '746571'): ['51', '52', '53', '54', '55'], (680767, '744953'): ['57', '58', '59', '60'], (680767, '744952'): ['49', '50', '51', '52'], (680767, '745593'): ['43', '44', '48', '49', '50'], (680767, '746570'): ['38', '39', '40', '41', '42', '43', '44'], (680767, '746565'): ['39', '40', '41', '42', '46', '47', '48', '49'], (680767, '746567'): ['41', '42', '43', '44', '45'], (680767, '746560'): ['64', '65', '66', '67', '68', '69', '70'], (680767, '746072'): ['68'], (680767, '745511'): ['46', '47', '48', '49', '53', '54', '55'], (680767, '746563'): ['56', '57', '58', '62', '63', '64', '65'], (680767, '746555'): ['42', '43', '44', '48', '49', '50', '51'], (680767, '745418'): ['56', '57', '58', '59', '60', '61', '62'], (680767, '745335'): ['47', '48', '49', '50', '51', '52'], (680767, '745336'): ['43', '47', '48', '49', '50'], (680767, '745661'): ['78', '79', '80'], (680767, '745660'): ['52', '53', '54', '55', '56'], (680767, '746556'): ['50', '51', '52'], (680767, '746549'): ['60', '61', '65', '66', '67', '68'], (680767, '746552'): ['80', '81', '82'], (680767, '746140'): ['60', '61', '62'], (680767, '746544'): ['42', '43', '48', '49', '50'], (680767, '745174'): ['54', '58', '59', '60', '61', '62'], (680767, '745166'): ['48', '49', '50', '57', '58', '59', '60'], (680767, '745899'): ['52', '53', '54', '55'], (680767, '746545'): ['51', '56', '57', '58', '59', '60'], (680767, '746546'): ['43', '44', '45', '46', '47', '48'], (680767, '746542'): ['76', '77', '78', '79', '80'], (680767, '746540'): ['62', '63', '64', '65', '66'], (680767, '746374'): ['67', '68'], (680767, '746779'): ['55', '56', '60', '61', '62'], (680767, '746535'): ['73', '74', '75'], (680767, '746533'): ['57', '58', '59', '60'], (680767, '746529'): ['65', '66', '67', '68', '69'], (680767, '746689'): ['58', '64', '65', '66', '71'], (680767, '745798'): ['58', '59', '60', '65', '66', '67'], (680767, '745797'): ['73', '74', '75', '80', '81', '82', '83', '84'], (680767, '746531'): ['76', '77', '78'], (680767, '746527'): ['65', '66', '67'], (680767, '746526'): ['74', '75', '76', '80', '81', '82', '83'], (680767, '745305'): ['69', '70', '71', '72', '73'], (680767, '746198'): ['68', '69', '70'], (680767, '746196'): ['70', '71', '72', '73'], (680767, '745387'): ['70', '71', '72', '73'], (680767, '746519'): ['70', '71', '72', '73'], (680767, '746521'): ['67', '68', '69', '70', '71', '72', '73'], (680767, '746523'): ['73', '74', '75', '76'], (680767, '746517'): ['80', '81', '82'], (680767, '747160'): ['68', '69', '70', '71', '72', '73'], (680767, '746518'): ['72', '73', '74', '75', '76'], (680767, '746516'): ['69', '70', '71', '72'], (680767, '746428'): ['56', '57', '58'], (680767, '746427'): ['52', '53', '54'], (680767, '746509'): ['61', '62', '63', '64', '65', '66'], (680767, '746500'): ['47', '48', '49', '50', '51'], (680767, '746094'): ['63', '64', '65'], (680767, '746096'): ['55', '60', '61', '62', '63'], (680767, '746504'): ['49', '50', '51', '52', '53', '54'], (680767, '746495'): ['58', '59', '60'], (680767, '746496'): ['53', '54', '55', '56'], (641755, '746564'): ['65', '66', '67', '68', '69'], (641755, '746560'): ['76', '77', '78', '79', '80', '81', '82'], (641755, '745504'): ['51', '55', '56', '57', '58'], (641755, '746559'): ['69', '70', '71'], (641755, '746554'): ['58', '69', '70', '71'], (641755, '746555'): ['57', '58', '59', '60'], (641755, '745418'): ['74', '75', '76'], (641755, '745339'): ['82'], (641755, '745336'): ['54', '55', '56', '57'], (641755, '745662'): ['63', '64', '65', '66', '67', '68'], (641755, '745660'): ['60', '61', '62', '63', '64'], (641755, '746558'): ['81', '82', '83'], (641755, '746557'): ['65', '66', '67'], (641755, '746551'): ['67', '68', '69', '70', '71', '72', '73'], (641755, '746550'): ['59', '60', '61'], (641755, '746544'): ['66', '67', '68', '69', '70'], (641755, '745174'): ['74', '75'], (641755, '745166'): ['70', '71', '72', '73'], (641755, '745899'): ['69', '70', '71', '72', '73'], (641755, '746546'): ['63', '64', '65', '66', '67'], (641755, '746542'): ['71', '72', '73', '74', '75'], (641755, '746539'): ['63', '64', '65', '66', '67'], (641755, '746540'): ['71', '72', '73', '74', '75'], (641755, '746374'): ['51', '52', '53', '54'], (641755, '746779'): ['82', '83', '85', '89', '90', '91'], (641755, '746535'): ['81', '82', '83', '84'], (641755, '746532'): ['55', '56', '57', '62', '63', '64', '65'], (641755, '746529'): ['57', '58', '59'], (641755, '746692'): ['55', '56', '57', '58', '59', '60', '61'], (641755, '746688'): ['45', '46', '47', '48'], (641755, '745798'): ['51', '52', '53', '54'], (641755, '745797'): ['50', '51', '52', '53'], (641755, '746531'): ['52', '53', '54', '55', '56'], (641755, '746528'): ['54', '55', '56', '57'], (641755, '746525'): ['66', '67', '68'], (641755, '745315'): ['47', '48', '49', '50', '51'], (641755, '745305'): ['54', '59', '60', '61', '62'], (641755, '746198'): ['61', '62', '63', '64'], (641755, '746196'): ['78', '79', '80'], (641755, '745387'): ['64', '65', '66'], (641755, '746519'): ['61', '62', '63'], (641755, '746521'): ['60', '61', '62', '63'], (641755, '746523'): ['65', '66', '67', '68'], (641755, '747165'): ['59', '63', '64', '65'], (641755, '747160'): ['62', '63', '64'], (641755, '746518'): ['60', '64', '65', '66', '67'], (641755, '746516'): ['61', '62', '63', '64'], (641755, '744814'): ['62', '63', '64', '65'], (641755, '746513'): ['68', '69', '70'], (641755, '746510'): ['72', '73', '74', '75'], (641755, '746507'): ['71', '72', '73', '74', '75'], (641755, '747073'): ['65', '66', '67'], (641755, '745941'): ['65', '66', '67'], (641755, '745940'): ['70', '71', '72'], (641755, '746427'): ['66', '67', '68', '69', '70'], (641755, '746508'): ['77', '78', '79'], (641755, '746500'): ['69', '70', '71', '72'], (656234, '746575'): ['53', '54', '55', '56', '57', '58'], (656234, '746569'): ['80'], (656234, '746573'): ['55', '56', '57', '58', '59'], (656234, '744953'): ['64', '65', '66', '67', '68'], (656234, '745597'): ['72', '73', '74'], (656234, '746568'): ['55', '56', '57', '61', '62', '63'], (656234, '746566'): ['71', '72', '73', '74'], (656234, '746561'): ['52', '53', '54', '55', '56', '57'], (656234, '746072'): ['49', '50', '54', '55', '56'], (656234, '745510'): ['61', '62', '68', '69'], (656234, '746562'): ['62', '66', '67', '68', '69'], (656234, '746553'): ['63', '64', '65'], (656234, '746554'): ['53', '54', '55', '56', '57'], (656234, '745420'): ['51', '55', '56', '57'], (656234, '745335'): ['59', '60', '61', '65', '66', '67', '68', '70', '71'], (656234, '746543'): ['44', '45', '46', '47', '48'], (656234, '746538'): ['62', '63', '64', '65', '66'], (656234, '746541'): ['52', '53', '54', '55'], (656234, '745310'): ['53', '54', '55', '56'], (656234, '746197'): ['63', '64', '65', '66', '67'], (656234, '746513'): ['43', '44', '45', '46'], (656234, '746514'): ['60', '61', '62', '63', '71', '72', '73', '77', '78', '79'], (656234, '746505'): ['32', '33', '34', '35', '39', '40', '41'], (656234, '747075'): ['53', '54', '55', '56', '57'], (656234, '745942'): ['33', '34', '38', '39', '40', '41'], (656234, '746425'): ['35', '36', '44', '45', '46', '47', '51', '52'], (656234, '746508'): ['57', '58', '59'], (656234, '746506'): ['47', '48', '49', '50', '51', '55', '56', '57'], (656234, '746504'): ['64', '65', '70', '71', '72'], (656234, '746495'): ['44', '45', '50', '51', '52', '53'], (656234, '746499'): ['77', '78', '79', '80', '81'], (664875, '746566'): ['79', '80', '81', '82'], (664875, '746564'): ['81', '82', '83'], (664875, '746073'): ['65', '66', '67', '68', '69'], (664875, '746070'): ['65', '66', '73', '74', '75', '79'], (664875, '746562'): ['74', '75', '76', '77'], (664875, '746553'): ['55', '56', '57'], (664875, '745420'): ['58', '59', '63', '64', '65', '66', '67', '68'], (664875, '745417'): ['54', '55', '56', '57', '58'], (664875, '745339'): ['54', '55', '56', '57', '63', '64', '65', '66', '67'], (664875, '745662'): ['55', '56', '57', '58', '59'], (664875, '745660'): ['72', '73', '77', '78', '79', '80'], (664875, '746556'): ['68', '69', '70', '71', '72'], (664875, '746543'): ['68', '72', '73', '74', '75', '76', '77'], (664875, '746539'): ['72', '73', '74', '75'], (664875, '746541'): ['55', '56', '61', '62', '63', '64'], (664875, '746372'): ['48', '49', '54', '55', '56', '57'], (664875, '746776'): ['46', '47', '48', '49', '50'], (664875, '746779'): ['66', '67', '68'], (664875, '746535'): ['64', '65', '66', '67'], (664875, '746537'): ['79', '80'], (664875, '746530'): ['56', '57', '58', '59', '60', '61'], (664875, '746690'): ['53', '54', '55', '56', '57', '63', '64', '65', '66', '67', '68', '69'], (664875, '745800'): ['62'], (664875, '745798'): ['37', '44', '45', '46', '47'], (664875, '745797'): ['42', '43', '44'], (664875, '746526'): ['88', '89', '90', '95', '96', '97', '98'], (664875, '745315'): ['39', '40', '41', '42', '43'], (664875, '745307'): ['53', '54', '55', '56', '57', '58'], (664875, '746197'): ['71', '72', '73', '74', '75', '76', '77'], (664875, '745384'): ['41', '45', '46', '47', '48', '49', '50'], (664875, '746519'): ['40', '46', '47', '48', '49'], (664875, '746521'): ['74'], (664875, '746523'): ['46', '47', '48', '54', '55', '56'], (664875, '746517'): ['51', '52', '53', '54', '55'], (664875, '747163'): ['62', '68', '69', '70', '71', '72'], (664875, '746515'): ['67', '68', '73', '74', '75', '76', '77'], (664875, '744815'): ['48', '49', '50'], (664875, '745706'): ['72', '73'], (664875, '746513'): ['53', '54', '55', '56'], (664875, '746510'): ['53', '60', '61', '62'], (664875, '746514'): ['83', '84', '85', '86', '87', '88'], (664875, '746505'): ['64', '65', '66', '67'], (664875, '747079'): ['62', '63', '64', '65', '66'], (664875, '746428'): ['51', '52'], (664875, '746506'): ['72', '73', '74', '75', '76', '77', '78', '79', '80'], (664875, '746500'): ['73', '74', '75'], (664875, '746494'): ['78', '79', '80'], (664875, '746499'): ['63', '64', '65', '66', '67', '68', '72', '73', '74', '75', '76'], (663567, '747221'): ['47', '53', '54', '55', '56'], (663567, '746899'): ['51', '52', '53', '54', '55', '56', '60', '61', '62', '63', '64'], (663567, '746576'): ['37', '38', '39', '43', '44', '45', '46', '47'], (663567, '746569'): ['48', '49', '50', '60', '61', '62'], (663567, '744954'): ['57', '58', '59', '66', '67', '68', '73', '74', '75', '76'], (663567, '745591'): ['52', '53', '54', '59', '60', '61', '62'], (663567, '746572'): ['1', '2', '3', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '23', '24', '25', '26'], (663567, '746564'): ['37', '38', '42', '43', '44', '49', '50', '51'], (663567, '746070'): ['5', '6', '7', '8', '9', '10', '17', '18', '19', '20', '21', '25', '26', '27', '28', '32', '33', '34'], (663567, '746562'): ['1', '2', '3', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '23', '24', '25', '26', '27'], (663567, '745417'): ['72', '73', '74', '75'], (663567, '745339'): ['35', '36', '37', '43', '44', '45', '46', '52', '53'], (663567, '745660'): ['92', '93', '94', '95', '97'], (663567, '746552'): ['46', '50', '51', '52', '60', '61'], (663567, '746141'): ['33', '34', '35', '36', '37', '43', '44', '45', '49', '50', '51', '52', '53'], (663567, '746534'): ['39', '44', '45', '46', '50', '51', '52', '53', '57', '58', '59', '65', '66', '67', '68'], (663567, '746690'): ['24', '25', '26', '35', '36', '37', '38', '39', '43', '44', '45', '46', '47', '48'], (663567, '745800'): ['36', '37', '38', '39', '44', '45', '46', '47', '48', '53', '54', '55', '60', '61'], (663567, '746526'): ['50', '51'], (663567, '746525'): ['76', '77', '78', '79', '80', '84', '85', '86', '87', '88', '89'], (663567, '745307'): ['62', '63', '64', '65'], (663567, '746197'): ['45', '49', '50', '51', '52', '56', '57', '58'], (663567, '745388'): ['46', '47', '48', '49', '50', '51'], (663567, '746522'): ['29', '30', '31', '36', '37', '38', '39', '40', '41', '48', '49', '50', '51', '57'], (663567, '746517'): ['27', '35', '36', '37', '38', '43', '44', '45', '46', '47'], (663567, '747163'): ['13', '17', '18', '19', '20', '21', '26', '27', '28', '29', '30'], (663567, '746515'): ['45', '46', '47', '48', '49', '50', '57', '58', '59', '60', '65', '66'], (663567, '744815'): ['54', '55', '56', '57', '61', '62', '63', '64', '65', '66', '67', '68'], (660825, '745741'): ['67', '68', '69'], (660825, '745245'): ['61', '62', '63'], (660825, '745241'): ['69', '70', '71', '72', '73', '74', '75'], (660825, '746625'): ['55', '56', '57', '58', '62', '63', '64'], (660825, '746048'): ['73', '74', '75'], (660825, '745076'): ['50', '55', '56', '57', '58', '59'], (660825, '745239'): ['60', '65', '66', '67'], (660825, '745235'): ['73', '74', '75'], (660825, '746204'): ['69', '70', '71', '76', '77', '78', '79'], (660825, '746183'): ['63', '64', '65'], (660825, '745614'): ['75', '76', '77'], (660825, '745128'): ['68', '69', '70', '71', '72'], (660825, '745210'): ['62', '63', '64', '70', '71', '72', '73', '74'], (660825, '745206'): ['66', '67', '68'], (660825, '744960'): ['74', '75', '76', '77', '78'], (660825, '744958'): ['86', '87', '88', '89', '90'], (660825, '746334'): ['73', '74', '75'], (660825, '745203'): ['84', '85', '86'], (660825, '745201'): ['43'], (608638, '745560'): ['59'], (608638, '746047'): ['62', '63', '64', '65', '66', '70', '71', '72'], (608638, '746046'): ['64', '65', '66', '67'], (608638, '746039'): ['72', '73', '74', '75'], (608638, '746366'): ['53', '54', '55', '59', '60', '61'], (608638, '746685'): ['54', '55', '61', '62', '63', '64'], (608638, '746041'): ['68', '69', '70', '71', '72'], (608638, '746034'): ['71', '72', '73', '74', '75', '76'], (608638, '746037'): ['66', '67', '68', '69', '70'], (608638, '745225'): ['38', '44', '45', '46', '47'], (608638, '745222'): ['45', '46', '47'], (608638, '745223'): ['42', '43', '44', '45'], (608638, '745226'): ['59', '60', '61'], (608638, '745462'): ['59'], (608638, '745463'): ['50', '51', '55', '56', '57'], (608638, '746110'): ['56', '57', '61', '62', '63', '64'], (608638, '745218'): ['45', '50', '51', '52', '53'], (608638, '745213'): ['45', '46', '47', '48', '49'], (608638, '746184'): ['56', '57'], (608638, '746182'): ['59', '60', '61'], (608638, '745619'): ['54', '55', '56'], (608638, '745614'): ['62', '63', '64', '65'], (608638, '745128'): ['73', '81', '82', '83'], (608638, '745210'): ['43', '44', '45'], (608638, '745207'): ['67', '68', '69'], (608638, '745204'): ['52', '53'], (608638, '744960'): ['66', '67', '68'], (608638, '744955'): ['58', '59', '60', '61', '62'], (608638, '746331'): ['57', '58', '59'], (608638, '745203'): ['48', '49', '50'], (554340, '744946'): ['54', '60', '61', '62'], (554340, '744947'): ['57', '58', '59', '61'], (554340, '744943'): ['74', '75', '76'], (554340, '747040'): ['58', '59', '60'], (554340, '747038'): ['59', '61', '62', '66', '67', '69'], (554340, '744938'): ['53', '54', '55', '56'], (554340, '744935'): ['52', '53', '54'], (554340, '746469'): ['79', '80', '81', '82', '83', '84', '85'], (554340, '746790'): ['63', '64', '65'], (554340, '744928'): ['62', '63', '64'], (554340, '744929'): ['76', '77', '78', '79', '80'], (554340, '744927'): ['65', '66', '67'], (554340, '744923'): ['67', '68', '69', '70', '71'], (554340, '745651'): ['66', '67', '68'], (554340, '745981'): ['67', '74', '75', '76'], (554340, '744926'): ['73', '74', '75', '76', '77', '78', '79'], (554340, '744910'): ['67', '68', '69'], (554340, '744905'): ['65', '66', '67', '68', '69'], (554340, '746768'): ['64', '65', '66'], (554340, '746770'): ['65', '66', '67'], (554340, '746926'): ['75', '76', '77', '78'], (554340, '745225'): ['56', '57', '58', '59', '60'], (554340, '745222'): ['51', '52', '53', '54'], (554340, '745226'): ['49', '53', '54', '55'], (554340, '745219'): ['58', '59', '60', '61'], (554340, '746436'): ['61', '62', '63', '64', '65'], (554340, '746437'): ['56', '57', '58'], (554340, '746109'): ['49', '50', '51'], (642048, '745028'): ['43', '44', '45', '50', '51'], (642048, '745265'): ['61', '62', '63', '64'], (642048, '745261'): ['66', '67', '68'], (642048, '746394'): ['68', '69', '70', '71'], (642048, '746395'): ['58', '59', '60', '61', '62'], (642048, '745915'): ['65', '66', '67'], (642048, '745258'): ['53', '54', '55', '56'], (642048, '745253'): ['52', '53', '54'], (642048, '745656'): ['77', '78'], (642048, '746302'): ['58', '59', '60', '61'], (642048, '746297'): ['81'], (642048, '745244'): ['66'], (642048, '745246'): ['61'], (642048, '745241'): ['61'], (642048, '746627'): ['64', '65', '66', '67', '68'], (642048, '746626'): ['49', '50', '51'], (642048, '746051'): ['40', '41', '42', '43'], (642048, '745078'): ['30', '31', '32'], (642048, '745239'): ['57', '58', '59'], (642048, '745234'): ['45', '46', '51', '52', '53'], (642048, '745235'): ['44', '45', '46'], (642048, '745393'): ['66'], (642048, '746201'): ['62', '63'], (642048, '745228'): ['56', '57', '58', '59'], (642048, '745230'): ['42', '43', '44'], (642048, '745231'): ['65', '66', '67', '68'], (642048, '746928'): ['74', '75', '76', '77'], (642048, '746926'): ['64', '65', '66', '67', '68'], (642048, '746925'): ['59', '60', '61', '62'], (642048, '745225'): ['35', '36', '37'], (642048, '745222'): ['55', '56', '57'], (642048, '746435'): ['65', '66', '67'], (642048, '745463'): ['58', '59', '60'], (642048, '746109'): ['52', '53', '54'], (642048, '746107'): ['52', '57', '58'], (642048, '745218'): ['76', '77', '78'], (642048, '745614'): ['66'], (642048, '745128'): ['52', '53', '54', '55', '60', '61', '62'], (642048, '745210'): ['55'], (642048, '745208'): ['64', '65'], (642048, '745207'): ['58', '59'], (642048, '744958'): ['68', '69'], (642048, '746331'): ['66'], (642048, '745202'): ['56', '57', '58'], (642048, '745203'): ['51'], (676092, '746300'): ['58', '59', '60', '61'], (676092, '745238'): ['53', '54', '55'], (676092, '745239'): ['49', '50', '51', '52'], (676092, '745240'): ['56', '57', '58', '59'], (676092, '745234'): ['65', '66', '67'], (676092, '745235'): ['34', '35', '36', '37', '42', '43'], (676092, '745233'): ['83', '84', '85', '87'], (676092, '746207'): ['50', '51', '52'], (676092, '746202'): ['61', '62', '63', '64'], (676092, '745228'): ['63', '64', '65'], (676092, '745230'): ['52', '53', '54', '55'], (676092, '745231'): ['49', '50', '51'], (676092, '746768'): ['46', '47', '48', '49', '50'], (676092, '746928'): ['64', '65', '66'], (676092, '746926'): ['69', '70', '71'], (676092, '746925'): ['69', '74', '75', '76', '77', '78'], (676092, '745225'): ['79', '80', '81', '82'], (676092, '745223'): ['51', '52', '53'], (676092, '745220'): ['45', '46', '47', '48'], (676092, '745216'): ['57', '58', '59'], (676092, '746436'): ['75', '76', '77'], (676092, '745464'): ['63', '64', '65'], (676092, '746107'): ['44', '45', '46', '50', '51'], (676092, '745217'): ['80', '82', '83'], (676092, '745212'): ['56', '57', '58', '59', '60', '61'], (676092, '745213'): ['53', '54', '55', '56', '57'], (676092, '745211'): ['57', '58', '59'], (676092, '746182'): ['66', '67', '68'], (676092, '745619'): ['62', '63', '64', '65'], (676092, '745612'): ['60', '61', '62', '63'], (676092, '745131'): ['63', '64', '65', '66'], (676092, '745214'): ['57', '58', '59', '60'], (676092, '745208'): ['59', '60', '61', '62', '63'], (676092, '745209'): ['61', '62', '63'], (676092, '745199'): ['64', '65', '66', '71', '72', '73', '74'], (676092, '744958'): ['59', '60', '65', '66', '67'], (676092, '746331'): ['63', '64', '65'], (676092, '745203'): ['73', '74', '75', '76', '77', '78'], (662253, '745260'): ['60', '62', '63', '68', '69', '70'], (662253, '746395'): ['72', '77', '78', '79'], (662253, '745915'): ['79', '80', '81'], (662253, '745255'): ['69', '70', '71'], (662253, '745256'): ['64', '70', '71', '72'], (662253, '745259'): ['59', '60', '65', '66', '67', '68', '69'], (662253, '747036'): ['73', '74', '75', '76'], (662253, '745746'): ['74', '75', '76', '77'], (662253, '745742'): ['70', '71', '72', '73', '74'], (662253, '744855'): ['75', '76', '77', '78'], (662253, '745254'): ['68', '69', '70'], (662253, '745250'): ['60', '61', '62'], (662253, '745253'): ['67', '68', '69', '70'], (662253, '745656'): ['73', '74', '75', '76', '77'], (662253, '746297'): ['59', '60', '61'], (662253, '745245'): ['68', '69', '70', '72'], (662253, '745247'): ['57', '58', '59', '60'], (662253, '746627'): ['78', '79', '84', '85', '86', '87'], (662253, '746052'): ['62', '63', '64', '65'], (662253, '745078'): ['70', '71', '72'], (662253, '745073'): ['65', '66', '67'], (662253, '745238'): ['63', '64', '65'], (662253, '745236'): ['67', '68', '69', '70'], (662253, '745237'): ['66', '67', '68'], (662253, '745233'): ['74', '75', '76'], (662253, '745394'): ['61', '66', '67', '68', '69', '70'], (662253, '746207'): ['73', '74', '75', '76'], (662253, '745230'): ['68', '69', '70'], (662253, '745232'): ['56', '57'], (662253, '746768'): ['79', '84', '85', '86'], (662253, '746770'): ['72', '73', '74', '75'], (662253, '746926'): ['82', '83', '84'], (662253, '745225'): ['72', '73', '74', '75'], (662253, '745219'): ['66', '67', '68'], (662253, '746436'): ['69', '70', '71'], (662253, '746437'): ['59', '60'], (662253, '746107'): ['59', '60', '62', '66', '67', '68', '69', '70'], (662253, '745217'): ['74', '75', '76'], (662253, '745212'): ['67', '68', '69', '70', '71'], (662253, '745211'): ['64', '65', '66'], (662253, '746182'): ['74', '75', '76'], (662253, '745612'): ['78', '79', '80', '81', '82'], (662253, '745214'): ['64', '65', '66'], (662253, '745207'): ['76', '77', '78'], (662253, '745209'): ['68', '69', '70'], (662253, '745199'): ['56', '57', '58', '59'], (662253, '745204'): ['73', '74', '75'], (662253, '744955'): ['72', '73', '74', '75'], (662253, '746336'): ['63', '64', '65', '66'], (608723, '747035'): ['58', '59', '60'], (608723, '745743'): ['63', '64', '65', '66', '67'], (608723, '744853'): ['59', '60', '61', '62'], (608723, '744855'): ['64', '65', '66'], (608723, '745258'): ['47', '48', '49'], (608723, '745250'): ['56'], (608723, '745251'): ['55', '56', '57'], (608723, '745656'): ['50', '51', '52', '53'], (608723, '745657'): ['51', '52', '53'], (608723, '746297'): ['77', '78', '79', '80'], (608723, '745246'): ['51', '52', '53'], (608723, '745245'): ['55', '56', '57'], (608723, '745242'): ['62', '63', '64'], (608723, '746627'): ['57'], (608723, '746052'): ['69'], (608723, '746051'): ['56', '57', '58', '59', '60'], (608723, '745078'): ['65', '66', '67', '68', '69'], (608723, '745073'): ['51', '52', '53'], (608723, '745238'): ['46', '47', '48', '49'], (608723, '745243'): ['47', '48', '49'], (608723, '745240'): ['49', '50', '51'], (608723, '745236'): ['45'], (608723, '745237'): ['51'], (608723, '745233'): ['64', '65', '66', '67'], (608723, '745394'): ['51', '52', '53'], (608723, '746207'): ['82'], (608723, '746202'): ['54', '55', '56', '57'], (608723, '746201'): ['59', '60', '61'], (608723, '745229'): ['66', '67', '68'], (608723, '745227'): ['51', '55', '56', '57'], (608723, '746768'): ['55', '56', '57', '58', '59'], (608723, '746770'): ['60', '61'], (608723, '746925'): ['51', '52'], (608723, '745225'): ['65', '66', '67'], (608723, '745221'): ['48', '49', '50', '51'], (608723, '745223'): ['57', '58', '59', '60', '61'], (608723, '745220'): ['58', '59', '60', '61', '62'], (608723, '745216'): ['68', '69', '70', '71', '72'], (608723, '745462'): ['56', '57', '58'], (608723, '745464'): ['70', '71', '72', '73'], (608723, '746107'): ['71', '72'], (608723, '745218'): ['57', '58', '59', '60'], (608723, '745215'): ['55', '56', '57'], (608723, '745211'): ['46', '51', '52', '53'], (608723, '746184'): ['64', '65', '66'], (608723, '745619'): ['69'], (608723, '745612'): ['69', '70', '71', '72'], (608723, '745210'): ['49', '50', '51', '52', '53', '54'], (608723, '745208'): ['69', '70', '71'], (608723, '745207'): ['54', '55', '56', '57'], (608723, '745205'): ['44', '49', '50', '51', '52', '53'], (608723, '744958'): ['74', '75', '76', '77', '78'], (608723, '745203'): ['65', '66', '67', '68'], (668909, '746441'): ['4', '5', '6', '12', '13', '14', '15', '16', '20', '21', '22', '23', '28', '29', '30', '35', '36', '37', '38'], (668909, '746606'): ['1', '2', '3', '4', '5', '10', '11', '12', '13', '17', '18', '19', '20', '21', '22', '23', '30', '31', '32', '33', '34', '35'], (668909, '745873'): ['5', '6', '7', '8', '12', '13', '14', '18', '19', '20', '21', '22', '27', '28', '29', '33', '34', '35', '40', '41', '42', '43', '44'], (668909, '745949'): ['4', '5', '6', '7', '8', '9', '15', '16', '17', '18', '23', '24', '25', '26', '27', '28', '34', '35', '36', '37', '38'], (668909, '745705'): ['5', '6', '7', '8', '13', '14', '15', '21', '22', '23', '27', '28', '29', '30', '31', '32', '33', '37', '38', '39'], (668909, '746594'): ['1', '2', '3', '7', '8', '9', '10', '11', '17', '18', '19', '23', '24', '25', '26', '33', '34', '35', '36', '37'], (668909, '746265'): ['5', '6', '7', '8', '12', '13', '14', '15', '16', '20', '21', '22', '26', '27', '28', '36', '37', '38', '44', '45', '46', '50', '51', '52'], (668909, '746099'): ['5', '6', '7', '8', '9', '10', '11'], (668909, '746591'): ['1', '2', '3', '4', '5', '12', '13', '14', '15', '22', '23', '24', '28', '29', '30', '31', '32', '33', '37', '38', '39', '43', '44', '45'], (668909, '746587'): ['1', '2', '3', '7', '8', '9', '15', '16', '17', '18', '19', '20', '25', '26', '27', '28', '33', '34', '35', '36', '37', '38'], (668909, '745118'): ['4', '5', '6', '10', '11', '12', '16', '17', '18', '19', '20', '24', '25', '26', '27', '31', '32', '33', '39', '40', '41', '42'], (669212, '746617'): ['40', '41', '42', '43', '47', '48', '49'], (669212, '746606'): ['40', '41', '42', '43'], (669212, '746600'): ['72', '73', '74'], (669212, '746604'): ['42', '43', '44', '49', '50'], (669212, '745927'): ['46', '47', '53', '54', '55', '56', '57'], (669212, '745949'): ['50', '51', '56', '57', '58'], (669212, '745704'): ['52', '53', '54', '55', '61', '62', '63', '64'], (669212, '746597'): ['71', '72', '73', '74'], (669212, '746628'): ['11', '12', '13', '21', '22', '23', '27', '28', '29'], (669212, '746589'): ['50', '56', '57', '58', '59'], (669212, '746590'): ['58', '59', '63', '64', '65', '66', '67'], (669212, '746262'): ['40', '41', '42', '43', '47', '48', '49'], (669212, '746099'): ['67', '68', '69'], (669212, '746747'): ['35', '36', '37', '38', '45', '46', '47'], (669212, '746586'): ['53', '54', '55', '61', '62', '63'], (669212, '746588'): ['50', '51', '55', '56', '57'], (669212, '746587'): ['43', '44', '45', '46', '50', '51'], (669212, '746579'): ['74', '75', '76', '77', '78'], (669212, '745124'): ['39', '40', '41', '42', '46'], (669212, '746583'): ['28', '29', '30', '34', '35', '36'], (669212, '746580'): ['31', '35', '36', '37', '38'], (680704, '746655'): ['40', '41', '42'], (680704, '746656'): ['37', '38', '43', '44', '45', '46'], (680704, '746974'): ['57', '58', '59'], (680704, '746973'): ['40', '41', '42'], (680704, '746651'): ['48', '49'], (680704, '746652'): ['53', '54', '55'], (680704, '746646'): ['44', '45', '50', '51', '52'], (680704, '747131'): ['80', '81', '82'], (680704, '747125'): ['41', '42', '43'], (680704, '746398'): ['65', '66', '71', '72', '73', '74'], (680704, '746403'): ['50'], (680704, '746644'): ['53', '54', '55'], (680704, '746639'): ['45', '50', '51', '52'], (680704, '746640'): ['43', '44', '48', '49', '50'], (680704, '746798'): ['55', '56', '57', '58'], (680704, '745019'): ['42', '43'], (680704, '745015'): ['43', '44', '45'], (680704, '746642'): ['54', '55', '56', '57'], (680704, '746637'): ['44', '45'], (680704, '746633'): ['37', '38', '39'], (680704, '746549'): ['39', '40', '44', '45', '46'], (680704, '746552'): ['53', '54', '55'], (680704, '746632'): ['50', '51', '52'], (680704, '746631'): ['50', '51', '52'], (680704, '746054'): ['45', '46', '47'], (680704, '746701'): ['51', '52', '53', '54', '59'], (680704, '744926'): ['42', '43', '44', '45', '46'], (680704, '746619'): ['76', '77', '78', '79'], (680704, '746620'): ['71', '72', '73'], (680704, '746455'): ['47', '48', '49'], (680704, '746451'): ['57', '58', '59', '60'], (680704, '746452'): ['55', '56', '57', '58', '59'], (680704, '745067'): ['58', '59', '60'], (680704, '746618'): ['66', '67', '68', '69'], (680704, '746610'): ['40', '41', '45', '46'], (680704, '746614'): ['55', '56', '57'], (680704, '745553'): ['33', '34'], (680704, '745550'): ['49', '50', '51'], (680704, '746441'): ['49', '50', '51'], (680704, '746607'): ['73', '74', '75', '76'], (680704, '746608'): ['46'], (680704, '746606'): ['54', '55', '56'], (680704, '746602'): ['44', '49', '50', '51'], (680704, '745871'): ['51', '52', '53'], (680704, '745872'): ['70', '71'], (680704, '746603'): ['67', '68', '70'], (680704, '746598'): ['44', '48', '49', '50'], (680704, '745952'): ['42', '43', '44'], (680704, '745707'): ['57', '58'], (680704, '745705'): ['40', '41', '42', '43'], (680704, '746628'): ['1', '2', '3', '4', '5'], (680704, '746594'): ['41', '42', '43'], (680704, '746590'): ['43', '44', '45', '46', '47'], (680704, '746264'): ['56', '57', '58'], (680704, '746098'): ['46', '47'], (680704, '746746'): ['62', '63', '64'], (680704, '746745'): ['48', '49', '54', '55', '56', '57'], (680704, '746584'): ['65', '66', '67', '68'], (680704, '746587'): ['56', '57', '58', '59', '60'], (680704, '746579'): ['36', '37', '38'], (680704, '745124'): ['55', '56', '57', '58', '59', '60', '61'], (680704, '746580'): ['54', '55', '56', '57', '61', '62', '63'], (682120, '746654'): ['47', '48'], (682120, '746655'): ['47', '48', '49'], (682120, '746656'): ['58', '64', '65', '66'], (682120, '746974'): ['39', '40', '41', '42', '48', '49', '50'], (682120, '746970'): ['60', '61', '62'], (682120, '746650'): ['44', '45', '46'], (682120, '746652'): ['47', '51', '52'], (682120, '746646'): ['41', '42', '43'], (682120, '746398'): ['56', '63', '64'], (682120, '746399'): ['62', '67', '68'], (682120, '746645'): ['45', '46', '47', '52', '53'], (682120, '746643'): ['55', '56', '57'], (682120, '746638'): ['60', '61', '65', '66', '67', '68', '69', '70'], (682120, '746802'): ['64', '65', '66'], (682120, '745019'): ['61', '62', '63', '64'], (682120, '746641'): ['56', '57', '58'], (682120, '746633'): ['66'], (682120, '746634'): ['40', '41'], (682120, '746227'): ['61', '65', '66', '67'], (682120, '746552'): ['56', '57', '58'], (682120, '746551'): ['45', '46', '51', '52'], (682120, '746629'): ['44'], (682120, '746632'): ['56', '57', '58'], (682120, '746631'): ['78'], (682120, '746054'): ['39'], (682120, '746702'): ['49', '50', '51'], (682120, '746701'): ['60', '61'], (682120, '744925'): ['49', '50', '51', '52'], (682120, '746627'): ['24', '25', '26', '30', '31', '32', '33', '34'], (682120, '746626'): ['52', '53', '54'], (682120, '746621'): ['35', '36', '37'], (682120, '747021'): ['55', '56', '57'], (682120, '746288'): ['56', '57', '58'], (682120, '746290'): ['47', '48', '49', '50'], (682120, '746617'): ['58', '63', '64'], (682120, '746620'): ['41', '46', '47', '48'], (682120, '746616'): ['55', '56', '57'], (682120, '746455'): ['57', '58'], (682120, '745064'): ['46', '47', '48', '49'], (682120, '746613'): ['43', '44', '45', '46'], (682120, '746612'): ['58', '63', '64', '65', '66'], (682120, '746611'): ['45', '46'], (682120, '745553'): ['46', '47', '48', '49', '50'], (682120, '745550'): ['52'], (682120, '746441'): ['42', '43', '44'], (682120, '746607'): ['63', '64', '65', '66', '67', '68'], (682120, '746606'): ['50', '51', '52', '53'], (682120, '746600'): ['57', '62', '63'], (682120, '746602'): ['38', '42', '43'], (682120, '745871'): ['60', '64', '65', '66'], (682120, '746603'): ['54', '55', '56', '57'], (682120, '746598'): ['55', '56', '57', '58'], (682120, '745952'): ['55', '56', '57'], (682120, '745707'): ['94', '96', '97'], (682120, '746597'): ['59'], (682120, '746596'): ['52', '53', '54', '60', '61', '62', '63', '64'], (682120, '746589'): ['47', '48', '49'], (682120, '746593'): ['64', '65', '66'], (682120, '746590'): ['57'], (682120, '746592'): ['67', '68', '69', '70'], (682120, '746099'): ['61', '62', '63'], (682120, '746098'): ['43', '44', '45'], (682120, '746747'): ['56', '57', '58'], (682120, '746584'): ['47', '48', '49', '50'], (682120, '746585'): ['49', '50', '51'], (682120, '746587'): ['52'], (682120, '746579'): ['44', '45', '46', '53'], (682120, '745124'): ['47', '48', '49'], (682120, '745118'): ['43', '47', '48', '49'], (682120, '746583'): ['41', '42', '43', '44'], (682120, '746578'): ['50', '51', '52', '53'], (661403, '746970'): ['73', '74', '75'], (661403, '746651'): ['73', '74', '75', '76'], (661403, '746652'): ['72', '73', '74'], (661403, '746647'): ['65', '66', '67'], (661403, '746646'): ['72', '73', '74'], (661403, '747131'): ['63', '64', '65'], (661403, '747125'): ['78', '79'], (661403, '746398'): ['79', '80', '81', '82'], (661403, '746399'): ['76', '77'], (661403, '746643'): ['63', '64', '65'], (661403, '746639'): ['65', '66', '67'], (661403, '746640'): ['70', '71', '72'], (661403, '745019'): ['71', '72', '73'], (661403, '745014'): ['78', '79', '80', '81'], (661403, '746641'): ['63', '64', '65'], (661403, '746636'): ['57', '58', '59', '60', '61', '62'], (661403, '746637'): ['61', '62', '63'], (661403, '746633'): ['71', '72', '73'], (661403, '746226'): ['66', '67', '68', '69'], (661403, '746230'): ['75', '76', '77', '78'], (661403, '746635'): ['68', '69', '70', '71'], (661403, '746629'): ['67', '68', '69'], (661403, '746630'): ['71', '72', '73', '74'], (661403, '746054'): ['69', '70', '71', '72'], (661403, '746702'): ['73', '74', '75', '76', '77', '78'], (661403, '744924'): ['63', '64', '65'], (661403, '746625'): ['65', '66', '67'], (661403, '746626'): ['67', '68', '69'], (661403, '746622'): ['66', '67', '68'], (661403, '746621'): ['78', '79', '80', '81', '82'], (661403, '747022'): ['84', '85', '86', '87'], (661403, '746290'): ['67', '68', '69'], (661403, '746623'): ['66', '67', '68'], (661403, '746615'): ['72', '73', '74', '75'], (661403, '746616'): ['69', '70', '71', '72', '73'], (661403, '746451'): ['91', '92', '93', '94', '95'], (661403, '745067'): ['75', '76', '77', '78'], (661403, '746610'): ['63', '64', '65'], (661403, '746614'): ['73', '74', '75'], (661403, '746611'): ['59', '60', '61', '62'], (661403, '745554'): ['72', '73', '74', '75'], (661403, '745550'): ['72', '73', '74'], (661403, '746441'): ['64', '65', '66'], (661403, '746605'): ['73', '74', '75', '76'], (661403, '746600'): ['79', '80', '81'], (661403, '745873'): ['62', '63', '64'], (661403, '745872'): ['77', '78', '79', '80', '81'], (661403, '746603'): ['76', '77', '78'], (661403, '746601'): ['63', '64', '65', '66'], (661403, '745948'): ['56', '57', '58'], (661403, '745707'): ['79', '80', '81', '86', '88', '89'], (661403, '746595'): ['64', '65', '66'], (661403, '746596'): ['76', '77', '78', '79', '80'], (661403, '746589'): ['78', '79', '80', '81'], (661403, '746593'): ['83', '84', '85', '86'], (661403, '746265'): ['65', '66', '67'], (661403, '746104'): ['66', '67', '68'], (661403, '746746'): ['72', '73', '74', '75'], (661403, '746745'): ['77', '78', '79', '80'], (661403, '746585'): ['63', '64', '65'], (661403, '746588'): ['74', '75', '76'], (661403, '746582'): ['62', '63', '64'], (661403, '746579'): ['68', '69', '70'], (661403, '746581'): ['66', '67', '68'], (661403, '746583'): ['59', '60', '61', '62', '63', '64'], (661403, '746578'): ['73', '74', '75'], (683769, '745926'): ['49', '50', '51'], (683769, '746654'): ['53', '54', '55', '56'], (683769, '746653'): ['57', '58', '59'], (683769, '746648'): ['58', '59', '63', '64', '65'], (683769, '746976'): ['79', '80', '81'], (683769, '746970'): ['67', '68', '69'], (683769, '746651'): ['65', '66', '67', '68'], (683769, '746647'): ['54', '55', '56', '57'], (683769, '746646'): ['53', '54', '55'], (683769, '747131'): ['54', '55', '56', '57', '58', '59'], (683769, '747125'): ['61', '62', '63', '64', '65', '66'], (683769, '746398'): ['87', '88', '89'], (683769, '746645'): ['54', '55'], (683769, '746639'): ['56', '57', '58', '59', '60'], (683769, '746640'): ['55', '56', '57', '58', '63', '64', '65'], (683769, '746798'): ['63', '64', '65', '66'], (683769, '745019'): ['48', '49', '50'], (683769, '746641'): ['49'], (683769, '746636'): ['51', '52', '53'], (683769, '746633'): ['56', '57', '58'], (683769, '746634'): ['48', '49', '50', '51', '58'], (683769, '746226'): ['60', '61', '62'], (683769, '746552'): ['77', '78', '79'], (683769, '746635'): ['60', '61', '62'], (683769, '746629'): ['58', '59', '60', '61'], (683769, '746631'): ['66', '67', '68', '69', '70'], (683769, '746053'): ['52', '53', '54'], (683769, '746054'): ['61', '62', '63'], (683769, '744924'): ['57', '58', '59'], (683769, '744926'): ['71', '72'], (683769, '746627'): ['80', '81', '82', '83'], (683769, '746625'): ['59', '60', '61'], (683769, '746622'): ['61', '62'], (683769, '747021'): ['62', '63', '64', '65'], (683769, '746288'): ['62', '63', '64'], (683769, '746289'): ['62', '63', '64'], (683769, '746623'): ['53', '54', '55'], (683769, '746619'): ['68', '69', '70', '71'], (683769, '746615'): ['65', '66', '67'], (683769, '746616'): ['62', '63', '64'], (683769, '746451'): ['74', '75', '76'], (683769, '745067'): ['66', '67', '68', '69', '70'], (683769, '745066'): ['56', '57', '58', '59'], (683769, '746618'): ['52', '53', '54', '55'], (683769, '746614'): ['58', '59', '60'], (683769, '746611'): ['50', '51', '52'], (683769, '745554'): ['66', '67', '68'], (683769, '745550'): ['57', '58', '59', '60', '61'], (683769, '746444'): ['72', '73', '74', '75', '76'], (683769, '746605'): ['65', '66', '67'], (683769, '746600'): ['54', '55', '56'], (683769, '746602'): ['59', '60', '61'], (683769, '745873'): ['55', '56', '57'], (683769, '745872'): ['62', '67', '68', '69'], (683769, '746601'): ['57', '58', '59'], (683769, '745948'): ['49', '50', '51'], (683769, '745707'): ['63', '64', '65', '66'], (683769, '746595'): ['56', '57', '58', '59'], (683769, '746628'): ['63', '64', '65'], (683769, '746589'): ['69', '70', '71'], (683769, '746593'): ['71', '72', '73', '74', '75'], (683769, '746592'): ['60', '61', '62'], (683769, '746104'): ['58', '59', '60', '62'], (683769, '746746'): ['65', '66'], (683769, '746747'): ['63', '64', '65'], (683769, '746745'): ['70', '71', '72'], (683769, '746584'): ['54', '55', '56', '57', '58'], (683769, '746585'): ['56', '57', '58'], (683769, '746582'): ['68', '69', '70', '71', '72'], (683769, '746579'): ['61', '62', '63'], (683769, '745118'): ['57', '58', '59', '60'], (683769, '746581'): ['60', '61', '62'], (683769, '746583'): ['48', '49', '50', '51', '52'], (683769, '746578'): ['66', '67', '68'], (681982, '745032'): ['51'], (681982, '745034'): ['40', '44', '45', '46', '47', '48', '49'], (681982, '746408'): ['73', '74', '75'], (681982, '745028'): ['71', '72', '73', '74'], (681982, '746553'): ['61', '62', '66', '67', '68', '69'], (681982, '745014'): ['74', '76', '77'], (681982, '745016'): ['32', '36', '37', '38', '39', '40', '41'], (681982, '745012'): ['56', '57', '58', '65', '66', '67'], (681982, '745576'): ['72', '73', '74', '75'], (681982, '745573'): ['36', '41', '42', '43', '44'], (681982, '745017'): ['47', '51', '52', '53'], (681982, '746059'): ['63', '64', '66', '67'], (681982, '746061'): ['70', '71', '72'], (681982, '745010'): ['56', '57', '58', '62', '63', '64'], (681982, '745008'): ['57', '61', '62', '63'], (681982, '746139'): ['49', '50', '51', '52', '53', '54', '55', '56', '57'], (681982, '744993'): ['63', '64', '65', '66', '74', '75', '76'], (681982, '745712'): ['52', '53', '54', '59', '60', '61', '65', '66', '67', '68', '73', '74', '75', '76'], (681982, '746599'): ['53', '54', '55', '56', '57', '64', '65', '66', '67'], (681982, '746752'): ['81'], (681982, '744969'): ['44', '45', '46'], (681982, '744968'): ['56', '57', '58', '59', '60'], (681982, '744964'): ['80', '81', '82'], (502085, '746554'): ['62', '63', '64', '65'], (502085, '745015'): ['59', '60', '61', '62'], (502085, '745013'): ['71', '72', '73', '77', '78', '79'], (502085, '745901'): ['56', '57', '58', '62', '63', '64', '65'], (502085, '745017'): ['57', '58', '59'], (502085, '745011'): ['61', '62'], (502085, '746061'): ['60', '61', '62'], (502085, '745007'): ['55', '60', '61', '62', '63'], (502085, '745008'): ['52', '53', '54', '55', '56'], (502085, '746137'): ['63', '64', '65'], (502085, '746136'): ['59', '60', '61', '62', '63'], (502085, '745004'): ['55', '56', '60', '61', '62', '63', '64'], (502085, '745006'): ['64'], (502085, '744997'): ['51', '52', '53'], (502085, '745975'): ['67', '68', '69', '70'], (502085, '747018'): ['74'], (502085, '744992'): ['55', '60', '61', '62', '63', '64'], (502085, '744995'): ['61', '62', '63'], (502085, '746208'): ['64'], (502085, '746206'): ['64', '65', '66'], (502085, '746369'): ['66', '67', '68', '73', '74', '75', '76'], (502085, '744991'): ['56', '57', '58', '59', '60', '61'], (502085, '744988'): ['56', '57', '58'], (502085, '744984'): ['56', '57', '58', '64', '65', '66'], (502085, '745147'): ['62', '63', '64'], (502085, '744987'): ['61', '62', '67', '68', '69', '70'], (502085, '744982'): ['54', '55', '57', '58', '59', '60'], (502085, '744981'): ['77', '78', '79', '80'], (502085, '745708'): ['80', '81', '82'], (502085, '746921'): ['61', '62', '63', '64', '69', '70', '71'], (502085, '746919'): ['65', '66', '67', '69', '70'], (502085, '744977'): ['57', '58', '59'], (502085, '744975'): ['63', '64', '65', '66'], (502085, '744973'): ['56', '57', '58', '59', '60'], (502085, '746597'): ['66', '67', '68', '69', '70'], (502085, '746755'): ['65', '66', '67'], (502085, '746748'): ['57', '58', '59', '60', '61'], (502085, '744971'): ['54', '55', '56'], (502085, '744970'): ['64', '65', '66', '67'], (502085, '744966'): ['56', '57', '58', '59'], (502085, '744963'): ['59', '60', '61'], (502085, '744965'): ['58', '59', '60', '61', '62'], (502085, '745208'): ['66', '67', '68'], (502085, '745207'): ['70', '71', '72', '73', '74', '75'], (502085, '744961'): ['75', '76', '77', '78', '79', '80'], (502085, '744959'): ['55', '56', '57'], (502085, '744958'): ['70', '71', '72', '73'], (502085, '744955'): ['63', '64', '65'], (502085, '745605'): ['64', '65', '66', '67'], (502085, '746173'): ['63', '64', '65', '66'], (502085, '746174'): ['73', '74', '75', '76', '77'], (656641, '745024'): ['71', '72', '73', '74'], (656641, '745029'): ['37', '38', '39', '43', '44', '45'], (656641, '746405'): ['38', '39', '43', '44', '45', '46'], (656641, '746486'): ['60', '61', '62'], (656641, '746487'): ['47', '48', '49'], (656641, '747136'): ['45', '46', '47', '48', '49', '50'], (656641, '745026'): ['48', '49', '50', '51', '52'], (656641, '745025'): ['49', '50', '51'], (656641, '745022'): ['57', '58', '59'], (656641, '745023'): ['63', '64'], (656641, '745021'): ['42', '43', '49', '50', '51'], (656641, '746311'): ['54'], (656641, '745659'): ['48', '49', '50', '51', '52', '53'], (656641, '746553'): ['58', '59', '60'], (656641, '746555'): ['55', '56', '61'], (656641, '745019'): ['55', '56', '58', '59', '60'], (656641, '745578'): ['44', '45', '46', '47', '52', '53'], (656641, '745905'): ['55', '56', '60', '61', '62'], (656641, '745902'): ['47', '51', '52', '53', '60', '61'], (656641, '745011'): ['70', '71', '72', '73', '74'], (656641, '746062'): ['64', '65'], (656641, '746061'): ['51', '52', '53', '54', '55'], (656641, '745010'): ['50'], (656641, '745001'): ['67'], (656641, '746137'): ['31', '32', '33', '41', '42', '43', '48'], (656641, '745242'): ['39', '43', '44', '45'], (656641, '745004'): ['66'], (656641, '744997'): ['44', '45', '46'], (656641, '745974'): ['47', '48', '49', '50'], (656641, '745975'): ['83', '85', '86'], (656641, '747017'): ['44', '45', '46'], (656641, '744998'): ['52', '53', '54'], (656641, '744992'): ['51', '52', '53', '54'], (656641, '744995'): ['51', '52', '53'], (656641, '746205'): ['50', '51', '52', '53'], (656641, '746370'): ['61', '62'], (656641, '744994'): ['50', '51', '52', '53', '54', '55'], (656641, '744984'): ['47', '48', '49', '50'], (656641, '744906'): ['65'], (656641, '744902'): ['27', '28', '29', '30', '35', '36', '37'], (656641, '745148'): ['61', '62', '63', '64', '68', '69', '70'], (656641, '744986'): ['36', '37', '38', '39', '43', '44', '45', '46', '47', '48'], (489446, '745901'): ['66', '67', '68'], (489446, '745017'): ['64', '65', '66', '67', '68'], (489446, '745011'): ['75'], (489446, '745007'): ['72', '73', '74'], (489446, '745003'): ['66', '67', '68'], (489446, '746137'): ['69', '70', '71', '72', '73'], (489446, '746136'): ['67', '68', '69', '70'], (489446, '745004'): ['70', '71', '72', '73', '74', '75'], (489446, '745002'): ['67', '68', '69'], (489446, '745006'): ['68', '69', '70', '71'], (489446, '745975'): ['74', '75', '76', '78'], (489446, '744998'): ['75', '76', '77'], (489446, '744992'): ['70', '71', '72'], (489446, '744995'): ['67', '68', '69'], (489446, '746206'): ['71', '72', '73', '74'], (489446, '746369'): ['81', '82', '84'], (489446, '746370'): ['66', '67', '68', '69', '70'], (489446, '744991'): ['66', '67', '68'], (489446, '744988'): ['62', '63', '64', '65', '66'], (489446, '744985'): ['64', '65'], (489446, '744906'): ['80', '81', '82'], (489446, '745147'): ['74', '75', '76'], (489446, '744987'): ['71', '72', '77', '78', '79', '80'], (489446, '744982'): ['65', '66', '67', '72', '74', '75', '76', '77'], (489446, '746921'): ['72', '73', '74', '78', '79', '80'], (489446, '746919'): ['87', '88', '89'], (489446, '744980'): ['70', '71', '72', '73', '74'], (489446, '744975'): ['70', '71', '72', '73'], (489446, '744976'): ['65', '66', '67'], (489446, '744973'): ['64', '65', '66'], (489446, '746597'): ['75', '76', '77'], (489446, '746755'): ['71', '72', '73', '74'], (489446, '746748'): ['66', '67', '68', '69'], (489446, '744971'): ['60', '61', '62', '63', '64'], (489446, '744964'): ['83'], (489446, '744966'): ['65', '66', '67'], (489446, '744963'): ['68', '69', '70'], (489446, '744965'): ['69', '70', '71', '72'], (489446, '745208'): ['72', '73', '74', '75'], (489446, '744961'): ['90', '91', '92', '93', '94'], (489446, '744959'): ['61', '62', '63'], (489446, '744955'): ['69', '70', '71'], (489446, '746173'): ['71', '72', '73', '74'], (489446, '746171'): ['85', '86', '87'], (605177, '745751'): ['56', '57', '58', '59'], (605177, '746638'): ['71', '72', '73', '74'], (605177, '746640'): ['66', '67', '68'], (605177, '746479'): ['57', '58', '59'], (605177, '747202'): ['69', '70', '71', '72'], (605177, '746304'): ['71', '72', '73'], (605177, '746475'): ['69', '70', '71', '72', '73', '74'], (605177, '746468'): ['59', '60', '61', '62', '63', '64'], (605177, '746952'): ['52', '53', '54', '55', '56', '57'], (605177, '746953'): ['77', '78', '79', '87', '88', '89'], (605177, '745007'): ['64', '65', '66', '67', '68', '69'], (605177, '746461'): ['71', '72', '73', '74', '75'], (605177, '746464'): ['75', '76', '77'], (605177, '746465'): ['69', '70', '71'], (605177, '747107'): ['55', '56', '61'], (605177, '746458'): ['71', '72', '73', '74', '75', '76'], (605177, '746214'): ['52', '53', '54'], (605177, '746209'): ['60', '61', '62', '63'], (605177, '746693'): ['64', '65', '66'], (605177, '746694'): ['73'], (605177, '746451'): ['72', '73', '77', '78'], (605177, '746452'): ['60', '61', '62', '63'], (605177, '746447'): ['69', '70', '71', '72', '73'], (605177, '746449'): ['54', '58', '59', '60'], (605177, '744911'): ['53', '54', '55'], (605177, '746612'): ['79', '80'], (605177, '746611'): ['47', '48', '49'], (605177, '746609'): ['53', '57', '58', '59'], (605177, '746445'): ['67', '68', '69', '70'], (605177, '744986'): ['72', '73', '74', '75', '76', '77'], (605177, '744987'): ['41', '42', '46'], (605177, '744982'): ['48', '49', '50'], (605177, '744981'): ['49', '57', '58'], (605177, '745711'): ['52', '60', '61'], (605177, '746921'): ['45', '46', '47', '48'], (605177, '746919'): ['47', '48', '49', '55', '56', '57'], (605177, '744974'): ['59', '60', '61', '62'], (605177, '744975'): ['77', '78', '79'], (605177, '744973'): ['50', '51', '52'], (605177, '746597'): ['60', '61', '62'], (605177, '746595'): ['55'], (605177, '746752'): ['76', '77', '78', '79', '80'], (605177, '744970'): ['57', '58', '59', '60'], (605177, '744968'): ['48', '49', '54', '55'], (605177, '744963'): ['40', '41', '42'], (605177, '745208'): ['40', '41', '42', '43', '48', '49', '50'], (605177, '744962'): ['56', '57', '58'], (605177, '744958'): ['32', '33', '39', '40', '41', '45', '46', '47'], (605177, '745607'): ['63', '64', '65', '66', '71'], (605177, '746173'): ['56', '57', '58'], (600917, '746488'): ['44', '45', '46', '47', '52', '53', '54'], (600917, '746485'): ['69', '76', '77', '78', '79'], (600917, '745026'): ['58', '59', '60'], (600917, '745025'): ['56', '57', '58'], (600917, '745023'): ['55', '56', '57', '62'], (600917, '746311'): ['62', '63', '64', '65', '66'], (600917, '745669'): ['51', '52', '53', '54'], (600917, '745659'): ['79', '80', '81', '82', '83'], (600917, '746554'): ['49', '50', '51', '52', '59', '60', '61'], (600917, '745019'): ['51', '52', '53', '54'], (600917, '745015'): ['53', '54', '55'], (600917, '745013'): ['48', '49', '50', '51', '52'], (600917, '745573'): ['66', '67', '68', '69'], (600917, '745905'): ['63', '64', '68', '69', '70'], (600917, '745902'): ['62', '66', '67', '68', '72', '73', '74'], (600917, '745011'): ['44', '45', '46'], (600917, '746062'): ['57', '58', '63'], (600917, '745009'): ['42', '48', '49', '50', '54', '55', '56', '57'], (600917, '745003'): ['60', '61', '62'], (600917, '746137'): ['49', '50', '51', '56', '57', '58', '59'], (600917, '745242'): ['49', '50', '51', '52'], (600917, '745004'): ['51', '52', '53', '54'], (600917, '745006'): ['53', '54', '55', '60', '61', '62', '63'], (600917, '745974'): ['72', '73'], (600917, '747017'): ['54', '55', '56', '62', '63', '64', '65'], (600917, '744998'): ['41', '42', '43', '44', '50', '51'], (600917, '744992'): ['43', '44', '45', '46', '47'], (600917, '746208'): ['69', '70', '71'], (600917, '746206'): ['58', '59'], (600917, '746370'): ['50', '51', '52', '58', '59', '60'], (600917, '744994'): ['80', '81', '82', '83'], (600917, '744988'): ['43', '44', '45', '49', '50', '51'], (600917, '744989'): ['60', '61', '62'], (600917, '744984'): ['51'], (600917, '745148'): ['39', '40', '41', '42'], (600917, '744987'): ['29', '30', '38', '39', '40'], (600917, '744982'): ['45', '46', '47'], (600917, '744981'): ['37', '38', '39', '43', '44', '45', '46', '47', '48'], (600917, '745708'): ['51', '52', '53', '54', '63'], (600917, '745711'): ['40', '41'], (600917, '746924'): ['65', '66', '67', '68', '69', '70'], (600917, '744980'): ['55', '56', '57', '58', '59'], (600917, '744977'): ['66', '67'], (600917, '744973'): ['42', '43', '44', '49'], (600917, '746595'): ['51', '52', '53', '54'], (600917, '746755'): ['56', '57', '58', '59', '60'], (600917, '744970'): ['71', '72', '73', '74'], (600917, '744968'): ['39', '40', '45', '46', '47'], (600917, '744966'): ['49', '50', '51', '52'], (600917, '744963'): ['51', '52'], (600917, '744965'): ['52', '53', '54'], (600917, '745207'): ['65', '66'], (600917, '745209'): ['71', '72', '73', '74', '75'], (600917, '744961'): ['68', '69', '70'], (600917, '744958'): ['48', '49', '50', '51', '52', '61', '62', '63', '64'], (600917, '745607'): ['72', '73', '77', '78', '79', '80'], (600917, '746173'): ['59'], (600917, '746174'): ['62', '63', '64'], (670036, '744974'): ['37', '38', '43', '44', '45', '52', '53', '54'], (670036, '746595'): ['60', '61', '62', '63'], (670036, '746755'): ['46', '47', '48'], (670036, '744970'): ['79', '80', '81', '82', '83'], (670036, '744964'): ['71', '75', '76', '77', '78', '79'], (670036, '744963'): ['43', '44', '49', '50'], (670036, '747152'): ['57', '58', '62', '63', '64', '65', '66'], (670036, '745208'): ['55', '56', '57', '58'], (670036, '745206'): ['42', '43', '44', '45', '46', '47'], (670036, '744961'): ['51', '58', '59', '60'], (670036, '744962'): ['45', '49', '50', '51', '52'], (670036, '744960'): ['69', '70', '71', '72', '73'], (670036, '744955'): ['44', '45', '55', '56', '57'], (670036, '745607'): ['43', '47', '48', '49', '50', '56', '57', '58', '59'], (670036, '746173'): ['40', '41', '42', '43', '44'], (681869, '746889'): ['39', '45', '46', '47', '53', '54', '55'], (681869, '746399'): ['71', '72', '73', '74', '75'], (681869, '746395'): ['52', '53', '54', '55'], (681869, '745749'): ['50', '51', '52', '53', '54', '55', '56'], (681869, '746480'): ['71', '72', '73', '74', '75'], (681869, '746391'): ['54', '55', '56', '57', '61', '62', '63', '64', '68', '69', '70'], (681869, '746379'): ['44', '48', '49', '50', '54', '55', '56'], (681869, '745328'): ['58', '59', '60', '61', '65', '66', '67', '68'], (681869, '746381'): ['16', '17', '18', '19', '20', '21', '26', '27', '28', '29', '30'], (681869, '746375'): ['33', '38', '39', '40', '41', '42', '49', '50', '51'], (681869, '746374'): ['58', '59', '60', '61', '62'], (681869, '745802'): ['4', '5', '6', '7', '12', '13', '14', '15', '19', '20', '21', '28', '29'], (681869, '745884'): ['4', '5', '6', '15', '16', '17', '18', '19', '20', '21', '22', '26', '27', '28', '29'], (681869, '746367'): ['40', '41', '42', '43', '48', '49', '50'], (681869, '746369'): ['65'], (681869, '746370'): ['46', '47', '48', '49'], (681869, '745230'): ['65', '66', '67'], (681869, '745632'): ['39', '40', '41', '42', '43', '44'], (681869, '746362'): ['43', '44', '45', '46', '47', '48'], (681869, '746364'): ['58', '59', '60', '61', '62'], (681869, '746365'): ['65', '71', '72', '73', '74'], (681869, '746356'): ['57', '58', '59', '63', '64', '65', '66'], (681869, '746923'): ['75', '76', '77'], (681869, '746357'): ['56', '57', '58', '59', '60'], (681869, '746349'): ['72', '73', '74', '75', '76', '77'], (681869, '746998'): ['53', '54', '55', '61', '62', '63', '64', '65'], (681869, '746331'): ['49', '50', '51', '52', '53', '54', '55', '56'], (681869, '746578'): ['76', '77', '78'], (661527, '745034'): ['17', '23', '24', '25', '26', '30', '31', '32', '41', '42', '43'], (661527, '746322'): ['19', '26', '27', '28', '32', '33', '34'], (661527, '746404'): ['71', '72', '73', '74', '75'], (661527, '746406'): ['37', '38', '39', '40'], (661527, '746401'): ['78', '80', '81', '82'], (661527, '744865'): ['81'], (661527, '746888'): ['57', '58', '59', '60', '66', '67', '68', '69'], (661527, '746887'): ['63', '64', '65', '66'], (661527, '746560'): ['54', '55'], (661527, '746403'): ['65', '66', '67', '68', '69', '73', '74', '75', '76'], (661527, '746402'): ['47', '48', '49'], (661527, '746395'): ['37', '38'], (661527, '745752'): ['70', '71', '72', '73', '74'], (661527, '746476'): ['66', '67', '68', '69', '76', '77', '78', '79', '80', '81', '82'], (661527, '746397'): ['53', '54', '55', '60', '61', '62', '63'], (661527, '746389'): ['36', '37', '43', '44', '45', '50', '51', '52', '53'], (661527, '746390'): ['59', '60', '61'], (661527, '746386'): ['55', '56', '57'], (661527, '745658'): ['63', '64', '65', '66'], (661527, '746387'): ['46', '47', '48', '52', '53', '54'], (661527, '746382'): ['54', '55', '56'], (661527, '746380'): ['57', '61', '62', '63', '64'], (661527, '746216'): ['54', '55', '56'], (661527, '745334'): ['45'], (661527, '745328'): ['41', '42', '43', '44', '45', '51', '52', '53', '54'], (661527, '746381'): ['61', '62', '63', '64', '65'], (661527, '746372'): ['58', '59', '60', '61'], (661527, '745801'): ['63', '64', '65'], (661527, '745802'): ['43', '44', '45', '46', '47', '48'], (661527, '744913'): ['58', '59', '60', '65', '66', '67'], (661527, '745884'): ['30', '34', '35', '36', '37'], (661527, '746369'): ['60', '61', '62', '63', '64'], (661527, '746370'): ['63', '64', '65'], (661527, '745230'): ['48', '49', '50', '51', '57', '58', '59'], (661527, '745632'): ['49', '50', '51', '52', '53', '58', '59', '60', '61', '62'], (661527, '746364'): ['48', '49', '50'], (661527, '746365'): ['59', '60', '61', '62', '63', '64'], (661527, '746350'): ['66', '67', '68', '69'], (661527, '746338'): ['82', '83', '84', '85', '86', '87', '88'], (661527, '746334'): ['61', '67', '68', '69', '70', '71', '72'], (519151, '746408'): ['76', '77', '78', '79'], (519151, '746401'): ['62', '63', '64', '65', '66', '67', '68'], (519151, '744872'): ['63', '64', '65', '66'], (519151, '744865'): ['73', '74', '75', '76', '77', '78'], (519151, '746889'): ['67', '68', '69'], (519151, '746561'): ['60', '61', '62', '63'], (519151, '746398'): ['67', '68', '69', '70'], (519151, '746399'): ['63', '64', '65', '66'], (519151, '746402'): ['60', '61', '62', '63'], (519151, '745748'): ['63', '64', '65', '66'], (519151, '746393'): ['59', '60', '61', '62', '63'], (519151, '746396'): ['55', '56', '57', '58', '59'], (519151, '746390'): ['66', '67', '68'], (519151, '746385'): ['66', '67', '68', '69'], (519151, '745663'): ['71', '72', '73'], (519151, '745258'): ['57', '58', '59', '60', '61', '62', '63'], (519151, '745249'): ['60', '61', '62'], (519151, '746383'): ['58', '59', '60', '61', '62'], (519151, '746380'): ['80', '81', '82'], (519151, '746223'): ['69', '70', '71', '72', '73'], (519151, '746216'): ['73', '74', '75', '76', '77'], (519151, '745327'): ['64', '65', '66'], (519151, '746377'): ['52', '53', '54', '55'], (519151, '746781'): ['64', '65', '66', '67'], (519151, '746782'): ['70', '71', '72', '73'], (519151, '746376'): ['54', '55', '56'], (519151, '746372'): ['62'], (519151, '745801'): ['74', '75', '76', '77'], (519151, '745802'): ['64', '65', '66'], (519151, '744912'): ['67', '68', '69'], (519151, '745883'): ['62', '63', '64', '65'], (519151, '746367'): ['64', '65', '66', '67'], (519151, '746371'): ['64', '65', '66'], (519151, '745228'): ['60', '61', '62'], (519151, '745229'): ['62', '63', '64', '65'], (519151, '746361'): ['60', '61', '62', '63'], (519151, '746360'): ['55', '56', '57', '58', '59', '60', '61'], (519151, '746363'): ['58', '59', '60', '61', '62'], (519151, '746354'): ['60', '61', '62', '63', '64', '65'], (519151, '744982'): ['61', '62', '63', '64'], (519151, '744981'): ['71', '72', '73', '74', '75', '76'], (519151, '746927'): ['82', '83', '84', '85', '86'], (519151, '745055'): ['61', '62', '63', '64', '65', '66'], (519151, '746352'): ['56', '57', '58'], (519151, '746348'): ['63', '64', '65', '66'], (519151, '746666'): ['76', '77', '78'], (519151, '746346'): ['59', '60', '61', '62'], (519151, '746337'): ['56', '57', '58', '59', '60'], (519151, '746177'): ['67', '68', '75', '76', '77'], (519151, '745366'): ['62', '63', '64'], (519151, '746340'): ['74', '75', '76'], (519151, '746336'): ['58', '59', '60', '61'], (519151, '746578'): ['69', '70', '71', '72'], (657571, '744948'): ['47', '48', '52', '53', '54'], (657571, '745760'): ['65', '66', '67', '68', '69'], (657571, '745754'): ['52', '53', '57'], (657571, '746000'): ['63', '64', '65', '66'], (657571, '746001'): ['85', '86'], (657571, '747046'): ['60', '61', '62', '63'], (657571, '747048'): ['48', '49'], (657571, '745750'): ['44', '45', '46'], (657571, '745749'): ['60', '61'], (657571, '745092'): ['52', '53', '54', '55', '56'], (657571, '745908'): ['73', '74', '75'], (657571, '745747'): ['63', '64'], (657571, '745746'): ['73'], (657571, '745742'): ['67', '68', '69'], (657571, '745413'): ['58', '59', '60', '61'], (657571, '746225'): ['57', '58'], (657571, '745332'): ['50', '51', '52', '53', '54'], (657571, '745737'): ['54', '55', '56'], (657571, '745736'): ['44', '52', '53'], (657571, '746294'): ['80', '81', '82', '83', '84'], (657571, '746947'): ['56', '60', '61', '62'], (657571, '746945'): ['55', '56', '57'], (657571, '745732'): ['65', '66', '67', '68'], (657571, '745734'): ['75', '76', '77', '78', '79', '80'], (657571, '745808'): ['42', '43'], (657571, '744921'): ['56', '57', '58', '62', '63', '64'], (657571, '745730'): ['36', '37', '38', '39', '40'], (657571, '745726'): ['71', '72', '73', '74'], (657571, '745065'): ['68', '69', '70'], (657571, '745721'): ['71', '72', '73', '74'], (657571, '745722'): ['70'], (657571, '745715'): ['55', '56'], (657571, '745716'): ['66', '67', '68', '69', '70'], (657571, '746931'): ['38', '39', '40'], (657571, '746929'): ['71', '72', '78', '79', '80', '81', '82'], (657571, '746358'): ['58', '59', '60', '64', '65', '66', '67', '68', '69'], (657571, '744982'): ['78', '79', '80'], (657571, '746927'): ['61', '62', '63'], (657571, '746923'): ['56', '57', '61', '62', '63'], (657571, '746357'): ['46', '47', '48'], (657571, '746351'): ['63', '64', '65'], (657571, '746349'): ['50', '51', '52', '53', '54'], (657571, '746998'): ['70', '71', '72', '73'], (657571, '746996'): ['57', '62', '63', '64'], (657571, '745540'): ['53', '54', '55', '56', '57', '58', '64', '65', '66'], (657571, '746668'): ['55', '56', '57', '58', '64', '65', '66', '67', '68', '69'], (657571, '746664'): ['60', '61', '62', '63'], (657571, '746344'): ['61', '62', '63', '64', '65'], (657571, '746345'): ['31', '32', '33', '34', '35'], (657571, '746342'): ['48', '49', '50', '51', '56', '57', '58'], (657571, '746180'): ['72', '73', '74', '75'], (657571, '745367'): ['67', '68'], (657571, '746338'): ['74', '75', '76'], (657571, '746334'): ['44', '45', '46'], (657571, '746331'): ['60', '61', '62'], (593576, '746885'): ['66', '67', '68', '69', '70'], (593576, '745499'): ['72', '73', '74', '75', '76', '77'], (593576, '746878'): ['72', '73', '74', '75'], (593576, '746882'): ['58', '59', '60'], (593576, '746875'): ['73', '74', '75'], (593576, '745987'): ['64', '65', '66', '67'], (593576, '746877'): ['70', '71', '72', '73', '74'], (593576, '746870'): ['76', '77', '78', '79'], (593576, '746872'): ['69', '70', '71'], (593576, '746873'): ['75', '76', '77', '78'], (593576, '746705'): ['74', '75', '76'], (593576, '745077'): ['71', '72', '73', '74', '75', '76'], (593576, '745079'): ['73', '74', '75', '76', '77', '78'], (593576, '746868'): ['77', '78', '79', '80', '81', '82'], (593576, '746863'): ['68', '69', '70', '71', '72'], (593576, '745324'): ['72', '73', '74'], (593576, '745972'): ['73', '74', '75', '76', '77'], (593576, '746857'): ['71', '72', '73', '74'], (593576, '746855'): ['67'], (593576, '747015'): ['74', '75', '76'], (593576, '745153'): ['70', '71', '72'], (593576, '746853'): ['65', '66', '67', '68'], (593576, '746850'): ['69', '70', '71', '72'], (593576, '746851'): ['67', '68', '69', '70', '71'], (593576, '746278'): ['73', '74', '75', '76'], (593576, '746274'): ['75', '76', '77'], (593576, '746845'): ['64', '65', '70', '71', '72'], (593576, '746846'): ['65', '66', '67', '68'], (593576, '746760'): ['77', '78', '79', '80', '81', '82'], (593576, '746761'): ['78', '79', '80', '81'], (593576, '746843'): ['66', '67', '68', '69', '70', '71', '72'], (593576, '746838'): ['63', '64', '65', '66', '67'], (593576, '746994'): ['49', '55', '56', '57'], (593576, '745538'): ['51'], (593576, '746348'): ['55', '56', '57', '58'], (593576, '746347'): ['51', '52', '53', '54'], (593576, '746666'): ['70', '71', '72'], (593576, '746343'): ['60', '61', '62', '63'], (593576, '746345'): ['60', '61', '62', '63', '64'], (593576, '746346'): ['88', '89', '90', '91'], (593576, '746177'): ['51', '52', '53'], (593576, '746180'): ['56', '57', '58'], (593576, '745366'): ['78', '79', '80', '81', '82'], (593576, '746341'): ['50', '51', '52'], (593576, '746340'): ['59', '65', '66', '67', '68', '69', '70'], (593576, '746339'): ['50', '51', '52', '53'], (593576, '746331'): ['67', '68', '69', '70', '71', '72'], (593576, '746578'): ['62', '63', '64', '65'], (623352, '746418'): ['75', '76', '77'], (623352, '746410'): ['66', '67', '68', '69'], (623352, '746413'): ['69', '70', '71', '72', '73'], (623352, '746411'): ['61', '62', '63', '64', '65'], (623352, '745032'): ['65', '66', '67', '68'], (623352, '746326'): ['78', '79', '80'], (623352, '746408'): ['83', '84', '85'], (623352, '746404'): ['76'], (623352, '746406'): ['68', '69', '70', '71', '72', '73'], (623352, '746401'): ['72', '73', '74'], (623352, '744872'): ['70', '71', '72', '73', '74', '75'], (623352, '746561'): ['67', '68', '69', '70'], (623352, '746398'): ['75', '76', '77', '78', '83', '84', '85', '86'], (623352, '746402'): ['68', '69', '70', '71'], (623352, '746395'): ['73', '74', '75', '76'], (623352, '745748'): ['67', '73', '74', '75', '76'], (623352, '746479'): ['72', '73', '74', '75'], (623352, '746396'): ['64', '65', '66', '70', '71', '72', '73'], (623352, '746388'): ['70', '71', '72'], (623352, '746385'): ['73', '74', '75', '80', '81', '82'], (623352, '745663'): ['78', '79', '80'], (623352, '745653'): ['71', '72', '73', '74'], (623352, '745250'): ['63', '64', '65', '66', '67'], (623352, '745249'): ['67', '68', '69', '70', '71'], (623352, '746384'): ['66', '67', '68'], (623352, '746382'): ['74', '75', '76'], (623352, '746216'): ['78', '79', '83', '84', '85'], (623352, '745334'): ['67', '68', '69', '70'], (623352, '746378'): ['62', '63', '64'], (623352, '746377'): ['60', '61', '62', '63', '64'], (623352, '746781'): ['72', '73', '74', '75'], (623352, '746782'): ['78', '79', '80'], (623352, '746376'): ['61', '62', '63'], (623352, '746372'): ['68', '69', '70', '71', '72'], (623352, '745801'): ['84', '85', '86'], (623352, '744914'): ['65', '66', '67', '68'], (623352, '744912'): ['74', '75', '76'], (623352, '745884'): ['98', '99'], (623352, '745883'): ['71'], (623352, '746368'): ['64', '65', '66', '67'], (623352, '746367'): ['71', '72', '73'], (623352, '746371'): ['70', '71', '72'], (623352, '745228'): ['66', '67', '68'], (623352, '745229'): ['69', '70', '71', '72'], (623352, '746362'): ['77', '78', '79'], (623352, '746360'): ['67', '68', '69', '70', '71', '72'], (623352, '746363'): ['66', '67', '68', '69'], (623352, '746354'): ['69', '70', '71', '72'], (623352, '744982'): ['68', '69', '70', '71'], (623352, '744983'): ['71', '72'], (623352, '744981'): ['81', '82', '83'], (623352, '746922'): ['70', '71', '72', '73'], (623352, '745055'): ['70', '71', '72'], (623352, '745053'): ['61', '62', '63', '69', '70', '71'], (623352, '746359'): ['61', '62', '63'], (623352, '746351'): ['72', '73', '74'], (623352, '746994'): ['69', '70', '71'], (623352, '745538'): ['71', '72', '73', '77', '78'], (623352, '746352'): ['68', '69', '70'], (623352, '746353'): ['62', '63', '64', '65', '66'], (623352, '746345'): ['80', '81', '82', '83'], (623352, '746346'): ['67', '68', '69', '70'], (623352, '746337'): ['70', '71', '72', '73'], (623352, '746181'): ['73', '74', '75'], (623352, '746180'): ['76', '77', '78'], (623352, '745366'): ['65', '70', '71', '72'], (623352, '746341'): ['69', '70', '71'], (623352, '746340'): ['80', '81', '82'], (623352, '746339'): ['69', '70', '71', '72', '73', '74'], (623352, '746336'): ['62', '67', '68', '69'], (623352, '746580'): ['75', '76', '77', '78', '79', '80'], (605463, '746407'): ['67', '68', '69'], (605463, '745031'): ['75', '76', '77'], (605463, '745034'): ['50', '51', '52', '53', '57', '58', '59', '60'], (605463, '746323'): ['74'], (605463, '746322'): ['39', '40', '41', '47', '48', '49', '50'], (605463, '746404'): ['63', '64', '65', '66', '67'], (605463, '744866'): ['47', '48', '49', '50'], (605463, '746888'): ['48', '49', '50', '51', '52', '53'], (605463, '746560'): ['71', '72', '73', '74', '75'], (605463, '746403'): ['45', '51', '52', '53', '54'], (605463, '746394'): ['43', '44', '45'], (605463, '745752'): ['50', '51', '52', '53', '54'], (605463, '745749'): ['69', '70', '71', '72', '73'], (605463, '746480'): ['20', '21'], (605463, '746393'): ['72', '73', '74', '75'], (605463, '746396'): ['28', '29', '30', '35', '36', '37'], (605463, '746388'): ['45', '46', '47', '48'], (605463, '746392'): ['73', '74', '75', '76', '81', '82', '83'], (605463, '746385'): ['40', '41', '42', '43', '48', '49', '50', '51'], (605463, '745658'): ['46', '47', '48', '49', '50'], (605463, '745254'): ['64', '65', '66', '67'], (605463, '745250'): ['71', '72', '73', '74'], (605463, '746380'): ['35', '36', '37', '38', '39', '45', '46', '47', '48'], (605463, '746223'): ['77', '78', '79', '80'], (605463, '745334'): ['51', '52', '53', '58', '59', '60', '61'], (605463, '745327'): ['51', '52', '53', '54'], (605463, '746381'): ['85', '86', '87', '88'], (605463, '746783'): ['52', '53', '54'], (605463, '746782'): ['45', '46', '50', '51', '52'], (605463, '746375'): ['82'], (605463, '746372'): ['50', '51', '52', '53'], (605463, '745804'): ['55'], (605463, '745801'): ['50', '55', '56', '57', '58'], (605463, '745802'): ['70', '71', '72', '76', '77', '79', '80'], (605463, '744915'): ['48'], (605463, '744912'): ['61', '62', '63'], (605463, '746367'): ['51', '52'], (605463, '746369'): ['69', '70', '71', '72', '77', '78', '79', '80'], (605463, '745229'): ['48'], (605463, '745634'): ['67', '68', '69', '70'], (605463, '746361'): ['68', '69', '70'], (605463, '746362'): ['64', '65', '66', '67', '68'], (605463, '746360'): ['1', '2', '3', '8', '9', '10', '14', '15', '16'], (605463, '746354'): ['45', '46'], (605463, '746358'): ['50', '51', '52', '53', '54'], (605463, '744981'): ['53', '54', '55'], (605463, '746927'): ['64', '65'], (605463, '745058'): ['51', '56', '57', '58', '59'], (605463, '745053'): ['49', '50'], (605463, '746351'): ['48', '49', '55', '56', '57'], (605463, '746350'): ['48', '49', '50', '51', '52', '53', '60', '61', '62'], (605463, '746995'): ['55', '56', '57'], (605463, '746996'): ['48', '49', '54', '55', '56'], (605463, '745540'): ['46', '47', '48', '49'], (605463, '746347'): ['41', '42', '43', '44', '45', '46'], (605463, '746666'): ['15', '16', '22', '23', '24', '25', '26', '32', '33', '34', '35'], (605463, '746342'): ['62', '63', '64', '65'], (605463, '746177'): ['63', '64', '65', '66'], (605463, '745367'): ['56', '63', '64', '65', '66'], (669211, '746980'): ['64', '65', '66', '67', '72', '73', '74'], (669211, '747054'): ['48', '49', '50', '55', '56', '57'], (669211, '747053'): ['56', '57', '58'], (669211, '747049'): ['48'], (669211, '746325'): ['46', '47', '48', '49', '50'], (669211, '746324'): ['76', '77', '78'], (669211, '746246'): ['62', '63', '64'], (669211, '747050'): ['67', '68'], (669211, '747045'): ['41', '42', '43', '48', '49', '50'], (669211, '747047'): ['59', '60', '61'], (669211, '747044'): ['41', '49', '50', '51', '52'], (669211, '744859'): ['60', '61', '62', '63', '64'], (669211, '744858'): ['71', '72'], (669211, '747041'): ['58', '59', '60'], (669211, '747040'): ['69', '70', '71'], (669211, '747037'): ['71', '72', '73'], (669211, '745181'): ['63', '68', '69', '70'], (669211, '745180'): ['47', '48', '53', '54', '55', '59', '60', '61'], (669211, '746794'): ['35', '36', '37', '38', '39', '40', '41', '45', '46', '47', '48'], (669211, '747033'): ['70', '71', '72', '73', '74', '75'], (669211, '747029'): ['76', '77', '78', '79'], (669211, '744930'): ['71', '72', '73', '74'], (669211, '744923'): ['54', '55', '56', '57', '63', '64', '65', '66'], (669211, '745085'): ['66', '67', '68', '69', '70', '75', '76', '77', '78', '79'], (669211, '747028'): ['51', '57', '58', '59', '60', '61'], (669211, '747025'): ['40', '41', '42', '47', '48', '49', '55', '56', '57'], (669211, '745733'): ['39', '44', '45', '46', '47', '52', '53', '54'], (669211, '745734'): ['72', '73', '74', '81', '82', '83'], (669211, '746373'): ['63', '64', '65'], (669211, '747021'): ['66', '67', '68'], (669211, '747016'): ['76', '77', '78', '79'], (669211, '745234'): ['47', '48', '49'], (669211, '745236'): ['57', '58', '59', '64', '65', '66'], (669211, '747014'): ['54', '55', '56', '57', '58', '63', '64', '65', '66', '67', '68'], (669211, '747012'): ['53', '58', '59'], (669211, '747013'): ['49', '53', '54', '55', '56', '57'], (669211, '744990'): ['57', '58', '59', '63', '64', '65', '66'], (669211, '744991'): ['62', '63', '64'], (669211, '746036'): ['32', '33', '34', '38'], (669211, '746035'): ['65', '66', '67', '68'], (669211, '747010'): ['51', '52', '53'], (669211, '747007'): ['62', '63', '64', '65'], (669211, '747003'): ['64', '65', '66', '67'], (669211, '744899'): ['44', '45', '50', '51', '52', '53'], (669211, '747004'): ['62', '63', '64', '65', '66', '71', '72', '73', '74', '75'], (669211, '747001'): ['48', '49', '50', '51', '59', '60', '61'], (669211, '745780'): ['54', '55', '56', '61', '62', '63'], (669211, '746996'): ['42', '50', '51', '52'], (669211, '746103'): ['51', '52', '53'], (669211, '746102'): ['73', '74'], (669211, '746507'): ['32', '33', '34', '35'], (669211, '746992'): ['51', '52', '53'], (669211, '746990'): ['37', '38', '39'], (669211, '746989'): ['58', '64', '65', '66', '67'], (669211, '746908'): ['76', '77', '78'], (669211, '746419'): ['53', '54', '55', '56'], (669211, '746987'): ['29', '30', '34', '35', '36', '41', '42', '43'], (669211, '746983'): ['52', '53', '54', '55', '56'], (669211, '746982'): ['44', '45', '46'], (669211, '745688'): ['45', '46', '47'], (669211, '745690'): ['85', '86'], (669211, '745849'): ['78', '79', '80', '81', '82'], (657097, '746981'): ['73', '74', '75'], (657097, '746977'): ['49', '53', '54', '55'], (657097, '747055'): ['72', '73', '74', '75', '76', '77'], (657097, '747056'): ['45', '46', '47', '48'], (657097, '747049'): ['45', '46', '47'], (657097, '746316'): ['59', '63', '64'], (657097, '746242'): ['64'], (657097, '746244'): ['53', '54', '55', '60'], (657097, '747050'): ['72', '73', '74', '75'], (657097, '747045'): ['58'], (657097, '747047'): ['62', '67', '68', '69'], (657097, '747044'): ['59', '63', '64', '65'], (657097, '744859'): ['52', '53', '54', '59'], (657097, '744858'): ['93', '94', '95', '96'], (657097, '747043'): ['60', '61', '62'], (657097, '747040'): ['61', '62', '63', '67', '68'], (657097, '747038'): ['37', '38', '39', '40', '41', '42'], (657097, '747037'): ['51', '52', '53'], (657097, '747035'): ['52', '53', '54'], (657097, '745180'): ['44', '45', '46'], (657097, '746792'): ['65', '66', '67', '68'], (657097, '747034'): ['46', '47', '48', '53', '54', '55', '56'], (657097, '747032'): ['67'], (657097, '747029'): ['32', '40', '41', '42', '43', '50', '51', '52'], (657097, '744931'): ['65', '66', '67'], (657097, '744927'): ['47', '48', '49', '50', '51'], (657097, '745083'): ['53', '57', '58', '59', '60'], (657097, '747027'): ['44', '45', '52', '53', '54', '55'], (657097, '747026'): ['62', '63'], (657097, '747025'): ['79', '81', '82', '83'], (657097, '747023'): ['77', '78', '79', '80'], (657097, '745732'): ['44', '48', '49', '50'], (657097, '746375'): ['54', '55', '56', '57'], (657097, '747021'): ['58', '59', '60', '61'], (657097, '747017'): ['51', '52', '53'], (657097, '747018'): ['64', '65', '66', '67'], (657097, '745234'): ['50', '54', '55', '56', '57'], (657097, '745639'): ['54', '55', '56'], (657097, '745637'): ['57', '58', '59'], (657097, '747015'): ['62', '63', '64'], (657097, '747013'): ['46', '47', '48'], (657097, '747009'): ['41', '46', '47', '48'], (657097, '744994'): ['65', '66', '67', '68'], (657097, '746035'): ['48', '49', '50', '54'], (657097, '746037'): ['63', '64', '65'], (657097, '747008'): ['44', '50', '51', '52', '53', '54'], (657097, '747039'): ['42', '43', '44', '45'], (657097, '746607'): ['57', '58', '59', '60', '61', '62'], (657097, '746605'): ['53', '54', '62', '63', '64'], (657097, '746419'): ['44', '45', '46', '47', '48'], (657097, '746988'): ['69', '70', '71', '72'], (657097, '746984'): ['67', '68', '69', '70', '71', '72'], (657097, '745688'): ['48'], (657097, '745690'): ['51', '52', '56', '57', '58', '59'], (657097, '745847'): ['49', '54', '55', '56'], (657097, '745846'): ['62', '63', '64', '65'], (672335, '747043'): ['63', '64'], (672335, '747042'): ['46', '47', '48', '49', '50'], (672335, '747038'): ['47', '48', '49', '53', '54'], (672335, '747036'): ['62', '63', '64', '69', '70', '71', '72'], (672335, '745181'): ['51', '52', '53', '57', '58', '59', '60', '61', '62'], (672335, '746794'): ['74', '75', '76'], (672335, '747034'): ['70', '71', '72'], (672335, '747033'): ['56', '57', '58', '62'], (672335, '747032'): ['43', '44', '45'], (672335, '747029'): ['64', '65', '66'], (672335, '745082'): ['68', '69', '70', '71'], (672335, '745085'): ['56', '57', '58'], (672335, '747027'): ['60', '61', '62', '63'], (672335, '747026'): ['68', '69', '70', '71', '72', '73', '74'], (672335, '747025'): ['73'], (672335, '747023'): ['59', '60', '61', '62', '63'], (672335, '745732'): ['51', '52', '53', '59', '60'], (672335, '747021'): ['47', '51', '52', '53', '54'], (672335, '747019'): ['57', '58', '59'], (672335, '747017'): ['57', '58', '59', '60', '61'], (672335, '747018'): ['68', '69'], (672335, '745240'): ['60', '61', '62'], (672335, '745234'): ['61', '62', '63', '64'], (672335, '747015'): ['53', '54', '55', '56'], (672335, '747012'): ['69', '70', '71'], (672335, '747009'): ['49', '50', '54', '55'], (672335, '744994'): ['74', '75', '76'], (672335, '746035'): ['55', '56', '57', '58', '59', '60'], (672335, '746037'): ['82', '83', '84'], (672335, '747008'): ['58', '59', '60', '61'], (672335, '747005'): ['56', '57', '64'], (672335, '746608'): ['56', '57', '66'], (672335, '746606'): ['67', '68'], (672335, '744901'): ['62', '63', '64'], (672335, '744903'): ['68', '69', '70'], (672335, '745056'): ['75', '76', '77'], (672335, '745057'): ['54', '55', '56'], (672335, '747000'): ['58', '59', '60'], (672335, '747002'): ['57', '58', '59', '60'], (672335, '746999'): ['49', '50', '51', '52', '56', '57'], (672335, '745782'): ['55', '56', '57'], (672335, '745781'): ['62', '63', '64'], (672335, '746994'): ['58', '59', '60', '61', '62'], (672335, '746105'): ['55', '56', '57', '62'], (672335, '746511'): ['61', '62', '63'], (672335, '746507'): ['57', '58', '59'], (672335, '746990'): ['68', '69', '70', '71', '72', '73', '74'], (672335, '746993'): ['53', '54', '55'], (672335, '746907'): ['58', '59', '60', '64', '65'], (672335, '746908'): ['60', '61', '62', '63'], (672335, '746422'): ['56', '57', '58', '59'], (672335, '746985'): ['50', '51', '52', '53', '54'], (672335, '746983'): ['60', '61', '62'], (672335, '746982'): ['55', '60', '61', '62', '63'], (672335, '745688'): ['54', '55', '56', '57'], (672335, '745691'): ['43', '44', '45'], (672335, '745849'): ['70', '71', '72'], (572143, '745197'): ['56', '57', '58'], (572143, '745761'): ['43', '47', '48', '49'], (572143, '746084'): ['55', '56', '57'], (572143, '746081'): ['54', '55', '56', '57'], (572143, '746892'): ['51', '52', '53', '57', '58', '59'], (572143, '747134'): ['50', '55', '56', '57', '58', '59'], (572143, '747132'): ['25', '26', '31', '32', '33', '34'], (572143, '746078'): ['60', '64', '65', '66'], (572143, '746077'): ['56', '57', '58', '59', '60', '61'], (572143, '746073'): ['32', '33', '34', '35'], (572143, '746070'): ['76', '77', '78'], (572143, '745667'): ['42', '43', '44'], (572143, '746147'): ['48', '49', '50', '51'], (572143, '746069'): ['52', '53', '54', '55', '56', '61', '62', '63', '64', '65'], (572143, '746065'): ['62', '63', '64', '65'], (572143, '746064'): ['45', '46', '47', '48', '54', '55'], (572143, '746058'): ['34', '39', '40', '41', '42'], (572143, '745416'): ['64', '65', '66'], (572143, '746059'): ['68', '69', '70', '71'], (572143, '746061'): ['63', '64', '65', '66', '67', '68', '69'], (572143, '746054'): ['57', '58', '59', '60', '64', '65', '66', '67', '68'], (572143, '745811'): ['66', '71', '72', '73', '74', '75', '76'], (572143, '747012'): ['45', '46', '51', '52'], (572143, '744990'): ['71', '72', '73'], (572143, '744991'): ['65'], (572143, '746036'): ['39', '40', '45', '46', '47', '48'], (572143, '747010'): ['41', '42', '43', '44', '45', '46'], (572143, '747007'): ['70', '71', '72', '80', '81', '82', '83', '84'], (572143, '747003'): ['82', '83', '84'], (572143, '746607'): ['69', '70', '71', '72'], (572143, '746606'): ['60', '65', '66'], (572143, '744901'): ['38', '39', '40'], (572143, '744903'): ['48', '49', '50'], (572143, '745056'): ['58', '59', '60', '61'], (572143, '747002'): ['51'], (572143, '747001'): ['79', '80', '81', '82', '83'], (572143, '746997'): ['59', '60', '61', '62', '63'], (572143, '745782'): ['62', '63', '64', '65', '66'], (572143, '746995'): ['45', '46', '47', '52', '53', '54'], (572143, '746994'): ['50', '51', '52', '53', '54'], (572143, '746102'): ['36', '37', '44', '45', '46', '47'], (572143, '746507'): ['44', '45', '46'], (572143, '746991'): ['67', '68'], (572143, '746990'): ['60', '61', '62', '63'], (572143, '746986'): ['65', '66', '67', '68'], (572143, '746909'): ['47', '52', '53', '54', '55', '56'], (572143, '746987'): ['48', '49', '50', '55', '56', '57'], (642397, '745591'): ['63', '64', '65', '66', '67', '68'], (642397, '746723'): ['56', '57', '58'], (642397, '746725'): ['66', '67', '68', '69', '70', '71'], (642397, '746236'): ['69', '70', '71', '72'], (642397, '746238'): ['71', '72', '73', '74', '75'], (642397, '745587'): ['56', '57', '58', '59'], (642397, '746071'): ['70', '71', '72', '73', '74', '75'], (642397, '746068'): ['83'], (642397, '745826'): ['61', '62', '63'], (642397, '745580'): ['49', '50', '51'], (642397, '745581'): ['73', '74', '75'], (642397, '745576'): ['60', '61', '62', '63'], (642397, '746558'): ['85', '86', '87'], (642397, '745333'): ['58', '59', '60'], (642397, '745575'): ['66', '67', '68', '69'], (642397, '745572'): ['77', '78', '79', '80', '81'], (642397, '745571'): ['45', '46', '47'], (642397, '746948'): ['74', '75', '76', '77', '78'], (642397, '746944'): ['44', '48', '49', '50', '51', '52'], (642397, '747024'): ['50', '55', '56', '57'], (642397, '745561'): ['62', '63', '64', '65'], (642397, '745564'): ['62', '63', '64'], (642397, '746454'): ['60', '61', '62'], (642397, '746456'): ['31', '32', '33', '34'], (642397, '745555'): ['76', '77', '78'], (642397, '746860'): ['46', '47', '48'], (642397, '746857'): ['65', '66', '67', '68', '69', '70'], (642397, '747099'): ['55', '56', '57', '58'], (642397, '745557'): ['58', '59', '60', '61', '62', '63'], (642397, '745552'): ['70', '71', '72'], (642397, '745549'): ['57'], (642397, '745481'): ['57', '58', '59', '60', '61'], (642397, '745878'): ['59', '60'], (642397, '745877'): ['78', '79', '80'], (642397, '745550'): ['38', '39', '40'], (642397, '745547'): ['63', '64', '65', '66', '67', '68', '69'], (642397, '746605'): ['47', '48', '49', '50', '51', '52'], (642397, '746606'): ['57', '58', '59'], (642397, '744901'): ['41', '42', '43', '44', '45', '46'], (642397, '744899'): ['65', '66', '67'], (642397, '747004'): ['56', '57', '58'], (642397, '747002'): ['48', '49', '50'], (642397, '746997'): ['68', '69', '70', '71'], (642397, '746998'): ['56', '57', '58', '59', '60'], (642397, '746103'): ['58', '59', '60'], (642397, '746102'): ['52', '53', '54'], (642397, '746507'): ['51', '52', '53'], (642397, '746505'): ['70', '71', '72', '73', '74'], (642397, '746990'): ['52', '53', '54', '55'], (642397, '746986'): ['50', '55', '56'], (642397, '746909'): ['57', '58', '59', '60'], (642397, '746423'): ['47', '52', '53', '54'], (642397, '746419'): ['62', '63', '64', '65'], (642397, '746988'): ['52', '53', '54', '55'], (642397, '746985'): ['72'], (642397, '746983'): ['66'], (642397, '745688'): ['65', '66', '71', '72'], (642397, '745690'): ['73', '74', '75'], (642397, '745846'): ['57'], (666974, '745523'): ['62', '63', '64', '65', '66'], (666974, '746977'): ['64'], (666974, '747056'): ['53', '54', '55', '56', '62', '63'], (666974, '747053'): ['59', '60', '61', '65', '66', '67'], (666974, '747049'): ['55', '56', '57', '58'], (666974, '746324'): ['66', '67', '68', '69', '70'], (666974, '746242'): ['56', '57', '58', '62', '63'], (666974, '746244'): ['61', '62', '67', '68'], (666974, '747050'): ['51', '55', '56', '57'], (666974, '747045'): ['63', '64', '65', '66'], (666974, '747046'): ['53', '57', '58', '59', '66'], (666974, '746721'): ['55', '56'], (666974, '746720'): ['66', '67', '68'], (666974, '744858'): ['53', '54', '55'], (666974, '747043'): ['69', '70', '71', '72'], (666974, '747041'): ['65', '66', '67', '68'], (666974, '747040'): ['53', '54', '55', '56'], (666974, '747037'): ['54'], (666974, '747036'): ['59', '60', '61'], (666974, '746792'): ['84', '85', '86'], (666974, '746793'): ['57', '58', '59', '60'], (666974, '746789'): ['59'], (666974, '747033'): ['63', '64', '65'], (666974, '747032'): ['54', '55', '56', '57'], (666974, '747029'): ['70', '71', '72'], (666974, '744930'): ['58', '63', '64', '65'], (666974, '744927'): ['62', '63', '64'], (666974, '745083'): ['66', '67', '68', '69'], (666974, '745080'): ['62', '63', '64', '65'], (666974, '747027'): ['56'], (666974, '747025'): ['69', '70', '71', '72'], (666974, '747023'): ['81'], (666974, '745732'): ['61', '62', '63', '64'], (666974, '747022'): ['69', '70', '71', '72', '73', '74'], (666974, '747017'): ['47', '48', '49', '50'], (666974, '747018'): ['54', '55', '56'], (666974, '745240'): ['54', '55'], (666974, '745639'): ['60', '61', '62', '63'], (666974, '745637'): ['63', '64', '65', '66'], (666974, '747009'): ['56', '57', '61', '62', '63'], (666974, '744994'): ['77', '78', '79'], (666974, '746037'): ['78'], (666974, '747008'): ['76', '77', '78', '79', '80', '81'], (666974, '747005'): ['65', '66'], (666974, '746608'): ['76', '77', '78'], (666974, '746606'): ['74', '75', '76', '77'], (666974, '744903'): ['56', '57', '58', '59', '67'], (666974, '745062'): ['69', '70', '71', '72'], (666974, '745056'): ['71', '72', '73', '74'], (666974, '747000'): ['48', '49', '50'], (666974, '747002'): ['65', '66', '67'], (666974, '746997'): ['49', '50', '51', '52'], (666974, '746999'): ['58', '59', '60', '61'], (666974, '745782'): ['67', '68', '75'], (666974, '746996'): ['53', '58', '59', '60', '61'], (666974, '746105'): ['63', '64', '65'], (666974, '746507'): ['60', '61'], (666974, '746993'): ['46', '47', '48'], (666974, '746907'): ['66', '67', '68', '69'], (666974, '746908'): ['64'], (666974, '746985'): ['61', '62', '63'], (666974, '746983'): ['80', '81', '82', '83', '84'], (666974, '746982'): ['67', '68', '69', '70'], (666974, '745688'): ['58', '59', '64'], (666974, '745691'): ['39', '40', '41', '42'], (666974, '745849'): ['64', '65', '66'], (622554, '745595'): ['46', '47', '48', '49'], (622554, '745597'): ['69', '70', '71'], (622554, '745590'): ['66', '67', '68', '69'], (622554, '746725'): ['51', '52', '53', '54', '55', '56'], (622554, '745424'): ['76', '77', '78'], (622554, '746236'): ['59', '60', '61', '62', '63', '64'], (622554, '745589'): ['48', '49', '50', '51'], (622554, '745583'): ['85', '86', '87', '88'], (622554, '745584'): ['50', '51', '52', '53'], (622554, '745826'): ['51', '52', '58', '59', '60'], (622554, '745582'): ['78', '79', '80'], (622554, '745577'): ['63', '64', '65', '66', '67'], (622554, '746556'): ['55', '56', '57', '58'], (622554, '746557'): ['61', '62', '63', '64'], (622554, '745333'): ['69', '70', '71'], (622554, '745575'): ['58', '59', '60'], (622554, '745568'): ['74', '75', '76', '77'], (622554, '745814'): ['67', '68', '69', '70'], (622554, '746948'): ['68', '69'], (622554, '747025'): ['84', '85', '86'], (622554, '747023'): ['73', '74', '75'], (622554, '745561'): ['58', '59', '60', '61'], (622554, '745562'): ['63', '64', '65', '66'], (622554, '746454'): ['53', '54', '55', '56'], (622554, '745555'): ['65', '66', '67', '68', '69'], (622554, '746857'): ['41', '42', '43', '44', '48', '49', '50'], (622554, '747099'): ['63', '64', '65', '66'], (622554, '745557'): ['50', '51', '52', '53', '54'], (622554, '745552'): ['53', '54', '55', '56'], (622554, '745551'): ['42', '43', '44', '45', '51', '52', '53', '54'], (622554, '745478'): ['52', '53', '54'], (622554, '745881'): ['54', '55', '56', '57', '58'], (622554, '747010'): ['59', '60', '61', '62', '63'], (622554, '747008'): ['67', '68', '69'], (622554, '747005'): ['72', '73', '74'], (622554, '746608'): ['67', '68', '69', '70'], (622554, '746606'): ['82', '83', '84', '85'], (622554, '744903'): ['74', '75', '76', '77'], (622554, '745062'): ['63', '64', '65'], (622554, '745056'): ['81', '82', '83', '84'], (622554, '747000'): ['66', '67', '68'], (622554, '746999'): ['62', '67', '68', '69', '70'], (622554, '745780'): ['67', '68'], (622554, '745781'): ['65', '69'], (622554, '746995'): ['72', '73', '74'], (622554, '746996'): ['65', '66', '67'], (622554, '746105'): ['70', '71', '72', '73', '74'], (622554, '746511'): ['70', '71', '72', '73', '74'], (622554, '746993'): ['56', '61', '62', '63', '64', '65'], (622554, '746907'): ['75', '76', '77', '78'], (622554, '746908'): ['68', '69', '70', '71'], (622554, '746422'): ['64', '65', '66', '67', '68', '69', '70'], (622554, '746985'): ['68', '69', '70', '71'], (622554, '746983'): ['63', '64', '65'], (622554, '745688'): ['73', '74'], (622554, '745847'): ['72', '73', '74', '75', '76', '77'], (622554, '745846'): ['53', '54', '55', '56'], (676684, '746818'): ['66', '70', '71', '72', '73', '74'], (676684, '745842'): ['43'], (676684, '746497'): ['51', '52', '53', '54'], (676684, '746498'): ['56', '57', '58', '64', '65', '66', '67'], (676684, '745520'): ['57'], (676684, '746491'): ['72', '73', '74', '75', '76', '77', '78'], (676684, '746492'): ['49', '50', '55', '56', '57'], (676684, '746487'): ['53', '59', '60', '61'], (676684, '745922'): ['53', '54', '55'], (676684, '745107'): ['54', '55', '56'], (676684, '745099'): ['49', '50', '51', '52', '53', '54'], (676684, '746482'): ['67', '68', '69', '70'], (676684, '746478'): ['46', '47', '48'], (676684, '745750'): ['64', '65', '66'], (676684, '746479'): ['49', '50', '51', '52'], (676684, '746476'): ['64', '65', '70', '71', '72', '73', '74', '75'], (676684, '746472'): ['53', '54', '55', '56'], (676684, '747201'): ['61', '62', '63'], (676684, '746307'): ['48', '49', '50', '51', '52', '58', '59', '60'], (676684, '746304'): ['46', '47', '48'], (676684, '746475'): ['62', '63'], (676684, '746469'): ['55', '56', '57', '58', '64'], (676684, '746468'): ['52', '53', '54'], (676684, '746952'): ['45', '46', '47'], (676684, '745007'): ['56', '57', '58', '59'], (676684, '746466'): ['55', '56', '57', '58'], (676684, '746463'): ['62'], (676684, '746462'): ['74', '75', '76', '77', '78', '79', '80'], (676684, '746381'): ['81', '82', '83', '84'], (676684, '747108'): ['51', '52', '53', '54', '55'], (676684, '747107'): ['62', '63'], (676684, '746459'): ['40', '41'], (676684, '746457'): ['57', '58', '59'], (676684, '746456'): ['15', '19', '20', '21'], (676684, '746214'): ['46', '47', '51'], (676684, '746211'): ['68', '69', '70', '71'], (676684, '745886'): ['51', '52', '53', '54'], (676684, '746693'): ['51', '57', '58', '59'], (676684, '746691'): ['63', '64', '65'], (676684, '746451'): ['79', '80', '85', '86', '88', '89', '90'], (676684, '746450'): ['51', '52', '53', '54'], (676684, '746447'): ['90', '92', '93'], (676684, '744911'): ['46', '47'], (676684, '744909'): ['50', '51', '52', '53'], (676684, '746614'): ['52', '53', '54', '61', '62', '63', '64'], (676684, '746448'): ['65', '66', '73', '74', '75', '76', '77', '78', '79'], (676684, '746444'): ['69', '70', '71'], (676684, '746441'): ['45', '46', '47', '48'], (676684, '746440'): ['53', '57', '58', '59', '60'], (676684, '746439'): ['60', '61', '62', '63'], (676684, '745221'): ['52', '53', '54', '55'], (676684, '745226'): ['63', '64', '65', '66', '67'], (676684, '745303'): ['76', '77', '78', '79'], (676684, '746436'): ['54', '55', '56', '57', '58'], (676684, '746431'): ['55', '56', '57'], (676684, '746758'): ['53', '61', '62', '63', '64'], (676684, '746754'): ['54', '55', '61', '62', '63', '64'], (676684, '746432'): ['56', '57', '58', '59'], (676684, '746426'): ['50', '51', '55', '56', '57'], (676684, '745370'): ['73', '74', '75', '80', '81', '82'], (676684, '745611'): ['52', '58', '59', '60'], (676684, '746425'): ['58', '64', '65', '66'], (676684, '746419'): ['51', '52', '57', '58', '59', '60'], (676684, '746260'): ['65', '66', '67'], (676684, '746254'): ['76', '77'], (676684, '746983'): ['50', '51'], (676684, '746982'): ['58', '59', '64', '65', '66'], (676684, '746420'): ['62', '63', '64', '65'], (676684, '746417'): ['48', '49', '50', '58', '59'], (671345, '745844'): ['73', '74', '75', '76'], (671345, '746497'): ['59', '60', '61'], (671345, '745517'): ['71', '72', '73', '74'], (671345, '746489'): ['72', '73', '74', '78', '79', '80', '81'], (671345, '746492'): ['66', '67', '68', '69', '70'], (671345, '746486'): ['64', '65', '66', '67'], (671345, '745922'): ['71', '72', '73', '74'], (671345, '745102'): ['69', '70', '71'], (671345, '746484'): ['73', '74', '75', '76', '77', '78'], (671345, '746483'): ['65', '66', '67', '68', '69'], (671345, '746477'): ['54', '55', '56'], (671345, '745756'): ['68', '69', '70', '71'], (671345, '746640'): ['51', '52', '53'], (671345, '746479'): ['60', '61', '62', '63', '64'], (671345, '746471'): ['71', '72', '73'], (671345, '746473'): ['53', '54', '55'], (671345, '746309'): ['54', '55', '56', '57', '58'], (671345, '746475'): ['75'], (671345, '746470'): ['55', '56', '57'], (671345, '746469'): ['74', '75', '76', '77', '78'], (671345, '746952'): ['61', '62', '63', '64'], (671345, '746953'): ['69', '74', '75', '76'], (671345, '745010'): ['69'], (671345, '746461'): ['65', '66', '67'], (671345, '746464'): ['66', '67', '68', '69'], (671345, '746377'): ['56', '57', '58', '59'], (671345, '746460'): ['58', '62', '63', '64'], (671345, '746454'): ['63', '64', '65', '66'], (671345, '746211'): ['76', '77', '78', '82', '83'], (671345, '746693'): ['70', '71', '72', '73', '74', '75', '76'], (671345, '746694'): ['69', '70', '71', '72'], (671345, '746453'): ['67', '69'], (671345, '746450'): ['63', '64', '69', '70'], (671345, '746449'): ['61', '65', '66', '67', '68'], (671345, '744911'): ['62', '63', '64'], (671345, '744909'): ['69', '70'], (671345, '746609'): ['50', '51', '52'], (671345, '746446'): ['60', '61', '62', '63'], (671345, '746445'): ['63', '64', '65'], (671345, '746441'): ['52', '53', '54', '55', '56'], (671345, '746440'): ['74', '75', '76', '82', '83', '84', '85'], (671345, '746439'): ['73'], (671345, '745221'): ['60', '61', '62'], (671345, '745223'): ['62', '63', '64', '65'], (671345, '745308'): ['60', '64', '65', '66'], (671345, '746436'): ['66', '67', '68'], (671345, '746437'): ['61', '62', '63'], (671345, '746434'): ['65', '66', '67', '68', '69'], (671345, '746758'): ['79', '80', '81'], (671345, '746754'): ['81', '82', '83'], (671345, '746429'): ['66'], (671345, '746430'): ['65', '66', '67'], (671345, '746426'): ['61', '62', '63'], (671345, '745373'): ['60', '61', '62', '63'], (671345, '745370'): ['87', '88', '90'], (671345, '745611'): ['61', '62'], (671345, '745613'): ['69', '70', '71'], (671345, '746425'): ['71', '72', '73', '74'], (671345, '746427'): ['61', '62', '63', '64', '65'], (671345, '746419'): ['61', '66', '67', '68'], (671345, '746260'): ['79', '80', '81'], (671345, '746261'): ['78', '79', '80'], (671345, '746983'): ['73', '74', '75', '76'], (671345, '746982'): ['71', '72', '73'], (671345, '746415'): ['66', '67', '68'], (671345, '746417'): ['69'], (669724, '746439'): ['12', '19', '20', '21', '22', '26', '27', '28'], (669724, '745223'): ['74', '75', '76'], (669724, '745226'): ['4', '5', '6', '10', '11', '12', '13'], (669724, '745303'): ['62', '63', '67', '68', '69', '70'], (669724, '746438'): ['10', '11', '12', '13', '19', '20', '21'], (669724, '746431'): ['48', '49', '50', '51'], (669724, '746837'): ['50', '51', '55', '56', '57'], (669724, '746756'): ['54', '55', '56', '57', '61', '62', '63', '64', '65'], (669724, '746429'): ['41', '42', '43', '50', '51', '52', '56', '57'], (669724, '746433'): ['50', '51', '52'], (669724, '745373'): ['18', '19', '20', '21', '22'], (669724, '745611'): ['80', '81', '82'], (669724, '745613'): ['5', '6', '7'], (669724, '746427'): ['46', '47', '48', '49'], (669724, '746423'): ['49', '50', '51'], (669724, '746260'): ['43', '44', '45', '46', '47', '55', '56', '57', '63', '64'], (669724, '746254'): ['52', '53', '54', '59', '60', '61', '62', '66', '67'], (669724, '746982'): ['48', '49', '50', '51', '56', '57'], (669724, '746420'): ['39', '44', '45', '46', '47'], (669724, '746417'): ['1', '2', '3', '4', '9', '10', '11'], (656412, '746818'): ['38', '43', '44', '45', '46'], (656412, '745842'): ['48', '49', '50', '51', '55', '56', '57', '58'], (656412, '745843'): ['58', '64', '65', '66', '67'], (656412, '746493'): ['52', '53', '54', '55', '56', '61', '62', '63', '64'], (656412, '746491'): ['45', '46', '47', '55', '56', '57', '58'], (656412, '746488'): ['55', '56', '57', '63', '64', '65', '66'], (656412, '746485'): ['31', '35', '36', '37', '38', '46', '47', '48'], (656412, '745920'): ['72', '73', '74', '75', '76', '77'], (656412, '745102'): ['43', '44', '45', '46', '47', '52', '53', '54', '55'], (656412, '746484'): ['47', '48', '49', '53', '54', '55'], (656412, '746481'): ['55', '60', '61', '62', '63', '64', '65', '75', '76', '77'], (656412, '745756'): ['44', '45', '46', '53', '54'], (656412, '746638'): ['55', '56', '57', '62', '63', '64'], (656412, '746479'): ['69', '70', '71'], (656412, '746471'): ['50', '54', '55', '56', '60', '61', '62', '63', '64'], (656412, '747198'): ['58', '59', '60', '61', '62', '63', '69', '70', '71'], (656412, '746474'): ['47', '48', '52', '53', '54', '55', '56', '57', '58'], (656412, '746470'): ['49', '50', '51'], (656412, '746954'): ['49', '57', '58', '59'], (656412, '746953'): ['60', '61', '62', '66', '67', '68'], (656412, '745010'): ['41', '42', '43', '44', '45', '51', '52', '53', '54'], (656412, '746209'): ['47', '48', '49', '50', '54', '55', '56'], (656412, '745888'): ['58', '59', '60', '61', '65', '66', '67'], (656412, '746694'): ['4', '5', '6', '11', '12', '13'], (656412, '746691'): ['69', '70', '71', '72'], (656412, '746453'): ['57', '58', '64', '65', '66'], (656412, '746447'): ['52', '53', '58', '59', '60', '61', '62', '63'], (656412, '746449'): ['69', '70'], (656412, '744910'): ['20', '21', '22', '26', '27', '28'], (656412, '746614'): ['16', '17', '21', '22', '23', '24', '25', '31', '32', '33'], (656412, '746609'): ['37', '38', '39', '43', '44', '45'], (656412, '746445'): ['1', '2', '3', '4', '5', '9'], (656412, '746441'): ['19', '24', '25', '26', '27', '31', '32', '33'], (656412, '746439'): ['1', '2', '3', '4', '5', '9', '10', '11'], (656412, '745304'): ['4', '5', '6', '7', '8', '9'], (656412, '746437'): ['1', '2', '3', '4', '5', '6', '7'], (656412, '746839'): ['5', '6', '7', '8', '9', '16', '17'], (689225, '746468'): ['69', '70', '71', '72', '73'], (689225, '745009'): ['51', '52', '53', '58', '59', '60', '64', '65', '66', '67'], (689225, '746466'): ['39', '40', '41', '42', '43', '47', '48', '49', '50', '51'], (689225, '746464'): ['33', '34', '35', '36', '37'], (689225, '746378'): ['54', '55', '59', '60', '61'], (689225, '747107'): ['39', '40', '41', '47', '48', '49', '54'], (689225, '746460'): ['45', '49', '50', '51', '55', '56', '57'], (689225, '746458'): ['51', '52', '57', '58', '59'], (689225, '746210'): ['45', '49', '50', '51', '52', '53'], (689225, '746211'): ['58', '59', '60', '61', '62'], (689225, '745886'): ['55', '60', '61', '62', '63', '64', '65', '66'], (689225, '746694'): ['17', '18', '19', '24', '25', '26', '31', '32', '33', '34'], (689225, '746451'): ['49', '51', '52', '61', '62', '63', '68', '69', '70'], (689225, '746450'): ['59', '60', '61', '62'], (689225, '746449'): ['1', '2', '3', '4', '5', '6', '7'], (689225, '744911'): ['59', '60', '61'], (689225, '744909'): ['57', '58', '59', '60', '66', '67', '68'], (689225, '746611'): ['53', '54', '55', '57', '58'], (689225, '746446'): ['68', '69', '70'], (689225, '746444'): ['1', '2', '3', '4', '5', '6', '7'], (689225, '746443'): ['56', '57', '58', '59', '60', '64', '65', '66'], (689225, '745221'): ['66', '67', '68', '69'], (689225, '745308'): ['5', '6', '7', '8', '14', '15', '16', '21', '22', '23'], (689225, '746436'): ['1', '2', '3', '4', '8', '9', '10'], (689225, '746438'): ['1', '2', '3', '4', '5', '6'], (689225, '746431'): ['70', '71', '72'], (689225, '746837'): ['6', '7', '8', '9', '17', '18', '19'], (689225, '746754'): ['6', '7', '8', '9', '10', '11'], (689225, '746753'): ['65', '70', '71', '72', '73'], (689225, '746430'): ['53', '54', '58', '59', '60', '61'], (689225, '746426'): ['1', '2', '3', '7', '8', '9'], (689225, '745370'): ['54', '55', '56', '57', '58', '59'], (689225, '745611'): ['95', '96', '97', '98', '99', '105', '106'], (689225, '745616'): ['5', '6', '7', '8', '9'], (689225, '746423'): ['1', '2', '3', '8'], (689225, '746422'): ['1', '2', '3', '4'], (689225, '746260'): ['71', '72', '73', '74'], (689225, '746261'): ['56', '57', '58'], (689225, '746983'): ['77', '78', '79', '85', '86', '87', '88'], (689225, '746421'): ['52', '53', '54', '58', '59', '60', '61'], (689225, '746415'): ['51', '52', '53', '57', '58', '59'], (689225, '746414'): ['1', '2', '3', '8', '9', '10', '11'], (663947, '745520'): ['49', '50', '54', '55', '56'], (663947, '745517'): ['44', '45', '46', '47'], (663947, '746489'): ['51', '52', '53', '57', '58'], (663947, '746488'): ['50', '51'], (663947, '746485'): ['53', '54', '55', '59', '60', '61', '62'], (663947, '745920'): ['57', '58', '59', '64', '65', '66'], (663947, '746482'): ['61', '62', '63', '64', '65', '66'], (663947, '746481'): ['44', '45', '52', '53', '54'], (663947, '745750'): ['52', '53', '57', '58', '59'], (663947, '746638'): ['33', '34', '35', '40', '41', '42', '47', '48', '49'], (663947, '746480'): ['67', '68', '69', '70', '76', '77', '78'], (663947, '746472'): ['46', '47', '48'], (663947, '747198'): ['50', '51', '52', '57'], (663947, '746309'): ['22', '27', '28', '29', '30', '31', '39', '40', '41', '45', '46', '47', '48'], (663947, '746470'): ['62', '63', '64', '65'], (663947, '746469'): ['65', '66', '67', '71', '72', '73'], (663947, '746954'): ['60', '65', '66', '67', '68'], (663947, '746953'): ['41', '42', '43', '50', '51', '52', '59'], (663947, '745010'): ['65', '66', '67', '68'], (663947, '746466'): ['62', '63', '64'], (663947, '746464'): ['46', '47', '48', '52', '53', '54', '58', '59', '60'], (663947, '746465'): ['63', '64', '65'], (663947, '746377'): ['42', '43', '44', '49', '50', '51'], (663947, '746457'): ['64', '65', '66'], (663947, '746458'): ['38', '39', '45', '46', '47', '48', '49', '50'], (663947, '746456'): ['1', '2', '3', '4', '5', '6', '11', '12', '13', '14'], (663947, '746211'): ['63'], (663947, '746209'): ['74'], (663947, '745888'): ['68', '74', '75', '76', '77'], (663947, '746694'): ['35', '36', '37', '41', '42', '43', '44'], (663947, '746455'): ['45', '46', '50', '51', '52'], (663947, '746453'): ['49', '50', '51', '52', '56'], (663947, '746450'): ['71', '72', '73', '75'], (663947, '746449'): ['11', '12', '13', '17', '18', '19', '20'], (663947, '744911'): ['68', '69', '70'], (663947, '744910'): ['57', '58', '59', '60', '64', '65', '66'], (663947, '746614'): ['5', '6', '7', '8', '13', '14', '15'], (663947, '746609'): ['6', '7', '8', '9', '16', '17', '18', '19'], (663947, '746441'): ['1', '2', '3', '7', '8', '9', '10', '11', '17', '18'], (663947, '746440'): ['1', '2', '3', '8', '9', '10', '14', '15', '16'], (663947, '745221'): ['70'], (663947, '745226'): ['52', '56', '57', '58', '62'], (663947, '745303'): ['51', '52', '53', '54', '60', '61'], (663947, '746436'): ['72', '73', '74'], (663947, '746434'): ['43', '44', '45', '49', '50', '51', '56', '57', '58', '59'], (663947, '746837'): ['58', '59', '63', '64', '65'], (663947, '746836'): ['5', '6', '7'], (663947, '746754'): ['65', '72', '73', '74', '75', '76'], (663947, '746753'): ['77', '78', '79'], (663947, '746433'): ['75', '76', '77', '78'], (663947, '746424'): ['60', '61', '62', '63'], (663947, '745373'): ['4', '5', '6', '11', '12', '13', '14'], (663947, '745369'): ['68', '69', '70', '71'], (663947, '745611'): ['72', '73', '74', '79'], (663947, '745616'): ['60', '61'], (663947, '746425'): ['43', '48', '49', '50', '55', '56', '57'], (663947, '746423'): ['57', '58', '59', '60'], (663947, '746419'): ['40', '41', '42', '43', '49', '50'], (663947, '746261'): ['62', '63', '64', '69', '70', '71'], (663947, '746984'): ['5', '6', '7', '8', '9'], (663947, '746982'): ['4', '5', '6', '13', '14', '15', '16'], (663947, '746420'): ['48', '56', '57', '58'], (663947, '746417'): ['60', '66', '67', '68'], (656222, '744952'): ['56', '57', '58'], (656222, '745597'): ['57', '58', '59'], (656222, '746572'): ['56', '57', '58', '59', '64', '65', '66', '67', '68'], (656222, '746566'): ['47', '48', '49', '50', '57', '58', '59', '64', '65', '66'], (656222, '746560'): ['47', '48', '49', '56', '57', '58', '59'], (656222, '746073'): ['70', '72', '78', '79', '80'], (656222, '746070'): ['80', '81', '82'], (656222, '745510'): ['70', '71', '72'], (656222, '746563'): ['66', '72', '73', '74'], (656222, '746553'): ['72', '73', '74'], (656222, '746555'): ['67', '68', '69', '70'], (656222, '745420'): ['73', '74', '75', '76', '77'], (656222, '745336'): ['63', '64', '65', '66', '67'], (656222, '745661'): ['69', '70', '71', '72', '73', '74'], (656222, '745660'): ['68', '69', '70', '71'], (656222, '746556'): ['53', '54', '59', '60', '61', '67'], (656222, '746549'): ['72', '73', '74', '75', '76'], (656222, '746551'): ['60', '61', '62'], (656222, '746140'): ['63', '68', '69', '70'], (656222, '746544'): ['75', '76', '77', '78', '79', '80', '81'], (656222, '745174'): ['66', '67', '68', '72', '73'], (656222, '745166'): ['64', '65', '66'], (656222, '745899'): ['56', '62', '63', '64', '65'], (656222, '746547'): ['68', '69', '70', '71'], (656222, '746542'): ['59', '60', '65', '66', '67'], (656222, '746538'): ['71', '72', '73', '74'], (656222, '746541'): ['65', '71', '72', '73'], (656222, '746536'): ['56', '57', '58', '59', '60', '61'], (656222, '746778'): ['59', '60', '61', '62', '63'], (656222, '746779'): ['96', '97', '98', '99', '103', '104', '105'], (656222, '746537'): ['62', '63', '64', '65'], (656222, '746532'): ['66', '70', '71', '72'], (656222, '746533'): ['68', '69', '70', '71', '72'], (656222, '746689'): ['72', '73', '74', '75', '76', '77'], (656222, '745798'): ['68', '69', '70', '71'], (656222, '745797'): ['69', '70', '71', '72'], (656222, '746531'): ['57', '65', '66', '67'], (656222, '746528'): ['62', '63', '64'], (656222, '746526'): ['66', '67', '68', '69', '70'], (656222, '746363'): ['46', '47', '48'], (656222, '745471'): ['48', '49', '50'], (656222, '745474'): ['34', '35', '36', '37'], (656222, '745468'): ['63', '64', '65', '66'], (656222, '745469'): ['64', '65', '66'], (656222, '745466'): ['56'], (656222, '746114'): ['50', '51', '52', '53', '58', '59'], (656222, '745383'): ['57', '58', '59'], (656222, '745464'): ['74', '75', '76'], (656222, '744976'): ['63', '64'], (656222, '745457'): ['32', '33', '38', '39', '40'], (656222, '745459'): ['53', '55', '56'], (656222, '745461'): ['39', '40', '41'], (656222, '745452'): ['93', '94', '95', '96'], (656222, '746590'): ['49', '50', '51'], (656222, '746833'): ['52', '53'], (656222, '746835'): ['33', '34', '35', '39', '40', '41', '42', '43', '44'], (656222, '745455'): ['44', '48', '49', '50'], (656222, '745453'): ['60', '61', '62'], (656222, '745450'): ['67', '68'], (656222, '745448'): ['63', '64', '65'], (656222, '745130'): ['60', '61', '62', '63', '64'], (656222, '745121'): ['64', '65', '66', '67', '68', '69', '70'], (656222, '746660'): ['51', '52', '53', '54', '55', '56'], (656222, '745443'): ['70', '71', '72', '73', '74'], (656222, '745685'): ['73', '74', '75', '76'], (666214, '746478'): ['49', '53', '54', '55', '56'], (666214, '746639'): ['53', '54', '55', '61', '62', '63', '64'], (666214, '746640'): ['54', '59', '60', '61', '62'], (666214, '746476'): ['47', '48', '49', '50'], (666214, '747201'): ['67', '68', '69', '70', '71', '75', '76', '77', '78'], (666214, '746307'): ['43', '44', '45', '46', '47'], (666214, '746309'): ['62', '63', '64', '65', '66', '70', '71', '72'], (666214, '746469'): ['44', '45', '52', '53', '54'], (666214, '746468'): ['35', '37', '41', '42', '43', '44', '49', '50', '51'], (666214, '746955'): ['42', '48', '49', '50', '51', '52'], (666214, '745007'): ['4', '5', '6', '7', '8', '9', '13', '14', '15', '16', '17', '18', '22', '23', '24', '25'], (666214, '746463'): ['66', '67', '68'], (666214, '746462'): ['50', '54', '55', '56', '61', '62', '63'], (666214, '746381'): ['89', '90', '91'], (666214, '747106'): ['69', '70', '71'], (666214, '746459'): ['45', '46', '47', '48', '49'], (666214, '746457'): ['73', '74', '75'], (666214, '746456'): ['60', '61', '62', '66', '67', '68', '69'], (666214, '746210'): ['58', '59', '60'], (666214, '745890'): ['42', '48', '49', '50', '51', '52', '56', '57', '58', '59', '60', '61', '62'], (666214, '746451'): ['23', '24', '25', '31', '32', '33', '34', '35', '40', '41', '42', '47', '48'], (666214, '746447'): ['49', '50', '51'], (666214, '746614'): ['34', '35', '36', '41', '42', '43', '44', '45', '49', '50', '51'], (666214, '746448'): ['43', '44', '45', '46', '47', '52', '53', '54', '55', '62', '63', '64'], (666214, '746441'): ['60', '61', '62', '63'], (666214, '746443'): ['70', '71', '72'], (666214, '745304'): ['62', '63', '64', '65', '66'], (666214, '746435'): ['62', '63', '64', '68', '69', '70', '75', '76', '77'], (666214, '746837'): ['73', '74', '75'], (666214, '746836'): ['54', '55', '56', '57', '58', '59', '60'], (666214, '746756'): ['66', '71', '72', '73', '74'], (666214, '746432'): ['40', '41', '42', '47', '48', '49', '50'], (666214, '745451'): ['76', '77', '78', '79', '80'], (666214, '745446'): ['17', '18', '19', '20', '25', '26', '27', '32', '33', '34'], (666214, '745447'): ['36', '37', '38', '39', '45', '46', '47', '48'], (666214, '745121'): ['45'], (666214, '746661'): ['36', '37', '38', '43', '44', '45', '46', '50', '51', '52', '53'], (666214, '745441'): ['58', '59', '64', '65', '66'], (666214, '745692'): ['40', '41', '42', '43', '49'], (666214, '745686'): ['51', '52', '53', '57', '58'], (642701, '744951'): ['60', '61', '62'], (642701, '745758'): ['43', '44', '49', '50', '51'], (642701, '745759'): ['64', '65', '66', '67', '68', '69', '70'], (642701, '745754'): ['58', '59'], (642701, '745755'): ['58', '59', '60', '61', '66'], (642701, '746000'): ['57', '58', '59'], (642701, '747046'): ['49', '54', '55', '56'], (642701, '745756'): ['64', '65', '66', '67'], (642701, '745751'): ['61', '62', '63', '64'], (642701, '745748'): ['59', '60', '61', '62'], (642701, '745096'): ['48', '49', '55', '56', '57', '58'], (642701, '745744'): ['71', '72', '73'], (642701, '745742'): ['44', '45', '46', '47', '48', '49'], (642701, '745415'): ['55', '56', '60', '61', '62'], (642701, '745413'): ['53', '54'], (642701, '745330'): ['45', '46', '50', '51', '52', '53', '54', '55'], (642701, '745740'): ['53', '54', '55', '56', '57', '61', '62', '63', '64'], (642701, '745737'): ['79', '80', '81'], (642701, '745735'): ['64', '65', '71', '72', '73', '74', '75', '76'], (642701, '745168'): ['67', '68'], (642701, '746547'): ['72', '73', '74', '75', '76', '77', '78', '79', '80'], (642701, '745488'): ['60', '61', '62', '63'], (642701, '746696'): ['44', '45', '46'], (642701, '745482'): ['69', '70', '71', '72', '73'], (642701, '745484'): ['67', '68', '69', '70', '71'], (642701, '745476'): ['22', '23', '24', '25', '30', '31', '32'], (642701, '745968'): ['69', '70', '71', '72', '73', '78', '79', '80', '81'], (642701, '746766'): ['72', '73', '74'], (642701, '746769'): ['79', '80', '81', '82', '83'], (642701, '745481'): ['52', '53', '54', '55', '56'], (642701, '745470'): ['37', '42', '43', '44'], (642701, '747171'): ['28', '29', '33', '34', '35', '36'], (642701, '746365'): ['50', '51', '52', '56', '57', '58'], (642701, '745471'): ['76', '77', '78', '79', '80'], (642701, '745474'): ['31', '32', '33'], (642701, '745467'): ['67', '68', '69', '70', '71'], (642701, '745466'): ['53', '54', '55'], (642701, '746114'): ['41', '42', '43', '44', '49'], (642701, '745383'): ['60', '61'], (642701, '745463'): ['52', '53', '54', '62', '63', '64', '65'], (642701, '744976'): ['55', '56', '57', '62'], (642701, '745460'): ['43', '44', '45', '52', '53', '54'], (642701, '745459'): ['57', '58', '62', '63', '64'], (642701, '745461'): ['67', '68', '69'], (642701, '745452'): ['81'], (642701, '746590'): ['52', '60', '61', '62'], (642701, '746833'): ['70', '71', '72'], (642701, '745451'): ['60', '61', '62', '63', '64'], (642701, '745456'): ['66', '67', '68'], (642701, '745454'): ['58', '59', '60'], (642701, '745446'): ['63', '64', '65', '66', '67'], (642701, '745449'): ['55', '60', '61', '62', '63'], (642701, '745122'): ['60', '61', '62', '63', '64'], (642701, '745120'): ['62', '63', '64'], (642701, '746657'): ['58', '59', '60'], (642701, '745442'): ['52', '53', '54'], (642701, '745692'): ['63', '64', '65'], (642701, '745685'): ['62', '63', '64', '65', '66'], (547973, '745596'): ['69', '70', '71', '72'], (547973, '745594'): ['58', '59', '60', '61'], (547973, '745840'): ['66', '67', '68', '69'], (547973, '745514'): ['65', '66', '67', '68', '69'], (547973, '745519'): ['62', '63', '64', '65'], (547973, '745512'): ['67', '68', '69', '70', '71'], (547973, '745347'): ['59', '60', '61', '62', '63'], (547973, '745670'): ['58', '59', '60'], (547973, '745504'): ['59', '60', '61'], (547973, '745507'): ['54', '55', '56', '57'], (547973, '745505'): ['69', '70', '71', '72', '73'], (547973, '745499'): ['78', '79', '80', '81', '82', '83'], (547973, '746881'): ['56', '57', '58'], (547973, '746880'): ['65', '66', '67'], (547973, '745502'): ['62', '63', '64'], (547973, '745500'): ['67', '68', '69'], (547973, '745498'): ['57', '58', '59', '60', '61', '62'], (547973, '746468'): ['55', '56', '57', '58'], (547973, '744928'): ['57', '58', '59', '60', '61'], (547973, '745497'): ['57', '58', '59', '60', '61'], (547973, '745494'): ['56', '57', '58', '59', '60'], (547973, '745496'): ['49', '50', '51'], (547973, '745491'): ['74', '75', '76', '77'], (547973, '745170'): ['47', '48'], (547973, '746545'): ['61', '62', '63', '64', '65'], (547973, '745490'): ['51', '52', '53'], (547973, '745486'): ['64', '65', '66', '67', '68'], (547973, '746695'): ['67', '68', '69'], (547973, '747104'): ['71', '72', '73', '74'], (547973, '747101'): ['69', '70', '71', '72', '73'], (547973, '745487'): ['66', '67', '68', '72', '73', '74', '75', '76'], (547973, '745477'): ['64', '65', '66', '67', '68', '69', '70'], (547973, '745967'): ['66', '67', '68'], (547973, '746766'): ['61', '62', '63', '64'], (547973, '745478'): ['55', '56', '57'], (547973, '745473'): ['57', '58', '59'], (547973, '745472'): ['70', '71', '72', '73'], (547973, '747173'): ['64', '65', '66'], (547973, '746360'): ['62', '63', '64', '65', '66'], (547973, '745471'): ['60', '61', '62', '63'], (547973, '745474'): ['58', '59', '60', '61'], (547973, '745469'): ['70', '71', '72', '73', '74'], (547973, '745466'): ['62', '63', '64'], (547973, '746114'): ['69', '70', '71', '72'], (547973, '745462'): ['60', '61', '62'], (547973, '744978'): ['54', '55'], (547973, '744973'): ['61', '62', '63'], (547973, '745460'): ['60', '61', '62', '63', '64'], (547973, '745459'): ['69', '70', '71'], (547973, '745452'): ['68', '69', '76', '77', '78', '79', '80'], (547973, '746590'): ['68', '69', '70'], (547973, '746833'): ['61', '62', '63', '64'], (547973, '745451'): ['81', '82'], (547973, '745456'): ['76', '77', '78', '79', '80', '81', '82'], (547973, '745454'): ['65', '66', '67', '68', '69', '70'], (547973, '745446'): ['72', '73', '74', '75', '76', '77'], (547973, '745449'): ['75', '76', '77'], (547973, '745120'): ['69', '70', '71', '72'], (547973, '746657'): ['67', '68', '69'], (547973, '745442'): ['59', '60', '61'], (547973, '745692'): ['70', '71', '72'], (547973, '745685'): ['77', '78'], (670280, '745517'): ['62', '63', '64', '65', '66', '67', '68'], (670280, '745596'): ['76', '77', '78'], (670280, '745836'): ['61', '62', '63', '64', '65', '66'], (670280, '745514'): ['73', '74', '75'], (670280, '745519'): ['71', '72', '73', '74'], (670280, '745516'): ['62', '63', '64'], (670280, '745347'): ['69', '70', '71'], (670280, '745350'): ['74', '75', '76', '77', '78'], (670280, '745670'): ['66', '67', '68', '69', '70'], (670280, '745511'): ['56', '57', '58'], (670280, '745504'): ['65', '66', '67', '68'], (670280, '745505'): ['84', '85', '86', '87', '88'], (670280, '745499'): ['67', '68', '69', '70', '71'], (670280, '745991'): ['83', '84', '85'], (670280, '746881'): ['71', '72', '73'], (670280, '746882'): ['61', '62', '63'], (670280, '746880'): ['73', '74', '75', '76'], (670280, '745503'): ['83', '84', '85'], (670280, '745500'): ['73', '74', '75'], (670280, '745493'): ['68', '69', '70', '71', '72'], (670280, '746468'): ['74', '75', '76', '77'], (670280, '744928'): ['65', '66', '67', '68'], (670280, '745497'): ['65', '66', '67'], (670280, '745494'): ['77', '78', '79'], (670280, '745496'): ['63', '64', '65', '66'], (670280, '745492'): ['67', '68', '69', '70'], (670280, '745170'): ['64', '65', '66', '67', '68', '69'], (670280, '746545'): ['69', '70', '71', '72'], (670280, '745490'): ['66', '67', '68'], (670280, '745489'): ['59', '60', '61'], (670280, '746774'): ['68', '69', '70'], (670280, '745478'): ['64', '65', '66', '67'], (670280, '745473'): ['65', '66', '67', '68'], (670280, '747173'): ['55', '56', '57', '58'], (670280, '746360'): ['73', '74', '75', '76', '77', '78', '79'], (670280, '745474'): ['65', '66', '67', '68', '69', '70'], (670280, '745469'): ['80', '81', '82', '83', '84', '85'], (670280, '745466'): ['69', '70', '71', '72', '73', '74'], (670280, '746114'): ['76', '77', '78', '79', '83', '84', '85', '86', '87'], (670280, '745462'): ['67', '68', '69', '70', '71'], (670280, '745463'): ['72', '73', '74'], (670280, '744978'): ['62', '63', '64'], (670280, '744973'): ['67', '68', '69', '70', '71'], (670280, '745460'): ['69', '70', '71', '72', '73'], (670280, '745461'): ['75', '76', '77'], (670280, '745452'): ['86', '87', '88', '89', '90', '91', '92'], (670280, '746592'): ['63', '64', '65', '66'], (670280, '746834'): ['71', '72', '73'], (670280, '745455'): ['63', '64', '65', '66', '67', '68'], (670280, '745446'): ['56', '57', '58'], (670280, '745450'): ['62', '63', '64', '65', '66'], (670280, '745448'): ['72', '73', '74', '75', '76'], (670280, '745449'): ['68', '69', '70'], (670280, '745121'): ['60', '61', '62', '63'], (670280, '746660'): ['62', '63', '64', '65'], (670280, '745441'): ['71', '72', '73'], (670280, '745443'): ['53', '54', '55', '56'], (670280, '745685'): ['47', '48', '49', '50'], (693312, '745510'): ['63', '64', '65', '66', '67'], (693312, '745508'): ['72', '73', '74'], (693312, '745505'): ['42', '43', '44', '45', '46', '47'], (693312, '745499'): ['83', '84'], (693312, '745989'): ['54', '55', '56', '57', '61', '62', '63'], (693312, '745503'): ['62', '63', '64', '65', '66'], (693312, '745502'): ['87'], (693312, '745498'): ['66', '67', '68', '69', '70', '71'], (693312, '746467'): ['64', '65', '66', '67'], (693312, '744928'): ['101', '102', '103'], (693312, '744929'): ['72', '73', '74', '75'], (693312, '745495'): ['57', '58', '59', '60', '61', '62', '68', '69', '70', '75', '76'], (693312, '745491'): ['47', '48', '49', '50'], (693312, '745171'): ['56', '57', '58', '63', '64', '65'], (693312, '746547'): ['43', '51', '52', '53', '54', '55', '56'], (693312, '746546'): ['68', '69', '70', '75', '76', '77', '78', '79', '80'], (693312, '745485'): ['65', '66', '67'], (693312, '746695'): ['53', '54', '55', '56', '60', '61', '62'], (693312, '747104'): ['77'], (693312, '747101'): ['47', '48', '49', '54', '55', '56', '57', '58'], (693312, '745487'): ['39', '40', '41', '42'], (693312, '745484'): ['51', '52', '53'], (693312, '745477'): ['71'], (693312, '746766'): ['47', '54', '55', '56'], (693312, '746769'): ['52', '53', '54'], (693312, '745481'): ['37', '38', '42', '43', '44', '45', '46', '47'], (693312, '745470'): ['48', '49', '50', '51', '52', '53', '54'], (693312, '747174'): ['57', '58', '59', '61', '67', '68', '69', '70'], (693312, '746365'): ['66', '67', '68', '69', '70'], (693312, '746363'): ['63', '64', '65'], (693312, '745474'): ['41', '42', '43', '50', '51', '52', '53'], (693312, '745467'): ['33', '34', '35', '36', '40', '41', '42', '43', '48', '49', '50', '51', '52'], (693312, '745466'): ['75', '76'], (693312, '746114'): ['60', '61'], (693312, '745383'): ['50', '51', '52'], (693312, '745382'): ['59', '60', '61', '62', '63'], (693312, '745462'): ['51', '52', '53'], (693312, '745463'): ['46'], (693312, '744973'): ['53', '54', '55'], (693312, '745457'): ['61', '62', '63'], (693312, '745459'): ['50', '51', '52'], (693312, '745452'): ['56', '57', '58', '59', '63', '64', '65', '66', '67'], (693312, '746592'): ['56', '57', '58', '59'], (693312, '746835'): ['50', '51', '52', '53', '54', '55', '56'], (693312, '745451'): ['68', '69', '70', '71'], (693312, '745455'): ['56', '57', '58', '59'], (693312, '745446'): ['35', '36', '37', '45', '46', '47', '48', '49'], (693312, '745447'): ['55', '56', '57', '62', '63', '64', '65'], (693312, '745130'): ['59'], (670059, '745515'): ['70', '71', '72', '73', '74'], (670059, '745519'): ['53', '54', '55'], (670059, '745516'): ['55', '56', '57', '58'], (670059, '745512'): ['60', '61'], (670059, '745350'): ['50', '51', '52', '53'], (670059, '745510'): ['54', '55', '56', '57'], (670059, '745511'): ['50', '51', '52'], (670059, '745507'): ['61', '62', '63'], (670059, '745505'): ['50', '51', '52', '60', '61', '62', '63'], (670059, '746881'): ['62', '63', '64', '65', '66', '67'], (670059, '746882'): ['56', '57'], (670059, '746880'): ['57', '58', '59'], (670059, '745502'): ['65', '66', '67', '71', '72', '73', '74'], (670059, '745493'): ['53', '60', '61', '62', '63'], (670059, '746468'): ['65', '66', '67', '68'], (670059, '744928'): ['49', '50', '51', '52', '53'], (670059, '745497'): ['49', '50', '51', '52'], (670059, '745494'): ['73'], (670059, '745496'): ['55', '56', '57'], (670059, '745492'): ['56', '57', '58'], (670059, '745170'): ['52', '53', '54', '55', '56'], (670059, '746545'): ['52', '53', '54', '55'], (670059, '745490'): ['58', '59', '60', '61'], (670059, '745489'): ['52', '53', '54'], (670059, '745486'): ['56', '57', '58', '59', '60'], (670059, '745485'): ['53', '54', '55', '56', '57', '58', '59'], (670059, '747105'): ['65', '66', '67', '68'], (670059, '747104'): ['63', '64', '65'], (670059, '745487'): ['56', '57', '58', '59', '60', '61'], (670059, '745484'): ['59', '60', '61', '62', '63'], (670059, '745477'): ['51', '52', '53', '54', '55', '56'], (670059, '745967'): ['57', '58', '59', '60', '61', '62'], (670059, '746774'): ['59', '60', '61', '62'], (670059, '746769'): ['73', '74', '75'], (670059, '745481'): ['70', '71', '72'], (670059, '745472'): ['63', '64', '65', '66'], (670059, '747173'): ['76', '77', '78', '79', '80', '81'], (670059, '746363'): ['49', '50'], (670059, '745471'): ['67', '68', '69', '70', '71'], (670059, '745468'): ['53', '54', '55', '56', '57', '58'], (670059, '745469'): ['92', '93', '94', '95', '96', '97'], (670059, '746834'): ['48', '49', '50'], (670059, '745456'): ['83', '84', '85'], (670059, '745454'): ['52', '53', '54'], (670059, '745450'): ['56', '57', '58'], (670059, '745449'): ['49', '50', '51', '52', '53', '54'], (670059, '745130'): ['52', '53', '54', '55'], (670059, '745121'): ['53', '54', '55', '56'], (670059, '746657'): ['42', '43', '44', '45'], (670059, '745443'): ['47', '48', '49'], (670059, '745692'): ['50', '51'], (670059, '745686'): ['59', '64', '65', '66']}

# Obtain all pitch sequencing for all relievers
* Now that we have all the needed IDs, we can iterate over all hitters faced by the 210 relievers in 2024 and obtain the pitch-by-pitch sequences for each one
* Each sequence consists of:
    * Pitch order
    * Count before each pitch
    * All pitch types
    * Velocities of each pitch
    * Outcome of each pitch (called strike, hit, ground out etc.)
    * Hitter name
    * Inning

In [103]:
# Store the info mentioned in the above cell for all matchups
data = []

# For all relievers and games they appeared in:
for reliever, appearances in games.items():
    # Iterate over all games
    for game_id in appearances:
        # Get the IDs of all at-bats for the current reliever 
        at_bat_ids = at_bats.get((reliever, game_id), [])
        # Loop through each at-bat ID for the current reliever in the current game
        for at_bat_id in at_bat_ids:
            
            # Navigate to the page
            url = f'https://baseballsavant.mlb.com/visuals/pitch3d?player_id={reliever}#v=1&mainView=tracking&pov=umpire&g1={game_id}&g2={first_games[reliever]}&hitterSide=all&marks=none&plays1={at_bat_id}&plays2=all&pitchColors=pitch_type&dualActive=false&summaryViewActive=false&chartYear=2024&summaryYear=2024' 
            driver.get(url)
            driver.refresh()

            # Wait for the pitch list w/ pitch-by-pitch data to be visible 
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//div[@class='pitchList']")))
            pitch_list = driver.find_element(By.CLASS_NAME, "pitchList")

            # Extract the hitter's name, bat side, inning, and number of outs
            hitter_name = driver.find_element(By.XPATH, "//div[@class='pitchList']//span[@class='name']").text
            bat_side = driver.find_element(By.XPATH, "//div[@class='pitchList']//span[@class='batSide']").text
            inning = driver.find_element(By.XPATH, "//div[@class='pitchList']//span[@class='inningTB']").text
            outs_text = driver.find_element(By.XPATH, "//div[@class='pitchList']//span[not(@class)]").text
            
            # Iterate through the pitches and extract all contextual data
            pitches = pitch_list.find_elements(By.XPATH, ".//ul/li")
            for pitch in pitches:
                pitch_num = pitch.find_element(By.CLASS_NAME, "pitchNum").text
                pitch_type = pitch.find_element(By.CLASS_NAME, "pitchType").text
                count = pitch.find_element(By.CLASS_NAME, "count").text
                velocity = pitch.find_element(By.CLASS_NAME, 'pitch').text.split()[1]
                outcome = pitch.find_element(By.CLASS_NAME, 'outcome').text

                # Add the extracted data for this at-bat as a dictionary to the list
                data.append({
                    "game_id": game_id,
                    "reliever": reliever,
                    "hitter_name": hitter_name,
                    "bat_side": bat_side,
                    "inning": inning,
                    "pitch_number": pitch_num,
                    "count": count,
                    "pitch_type": pitch_type,
                    "velocity": velocity,
                    "outcome": outcome
                })

# Close the driver after finishing
driver.quit()

# Convert the data list to a pandas data frame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('pitch_data.csv', index=False)

print("Data saved to pitch_data.csv")

Data saved to pitch_data.csv


## Add unique IDs so we can distinguish between at-bats

In [104]:
# Load the pitch sequencing CSV we just created
df = pd.read_csv('pitch_data.csv')

# Ensure the data is sorted properly before assigning at_bat_id
df = df.sort_values(by=['game_id', 'reliever', 'hitter_name', 'inning', 'pitch_number'])

# Initialize at-bat tracking
at_bat_id = 0
previous_game = None
previous_reliever = None
previous_hitter = None
previous_inning = None

# List to store unique IDs
at_bat_ids = []

# Loop over the rows
for index, row in df.iterrows():
    # If it's a new at-bat, increment the ID
    if (row['pitch_number'] == 1 or 
        row['game_id'] != previous_game or 
        row['reliever'] != previous_reliever or 
        row['hitter_name'] != previous_hitter or 
        row['inning'] != previous_inning):
        at_bat_id += 1  

    # Label the current at-bat with this unique ID
    at_bat_ids.append(at_bat_id)

    # Update previous values for tracking
    previous_game = row['game_id']
    previous_reliever = row['reliever']
    previous_hitter = row['hitter_name']
    previous_inning = row['inning']

# Add the new ID column
df['at_bat_id'] = at_bat_ids

# Save the updated DataFrame back to a new CSV
df.to_csv('pitch_data_ids.csv', index=False)

print("CSV updated successfully with at_bat_id!")

CSV updated successfully with at_bat_id!
